<a href="https://colab.research.google.com/github/fbeilstein/machine_learning/blob/master/workbook_07_naive_bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Monty Hall

Use the following demonstration to tackle the Monty Hall paradox.
 * Do enough trials to proove to yourself that "switch" and "stay" strategies give inherently different outcomes
 * Draw graph of all events and calculate probabilities
 * Reobtain your result with Bayes theorem

In [ ]:
#@title Visualization code

import IPython
from google.colab import output

main_str = '''
<html>
<head>
<style type="text/css">
img {
        height: 450px;
        width: 220px;
        float:left;
}
</style>
<script type="text/javascript">

tempImg = new Image()
tempImg.src = "https://raw.githubusercontent.com/fbeilstein/machine_learning/master/lecture_07_naive_bayes/auto.png"

var state = 0;
var prize = Math.floor(Math.random() * 3) + 1;
var stay = 0;
var change = 0;

var stay_correct = 0;
var stay_incorrect = 0;
var change_correct = 0;
var change_incorrect = 0;

function click_img(num) {
  if (state == 0) {
    let intersection = [1, 2, 3].filter(x => ![num, prize].includes(x));
    var open = intersection[Math.floor(Math.random() * intersection.length)];
    stay = num;
    change = [1, 2, 3].filter(x => ![open, stay].includes(x))[0];
    //document.getElementById("stay_incorrect").innerHTML = "c:" + change + ", o:" + open + ",s:" + stay + ",P:" + prize + "," + intersection;
    
    document.getElementById("d" + open).src = "https://raw.githubusercontent.com/fbeilstein/machine_learning/master/lecture_07_naive_bayes/goat.png";
    document.getElementById("d" + stay).src = "https://raw.githubusercontent.com/fbeilstein/machine_learning/master/lecture_07_naive_bayes/door_click_to_stay.png";
    document.getElementById("d" + change).src = "https://raw.githubusercontent.com/fbeilstein/machine_learning/master/lecture_07_naive_bayes/door_click_to_swap.png";
    state = 1;
  } else if (state == 1 && (num == stay || num == change)) {
    for (var i = 1; i < 4; ++i)
      document.getElementById("d" + i).src = "https://raw.githubusercontent.com/fbeilstein/machine_learning/master/lecture_07_naive_bayes/goat.png";
    document.getElementById("d" + prize).src = "https://raw.githubusercontent.com/fbeilstein/machine_learning/master/lecture_07_naive_bayes/auto.png";
    if (num == prize && num == stay) stay_correct += 1;
    if (num == prize && num == change) change_correct += 1;
    if (num != prize && num == stay) stay_incorrect += 1;
    if (num != prize && num == change) change_incorrect += 1;

    document.getElementById("switch_correct").innerHTML = change_correct;
    document.getElementById("stay_correct").innerHTML = stay_correct;
    document.getElementById("switch_incorrect").innerHTML = change_incorrect;
    document.getElementById("stay_incorrect").innerHTML = stay_incorrect;
    document.getElementById("switch_proportion").innerHTML = Math.round((change_correct/(change_correct + change_incorrect + 0.000001))*1000)/1000;
    document.getElementById("stay_proportion").innerHTML = Math.round((stay_correct/(stay_correct + stay_incorrect + 0.000001))*1000)/1000;
    state = 2;
  } else if (state == 2) {
    for (var i = 1; i < 4; ++i)
      document.getElementById("d" + i).src = "https://raw.githubusercontent.com/fbeilstein/machine_learning/master/lecture_07_naive_bayes/door.png";
    prize = Math.floor(Math.random() * 3) + 1;
    state = 0;
  }
}

</script>
</head>

<body onload="getImages()">

<table>
<tr>
  <td><img src="https://raw.githubusercontent.com/fbeilstein/machine_learning/master/lecture_07_naive_bayes/door.png" id = "d1" onclick="click_img(1)"></td>
  <td><img src="https://raw.githubusercontent.com/fbeilstein/machine_learning/master/lecture_07_naive_bayes/door.png" id = "d2" onclick="click_img(2)"></td>
  <td><img src="https://raw.githubusercontent.com/fbeilstein/machine_learning/master/lecture_07_naive_bayes/door.png" id = "d3" onclick="click_img(3)"></td>
<tr/>
</table>
<table>
<tr>
  <td><b>Switch Trials</b></td><td></td>
  <td><b>Stay Trials</b></td><td></td>
</tr>
<tr>
  <td>Number Correct:</td>
  <td id="switch_correct">0</td>
  <td>Number Correct:</td>
  <td id="stay_correct">0</td>
<tr/>
<tr>
  <td>Number Incorrect:</td>
  <td id="switch_incorrect">0</td>
  <td>Number Incorrect:</td>
  <td id="stay_incorrect">0</td>
<tr/>
<tr>
  <td>Proportion Correct:</td>
  <td id="switch_proportion">0</td>
  <td>Proportion Correct:</td>
  <td id="stay_proportion">0</td>
<tr/>
</table>
</body>
</html>
'''



def generate_html():
  s = main_str
  return s
  
display(IPython.display.HTML(generate_html()))

BTW, that's how the 'doors' looked like in real life.

![](https://raw.githubusercontent.com/fbeilstein/machine_learning/master/lecture_07_naive_bayes/monty_hall.png)


#1D Naive Bayes

Use the following demo to uderstand how 1-dimensional naive Bayes works.
Be sure that you understand
* How red and blue points sample?
* How parameters of gaussians influence their positions?
* How the real line should be dissected in an ideal case?
* How many regions can there be (max, min)?
* Why Bayes sometime misses the "ideal" positions to cut?
* How number of points in samples influences the position of cut(s)?

In [ ]:
#@title Gaussian Naive Bayes for two classes

%%html

<html>

<head>
  <script src="https://cdn.plot.ly/plotly-latest.min.js"></script> 

  <script type="application/javascript">

   function random_point(mu, sigma) {
     var z = Math.sqrt(-2.0 * Math.log(Math.random())) * Math.cos(2.0 * Math.PI * Math.random());
     return z * sigma + mu;
   }

   function get_params(pts) {
     //console.log(pts);
     var mu = 0.0;
     for (var i = 0; i < pts.length; ++i)
        mu += pts[i];
     mu /= pts.length;
     var sigma = 0.0;
     for (var i = 0; i < pts.length; ++i)
        sigma += (pts[i] - mu) * (pts[i] - mu);
     sigma /= pts.length;
     sigma = Math.sqrt(sigma);
     return [mu, sigma];
   }

    var xs = [];
    var ys = [];
    var x_min = -5.0;
    var y_min =  0.0;
    var x_max = +5.0;
    var y_max = +1.0;
    for (var x = x_min; x <= x_max; x += 0.01)
      xs.push(x);
    for (var y = y_min; y <= y_max; y += 0.1)
      ys.push(y);
    var colors = ['blue', 'red'];
    var gen_nums = [0, 0];


    function loss_function_2d() {
      var canvas = document.getElementById("paint");
      var w = canvas.width;
      var h = canvas.height;

      mu = [];
      sigma = [];
      C = Math.sqrt(2 * Math.PI);
      for (var i = 0; i < pts.length; ++i) {
        var x = pts[i][0] / w * (x_max - x_min) + x_min;
        var y = (1.0 - pts[i][1] / h) * (y_max - y_min) + y_min;
        mu.push(x);
        sigma.push(1.0 / (y * C));
      }

      data = [];
      for (var trace_idx = 0; trace_idx < pts.length; ++trace_idx) {
        var trace = [];
        for (var i = 0; i < xs.length; ++i) {
            var delta = (xs[i] - mu[trace_idx]) / sigma[trace_idx]; 
            trace.push(Math.exp(-delta*delta / 2.0) / sigma[trace_idx] / C);
        }
        var name = ['true PDF for blue', 'true PDF for red'];
        data.push({
            x: xs,
            y: trace,
            showscale: false, 
            opacity: 1.0, 
            type: 'scatter',
            mode: 'line',
            name: name[trace_idx],
            line: {
              width: 3,
              color: colors[trace_idx]
            },
          });
      }

      var estimated_gaussians = [];
      var generated_pts = [];
      for (var trace_idx = 0; trace_idx < pts.length; ++trace_idx) {
        trace_x = [];
        trace_y = [];
        for (var i = 0; i < gen_nums[trace_idx]; ++i) {
            trace_x.push(random_point(mu[trace_idx], sigma[trace_idx]));
            trace_y.push(0.01);
        }
        data.push({
            x: trace_x,
            y: trace_y,
            showscale: false, 
            opacity: 0.5, 
            type: 'scatter',
            mode: 'markers',
            name: ['blue class datapoints', 'red class datapoints'][trace_idx],
            marker: {
              width: 20,
              color: colors[trace_idx]
            },
          });
        estimated_gaussians.push(get_params(trace_x));
        generated_pts.push(trace_x);
      }
      //console.log(estimated_gaussians);

      for (var trace_idx = 0; trace_idx < pts.length; ++trace_idx) {
        var trace = [];
        for (var i = 0; i < xs.length; ++i) {
            var delta = (xs[i] - estimated_gaussians[trace_idx][0]) / estimated_gaussians[trace_idx][1]; 
            trace.push(Math.exp(-delta*delta / 2.0) / estimated_gaussians[trace_idx][1] / C);
        }
        var name = ['estimated PDF for blue', 'estimated PDF for red'];
        data.push({
            x: xs,
            y: trace,
            showscale: false, 
            opacity: 1.0, 
            type: 'scatter',
            mode: 'line',
            name: name[trace_idx],
            line: {
              dash: 'dash',
              width: 1,
              color: colors[trace_idx]
            },
          });
      }

      p_red = [];
      p_blue = [];
      var prior_blue = gen_nums[0] / (gen_nums[0] + gen_nums[1]);
      var prior_red  = gen_nums[1] / (gen_nums[0] + gen_nums[1]);
      for (var i = 0; i < xs.length; ++i) {
        var delta = (xs[i] - estimated_gaussians[0][0]) / estimated_gaussians[0][1]; 
        var like_blue = Math.exp(-delta*delta / 2.0) / estimated_gaussians[0][1] / C;
        delta = (xs[i] - estimated_gaussians[1][0]) / estimated_gaussians[1][1]; 
        var like_red = Math.exp(-delta*delta / 2.0) / estimated_gaussians[1][1] / C;
        var prob_blue = like_blue * prior_blue / (like_blue * prior_blue + like_red * prior_red);
        var prob_red = like_red * prior_red / (like_blue * prior_blue + like_red * prior_red);
        p_blue.push(prob_blue);
        p_red.push(prob_red);
      }
     //console.log(p_red);
      data.push({
           x: xs,
           y: p_red,
           showscale: false, 
           opacity: 1.0, 
           type: 'scatter',
           mode: 'lines',
           name: 'estimated red probability',
           line: {
             width: 1,
             color: colors[1]
           },
         });
      data.push({
           x: xs,
           y: p_blue,
           showscale: false, 
           opacity: 1.0, 
           type: 'scatter',
           mode: 'lines',
           name: 'estimated red probability',
           line: {
             width: 1,
             color: colors[0]
           },
         });

      var ys = [0.0, 1.0];
      var zs = [];
      for (var i = 0; i < xs.length; ++i) {
        if (p_red[i] > p_blue[i])
            zs.push(0.0);
        else
            zs.push(1.0);
      }
      data.push({
           x: xs,
           y: ys,
           z: [zs, zs],
           showscale: false, 
           opacity: 0.2, 
           colorscale: [[0, 'red'], [1, 'blue']],
           type: 'heatmap'
         });

       
       for (var trace_idx = 0; trace_idx < pts.length; ++trace_idx) {
         var tp = 0;
         var fp = 0;
         for (var i = 0; i < generated_pts[trace_idx].length; ++i) {
            var x = generated_pts[trace_idx][i];
            var delta = (x - estimated_gaussians[0][0]) / estimated_gaussians[0][1]; 
            var like_blue = Math.exp(-delta*delta / 2.0) / estimated_gaussians[0][1] / C;
            delta = (x - estimated_gaussians[1][0]) / estimated_gaussians[1][1]; 
            var like_red = Math.exp(-delta*delta / 2.0) / estimated_gaussians[1][1] / C;
            if (trace_idx == 0 && prior_blue * like_blue >= prior_red * like_red)
                tp += 1;
            if (trace_idx == 0 && prior_blue * like_blue < prior_red * like_red)
                fp += 1;
            if (trace_idx == 1 && prior_blue * like_blue <= prior_red * like_red)
                tp += 1;
            if (trace_idx == 1 && prior_blue * like_blue > prior_red * like_red)
                fp += 1;
         }
         var elem = document.getElementById(['btp', 'rtp'][trace_idx]);
         elem.innerHTML = tp;
         elem = document.getElementById(['bfp', 'rfp'][trace_idx]);
         elem.innerHTML = fp;
       }
       var elem = document.getElementById('prior_label');
       elem.innerHTML = prior_red.toFixed(3) + ' / ' + prior_blue.toFixed(3);



      var cell_for_plot = document.getElementById("for_right_plot");
      var layout = {
          title: 'Naive Bayes',
          xaxis: {'range': [x_min, x_max], 'domain': [0.0, 1.0], 'fixedrange': true},
          yaxis: {'range': [y_min, y_max], 'domain': [0.0, 1.0], 'fixedrange': true},
          width: 800,
          height: 600,
          autosize: false,
          showlegend: true,
          margin: {
            l: 50,
            r: 50,
            b: 50,
            t: 50
          }
        };
      Plotly.newPlot(cell_for_plot, data, layout);
    }

    var pts = [[100, 300], [500, 300]];
    var active_idx = -1;

    function mdown_handle(event) {
      //console.log('down');
      event.stopPropagation();
      x = event.offsetX;
      y = event.offsetY;
      for (var i = 0; i < pts.length; ++i)
        if ((x - pts[i][0])*(x - pts[i][0]) + (y - pts[i][1])*(y - pts[i][1]) < 500)
            active_idx = i;
      console.log('ACTIVE', active_idx);
    }

    function mmove_handle(event) {
      event.stopPropagation();
      //console.log('move');
      if (active_idx < 0)
        return;
      pts[active_idx] = [event.offsetX, event.offsetY];
      loss_function_2d();
    }

    function mup_handle(event) {
      active_idx = -1;
    }

    function adjust_canvas() {
      var plt_div = document.getElementById('for_right_plot');
      var el = plt_div.getElementsByClassName('xy')[0];
      if (!el)
        return [600, 600];
      var rect = el.getBoundingClientRect();
      var canvas = document.getElementById("paint");
      //console.log(rect);
      canvas.style.cssText = "position:absolute; top:" + parseInt(rect.top)
                          + "px; left: " + parseInt(rect.left)
                          + "px; width:" + parseInt(rect.width)
                          + "px; height:" + parseInt(rect.height)
                          + "px; z-index:1000;";
      canvas.width = rect.width;
      canvas.height = rect.height;
      var ctx = canvas.getContext('2d');
      ctx.clearRect(0, 0, canvas.width, canvas.height);
      //loss_function_2d();
      return [canvas.width, canvas.height];
    }

    function calculate()
    {
      var elem = document.getElementById("red_val");
      var reds = elem.value;
      elem = document.getElementById("blue_val");
      var blues = elem.value;
      gen_nums = [parseInt(blues), parseInt(reds)];
      elem = document.getElementById("red_label");
      elem.innerHTML = reds;    
      elem = document.getElementById("blue_label");
      elem.innerHTML = blues;
      loss_function_2d();
    }

  </script>

<style>
  table.tstyle{
  border: 1px solid black;
  border-collapse: collapse;
  border-spacing: 0px;
  text-align: center;
}
  table.tstyle td{
  border: 1px solid black;
  border-collapse: collapse;
  border-spacing: 0px;
  text-align: center;
}
</style>

</head>

<body>

<table>
<tr>
  <td><b>red samples:</b> </td>
  <td><input type="range" class="slider" min = "0" max = "50" value="0" id="red_val" oninput="calculate();"></td>
  <td><div id="red_label" >0</div></td>
  <td><b>blue samples:</b> </td>
  <td><input type="range" class="slider" min = "0" max = "50" value="0" id="blue_val" oninput="calculate();"></td>
  <td><div id="blue_label" >0</div></td>
  <td><b>prior:</b></td>
  <td id="prior_label"></td>
</tr>
</table>

<div id="everything" style="height:600px; width:600px;">
  <div id="for_right_plot">
  </div>
  <div style="position:absolute;top:300px;left:600px">
    <table class='tstyle'>
        <tr><td>    </td><td><b>TP</b></td><td><b>FP</b></td></tr>
        <tr><td><b style="color:blue">BLUE</b></td><td id="rtp"></td><td id="rfp"></td></tr>
        <tr><td><b style="color:red">RED</b></td><td id="btp"></td><td id="bfp"></td></tr>
    </table>
  </div>
</div>

<canvas id="paint" width=600 height=600
    onmousedown="mdown_handle(event)"
    onmousemove="mmove_handle(event)"
    onmouseup="mup_handle(event)">
</canvas>

</body>

<script>
function init()
{
  loss_function_2d();
  adjust_canvas();
  setInterval(adjust_canvas, 1000);
}
init();
</script>

</html>

#Naive Bayes Visualization

Use the following visualization to understand 2D Naive Bayes classification

**Code organization**
* Code consists of two parts in separate cells -- the lower one and the upper one
* Code in the lower part can be hidden by double-clicking "Visualization code" text
* To use for the first time run cells one-by-one (upper than lower)
* Drag-and-drop colored circles representing features on the field
* Press "Invoke Bayes" button for the code to start -- you should see background colored according to classification
* You can change parameters in the upper part and restart it without restarting the lower part. Use this feature extensively -- this will retain features' positions, but you will be able to check different classifier configurations.

**Problem**
* Try different configurations of features, gain intuition how separating lines for NB look like
* Check what happens if there are more features of one class compared to another
* Try different arrangements of features, understand what Bayes is able to separate and what not
* Set some configuration of features and run classification. Google for documentation on `GaussianNB()` and check parameters it has. Modify the code in the upper cell adding some parameters (any you would like to), restart the upper cell without restarting the lower! Press "Invoke Bayes" button and see what changes. Try differen combinations of parameters, understand their influence on the output.


In [2]:
from sklearn.naive_bayes import GaussianNB

model = GaussianNB()

def fit_model(training_set, y):
  model.fit(training_set, y);

def predict_result(X):
  return model.predict(X)

In [3]:
#@title 2D Naive Bayes

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap


def visualize(X, y):
  fig = plt.figure(figsize=(10,10))
  ax = fig.gca()
  ax.tick_params(axis="y",direction="in", pad=-40)
  ax.tick_params(axis="x",direction="in", pad=-15)
  ax.set_xticks(np.arange(-.75, 1, .25))
  ax.set_yticks(np.arange(-.75, 1, .25))
  
  # Plot the training points
  rgb = [[210, 0, 0], [0, 0, 150], [107, 142, 35], [150, 0, 150], [220, 105, 0]]
    
  rgb=np.array(rgb)/255.
  ax.scatter(X[:, 0], X[:, 1], c=rgb[y], s=30, zorder=3)
  ax.axis('tight')
  xlim = [-1, +1]
  ylim = [-1, +1]
  ax.set_xlim(xlim)
  ax.set_ylim(ylim)
  
  # fit the estimator
  xx, yy = np.meshgrid(np.linspace(*xlim, num=700), np.linspace(*ylim, num=700))
  Z = predict_result(np.c_[xx.ravel(), yy.ravel()])

  # Put the result into a color plot
  n_classes = 5
  Z = Z.reshape(xx.shape)
  contours = ax.contourf(xx, yy, Z, alpha=0.3, levels=np.arange(n_classes + 1) - 0.5, cmap=ListedColormap(rgb), zorder=1)

  plt.close(fig)

  return fig

import base64
import io

def figure_to_base64(fig):
  buf = io.BytesIO()
  fig.savefig(buf, format='png', bbox_inches='tight', pad_inches=0);
  image_base64 = base64.b64encode(buf.getvalue()).decode('utf-8').replace('\n', '')
  buf.close()
  return image_base64

import json

def invoke(pts):
  data = np.array(pts).reshape((-1, 3))
  X = data[:, :2] / 300.0 - 1.0
  X[:, 1] *= -1.0
  y = data[:, 2]

  fit_model(X, y)
  fig = visualize(X, y)

  return IPython.display.JSON({'image': figure_to_base64(fig)})


import IPython
from google.colab import output

main_str = '''
  <script type="application/javascript">
  
    var active_pt = null;
    var all_pts = [];
    var class_btn = [[650, 50], [650, 150], [650, 250], [650, 350], [650, 450]];
    var class_name = ["class A", "class B", "class C", "class D", "class E"];
    var class_color = ["rgba(210, 0, 0, 0.75)", "rgba(0, 0, 150, 0.75)", 
         "rgba(107, 142, 35, 0.95)", "rgba(150, 0, 150, 0.75)", "rgba(220, 105, 0, 0.75)"]

    var ptR = 10.0;
    var params = null;
    var debug_txt = "";
    var bg_image = new Image();

    function is_close(pt1, pt2) {
      return   (pt1[0] - pt2[0])*(pt1[0] - pt2[0])
            +  (pt1[1] - pt2[1])*(pt1[1] - pt2[1])
            <= ptR*ptR;
    }
  
    function circ(ctx, pos, cls) {
      ctx.beginPath();
      ctx.arc(pos[0], pos[1], ptR, 0.0, 2.0 * Math.PI, 0);
      ctx.fillStyle = class_color[cls];
      ctx.fill();
    }
    
    function draw() {
      var canvas = document.getElementById('canvas');
      if (canvas.getContext) {
        var ctx = canvas.getContext('2d');
        
        ctx.clearRect(0, 0, canvas.width, canvas.height); // cleanup before start
        ctx.drawImage(bg_image, 0, 0, 600, 600);
        ctx.strokeRect(0, 0, 600, 600); // field

        ctx.fillText(debug_txt, 200, 200);

        if (active_pt) {
          circ(ctx, [active_pt[0], active_pt[1]], active_pt[2]);
        }
        all_pts.forEach(function (item, index) {
          circ(ctx, [item[0], item[1]], item[2]);
        });

      }
    }
    
    var timer
    function init() {
      timer = setInterval(draw, 10);
    }

    function get_state() {
      var rates = document.getElementsByName('rate');
      for(var i = 0; i < rates.length; i++)
        if(rates[i].checked)
          return rates[i].value;
      return "none";
    }

    
    function mdown_handle(evt) {
      x = evt.offsetX;
      y = evt.offsetY;
      state = get_state();
      if (state.slice(0,3) == "add") {
        var class_idx = parseInt(state.slice(3,4), 10);
        if (x < 600)
            all_pts.push([x, y, class_idx]);
        return;
      }

      idx = -1;
      all_pts.forEach(function (item, index) {
        if (is_close([x,y], [item[0], item[1]])) {
          idx = index;
        }
      });
      if (idx <= -1)
          return;
      if (state == "move") {
        active_pt = all_pts[idx];
        all_pts.splice(idx, 1);
      }
      if (state == "del")
          all_pts.splice(idx, 1);
    }
    
    function mmove_handle(evt) {
      if (get_state() != "move")
          return;
      x = evt.offsetX;
      y = evt.offsetY;
      if (active_pt) {
        active_pt[0] = x;
        active_pt[1] = y;
      }
    }
    
    function mup_handle(evt) {
      if (get_state() != "move")
          return;
      x = evt.offsetX;
      y = evt.offsetY;
      if (x >= 600) {
        active_pt = null;
      }
      if (active_pt) {
        all_pts.push(active_pt);
        active_pt = null;
      }
    }
    
    function clear_field() {
      active_pt = null;
      all_pts = [];
    }

    async function calculate() {
      if (!all_pts)
        return;
      var merged = [].concat.apply([], all_pts);
      const result = await google.colab.kernel.invokeFunction('notebook.InvSVM', [merged], {});
      params = result.data['application/json'];
      bg_image.src = 'data:image/  png;base64,' + params.image;
    }

  function getImageURL(class_idx)
  {
     var canvas = document.createElement('canvas');
     var ctx = canvas.getContext('2d');
     canvas.width  = 2.0 * ptR;
     canvas.height = 2.0 * ptR;
     circ(ctx, [ptR, ptR], class_idx);
     return canvas.toDataURL(); //image URL
  }

  </script>
  <canvas id="canvas" width="800" height="600" 
    onmousedown="mdown_handle(event)"
    onmousemove="mmove_handle(event)"
    onmouseup="mup_handle(event)"></canvas>
  <div style="position:absolute; top:200px; left:625px;">
   <div style="border-style: solid;">
    <input type="radio" id="r1a" name="rate" value="add0" checked="checked"> 
    <label for="r1a">
      <img src="" onerror="this.onerror=null; this.src = getImageURL(0); "> Add class A
    </label>
    <br>
    <input type="radio" id="r1b" name="rate" value="add1"> 
    <label for="r1b">
      <img src="" onerror="this.onerror=null; this.src = getImageURL(1); "> Add class B
    </label>
    <br>
    <input type="radio" id="r1c" name="rate" value="add2"> 
    <label for="r1c">
      <img src="" onerror="this.onerror=null; this.src = getImageURL(2); "> Add class C
    </label>
    <br>
    <input type="radio" id="r1d" name="rate" value="add3"> 
    <label for="r1d">
      <img src="" onerror="this.onerror=null; this.src = getImageURL(3); "> Add class D
    </label>
    <br>
    <input type="radio" id="r1e" name="rate" value="add4"> 
    <label for="r1e">
      <img src="" onerror="this.onerror=null; this.src = getImageURL(4); "> Add class E
    </label>
    <br>
    <input type="radio" id="r2" name="rate" value="move">
    <label for="r2">
      Move data point
    </label>
    <br>
    <input type="radio" id="r3" name="rate" value="del">
    <label for="r3">
      Delete data point
    </label>
    <br>
    <button type="button" onclick="clear_field();">
      Clear
    </button>
   </div>
   <br><br>
    <button type="button" onclick="calculate();">
      Invoke Bayes
    </button>
  </div>
  <script> init();</script>
'''

display(IPython.display.HTML(main_str))
    
output.register_callback('notebook.InvSVM', invoke)

#(Multinomial) Naive Bayes text classifier

##Road map

We are about to design our own Naive Bayes text classifier. For that we need to write few functions and a class
* **`calculate_priors(tags)`** - calculates prior probabilities of tags (labels/categories/classes);
* **`Vectorizer`** - a class. Given a corpus, creates a vocabulary and counts of word occurences for each word in the vocabulary in each document;
* **`get_params(counts, tags, alpha=1.0)`** - calculates model parameters for each tag;
* **`calculate_likelihoods(counts, params)`** - calculates likelihoods for a document with given counts to fit for some tag given parameters of the model;
* **`calculate_posteriors(likelihoods, priors)`** - calculates posterior probabilities of the document to belong to each tag;
* **`do_decision(posteriors)`** - chooses tag with the largest posterior probability.



In [ ]:
#@title ##Algorithm outline
%%html

<svg width="200mm" height="370mm" version="1.1" viewBox="0 0 337.09 611.16" xmlns="http://www.w3.org/2000/svg" xmlns:ns0="http://www.iki.fi/pav/software/textext/">
 <defs>
  <marker id="b" overflow="visible" orient="auto">
   <path transform="matrix(-.4 0 0 -.4 -4 0)" d="m0 0 5-5-17.5 5 17.5 5z" fill-opacity=".57971" fill-rule="evenodd" stroke="#000" stroke-opacity=".57971" stroke-width="1pt"/>
  </marker>
  <marker id="c" overflow="visible" orient="auto">
   <path transform="matrix(-.4 0 0 -.4 -4 0)" d="m0 0 5-5-17.5 5 17.5 5z" fill-opacity=".57971" fill-rule="evenodd" stroke="#000" stroke-opacity=".57971" stroke-width="1pt"/>
  </marker>
  <marker id="d" overflow="visible" orient="auto">
   <path transform="matrix(-.4 0 0 -.4 -4 0)" d="m0 0 5-5-17.5 5 17.5 5z" fill-opacity=".57971" fill-rule="evenodd" stroke="#000" stroke-opacity=".57971" stroke-width="1pt"/>
  </marker>
  <marker id="e" overflow="visible" orient="auto">
   <path transform="matrix(-.4 0 0 -.4 -4 0)" d="m0 0 5-5-17.5 5 17.5 5z" fill-opacity=".57971" fill-rule="evenodd" stroke="#000" stroke-opacity=".57971" stroke-width="1pt"/>
  </marker>
  <marker id="f" overflow="visible" orient="auto">
   <path transform="matrix(-.4 0 0 -.4 -4 0)" d="m0 0 5-5-17.5 5 17.5 5z" fill-opacity=".57971" fill-rule="evenodd" stroke="#000" stroke-opacity=".57971" stroke-width="1pt"/>
  </marker>
  <marker id="g" overflow="visible" orient="auto">
   <path transform="matrix(-.4 0 0 -.4 -4 0)" d="m0 0 5-5-17.5 5 17.5 5z" fill-opacity=".57971" fill-rule="evenodd" stroke="#000" stroke-opacity=".57971" stroke-width="1pt"/>
  </marker>
  <marker id="h" overflow="visible" orient="auto">
   <path transform="matrix(-.4 0 0 -.4 -4 0)" d="m0 0 5-5-17.5 5 17.5 5z" fill-opacity=".57971" fill-rule="evenodd" stroke="#000" stroke-opacity=".57971" stroke-width="1pt"/>
  </marker>
  <marker id="i" overflow="visible" orient="auto">
   <path transform="matrix(-.4 0 0 -.4 -4 0)" d="m0 0 5-5-17.5 5 17.5 5z" fill-opacity=".57971" fill-rule="evenodd" stroke="#000" stroke-opacity=".57971" stroke-width="1pt"/>
  </marker>
  <marker id="j" overflow="visible" orient="auto">
   <path transform="matrix(-.4 0 0 -.4 -4 0)" d="m0 0 5-5-17.5 5 17.5 5z" fill-opacity=".57971" fill-rule="evenodd" stroke="#000" stroke-opacity=".57971" stroke-width="1pt"/>
  </marker>
  <marker id="k" overflow="visible" orient="auto">
   <path transform="matrix(-.4 0 0 -.4 -4 0)" d="m0 0 5-5-17.5 5 17.5 5z" fill-opacity=".57971" fill-rule="evenodd" stroke="#000" stroke-opacity=".57971" stroke-width="1pt"/>
  </marker>
  <marker id="m" overflow="visible" orient="auto">
   <path transform="matrix(-.4 0 0 -.4 -4 0)" d="m0 0 5-5-17.5 5 17.5 5z" fill-opacity=".57971" fill-rule="evenodd" stroke="#000" stroke-opacity=".57971" stroke-width="1pt"/>
  </marker>
  <marker id="n" overflow="visible" orient="auto">
   <path transform="matrix(-.4 0 0 -.4 -4 0)" d="m0 0 5-5-17.5 5 17.5 5z" fill-opacity=".57971" fill-rule="evenodd" stroke="#000" stroke-opacity=".57971" stroke-width="1pt"/>
  </marker>
  <marker id="o" overflow="visible" orient="auto">
   <path transform="matrix(-.4 0 0 -.4 -4 0)" d="m0 0 5-5-17.5 5 17.5 5z" fill-opacity=".57971" fill-rule="evenodd" stroke="#000" stroke-opacity=".57971" stroke-width="1pt"/>
  </marker>
  <marker id="p" overflow="visible" orient="auto">
   <path transform="matrix(-.4 0 0 -.4 -4 0)" d="m0 0 5-5-17.5 5 17.5 5z" fill-opacity=".57971" fill-rule="evenodd" stroke="#000" stroke-opacity=".57971" stroke-width="1pt"/>
  </marker>
  <marker id="a" overflow="visible" orient="auto">
   <path transform="matrix(-.4 0 0 -.4 -4 0)" d="m0 0 5-5-17.5 5 17.5 5z" fill-opacity=".57971" fill-rule="evenodd" stroke="#000" stroke-opacity=".57971" stroke-width="1pt"/>
  </marker>
  <marker id="l" overflow="visible" orient="auto">
   <path transform="matrix(-.4 0 0 -.4 -4 0)" d="m0 0 5-5-17.5 5 17.5 5z" fill-opacity=".57971" fill-rule="evenodd" stroke="#000" stroke-opacity=".57971" stroke-width="1pt"/>
  </marker>
 </defs>
 <g transform="translate(25.554 1.7836)">
  <rect x="82.615" y="16.745" width="214.91" height="64.906" fill="#fff300" fill-opacity=".060302"/>
  <rect x="43.057" y="95.552" width="86.147" height="24.907" fill="#f00" fill-opacity=".12077"/>
  <flowRoot fill="#000000" font-family="Verdana" font-size="40px" letter-spacing="0px" word-spacing="0px" style="line-height:1.25" xml:space="preserve"><flowRegion><rect x="81.429" y="66.805" width="228.57" height="242.86"/></flowRegion><flowPara/></flowRoot>
  <rect x="89.517" y="23.374" width="39.835" height="49.41" fill="none" stroke="#000" stroke-linecap="square" stroke-linejoin="bevel" stroke-opacity=".48235" stroke-width=".7"/>
  <path d="m89.535 36.279h39.823" fill="none" stroke="#000" stroke-opacity=".48235" stroke-width=".7"/>
  <text x="96.885376" y="31.951431" fill="#000000" font-family="Verdana" font-size="8.4905px" letter-spacing="0px" stroke-width=".21226" word-spacing="0px" style="line-height:1.25" xml:space="preserve"><tspan x="96.885376" y="31.951431" font-family="'Helvetica Neue LT'" font-size="8.4905px" stroke-width=".21226" style="font-feature-settings:normal;font-variant-caps:normal;font-variant-ligatures:normal;font-variant-numeric:normal">corpus</tspan></text>
  <g stroke="#000">
   <g id="content" transform="matrix(.4625 0 0 -.4625 -31.64 365.13)" fill-rule="evenodd" stroke-miterlimit="10.433" stroke-width="0" ns0:preamble="default_packages.tex" ns0:text="$$doc \\, \\, 1$$\n$$doc \\, \\, 2$$\n$$\\cdots$$\n$$doc \\, \\, N$$\n" xml:space="preserve"><path d="m298.01 706.46v0.0156h-0.0156v0.0468l-0.0156 0.0156v0.0156h-0.0156v0.0156h-0.0156v0.0156h-0.0156l-0.0156 0.0156h-0.0156l-0.0156 0.0156h-0.078c-0.23406 0-1.7164-0.14044-1.9817-0.15604-0.12483-0.0156-0.21845-0.0936-0.21845-0.29648 0-0.18724 0.14043-0.18724 0.37449-0.18724 0.74899 0 0.7802-0.10923 0.7802-0.26527l-0.0468-0.31208-0.93623-3.6825c-0.28087 0.57735-0.73338 0.99865-1.4356 0.99865-1.81 0-3.7449-2.2782-3.7449-4.5407 0-1.4512 0.85821-2.481 2.0753-2.481l0.0156 0.34329c-0.76459 0-0.98304 0.85821-0.98304 1.4512 0 0.7802 0.49932 2.6839 0.84261 3.4016 0.48372 0.90502 1.1859 1.4824 1.81 1.4824 1.0142 0 1.2327-1.2639 1.2327-1.3575 0-0.0936-0.0312-0.18725-0.0468-0.26527l-0.78019-3.0428c-0.078-0.28087-0.078-0.31208-0.31208-0.57735-0.68657-0.84261-1.3263-1.0923-1.7632-1.0923l-0.0156-0.34329c0.29647 0 1.0767 0.0624 2.0129 1.1703 0.12483-0.65537 0.67097-1.1703 1.42-1.1703 0.54614 0 0.90503 0.35889 1.1547 0.85822 0.26527 0.56174 0.46812 1.498 0.46812 1.5292 0 0.15604-0.14044 0.15604-0.18725 0.15604-0.15604 0-0.17164-0.0624-0.21845-0.28087-0.26527-1.0143-0.54614-1.9193-1.1859-1.9193-0.4213 0-0.46812 0.4057-0.46812 0.71778 0 0.35889 0.0312 0.46811 0.0936 0.73338z"/><path d="m305.36 700.09v0.15604l-0.0156 0.14043-0.0156 0.14044-0.0156 0.14043-0.0312 0.14044-0.0624 0.24966-0.0936 0.24966-0.1248 0.21846-0.0624 0.10922-0.0624 0.0936-0.234 0.2808-0.078 0.078-0.1872 0.156-0.0936 0.0624-0.10923 0.0624-0.10922 0.0624-0.0936 0.0468-0.12483 0.0468-0.10923 0.0468-0.10922 0.0312-0.12483 0.0312-0.12484 0.0312-0.37449 0.0468h-0.12483c-2.0909 0-4.1818-2.2002-4.1818-4.4003 0-1.5292 1.0455-2.6215 2.4966-2.6215l0.0156 0.34329c-0.67096 0-1.3575 0.48372-1.3575 1.6852 0 0.76459 0.4057 2.4342 0.90502 3.23 0.7802 1.1859 1.6696 1.42 2.1065 1.42 0.88942 0 1.3575-0.74899 1.3575-1.6696 0-0.60855-0.31207-2.2314-0.88942-3.2456-0.54613-0.88942-1.4044-1.42-2.1221-1.42l-0.0156-0.34329c2.0753 0 4.1662 2.1377 4.1662 4.4003z"/><path d="m311.76 701.76-0.0468-0.0156h-0.234l-0.0312-0.0156h-0.0312l-0.0312-0.0156h-0.0312l-0.0312-0.0156h-0.0156l-0.0156-0.0156-0.0312-0.0156h-0.0156l-0.0156-0.0156-0.0312-0.0156-0.0312-0.0312-0.0312-0.0156-0.0312-0.0312-0.0312-0.0156c-0.24966-0.23406-0.28087-0.49932-0.28087-0.60855 0-0.37449 0.28087-0.54614 0.57734-0.54614 0.45252 0 0.87382 0.3745 0.87382 0.99865 0 0.74899-0.73338 1.3263-1.8256 1.3263-2.1065 0-4.1818-2.2158-4.1818-4.4003 0-1.4044 0.90502-2.6215 2.5278-2.6215 2.2158 0 3.5265 1.6384 3.5265 1.8257 0 0.0936-0.0936 0.20285-0.18724 0.20285-0.078 0-0.10923-0.0312-0.20285-0.15604-1.2171-1.5292-2.9179-1.5292-3.1052-1.5292-0.98304 0-1.4044 0.76459-1.4044 1.6852 0 0.63976 0.31208 2.1377 0.84261 3.1052 0.48372 0.87382 1.3419 1.5448 2.2002 1.5448 0.53053 0 1.1079-0.20285 1.3263-0.60855z"/><path d="m322.09 705.79v0.2184l-0.0156 0.0156v0.0468l-0.0156 0.0156v0.0312l-0.0156 0.0156h-0.0156v0.0156h-0.0156l-0.0156 0.0156h-0.0312v0.0156h-0.1248v0.0156h-0.1092c-0.96744-0.99865-2.3406-0.99865-2.8399-0.99865v-0.48372c0.31208 0 1.2327 0 2.0441 0.4057v-8.0204c0-0.56174-0.0468-0.74898-1.4512-0.74898h-0.49933v-0.48372c0.54614 0.0468 1.9037 0.0468 2.5278 0.0468 0.62416 0 1.9817 0 2.5278-0.0468v0.48372h-0.49932c-1.4044 0-1.4512 0.17164-1.4512 0.74898z"/><path d="m295.95 685.37v0.0326h-0.0163v0.0488l-0.0163 0.0163v0.0162h-0.0163l-0.0163 0.0163v0.0163h-0.0326v0.0162h-0.0326l-0.0163 0.0163h-0.065c-0.24385 0-1.7882-0.14631-2.0646-0.17882-0.13005-0.0163-0.22759-0.0975-0.22759-0.30888 0-0.19508 0.14631-0.19508 0.39016-0.19508 0.78031 0 0.81283-0.11379 0.81283-0.27636l-0.0488-0.30887-0.9754-3.8366c-0.29262 0.60149-0.76406 1.0404-1.4956 1.0404-1.8858 0-3.9016-2.3735-3.9016-4.7307 0-1.5281 0.89411-2.5848 2.1621-2.5848l0.0163 0.35764c-0.79657 0-1.0242 0.87786-1.0242 1.5119 0 0.81283 0.52021 2.7961 0.87785 3.5439 0.50396 0.94288 1.2355 1.5444 1.8858 1.5444 1.0567 0 1.2843-1.333 1.2843-1.4306 0-0.0975-0.0325-0.17882-0.0488-0.2601l-0.81282-3.17c-0.0813-0.29262-0.0813-0.32513-0.32514-0.6015-0.71529-0.87785-1.3818-1.138-1.837-1.138l-0.0163-0.35764c0.30888 0 1.1217 0.065 2.0971 1.203 0.13006-0.66652 0.69904-1.203 1.4794-1.203 0.56898 0 0.94288 0.3739 1.203 0.87785 0.27636 0.58524 0.48769 1.5769 0.48769 1.6094 0 0.16257-0.1463 0.16257-0.19507 0.16257-0.16257 0-0.17883-0.065-0.2276-0.29262-0.27636-1.0567-0.56898-1.9996-1.2355-1.9996-0.43892 0-0.48769 0.40642-0.48769 0.73155 0 0.39016 0.0325 0.50395 0.0975 0.78031z"/><path d="m303.61 678.76v0.14631l-0.0163 0.16257-0.0162 0.14631-0.0163 0.14631-0.0325 0.13005-0.0325 0.14631-0.0325 0.13005-0.0488 0.13005-0.0488 0.1138-0.065 0.13005-0.065 0.1138-0.065 0.0975-0.065 0.11379-0.2439 0.2925-0.0813 0.0813-0.195 0.1626-0.0975 0.065-0.2276 0.13-0.0975 0.0488-0.13005 0.0488-0.11379 0.0488-0.1138 0.0325-0.2601 0.065-0.39016 0.0489h-0.13006c-2.1784 0-4.3568-2.2922-4.3568-4.5844 0-1.6094 1.0892-2.7311 2.601-2.7311l0.0163 0.35764c-0.69903 0-1.4143 0.4877-1.4143 1.7557 0 0.79658 0.42267 2.536 0.94288 3.3489 0.81283 1.2518 1.7394 1.4956 2.1946 1.4956 0.92662 0 1.4143-0.78031 1.4143-1.7394 0-0.63401-0.32513-2.3247-0.92662-3.3814-0.56898-0.94288-1.4631-1.4794-2.2109-1.4794l-0.0163-0.35764c2.1621 0 4.3405 2.2272 4.3405 4.5844z"/><path d="m310.28 680.48h-0.2277l-0.0488-0.0163h-0.0651l-0.0163-0.0162h-0.0488l-0.0163-0.0163h-0.0325l-0.0162-0.0163h-0.0163l-0.0325-0.0163-0.0163-0.0163h-0.0163l-0.0325-0.0163-0.0326-0.0326-0.0325-0.0163-0.0651-0.0651c-0.26011-0.24385-0.29262-0.52021-0.29262-0.63401 0-0.3739 0.29262-0.55272 0.60149-0.55272 0.47144 0 0.91037 0.39016 0.91037 1.0242 0 0.79657-0.76406 1.3981-1.902 1.3981-2.1946 0-4.3568-2.3084-4.3568-4.5844 0-1.4631 0.94288-2.7311 2.6336-2.7311 2.3084 0 3.674 1.7069 3.674 1.902 0 0.0975-0.0975 0.21133-0.19508 0.21133-0.0813 0-0.1138-0.0325-0.21134-0.16256-1.268-1.5932-3.04-1.5932-3.235-1.5932-1.0242 0-1.4631 0.78032-1.4631 1.7557 0 0.66652 0.32514 2.2272 0.87786 3.2351 0.50395 0.91037 1.3981 1.6094 2.2922 1.6094 0.55272 0 1.1542-0.21133 1.3818-0.65026z"/><path d="m318.32 675.59 1.7232 1.6744c2.536 2.2272 3.4952 3.0888 3.4952 4.7144 0 1.837-1.4468 3.1213-3.4301 3.1213-1.837 0-3.0237-1.4794-3.0237-2.9262 0-0.89411 0.79657-0.89411 0.84534-0.89411 0.27636 0 0.84534 0.19508 0.84534 0.8616 0 0.40641-0.29261 0.82909-0.86159 0.82909-0.13006 0-0.16257 0-0.21134-0.0163 0.3739 1.0567 1.2518 1.6419 2.1946 1.6419 1.4794 0 2.1784-1.3005 2.1784-2.6173 0-1.3005-0.81283-2.5685-1.7069-3.5764l-3.105-3.4626c-0.17882-0.17882-0.17882-0.21134-0.17882-0.58524h5.9987l0.45518 2.8124h-0.40641c-0.065-0.4877-0.17883-1.203-0.34139-1.4468-0.1138-0.13005-1.1867-0.13005-1.5444-0.13005z"/><path d="m301.52 661.15v0.0257l-0.0129 0.0386v0.0643l-0.0129 0.0386-0.0129 0.0257-0.0129 0.0386-0.0258 0.0514-0.0129 0.0386-0.0258 0.0514-0.0257 0.0257-0.0129 0.0257-0.0771 0.0771-0.0257 0.0129-0.0257 0.0257-0.0257 0.0129-0.0257 0.0257-0.0514 0.0258-0.0386 0.0129-0.0257 0.0129-0.0386 0.0129-0.0257 0.0129h-0.0386l-0.0257 0.0129h-0.1029c-0.37324 0-0.68212-0.29601-0.68212-0.66924 0-0.37324 0.30888-0.68212 0.68212-0.68212 0.37323 0 0.68211 0.30888 0.68211 0.68212z"/><path d="m307.22 661.15v0.0643l-0.0129 0.0386v0.0257l-0.0129 0.0386-0.0129 0.0257v0.0386l-0.0258 0.0514-0.0257 0.0386-0.0258 0.0514-0.0257 0.0257-0.0129 0.0257-0.0771 0.0771-0.0257 0.0129-0.0257 0.0257-0.0257 0.0129-0.0257 0.0257-0.0514 0.0258-0.0386 0.0129-0.0514 0.0258-0.0386 0.0129h-0.0386l-0.0257 0.0129h-0.1029c-0.37323 0-0.68212-0.29601-0.68212-0.66924 0-0.37324 0.30889-0.68212 0.68212-0.68212s0.68211 0.30888 0.68211 0.68212z"/><path d="m312.91 661.15v0.0643l-0.0129 0.0386v0.0257l-0.0129 0.0386v0.0257l-0.0129 0.0386-0.0258 0.0514-0.0257 0.0386-0.0258 0.0514-0.0257 0.0257-0.0129 0.0257-0.1157 0.1157-0.0257 0.0129-0.0386 0.0257-0.0514 0.0258-0.0258 0.0129-0.0386 0.0129-0.0257 0.0129-0.0386 0.0129h-0.0257l-0.0386 0.0129h-0.1029c-0.37323 0-0.68211-0.29601-0.68211-0.66924 0-0.37324 0.30888-0.68212 0.68211-0.68212 0.37324 0 0.68212 0.30888 0.68212 0.68212z"/><path d="m295.29 651.41v0.0498l-0.0166 0.0167v0.0166l-0.0167 0.0166-0.0167 0.0167v0.0167h-0.0167l-0.0166 0.0166h-0.0166v0.0166h-0.0832v0.0167h-0.0333c-0.25017 0-1.8345-0.1501-2.1014-0.18345-0.15009-0.0167-0.25016-0.10004-0.25016-0.31688 0-0.20012 0.1501-0.20012 0.40026-0.20012 0.80052 0 0.83387-0.11674 0.83387-0.28352l-0.0501-0.33356-0.98396-3.9192c-0.30021 0.60038-0.78385 1.0507-1.5343 1.0507-1.9513 0-4.0193-2.4349-4.0193-4.8532 0-1.551 0.91727-2.6517 2.2181-2.6517l0.0333 0.3669c-0.83388 0-1.0674 0.91727-1.0674 1.551 0 0.83388 0.53369 2.8685 0.91727 3.6357 0.517 0.98398 1.2675 1.5844 1.9346 1.5844 1.0674 0 1.3175-1.3509 1.3175-1.451 0-0.10005-0.0501-0.20013-0.0501-0.28351l-0.83387-3.2521c-0.0834-0.28352-0.0834-0.33355-0.33356-0.6004-0.7338-0.91726-1.4176-1.1841-1.8846-1.1841l-0.0333-0.3669c0.33355 0 1.1674 0.0667 2.1681 1.2508 0.13343-0.70046 0.70047-1.2508 1.501-1.2508 0.58371 0 0.9673 0.38357 1.2341 0.91725 0.28351 0.6004 0.50032 1.601 0.50032 1.6344 0 0.16677-0.15009 0.16677-0.20012 0.16677-0.16678 0-0.18345-0.0667-0.23349-0.3002-0.28352-1.0674-0.58371-2.0513-1.2675-2.0513-0.4503 0-0.50034 0.43361-0.50034 0.76716 0 0.40027 0.0333 0.51701 0.10005 0.78385z"/><path d="m303.16 644.61-0.0167 0.16677v0.1501l-0.0166 0.1501-0.0167 0.15009-0.0333 0.15011-0.0333 0.13341-0.0501 0.13342-0.0501 0.13343-0.0501 0.13342-0.0501 0.11674-0.1334 0.23348-0.0834 0.10005-0.0667 0.11674-0.0834 0.0834-0.0834 0.10005-0.10005 0.0834-0.10004 0.0834-0.10005 0.0834-0.10004 0.0667-0.10005 0.0667-0.23348 0.1334-0.11675 0.0501-0.11674 0.0333-0.13342 0.0501-0.11674 0.0334-0.13343 0.0166-0.13341 0.0333-0.13342 0.0167h-0.28352c-2.2181 0-4.4529-2.3515-4.4529-4.7031 0-1.6344 1.1174-2.8018 2.6684-2.8018v0.3669c-0.70046 0-1.4343 0.51701-1.4343 1.8012 0 0.8172 0.43362 2.6017 0.9673 3.4522 0.8172 1.2842 1.7678 1.5176 2.2348 1.5176 0.96731 0 1.4676-0.78384 1.4676-1.7845 0-0.65042-0.33356-2.3849-0.9673-3.4523-0.58372-0.96729-1.501-1.5343-2.2682-1.5343v-0.3669c2.2014 0 4.4529 2.2848 4.4529 4.7031z"/><path d="m310 646.39h-0.1503l-0.0501-0.0166h-0.1334l-0.0333-0.0167h-0.0333l-0.0333-0.0167h-0.0167l-0.0167-0.0166h-0.0334l-0.0167-0.0166h-0.0166l-0.0333-0.0166-0.0333-0.0333-0.0333-0.0166-0.0333-0.0333-0.0166-0.0167-0.0333-0.0166-0.0166-0.0167c-0.26685-0.25016-0.30021-0.53368-0.30021-0.65043 0-0.40026 0.30021-0.58371 0.61707-0.58371 0.48365 0 0.93395 0.40026 0.93395 1.0674 0 0.8172-0.78384 1.4176-1.968 1.4176-2.2348 0-4.4529-2.3682-4.4529-4.7031 0-1.4843 0.9673-2.8018 2.6851-2.8018 2.3849 0 3.7858 1.7678 3.7858 1.9513 0 0.10004-0.10005 0.21681-0.20013 0.21681-0.0834 0-0.11674-0.0334-0.21681-0.1501-1.3175-1.6511-3.1354-1.6511-3.3355-1.6511-1.0507 0-1.4843 0.8172-1.4843 1.8012 0 0.68378 0.33355 2.2848 0.88391 3.3188 0.517 0.95062 1.4343 1.6511 2.3515 1.6511 0.56703 0 1.2008-0.2168 1.4343-0.65042z"/><path d="m328.71 649.66 0.0333 0.13343 0.0166 0.0501 0.0166 0.0667 0.0167 0.0501 0.0334 0.0667 0.0166 0.0501 0.0333 0.0501 0.0166 0.0501 0.0666 0.1002 0.0334 0.0501 0.0333 0.0501 0.0501 0.0501 0.0501 0.0333 0.0333 0.0501 0.0501 0.0333 0.0667 0.0501 0.0501 0.0334 0.1334 0.0666 0.0667 0.0334 0.0834 0.0166 0.0667 0.0334 0.0834 0.0166 0.10005 0.0166 0.0834 0.0333 0.10004 0.0166h0.11674l0.10005 0.0166 0.11674 0.0167h0.1334c0.0834 0 0.26684 0.0166 0.26684 0.33354 0 0.0167 0 0.18346-0.21681 0.18346-0.53368 0-1.1174-0.0501-1.6678-0.0501-0.56703 0-1.1508 0.0501-1.7011 0.0501-0.10005 0-0.3002 0-0.3002-0.33355 0-0.18345 0.16678-0.18345 0.3002-0.18345 0.95061-0.0167 1.1341-0.36692 1.1341-0.73382 0-0.0501-0.0333-0.30019-0.0501-0.35023l-1.8679-7.3715-3.6691 8.6556c-0.13342 0.30019-0.15009 0.31688-0.53368 0.31688h-2.2348c-0.33355 0-0.48365 0-0.48365-0.33355 0-0.18345 0.1501-0.18345 0.46698-0.18345 0.0834 0 1.1341 0 1.1341-0.15011l-2.2348-8.9058c-0.16678-0.65042-0.43362-1.1841-1.7845-1.2341-0.10005 0-0.28352-0.0167-0.28352-0.33355 0-0.11675 0.0834-0.18346 0.21681-0.18346 0.53368 0 1.1007 0.0501 1.6511 0.0501 0.56704 0 1.1674-0.0501 1.7178-0.0501 0.0834 0 0.30019 0 0.30019 0.33356 0 0.16676-0.1501 0.18345-0.33355 0.18345-0.96731 0.0334-1.1007 0.40026-1.1007 0.7338 0 0.11675 0.0166 0.20014 0.0667 0.36692l2.2014 8.7557c0.0667-0.10004 0.0667-0.13341 0.1501-0.30019l4.136-9.7564c0.11675-0.28352 0.16678-0.31687 0.31687-0.31687 0.18347 0 0.18347 0.0501 0.26685 0.35023z"/></g>
   <rect x="187.37" y="23.996" width="39.835" height="49.41" fill="none" stroke-linecap="square" stroke-linejoin="bevel" stroke-opacity=".48235" stroke-width=".7"/>
   <path d="m187.33 37.165h39.823" fill="none" stroke-opacity=".48235" stroke-width=".7"/>
  </g>
  <text x="198.68054" y="32.554943" fill="#000000" font-family="Verdana" font-size="8.7182px" letter-spacing="0px" stroke-width=".21796" word-spacing="0px" style="line-height:1.25" xml:space="preserve"><tspan x="198.68054" y="32.554943" font-family="'Helvetica Neue LT'" font-size="8.7182px" stroke-width=".21796" style="font-feature-settings:normal;font-variant-caps:normal;font-variant-ligatures:normal;font-variant-numeric:normal">tags</tspan></text>
  <g transform="matrix(.4997 0 0 -.4997 86.708 390.22)" fill-rule="evenodd" stroke="#000" stroke-miterlimit="10.433" stroke-width="0" ns0:preamble="default_packages.tex" ns0:text='$$0="Physics"$$\n$$1="Lyrics"$$\n$$\\cdots$$\n$$N="Religion"$$\n' xml:space="preserve"><path d="m270.09 698.07v0.62556l-0.0139 0.19461-0.0139 0.20853-0.0139 0.20852-0.0278 0.20851-0.0139 0.19463-0.0417 0.20851-0.0417 0.19463-0.0417 0.20851-0.0556 0.19463-0.0556 0.19461-0.139 0.38924-0.0417 0.0973-0.0556 0.0973c-0.62556 1.3345-1.7655 1.557-2.3493 1.557-0.83408 0-1.8489-0.36143-2.4049-1.6543-0.44484-0.94529-0.51435-2.0296-0.51435-3.1278 0-1.0426 0.0556-2.2798 0.61166-3.3224 0.59776-1.126 1.6126-1.404 2.2937-1.404v0.30583c-0.54215 0-1.3623 0.34753-1.6126 1.6682-0.15291 0.83408-0.15291 2.0991-0.15291 2.9193 0 0.87579 0 1.7933 0.11121 2.5439 0.26412 1.6404 1.3067 1.7655 1.6543 1.7655 0.45875 0 1.3762-0.25023 1.6404-1.6126 0.13901-0.77848 0.13901-1.835 0.13901-2.6969 0-1.0426 0-1.974-0.1529-2.8637-0.20853-1.3067-1.0009-1.7238-1.6265-1.7238v-0.30583c0.75067 0 1.8072 0.29193 2.4049 1.5986 0.44484 0.9592 0.51435 2.0296 0.51435 3.1278z"/><path d="m268.07 683.36v0.195h-0.013v0.052l-0.013 0.013v0.026h-0.013l-0.026 0.026h-0.013l-0.013 0.013h-0.026v0.013h-0.182c-0.80476-0.81774-1.947-0.81774-2.3623-0.81774v-0.40237c0.25959 0 1.0254 0 1.7004 0.33747v-6.6846c0-0.46728-0.0389-0.62304-1.2071-0.62304h-0.41535v-0.38941c0.45429 0.0389 1.5835 0.0389 2.1027 0.0389s1.6484 0 2.0898-0.0389v0.38941h-0.40238c-1.1682 0-1.2071 0.14278-1.2071 0.62304z"/><path d="m268.85 650.32v0.1988h-0.0133v0.0531l-0.0132 0.0133v0.0265h-0.0133l-0.0133 0.0132-0.0133 0.0133h-0.0132l-0.0133 0.0132h-0.0266v0.0133h-0.1858c-0.81848-0.83168-1.9802-0.83168-2.4026-0.83168v-0.40923c0.26401 0 1.0429 0 1.7294 0.34322v-6.7986c0-0.47525-0.0395-0.63367-1.2277-0.63367h-0.42243v-0.39604c0.46204 0.0395 1.6105 0.0395 2.1386 0.0395 0.52804 0 1.6765 0 2.1254-0.0395v0.39604h-0.40923c-1.1881 0-1.2277 0.14522-1.2277 0.63367z" fill-rule="evenodd" stroke-miterlimit="10.433"/></g>
  <g fill="#000000" letter-spacing="0px" word-spacing="0px">
   <text x="220.80444" y="9.1072206" fill-opacity=".49275" font-family="Verdana" font-size="14.991px" stroke-opacity=".35749" stroke-width=".37477" style="line-height:1.25" xml:space="preserve"><tspan x="220.80444" y="9.1072206" fill="#000000" fill-opacity=".49275" font-family="'Helvetica Neue LT'" font-size="14.991px" stroke-opacity=".35749" stroke-width=".37477" style="font-feature-settings:normal;font-variant-caps:normal;font-variant-ligatures:normal;font-variant-numeric:normal">Training set</tspan></text>
   <flowRoot font-family="Verdana" font-size="40px" style="line-height:1.25" xml:space="preserve"><flowRegion><rect x="65.66" y="137.62" width="176.78" height="61.619"/></flowRegion><flowPara/></flowRoot>
   <text x="-3.3610792" y="54.735699" font-family="'Helvetica Neue LT'" font-size="9.5674px" stroke-width=".23918" style="line-height:1.25" xml:space="preserve"><tspan x="-3.3610792" y="54.735699" style="font-feature-settings:normal;font-variant-caps:normal;font-variant-ligatures:normal;font-variant-numeric:normal">Document</tspan><tspan x="-3.3610792" y="66.737358" style="font-feature-settings:normal;font-variant-caps:normal;font-variant-ligatures:normal;font-variant-numeric:normal">to classify </tspan><tspan x="-3.3610792" y="78.739021" style="font-feature-settings:normal;font-variant-caps:normal;font-variant-ligatures:normal;font-variant-numeric:normal"/></text>
  </g>
  <rect x="-13.635" y="42.864" width="62.355" height="29.021" fill="#003900" fill-opacity=".072464"/>
  <text x="57.228676" y="115.43646" fill="#000000" font-family="'Helvetica Neue LT'" font-size="8.023px" letter-spacing="0px" stroke-width=".20057" word-spacing="0px" style="line-height:1.25" xml:space="preserve"><tspan x="57.228676" y="115.43646" style="font-feature-settings:normal;font-variant-caps:normal;font-variant-ligatures:normal;font-variant-numeric:normal">CountVectorizer </tspan><tspan x="57.228676" y="125.50076" style="font-feature-settings:normal;font-variant-caps:normal;font-variant-ligatures:normal;font-variant-numeric:normal"/></text>
  <rect x="-4.1561" y="142.66" width="40.966" height="49.408" fill="none" stroke="#000" stroke-linecap="square" stroke-linejoin="bevel" stroke-opacity=".48235" stroke-width=".7"/>
  <path d="m-4.0605 155.24h40.847" fill="none" stroke="#000" stroke-opacity=".48235" stroke-width=".7"/>
  <text x="-2.1002738" y="151.57451" fill="#000000" font-family="Verdana" font-size="7.3px" letter-spacing="0px" stroke-width=".1825" word-spacing="0px" style="line-height:1.25" xml:space="preserve"><tspan x="-2.1002738" y="151.57451" font-family="'Helvetica Neue LT'" font-size="7.3px" stroke-width=".1825" style="font-feature-settings:normal;font-variant-caps:normal;font-variant-ligatures:normal;font-variant-numeric:normal">doc counts</tspan></text>
  <g stroke="#000">
   <g>
    <g transform="matrix(.5149 0 0 -.5149 -150.39 519.04)" fill-opacity=".76471" fill-rule="evenodd" stroke-miterlimit="10.433" stroke-width="0" ns0:preamble="default_packages.tex" ns0:text="$$word \\, \\, 1$$\n$$word \\, \\, 2$$\n$$\\cdots$$\n$$word \\, \\, V$$\n" xml:space="preserve"><path d="m299.45 697.31v0.0189l9e-3 0.0189v0.0189l9e-3 0.0284v0.0189l9e-3 0.0189 0.018 0.0948 0.0189 0.0474 0.018 0.0948 9e-3 0.0189v0.0284l9e-3 0.0189v0.0189l9e-3 0.0189v0.0189l9e-3 0.0189v0.0279l9e-3 0.0189v0.037l9e-3 9e-3v9e-3c0 0.17049-0.1326 0.25574-0.27468 0.25574-0.11367 0-0.28416-0.0758-0.35047-0.26522-0.0284-0.0663-0.47359-1.8565-0.5399-2.1028-0.0663-0.28416-0.0853-0.45465-0.0853-0.62514 0-0.10416 0-0.12314 9e-3 -0.17051-0.21786-0.49253-0.50202-0.78616-0.87142-0.78616-0.75776 0-0.75776 0.69144-0.75776 0.85247 0 0.3031 0.0569 0.67251 0.49254 1.8281 0.10417 0.28416 0.16102 0.41676 0.16102 0.60621 0 0.41676-0.31258 0.76723-0.7767 0.76723-0.89037 0-1.2408-1.364-1.2408-1.4492 0-0.0948 0.0947-0.0948 0.11366-0.0948 0.0947 0 0.10416 0.0189 0.15155 0.1705 0.25575 0.87142 0.62515 1.165 0.95667 1.165 0.0853 0 0.22733-9e-3 0.22733-0.31258 0-0.22732-0.0947-0.49254-0.1705-0.65356-0.40729-1.1082-0.52095-1.5439-0.52095-1.9039 0-0.85248 0.63462-1.184 1.3355-1.184 0.16103 0 0.60621 0 0.9851 0.66304 0.24627-0.6062 0.89983-0.66304 1.184-0.66304 0.7104 0 1.1177 0.59674 1.364 1.1556 0.32204 0.73882 0.62515 1.9986 0.62515 2.4438 0 0.52097-0.25574 0.66304-0.41677 0.66304-0.2368 0-0.4736-0.24627-0.4736-0.45465 0-0.11366 0.0568-0.1705 0.14208-0.24628 0.10416-0.10416 0.341-0.35046 0.341-0.80511 0-0.32205-0.27469-1.2408-0.52097-1.7239-0.24627-0.49253-0.57779-0.82405-1.0324-0.82405-0.44518 0-0.70092 0.28415-0.70092 0.81459 0 0.26521 0.0663 0.55884 0.10416 0.69145z"/><path d="m306.52 696.68v0.0948l-0.027 0.2559-0.0189 0.0853-0.0189 0.0758-0.0852 0.2274-0.0284 0.0663-0.0758 0.1326-0.0379 0.0568-0.0474 0.0569-0.0948 0.1136-0.0568 0.0474-0.0569 0.0474-0.0568 0.0474-0.1138 0.0758-0.0663 0.0379-0.1989 0.0852-0.0758 0.0189-0.0663 0.019-0.0758 0.0189-0.2274 0.027h-0.0853c-1.2598 0-2.529-1.3355-2.529-2.6711 0-0.92826 0.63462-1.5913 1.5155-1.5913l9e-3 0.20839c-0.40729 0-0.82405 0.29363-0.82405 1.023 0 0.46413 0.24627 1.4776 0.54936 1.9607 0.47361 0.71987 1.004 0.86195 1.2692 0.86195 0.54938 0 0.83354-0.45466 0.83354-1.0135 0-0.36941-0.18944-1.3545-0.54937-1.9702-0.33152-0.5399-0.85249-0.86194-1.2787-0.86194l-9e-3 -0.20839c1.2503 0 2.529 1.2977 2.529 2.6711z"/><path d="m307.5 694.67v-0.0189l-9e-3 -9e-3v-0.0279l-9e-3 -0.0284-9e-3 -0.0379-9e-3 -0.0284v-0.0284l-0.018-0.0568v-0.0189l-9e-3 -9e-3v-0.0469l-9e-3 -9e-3v-0.0369l-9e-3 -9e-3v-0.036c0-0.1705 0.13261-0.25575 0.27469-0.25575 0.11366 0 0.28416 0.0758 0.35047 0.26523 0.019 0.0379 0.34098 1.3166 0.3694 1.4871 0.0853 0.31258 0.24627 0.96615 0.3031 1.2219 0.0379 0.12312 0.30311 0.56831 0.53044 0.76722 0.0758 0.0663 0.35046 0.31258 0.75775 0.31258 0.24627 0 0.38835-0.11367 0.39783-0.11367-0.28416-0.0474-0.49254-0.27469-0.49254-0.51149 0-0.15155 0.10416-0.33152 0.35993-0.33152 0.25574 0 0.52096 0.21786 0.52096 0.54938s-0.30311 0.61568-0.78618 0.61568c-0.61567 0-1.0324-0.46413-1.2124-0.71987-0.0758 0.41677-0.4073 0.71987-0.85248 0.71987-0.4357 0-0.61567-0.36941-0.70093-0.5399-0.17049-0.31258-0.29363-0.8809-0.29363-0.90931 0-0.0948 0.0948-0.0948 0.11366-0.0948 0.0947 0 0.10416 9e-3 0.16102 0.21785 0.16102 0.66304 0.35046 1.1177 0.69146 1.1177 0.16101 0 0.29363-0.0758 0.29363-0.42624 0-0.19891-0.0284-0.3031-0.15156-0.79565z"/><path d="m316.06 700.55v0.018l-9e-3 9e-3v0.018h-9e-3v9e-3l-0.018 0.018h-9e-3v9e-3h-0.018v9e-3h-0.054c-0.14208 0-1.0419-0.0853-1.2029-0.0948-0.0758-9e-3 -0.13262-0.0568-0.13262-0.17996 0-0.11366 0.0853-0.11366 0.22734-0.11366 0.45465 0 0.47359-0.0663 0.47359-0.16103l-0.0284-0.18944-0.56832-2.2354c-0.17049 0.35046-0.43571 0.60621-0.86195 0.60621-1.1082 0-2.2827-1.3829-2.2827-2.7564 0-0.8809 0.52095-1.506 1.2598-1.506l0.0189 0.20839c-0.4736 0-0.6062 0.52096-0.6062 0.88089 0 0.4736 0.30311 1.6292 0.52095 2.0649 0.29363 0.54938 0.71987 0.89984 1.0988 0.89984 0.60621 0 0.73881-0.76723 0.73881-0.82406 0-0.0568-0.0189-0.11366-0.0284-0.16102l-0.4736-1.847c-0.0379-0.17051-0.0379-0.18945-0.17997-0.35047-0.41677-0.51149-0.80512-0.66304-1.0703-0.66304l-0.0189-0.20839c0.18944 0 0.66304 0.0379 1.2314 0.71041 0.0663-0.39783 0.39782-0.71041 0.85248-0.71041 0.33152 0 0.54938 0.21786 0.70093 0.52097 0.16101 0.34099 0.28415 0.90931 0.28415 0.92825 0 0.0947-0.0853 0.0947-0.11366 0.0947-0.0947 0-0.10416-0.0379-0.1326-0.17049-0.16103-0.61569-0.33152-1.165-0.71988-1.165-0.25574 0-0.28416 0.24626-0.28416 0.4357 0 0.21786 0.019 0.28417 0.0569 0.44518z"/><path d="m322.02 700.14v0.126l-9e-3 9e-3v0.027l-9e-3 9e-3v0.018l-9e-3 9e-3h-9e-3v9e-3h-9e-3l-9e-3 9e-3h-0.018v9e-3h-0.072v9e-3h-0.063c-0.58726-0.60621-1.4208-0.60621-1.7239-0.60621v-0.29364c0.18944 0 0.74828 0 1.2408 0.24627v-4.8686c0-0.341-0.0284-0.45466-0.8809-0.45466h-0.3031v-0.29363c0.33152 0.0284 1.1556 0.0284 1.5345 0.0284 0.37889 0 1.2029 0 1.5345-0.0284v0.29363h-0.30309c-0.85248 0-0.8809 0.10416-0.8809 0.45466z"/><path d="m298.98 678.39v0.0299l0.01 0.0198v0.0198l0.01 0.0198v0.0299l0.01 0.0198 0.02 0.0994 0.0198 0.0497 0.02 0.0994 0.01 0.0199v0.0198l0.01 0.0299v0.0198l0.01 0.0198v0.0198l0.01 0.0199v0.0298l0.01 0.0198v0.0399l0.01 0.01v0.01c0 0.1791-0.13929 0.26864-0.28855 0.26864-0.11942 0-0.29849-0.0796-0.36814-0.27859-0.0299-0.0697-0.49748-1.9601-0.56714-2.2089-0.0697-0.29849-0.0895-0.47758-0.0895-0.65667 0-0.10951 0-0.12931 0.01-0.17912-0.22886-0.52732-0.52735-0.82584-0.91539-0.82584-0.79598 0-0.79598 0.72634-0.79598 0.8955 0 0.31838 0.0596 0.70644 0.51739 1.9203 0.1095 0.28856 0.16914 0.42784 0.16914 0.62684 0 0.44776-0.32835 0.8159-0.81589 0.8159-0.93529 0-1.3034-1.4328-1.3034-1.5223 0-0.0995 0.0995-0.0995 0.11943-0.0995 0.0995 0 0.1095 0.0198 0.15918 0.1791 0.26866 0.91539 0.6567 1.2238 1.005 1.2238 0.0895 0 0.23878-0.01 0.23878-0.32836 0-0.24875-0.0995-0.52734-0.17909-0.68653-0.42785-1.1741-0.54724-1.6218-0.54724-1.9999 0-0.90544 0.66664-1.2437 1.4029-1.2437 0.16916 0 0.63679 0 1.0348 0.68655 0.25871-0.62685 0.94525-0.68655 1.2437-0.68655 0.74624 0 1.1741 0.6169 1.4328 1.2139 0.33829 0.76615 0.65668 2.0994 0.65668 2.5671 0 0.54725-0.26864 0.69651-0.4378 0.69651-0.24873 0-0.49748-0.25871-0.49748-0.47761 0-0.1293 0.0598-0.18904 0.14925-0.26864 0.1095-0.10951 0.35819-0.35818 0.35819-0.83579 0-0.33829-0.28854-1.3034-0.54724-1.8109-0.25871-0.52734-0.60694-0.86564-1.0845-0.86564-0.46766 0-0.7363 0.28854-0.7363 0.85568 0 0.27861 0.0697 0.58706 0.10938 0.72634z"/><path d="m306.41 677.75v0.0895l-0.01 0.0995-0.02 0.179-0.0199 0.0796-0.0198 0.0895-0.0598 0.1592-0.0299 0.0697-0.0299 0.0796-0.0398 0.0697-0.0397 0.0597-0.0398 0.0697-0.0497 0.0598-0.0497 0.0597-0.0497 0.0598-0.1794 0.1491-0.0598 0.0397-0.0597 0.0398-0.0696 0.0398-0.1394 0.0598-0.0696 0.0299-0.0796 0.0198-0.0697 0.0199-0.0796 0.0198-0.2388 0.03h-0.0895c-1.3233 0-2.6566-1.4029-2.6566-2.8059 0-0.98504 0.66664-1.6716 1.592-1.6716l0.01 0.2189c-0.42784 0-0.86563 0.29851-0.86563 1.0746 0 0.48755 0.25869 1.5522 0.57708 2.0497 0.4975 0.76614 1.0547 0.91539 1.3333 0.91539 0.57709 0 0.87559-0.47759 0.87559-1.0646 0-0.38805-0.199-1.4228-0.5771-2.0696-0.34824-0.57709-0.89547-0.90545-1.3432-0.90545l-0.01-0.2189c1.3134 0 2.6566 1.3631 2.6566 2.8059z"/><path d="m307.44 675.63v-0.01l-0.01-0.0198v-0.0298l-0.03-0.0897v-0.0299l-0.01-0.0397-0.01-0.0299v-0.01l-0.01-0.0198v-0.0398l-0.01-0.01v-0.0398l-0.01-0.01v-0.04c0-0.17909 0.1393-0.26864 0.28855-0.26864 0.11943 0 0.29849 0.0796 0.36814 0.2786 0.0198 0.0397 0.35821 1.383 0.38804 1.5522 0.0895 0.32835 0.2587 1.0248 0.3184 1.2935 0.0397 0.1293 0.3184 0.58704 0.5572 0.80594 0.0796 0.0697 0.36814 0.32834 0.79598 0.32834 0.2587 0 0.40795-0.11943 0.41788-0.11943-0.29849-0.0497-0.51738-0.28855-0.51738-0.54723 0-0.14926 0.10951-0.3383 0.3781-0.3383 0.26864 0 0.54723 0.2189 0.54723 0.57708 0 0.34826-0.31838 0.64675-0.82583 0.64675-0.64674 0-1.0845-0.48754-1.2736-0.76614-0.0796 0.44775-0.42782 0.76614-0.89549 0.76614-0.45767 0-0.64672-0.38804-0.73629-0.56713-0.17909-0.3383-0.30842-0.92534-0.30842-0.95519 0-0.0995 0.0995-0.0995 0.11942-0.0995 0.0995 0 0.10951 0.01 0.16915 0.22886 0.16914 0.69648 0.36814 1.1741 0.72634 1.1741 0.16915 0 0.30845-0.0796 0.30845-0.45771 0-0.20894-0.0299-0.31839-0.15921-0.82584z"/><path d="m316.43 681.79v0.03l-0.01 0.01v0.02h-0.01v0.01h-0.01v0.01h-0.01v0.01h-0.01v0.01h-0.03v0.01h-0.05c-0.14926 0-1.0945-0.0895-1.2636-0.1095-0.0796-0.01-0.1393-0.0598-0.1393-0.18904 0-0.11943 0.0895-0.11943 0.23879-0.11943 0.47761 0 0.49749-0.0697 0.49749-0.16915l-0.0299-0.18905-0.59698-2.3482c-0.17911 0.36814-0.45769 0.63679-0.90544 0.63679-1.1641 0-2.3979-1.4527-2.3979-2.8954 0-0.93529 0.54725-1.582 1.3233-1.582l0.0198 0.2189c-0.4975 0-0.6368 0.53728-0.6368 0.92533 0 0.4975 0.3184 1.7114 0.54725 2.1691 0.30843 0.5771 0.75619 0.94524 1.1542 0.94524 0.63679 0 0.77608-0.81589 0.77608-0.87559 0-0.0598-0.0198-0.1095-0.0299-0.15919l-0.49752-1.9402c-0.0397-0.17909-0.0397-0.19899-0.18903-0.36814-0.43779-0.53729-0.84574-0.69649-1.1243-0.69649l-0.0198-0.2189c0.199 0 0.69648 0.0398 1.2935 0.7363 0.0697-0.40794 0.4179-0.7363 0.89548-0.7363 0.34826 0 0.5771 0.22886 0.7363 0.53728 0.16913 0.35821 0.2985 0.96515 0.2985 0.98506 0 0.0995-0.0895 0.0995-0.11943 0.0995-0.0995 0-0.10938-0.0398-0.13931-0.17909-0.16914-0.64674-0.34823-1.2238-0.75618-1.2238-0.26866 0-0.29849 0.24876-0.29849 0.44776 0 0.23879 0.0198 0.30844 0.0598 0.47758z"/><path d="m321.03 675.81 1.0547 1.0248c1.5522 1.3631 2.1392 1.8905 2.1392 2.8854 0 1.1243-0.88554 1.9104-2.0994 1.9104-1.1243 0-1.8507-0.90543-1.8507-1.791 0-0.54723 0.48755-0.54723 0.51741-0.54723 0.16914 0 0.51739 0.11942 0.51739 0.52732 0 0.24875-0.1791 0.50745-0.52735 0.50745-0.0796 0-0.0995 0-0.12931-0.01 0.22885 0.64672 0.76614 1.0049 1.3432 1.0049 0.90544 0 1.3333-0.79599 1.3333-1.6019 0-0.79598-0.4975-1.5721-1.0447-2.189l-1.9104-2.1193c-0.0995-0.10938-0.0995-0.1293-0.0995-0.3582h3.6715l0.27859 1.7213h-0.24874c-0.0398-0.29849-0.11943-0.73629-0.20894-0.88553-0.0697-0.0796-0.72635-0.0796-0.94525-0.0796z"/><path d="m303.74 661.08v0.02l-0.01 0.03v0.05l-0.01 0.03-0.01 0.02-0.01 0.03-0.02 0.04-0.01 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.06 0.06-0.02 0.01-0.02 0.02-0.02 0.01-0.02 0.02-0.04 0.02-0.03 0.01-0.02 0.01-0.03 0.01-0.02 0.01h-0.03l-0.02 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m308.17 661.08v0.05l-0.01 0.03v0.02l-0.01 0.03-0.01 0.02v0.03l-0.02 0.04-0.02 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.06 0.06-0.02 0.01-0.02 0.02-0.02 0.01-0.02 0.02-0.04 0.02-0.03 0.01-0.04 0.02-0.03 0.01h-0.03l-0.02 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m312.59 661.08v0.05l-0.01 0.03v0.02l-0.01 0.03v0.02l-0.01 0.03-0.02 0.04-0.02 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.09 0.09-0.02 0.01-0.03 0.02-0.06 0.03-0.03 0.01-0.02 0.01-0.03 0.01h-0.02l-0.03 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m297.49 645.39 0.0101 0.0192v0.0192l0.0101 0.0192 0.0101 0.0288v0.0192l0.0101 0.0289 0.0101 0.0385 0.0101 0.0481 0.0101 0.048 0.0192 0.0481 0.0101 0.0481v0.0288l0.0202 0.0384v0.0192l0.0101 0.0192v0.0384l0.0101 0.0192v0.0192l0.0101 0.0101v0.0697c0 0.16328-0.12486 0.24973-0.26894 0.24973-0.11526 0-0.28815-0.0768-0.35538-0.25933-0.0288-0.0672-0.48025-1.8922-0.5475-2.1419-0.0672-0.28815-0.0864-0.46104-0.0864-0.63393 0-0.096 0-0.12487 0.0101-0.16328-0.22092-0.50907-0.50907-0.80682-0.88367-0.80682-0.7684 0-0.7684 0.71077-0.7684 0.87405 0 0.29776 0.0481 0.67235 0.49946 1.8442 0.10567 0.28815 0.16329 0.42263 0.16329 0.61473 0 0.43223-0.31697 0.778-0.78762 0.778-0.90286 0-1.2582-1.3831-1.2582-1.4696 0-0.096 0.096-0.096 0.11525-0.096 0.096 0 0.10567 0.0192 0.15369 0.17289 0.25933 0.89327 0.62433 1.1814 0.9605 1.1814 0.0864 0 0.24013-0.0101 0.24013-0.30736 0-0.24012-0.10567-0.50906-0.17289-0.67235-0.41302-1.1238-0.52828-1.5656-0.52828-1.9306 0-0.86445 0.63393-1.2006 1.3543-1.2006 0.16329 0 0.61472 0 0.99892 0.67234 0.24973-0.61471 0.91248-0.67234 1.191-0.67234 0.72039 0 1.143 0.60511 1.3927 1.1718 0.32658 0.74919 0.63393 2.0266 0.63393 2.4877 0 0.51867-0.25933 0.66275-0.42262 0.66275-0.24012 0-0.48025-0.24012-0.48025-0.45143 0-0.12486 0.0577-0.1825 0.14408-0.25934 0.10566-0.10566 0.34578-0.35538 0.34578-0.81642 0-0.31697-0.27855-1.2583-0.52828-1.7481-0.24973-0.49946-0.5859-0.83563-1.0565-0.83563-0.44184 0-0.70117 0.28815-0.70117 0.82603 0 0.26893 0.0672 0.56669 0.10566 0.70116z"/><path d="m304.67 644.76-0.0101 0.096v0.0864l-0.0101 0.0864-0.0192 0.0864-0.0101 0.0864-0.0192 0.0768-0.0288 0.0768-0.0289 0.0768-0.0288 0.0768-0.0289 0.0672-0.077 0.1344-0.048 0.0577-0.0385 0.0672-0.1538 0.1538-0.1152 0.0962-0.0576 0.0385-0.0577 0.0385-0.1344 0.077-0.0672 0.0289-0.0672 0.0192-0.0768 0.0289-0.0768 0.0192-0.0672 0.0101-0.0768 0.0192-0.0768 0.0101h-0.1632c-1.2871 0-2.5645-1.3543-2.5645-2.7086 0-0.94131 0.64354-1.6136 1.5272-1.6136l0.0101 0.21131c-0.40341 0-0.82603 0.29776-0.82603 1.0373 0 0.47065 0.24973 1.4984 0.54748 1.9882 0.48025 0.7396 1.0278 0.87406 1.2967 0.87406 0.55708 0 0.84523-0.45143 0.84523-1.0277 0-0.3746-0.19209-1.3735-0.55708-1.9882-0.33618-0.55709-0.86446-0.88366-1.3063-0.88366l-0.0101-0.21131c1.2775 0 2.5742 1.3159 2.5742 2.7086z"/><path d="m305.66 642.72-0.0101-0.0192v-0.0293l-0.0101-0.0101v-0.0288l-0.0101-0.0289-0.0101-0.0385-0.0101-0.0289v-0.0288l-0.0101-0.0288v-0.0192l-0.0101-0.0101v-0.0293l-0.0101-0.0101v-0.0394l-0.0101-0.0101v-0.0606c0-0.1729 0.13448-0.25934 0.27856-0.25934 0.11525 0 0.27854 0.0768 0.34577 0.26894 0.0192 0.0385 0.34578 1.3351 0.38421 1.508 0.0768 0.31698 0.24973 0.97971 0.30736 1.2391 0.0385 0.12486 0.30735 0.5763 0.53787 0.78761 0.0768 0.0672 0.3554 0.30735 0.76841 0.30735 0.24973 0 0.3938-0.11525 0.40341-0.11525-0.28815-0.0385-0.49946-0.26894-0.49946-0.51868 0-0.15367 0.10566-0.33617 0.36499-0.33617 0.25934 0 0.52828 0.22091 0.52828 0.56669 0 0.33618-0.30736 0.61473-0.79722 0.61473-0.62433 0-1.047-0.46105-1.2294-0.72998-0.0768 0.43222-0.42262 0.72998-0.87405 0.72998-0.43222 0-0.61472-0.36499-0.70116-0.53788-0.1729-0.32658-0.29777-0.90288-0.29777-0.93169 0-0.096 0.096-0.096 0.11527-0.096 0.096 0 0.10566 0.0101 0.16328 0.22091 0.16329 0.68196 0.35539 1.1334 0.69156 1.1334 0.16329 0 0.29776-0.0672 0.29776-0.43223 0-0.20171-0.0288-0.30736-0.15368-0.80682z"/><path d="m314.34 648.68v0.0202l-0.0101 0.0101v0.0202h-0.0101v0.0101h-0.0101v0.0101l-0.0101 0.0101h-0.0101v0.0101h-0.0101l-0.0101 0.0101h-0.0404v0.0101h-0.0202c-0.14407 0-1.0566-0.0864-1.2198-0.10566-0.0768-0.0101-0.13447-0.0576-0.13447-0.1825 0-0.11526 0.0864-0.11526 0.23051-0.11526 0.46104 0 0.48026-0.0672 0.48026-0.16329l-0.0288-0.1921-0.5763-2.2572c-0.17289 0.34577-0.45144 0.60511-0.88366 0.60511-1.1142 0-2.3052-1.4023-2.3052-2.7951 0-0.89327 0.52828-1.5272 1.2775-1.5272l0.0192 0.21132c-0.48025 0-0.61472 0.52827-0.61472 0.89325 0 0.48025 0.30737 1.6521 0.52827 2.0939 0.29777 0.5667 0.72039 0.91248 1.1046 0.91248 0.62433 0 0.7588-0.77801 0.7588-0.83563 0-0.0576-0.0192-0.11527-0.0288-0.16329l-0.48025-1.873c-0.0481-0.16329-0.0481-0.1921-0.1921-0.34579-0.41301-0.52827-0.80682-0.68194-1.0758-0.68194l-0.0192-0.21132c0.1921 0 0.67235 0.0385 1.239 0.72038 0.0768-0.40341 0.41302-0.72038 0.87406-0.72038 0.33617 0 0.55709 0.22092 0.71077 0.52828 0.16329 0.34577 0.28815 0.92208 0.28815 0.94129 0 0.096-0.0864 0.096-0.11525 0.096-0.096 0-0.10566-0.0385-0.13448-0.17288-0.16329-0.61472-0.33617-1.1814-0.72998-1.1814-0.25933 0-0.28815 0.24973-0.28815 0.44182 0 0.23052 0.0192 0.29776 0.0576 0.45143z"/><path d="m323.58 647.58 0.0385 0.0672 0.0481 0.0672 0.0385 0.0576 0.0481 0.0576 0.0385 0.0577 0.0865 0.0865 0.0385 0.0481 0.0385 0.0385 0.0481 0.0288 0.0385 0.0385 0.0385 0.0288 0.0385 0.0289 0.0385 0.0192 0.0385 0.0288 0.1155 0.0576 0.077 0.0202 0.0288 0.0101 0.1155 0.0303 0.0289 0.0101h0.0385l0.0672 0.0101h0.0672l0.0672 0.0101c0.11525 0.0101 0.12486 0.17288 0.12486 0.18249 0 0.0768-0.0481 0.11526-0.12486 0.11526-0.24973 0-0.51868-0.0288-0.77802-0.0288-0.31696 0-0.64352 0.0288-0.95089 0.0288-0.0576 0-0.1825 0-0.1825-0.18249 0-0.10567 0.0864-0.11526 0.15368-0.11526 0.25934-0.0192 0.44184-0.11527 0.44184-0.31697 0-0.14407-0.14408-0.35538-0.14408-0.36499l-2.9391-4.6488-0.65315 5.0426c0 0.16329 0.22092 0.28816 0.66275 0.28816 0.13446 0 0.24013 0 0.24013 0.1921 0 0.0864-0.0768 0.10566-0.13448 0.10566-0.38419 0-0.79721-0.0288-1.191-0.0288-0.17289 0-0.35538 0.0101-0.52827 0.0101-0.1729 0-0.3554 0.0192-0.51867 0.0192-0.0672 0-0.1825 0-0.1825-0.1825 0-0.11525 0.0864-0.11525 0.24012-0.11525 0.53788 0 0.54749-0.0864 0.5763-0.32658l0.75881-5.8879c0.0288-0.1921 0.0672-0.22091 0.19209-0.22091 0.15368 0 0.1921 0.0481 0.26895 0.17289z"/></g>
    <path d="m21.31 156.05v35.39" fill="none" stroke-opacity=".19608" stroke-width=".9"/>
    <g transform="matrix(.52242 0 0 -.52242 -106.9 524.68)" fill-rule="evenodd" stroke-miterlimit="10.433" ns0:preamble="default_packages.tex" ns0:text="$$2$$\n$$0$$\n$$\\cdots$$\n$$1$$\n" xml:space="preserve"><g stroke-width="0"><path d="m258.45 696.16 1.06 1.03c1.55 1.37 2.15 1.9 2.15 2.9 0 1.13-0.9 1.93-2.11 1.93-1.13 0-1.87-0.92-1.87-1.8 0-0.56 0.5-0.56 0.53-0.56 0.17 0 0.52 0.12 0.52 0.53 0 0.26-0.18 0.51-0.53 0.51h-0.13c0.23 0.64 0.77 1.01 1.35 1.01 0.9 0 1.33-0.81 1.33-1.62 0-0.8-0.5-1.58-1.04-2.2l-1.92-2.12c-0.11-0.11-0.11-0.13-0.11-0.37h3.7l0.28 1.73h-0.25c-0.05-0.3-0.12-0.74-0.22-0.89-0.07-0.08-0.73-0.08-0.95-0.08z"/><path d="m261.77 680.65v0.29l-0.01 0.15v0.15l-0.02 0.3-0.02 0.14-0.04 0.3-0.03 0.14-0.03 0.15-0.08 0.28-0.05 0.14-0.12 0.28c-0.46 0.95-1.27 1.11-1.69 1.11-0.6 0-1.33-0.26-1.74-1.18-0.32-0.69-0.37-1.46-0.37-2.25 0-0.75 0.04-1.64 0.45-2.4 0.43-0.8 1.16-1 1.65-1v0.22c-0.39 0-0.98 0.24-1.16 1.2-0.11 0.59-0.11 1.51-0.11 2.09 0 0.64 0 1.3 0.08 1.83 0.19 1.18 0.94 1.27 1.19 1.27 0.33 0 0.98-0.17 1.17-1.16 0.1-0.55 0.1-1.31 0.1-1.94 0-0.74 0-1.42-0.11-2.05-0.15-0.95-0.71-1.24-1.16-1.24v-0.22c0.53 0 1.29 0.21 1.73 1.15 0.32 0.69 0.37 1.46 0.37 2.25z"/><path d="m254.97 662v0.02l-0.01 0.03v0.05l-0.01 0.03-0.01 0.02-0.01 0.03-0.02 0.04-0.01 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.06 0.06-0.02 0.01-0.02 0.02-0.02 0.01-0.02 0.02-0.04 0.02-0.03 0.01-0.02 0.01-0.03 0.01-0.02 0.01h-0.03l-0.02 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m259.38 662.02v0.05l-0.01 0.03v0.02l-0.01 0.03-0.01 0.02v0.03l-0.02 0.04-0.02 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.06 0.06-0.02 0.01-0.02 0.02-0.02 0.01-0.02 0.02-0.04 0.02-0.03 0.01-0.04 0.02-0.03 0.01h-0.03l-0.02 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m263.78 661.94v0.05l-0.01 0.03v0.02l-0.01 0.03v0.02l-0.01 0.03-0.02 0.04-0.02 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.09 0.09-0.02 0.01-0.03 0.02-0.06 0.03-0.03 0.01-0.02 0.01-0.03 0.01h-0.02l-0.03 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m260.12 647.96v0.01l-0.01 0.01v0.16l-0.01 0.01v0.01l-0.01 0.01v0.01h-0.01l-0.02 0.02h-0.01l-0.01 0.01h-0.05l-0.01 0.01h-0.1c-0.61-0.64-1.49-0.64-1.81-0.64v-0.31c0.2 0 0.79 0 1.31 0.26v-5.14c0-0.36-0.03-0.48-0.93-0.48h-0.32v-0.31c0.35 0.03 1.22 0.03 1.62 0.03s1.26 0 1.61-0.03v0.31h-0.32c-0.89 0-0.92 0.11-0.92 0.48z"/></g><g fill-rule="evenodd" stroke-miterlimit="10.433"><g stroke-width="0"><path d="m394.04 685.04v0.01l-0.01 0.01v0.16l-0.01 0.01v0.01l-0.01 0.01v0.01h-0.01l-0.02 0.02h-0.01l-0.01 0.01h-0.05l-0.01 0.01h-0.1c-0.61001-0.64-1.49-0.64-1.81-0.64v-0.31c0.2 0 0.79 0 1.31 0.26v-5.14c0-0.36-0.03-0.48-0.93-0.48h-0.32v-0.31c0.35 0.03 1.22 0.03 1.62 0.03s1.26 0 1.61-0.03v0.31h-0.32c-0.89 0-0.92 0.11-0.92 0.48z"/><path d="m395.85 639.82v0.01l-0.01 0.01v0.16l-0.01 0.01v0.01l-0.01 0.01v0.01h-0.01l-0.02 0.02h-0.01l-0.01 0.01h-0.05l-0.01 0.01h-0.1c-0.61001-0.64-1.49-0.64-1.81-0.64v-0.31c0.2 0 0.79 0 1.31 0.26v-5.14c0-0.36-0.03-0.48-0.93-0.48h-0.32v-0.31c0.35 0.03 1.22 0.03 1.62 0.03s1.26 0 1.61-0.03v0.31h-0.32c-0.89 0-0.92 0.11-0.92 0.48z"/><path d="m425.35 670.58v0.01l-0.01 0.01v0.16l-0.01 0.01v0.01l-0.01 0.01v0.01h-0.01l-0.02 0.02h-0.01l-0.01 0.01h-0.05l-0.01 0.01h-0.1c-0.61002-0.64-1.49-0.64-1.81-0.64v-0.31c0.2 0 0.79 0 1.31 0.26v-5.14c0-0.36-0.03-0.48-0.93-0.48h-0.32v-0.31c0.35 0.03 1.22 0.03 1.62 0.03s1.26 0 1.61-0.03v0.31h-0.32c-0.89 0-0.92 0.11-0.92 0.48z"/><path d="m426.67 637.12v0.29l-0.01 0.15v0.15l-0.02 0.3-0.02 0.14-0.04 0.3-0.03 0.14-0.03 0.15-0.08 0.28-0.05 0.14-0.12 0.28c-0.46 0.95-1.27 1.11-1.69 1.11-0.6 0-1.33-0.26-1.74-1.18-0.32-0.69-0.37-1.46-0.37-2.25 0-0.74998 0.04-1.64 0.45-2.4 0.43-0.8 1.16-1 1.65-1v0.22c-0.39 0-0.98 0.24-1.16 1.2-0.11 0.59-0.11 1.51-0.11 2.09 0 0.64 0 1.3 0.08 1.83 0.19 1.18 0.94 1.27 1.19 1.27 0.33 0 0.98-0.17 1.17-1.16 0.1-0.55 0.1-1.31 0.1-1.94 0-0.73998 0-1.42-0.11-2.05-0.15-0.95-0.71-1.24-1.16-1.24v-0.22c0.53 0 1.29 0.21 1.73 1.15 0.32 0.69 0.37 1.46 0.37 2.25z"/><path d="m480.02 682.15v0.29l-0.01 0.15v0.15l-0.02 0.3-0.02 0.14-0.04 0.3-0.03 0.14-0.03 0.15-0.08 0.28-0.05 0.14-0.12 0.28c-0.46 0.95-1.27 1.11-1.69 1.11-0.6 0-1.33-0.26-1.74-1.18-0.32-0.69-0.37-1.46-0.37-2.25 0-0.74998 0.04-1.64 0.45-2.4 0.43-0.8 1.16-1 1.65-1v0.22c-0.39 0-0.98 0.24-1.16 1.2-0.11 0.59-0.11 1.51-0.11 2.09 0 0.64 0 1.3 0.08 1.83 0.19 1.18 0.94 1.27 1.19 1.27 0.33 0 0.98-0.17 1.17-1.16 0.1-0.55 0.1-1.31 0.1-1.94 0-0.73998 0-1.42-0.11-2.05-0.15-0.95-0.71-1.24-1.16-1.24v-0.22c0.53 0 1.29 0.21 1.73 1.15 0.32 0.69 0.37 1.46 0.37 2.25z"/><path d="m422.98 679.8 1.06 1.03c1.55 1.37 2.15 1.9 2.15 2.9 0 1.13-0.9 1.93-2.11 1.93-1.13 0-1.87-0.92-1.87-1.8 0-0.56 0.5-0.56 0.53-0.56 0.17 0 0.52 0.12 0.52 0.53 0 0.26-0.18 0.51-0.53 0.51h-0.13c0.23 0.64 0.77 1.01 1.35 1.01 0.9 0 1.33-0.81 1.33-1.62 0-0.8-0.5-1.58-1.04-2.2l-1.92-2.12c-0.11-0.11-0.11-0.13-0.11-0.37h3.7l0.28 1.73h-0.25c-0.05-0.3-0.12-0.74-0.22-0.89-0.07-0.08-0.73-0.08-0.95-0.08z"/><path d="m477.79 635.49 1.06 1.03c1.55 1.37 2.15 1.9 2.15 2.9 0 1.13-0.9 1.93-2.11 1.93-1.13 0-1.87-0.92-1.87-1.8 0-0.56 0.5-0.56 0.53-0.56 0.17 0 0.52 0.12 0.52 0.53 0 0.26-0.18 0.51-0.53 0.51h-0.13c0.23 0.64 0.77 1.01 1.35 1.01 0.9 0 1.33-0.81 1.33-1.62 0-0.8-0.5-1.58-1.04-2.2l-1.92-2.12c-0.11-0.11-0.11-0.13-0.11-0.37h3.7l0.28 1.73h-0.25c-0.05-0.3-0.12-0.74-0.22-0.89-0.07-0.08-0.73-0.08-0.95-0.08z"/><path d="m476.88 663.88 1.06 1.03c1.55 1.37 2.15 1.9 2.15 2.9 0 1.13-0.9 1.93-2.11 1.93-1.13 0-1.87-0.92-1.87-1.8 0-0.56 0.5-0.56 0.53-0.56 0.17 0 0.52 0.12 0.52 0.53 0 0.26-0.18 0.51-0.53 0.51h-0.13c0.23 0.64 0.77 1.01 1.35 1.01 0.9 0 1.33-0.81 1.33-1.62 0-0.8-0.5-1.58-1.04-2.2l-1.92-2.12c-0.11-0.11-0.11-0.13-0.11-0.37h3.7l0.28 1.73h-0.25c-0.05-0.3-0.12-0.74-0.22-0.89-0.07-0.08-0.73-0.08-0.95-0.08z"/></g><g transform="matrix(1.0577 0 0 1.0577 70.922 -71.008)" stroke="#000" ns0:preamble="default_packages.tex" ns0:text="$$3$$\n" xml:space="preserve"><path d="m306.02 698.67 0.08 0.02 0.08 0.03 0.07 0.03 0.07 0.04 0.07 0.03 0.07 0.04 0.06 0.04 0.07 0.04 0.06 0.05 0.06 0.04 0.12 0.1 0.05 0.05 0.05 0.06 0.05 0.05 0.05 0.06 0.04 0.06 0.05 0.05 0.04 0.07 0.03 0.06 0.04 0.06 0.03 0.07 0.03 0.06 0.03 0.07 0.04 0.14 0.02 0.06 0.01 0.08 0.02 0.07v0.07l0.01 0.07v0.07c0 0.82-0.88 1.38-1.83 1.38-1.01 0-1.77-0.6-1.77-1.36 0-0.32 0.22-0.51 0.51-0.51 0.31 0 0.51 0.22 0.51 0.5 0 0.5-0.47 0.5-0.62 0.5 0.31 0.49 0.97 0.62 1.33 0.62 0.4 0 0.95-0.22 0.95-1.12 0-0.11-0.02-0.69-0.28-1.13-0.3-0.47-0.63-0.5-0.88-0.51-0.08-0.01-0.32-0.03-0.39-0.03-0.08-0.01-0.15-0.02-0.15-0.12 0-0.11 0.07-0.11 0.24-0.11h0.44c0.81 0 1.18-0.68 1.18-1.65 0-1.35-0.68-1.64-1.12-1.64-0.43 0-1.18 0.17-1.53 0.75 0.35-0.05 0.66 0.17 0.66 0.55 0 0.36-0.27 0.56-0.56 0.56-0.24 0-0.56-0.14-0.56-0.58 0-0.9 0.93-1.56 2.02-1.56 1.21 0 2.12 0.9 2.12 1.92 0 0.81-0.63 1.59-1.67 1.8z" stroke-width="0"/></g></g></g>
   </g>
   <rect x="67.827" y="142.89" width="84.143" height="54.413" fill="none" stroke-linecap="square" stroke-linejoin="bevel" stroke-opacity=".48309" stroke-width=".7"/>
   <path d="m67.932 155.01h84.053" fill="none" stroke-opacity=".48235" stroke-width=".7"/>
  </g>
  <text x="96.872986" y="151.78587" fill="#000000" font-family="Verdana" font-size="9.3932px" letter-spacing="0px" stroke-width=".23483" word-spacing="0px" style="line-height:1.25" xml:space="preserve"><tspan x="96.872986" y="151.78587" font-family="'Helvetica Neue LT'" font-size="9.3932px" stroke-width=".23483" style="font-feature-settings:normal;font-variant-caps:normal;font-variant-ligatures:normal;font-variant-numeric:normal">counts</tspan></text>
  <g stroke="#000">
   <path d="m89.337 155.57v41.547" fill="none" stroke-opacity=".19608" stroke-width=".9"/>
   <path d="m151.57 162.91h-82.107" fill="none" stroke-opacity=".19608" stroke-width=".9"/>
   <g transform="matrix(.5149 0 0 -.5149 -78.886 525.96)" fill-opacity=".86275" fill-rule="evenodd" stroke-miterlimit="10.433" stroke-width="0" ns0:preamble="default_packages.tex" ns0:text="$$word \\, \\, 1$$\n$$word \\, \\, 2$$\n$$\\cdots$$\n$$word \\, \\, V$$\n" xml:space="preserve"><path d="m335.59 712.36v0.0184l9e-3 0.0183v0.0183l9e-3 0.0276v0.0184l9e-3 0.0183 0.018 0.0916 0.0183 0.0458 0.018 0.0916 9e-3 0.0183v0.0276l9e-3 0.0183v0.0184l9e-3 0.0183v0.0184l9e-3 0.0183v0.0274l9e-3 0.0184v0.0364l9e-3 9e-3v9e-3c0 0.1651-0.12845 0.24764-0.266 0.24764-0.11009 0-0.27517-0.0734-0.33938-0.25682-0.0275-0.0642-0.45861-1.7978-0.52282-2.0362-0.0641-0.27516-0.0825-0.44026-0.0825-0.60536 0-0.10086 0-0.11921 9e-3 -0.1651-0.21098-0.47696-0.48613-0.76131-0.84386-0.76131-0.73378 0-0.73378 0.66959-0.73378 0.82551 0 0.29351 0.055 0.65124 0.47696 1.7703 0.10085 0.27516 0.15592 0.40357 0.15592 0.58701 0 0.40359-0.30268 0.74297-0.75213 0.74297-0.86218 0-1.2016-1.3208-1.2016-1.4034 0-0.0917 0.0917-0.0917 0.11009-0.0917 0.0918 0 0.10086 0.0183 0.14676 0.1651 0.24766 0.84385 0.60537 1.1282 0.9264 1.1282 0.0825 0 0.22014-9e-3 0.22014-0.30268 0-0.22013-0.0917-0.47696-0.1651-0.63288-0.39442-1.0732-0.50448-1.4951-0.50448-1.8436 0-0.82551 0.61455-1.1465 1.2933-1.1465 0.15593 0 0.58703 0 0.95392 0.64206 0.23847-0.58703 0.87137-0.64206 1.1465-0.64206 0.68793 0 1.0823 0.57785 1.3208 1.119 0.31185 0.71544 0.60536 1.9354 0.60536 2.3664 0 0.50447-0.24765 0.64206-0.40357 0.64206-0.22932 0-0.45863-0.23848-0.45863-0.44027 0-0.11009 0.055-0.1651 0.1376-0.23849 0.10085-0.10085 0.3302-0.33936 0.3302-0.77963 0-0.31186-0.266-1.2016-0.50448-1.6694-0.23847-0.47695-0.5595-0.79798-0.99978-0.79798-0.4311 0-0.67874 0.27517-0.67874 0.78881 0 0.25683 0.0642 0.54117 0.10085 0.66958z"/><path d="m342.44 711.76v0.0917l-0.027 0.2475-0.0183 0.0825-0.0183 0.0734-0.0275 0.0734-0.0274 0.0734-0.0275 0.0734-0.0276 0.0642-0.0734 0.1284-0.0367 0.055-0.0458 0.0551-0.0916 0.11-0.055 0.0458-0.055 0.0459-0.055 0.0458-0.11 0.0734-0.0642 0.0367-0.0642 0.0275-0.0641 0.0276-0.0642 0.0275-0.0734 0.0183-0.0641 0.0184-0.0734 0.0183-0.2202 0.027h-0.0825c-1.2199 0-2.449-1.2933-2.449-2.5866 0-0.89888 0.61455-1.5409 1.4676-1.5409l9e-3 0.20178c-0.39442 0-0.79799 0.28435-0.79799 0.99062 0 0.44943 0.23847 1.4309 0.53199 1.8987 0.45861 0.69709 0.97225 0.83467 1.2291 0.83467 0.53199 0 0.80715-0.44026 0.80715-0.98143 0-0.35773-0.18344-1.3116-0.53199-1.9078-0.32103-0.52283-0.8255-0.83469-1.2382-0.83469l-9e-3 -0.20178c1.2107 0 2.449 1.2566 2.449 2.5866z"/><path d="m343.39 709.81v-0.0184l-9e-3 -9e-3v-0.0273l-9e-3 -0.0275-9e-3 -0.0367-9e-3 -0.0275v-0.0276l-9e-3 -0.0275-9e-3 -0.0276v-0.0183l-9e-3 -9e-3v-0.0457l-9e-3 -9e-3v-0.0364l-9e-3 -9e-3v-0.036c0-0.16509 0.12844-0.24764 0.26599-0.24764 0.1101 0 0.27516 0.0734 0.33937 0.25682 0.0183 0.0367 0.3302 1.275 0.35773 1.44 0.0825 0.30269 0.23847 0.93558 0.2935 1.1832 0.0367 0.1192 0.29351 0.55034 0.51365 0.74295 0.0734 0.0642 0.33938 0.30268 0.73379 0.30268 0.23847 0 0.37607-0.11009 0.38523-0.11009-0.27516-0.0458-0.47696-0.266-0.47696-0.4953 0-0.14676 0.10085-0.32103 0.34855-0.32103 0.24765 0 0.50447 0.21096 0.50447 0.53199 0 0.32104-0.2935 0.59621-0.76129 0.59621-0.5962 0-0.99979-0.44945-1.174-0.6971-0.0734 0.40358-0.39442 0.6971-0.82551 0.6971-0.42193 0-0.59621-0.35773-0.67875-0.52284-0.1651-0.30268-0.28435-0.85302-0.28435-0.88053 0-0.0917 0.0917-0.0917 0.1101-0.0917 0.0917 0 0.10085 9e-3 0.15593 0.21096 0.15592 0.64206 0.33938 1.0823 0.66958 1.0823 0.15593 0 0.28433-0.0734 0.28433-0.41275 0-0.19261-0.0275-0.29351-0.14675-0.77047z"/><path d="m351.68 715.5v0.018l-9e-3 9e-3v0.018h-9e-3v9e-3l-0.018 0.018h-9e-3v9e-3h-0.018v9e-3h-0.054c-0.13759 0-1.009-0.0825-1.1649-0.0917-0.0734-9e-3 -0.12844-0.055-0.12844-0.17428 0-0.11009 0.0825-0.11009 0.22013-0.11009 0.44026 0 0.45861-0.0642 0.45861-0.15595l-0.0275-0.18343-0.55034-2.1647c-0.1651 0.33937-0.42192 0.58703-0.83467 0.58703-1.0732 0-2.2105-1.3392-2.2105-2.6691 0-0.85302 0.50448-1.4584 1.2199-1.4584l0.0183 0.20179c-0.45862 0-0.58702 0.50448-0.58702 0.85302 0 0.45862 0.2935 1.5776 0.50447 1.9996 0.28434 0.53199 0.69709 0.87136 1.064 0.87136 0.58703 0 0.71544-0.74296 0.71544-0.79799 0-0.0551-0.0183-0.1101-0.0275-0.15593l-0.45863-1.7886c-0.0367-0.1651-0.0367-0.18345-0.17427-0.33938-0.40358-0.49529-0.77964-0.64206-1.0365-0.64206l-0.0183-0.20179c0.18345 0 0.64206 0.0367 1.1924 0.68792 0.0642-0.38524 0.38524-0.68792 0.8255-0.68792 0.32103 0 0.53199 0.21096 0.67876 0.50448 0.15592 0.3302 0.27515 0.88054 0.27515 0.89889 0 0.0917-0.0825 0.0917-0.11009 0.0917-0.0917 0-0.10085-0.0367-0.12844-0.1651-0.15593-0.5962-0.32103-1.1282-0.6971-1.1282-0.24765 0-0.27517 0.23848-0.27517 0.42192 0 0.21096 0.0183 0.27517 0.055 0.4311z"/><path d="m357.45 715.11v0.126l-9e-3 9e-3v0.027l-9e-3 9e-3v0.018l-9e-3 9e-3h-9e-3v9e-3h-9e-3l-9e-3 9e-3h-0.018v9e-3h-0.072v9e-3h-0.063c-0.56868-0.58702-1.3758-0.58702-1.6694-0.58702v-0.28435c0.18345 0 0.72462 0 1.2016 0.23849v-4.7146c0-0.33021-0.0275-0.44028-0.85303-0.44028h-0.29352v-0.28434c0.32103 0.0276 1.119 0.0276 1.4859 0.0276s1.1649 0 1.4859-0.0276v0.28434h-0.2935c-0.82552 0-0.85303 0.10086-0.85303 0.44028z"/><path d="m368.08 712.48v0.0274l9e-3 0.0183v0.0183l9e-3 0.0183v0.0274l9e-3 0.0183 0.018 0.0914 0.0183 0.0458 0.018 0.0914 9e-3 0.0183v0.0183l9e-3 0.0275v0.0183l9e-3 0.0183v0.0183l9e-3 0.0183v0.0273l9e-3 0.0183v0.0363l9e-3 9e-3v9e-3c0 0.16473-0.12816 0.2471-0.2654 0.2471-0.10985 0-0.27454-0.0732-0.3386-0.25626-0.0274-0.064-0.45759-1.8029-0.52164-2.0316-0.064-0.27456-0.0823-0.43929-0.0823-0.60402 0-0.10063 0-0.11893 9e-3 -0.16473-0.21049-0.48504-0.48504-0.75958-0.84195-0.75958-0.73213 0-0.73213 0.66807-0.73213 0.82365 0 0.29284 0.0549 0.64977 0.47588 1.7663 0.10063 0.2654 0.15558 0.39351 0.15558 0.57655 0 0.41182-0.30202 0.75044-0.75044 0.75044-0.86026 0-1.1989-1.3178-1.1989-1.4002 0-0.0915 0.0915-0.0915 0.10985-0.0915 0.0915 0 0.10062 0.0183 0.14643 0.16473 0.24709 0.84195 0.604 1.1257 0.92431 1.1257 0.0823 0 0.21964-9e-3 0.21964-0.30202 0-0.22878-0.0915-0.48503-0.16473-0.63146-0.39351-1.0799-0.50333-1.4917-0.50333-1.8395 0-0.8328 0.61315-1.144 1.2904-1.144 0.15558 0 0.58571 0 0.95177 0.63146 0.23795-0.57655 0.86941-0.63146 1.144-0.63146 0.68637 0 1.0799 0.56741 1.3178 1.1165 0.31116 0.70468 0.60402 1.931 0.60402 2.3611 0 0.50333-0.24711 0.64061-0.40268 0.64061-0.22879 0-0.45758-0.23795-0.45758-0.43928 0-0.11894 0.0549-0.17389 0.13727-0.24709 0.10063-0.10063 0.32946-0.32946 0.32946-0.76875 0-0.31115-0.2654-1.1989-0.50333-1.6656-0.23795-0.48504-0.55826-0.79619-0.99754-0.79619-0.43012 0-0.67722 0.2654-0.67722 0.78704 0 0.25624 0.064 0.53995 0.10062 0.66806z"/><path d="m374.91 711.89v0.0823l-9e-3 0.0915-0.018 0.1646-0.0183 0.0732-0.0183 0.0823-0.0548 0.1464-0.0275 0.064-0.0274 0.0732-0.0366 0.064-0.0366 0.0549-0.0366 0.064-0.1371 0.1647-0.1647 0.1371-0.1098 0.0732-0.064 0.0366-0.064 0.0274-0.064 0.0275-0.064 0.0274-0.0732 0.0183-0.064 0.0183-0.0732 0.0183-0.2196 0.027h-0.0823c-1.2172 0-2.4435-1.2904-2.4435-2.5808 0-0.90601 0.61316-1.5375 1.4643-1.5375l9e-3 0.20135c-0.39351 0-0.7962 0.27454-0.7962 0.98837 0 0.44843 0.23795 1.4277 0.5308 1.8852 0.45759 0.70468 0.97008 0.84196 1.2263 0.84196 0.5308 0 0.80535-0.43928 0.80535-0.97923 0-0.35691-0.18304-1.3087-0.5308-1.9035-0.3203-0.5308-0.82365-0.8328-1.2355-0.8328l-9e-3 -0.20135c1.208 0 2.4435 1.2538 2.4435 2.5808z"/><path d="m375.86 709.94v-9e-3l-9e-3 -0.0183v-0.0273l-9e-3 -0.0274-9e-3 -0.0275-9e-3 -0.0274v-0.0274l-9e-3 -0.0366-9e-3 -0.0274v-9e-3l-9e-3 -0.0183v-0.0363l-9e-3 -9e-3v-0.0363l-9e-3 -9e-3v-0.036c0-0.16473 0.12816-0.2471 0.2654-0.2471 0.10985 0 0.27455 0.0732 0.33862 0.25624 0.0183 0.0366 0.32946 1.2721 0.35691 1.4277 0.0823 0.302 0.23795 0.94262 0.29286 1.1897 0.0366 0.11894 0.29285 0.53995 0.51249 0.74128 0.0732 0.064 0.3386 0.30202 0.73213 0.30202 0.23795 0 0.37521-0.10985 0.38437-0.10985-0.27455-0.0458-0.47589-0.2654-0.47589-0.50335 0-0.13727 0.10063-0.31115 0.34776-0.31115 0.2471 0 0.50335 0.20133 0.50335 0.53079 0 0.32031-0.29286 0.59486-0.75959 0.59486-0.59486 0-0.99753-0.44843-1.1714-0.70468-0.0732 0.41183-0.39351 0.70468-0.82364 0.70468-0.42097 0-0.59486-0.35692-0.67723-0.52164-0.16472-0.31116-0.28369-0.8511-0.28369-0.87856 0-0.0915 0.0915-0.0915 0.10984-0.0915 0.0915 0 0.10063 9e-3 0.15558 0.21049 0.15558 0.64061 0.3386 1.0799 0.66807 1.0799 0.15557 0 0.2837-0.0732 0.2837-0.42099 0-0.19218-0.0274-0.29285-0.14642-0.75958z"/><path d="m384.13 715.61v0.027l-9e-3 9e-3v0.018h-9e-3v9e-3h-9e-3v9e-3h-9e-3v9e-3h-9e-3v9e-3h-0.027v9e-3h-0.045c-0.13727 0-1.0067-0.0823-1.1623-0.10063-0.0732-9e-3 -0.12815-0.0549-0.12815-0.17388 0-0.10985 0.0823-0.10985 0.21964-0.10985 0.43928 0 0.45759-0.064 0.45759-0.15557l-0.0275-0.17389-0.5491-2.1598c-0.16472 0.33861-0.42098 0.58571-0.8328 0.58571-1.0707 0-2.2055-1.3361-2.2055-2.6631 0-0.86026 0.50333-1.4551 1.2172-1.4551l0.0183 0.20135c-0.45758 0-0.58571 0.49418-0.58571 0.85109 0 0.45759 0.29286 1.5741 0.50335 1.9951 0.2837 0.53079 0.69552 0.86941 1.0616 0.86941 0.58571 0 0.71383-0.75044 0.71383-0.80535 0-0.0549-0.0183-0.10063-0.0274-0.14644l-0.45758-1.7846c-0.0366-0.16473-0.0366-0.18303-0.17389-0.33862-0.40266-0.49418-0.77788-0.6406-1.0341-0.6406l-0.0183-0.20135c0.18303 0 0.64062 0.0366 1.1897 0.67723 0.064-0.37522 0.38437-0.67723 0.82366-0.67723 0.3203 0 0.53079 0.21049 0.67722 0.49419 0.15557 0.32947 0.27455 0.88771 0.27455 0.90602 0 0.0915-0.0823 0.0915-0.10985 0.0915-0.0915 0-0.10063-0.0366-0.12815-0.16473-0.15558-0.59485-0.32031-1.1256-0.69553-1.1256-0.24709 0-0.27454 0.22879-0.27454 0.41182 0 0.21965 0.0183 0.2837 0.0549 0.43927z"/><path d="m388.36 710.1 0.97008 0.94263c1.4277 1.2538 1.9676 1.7388 1.9676 2.654 0 1.0341-0.8145 1.7571-1.931 1.7571-1.0341 0-1.7022-0.83281-1.7022-1.6473 0-0.50335 0.44843-0.50335 0.47588-0.50335 0.15558 0 0.47589 0.10985 0.47589 0.48504 0 0.2288-0.16473 0.46673-0.48504 0.46673-0.0732 0-0.0915 0-0.11894-9e-3 0.21049 0.59486 0.70467 0.92433 1.2355 0.92433 0.83281 0 1.2263-0.73213 1.2263-1.4734 0-0.73214-0.45758-1.446-0.96093-2.0134l-1.7571-1.9493c-0.0915-0.10063-0.0915-0.11894-0.0915-0.32946h3.377l0.25624 1.5832h-0.22879c-0.0366-0.27455-0.10985-0.67723-0.19218-0.8145-0.064-0.0732-0.66808-0.0732-0.86941-0.0732z"/><path d="m399.32 710.76v0.0146l-7e-3 0.0219v0.0365l-7e-3 0.0219-7e-3 0.0146-7e-3 0.0219-0.014 0.0292-7e-3 0.0219-0.014 0.0292-0.0146 0.0146-7e-3 0.0146-0.0438 0.0438-0.0146 7e-3 -0.0146 0.0146-0.0146 7e-3 -0.0146 0.0146-0.0292 0.014-0.0219 7e-3 -0.0146 7e-3 -0.022 7e-3 -0.0146 7e-3h-0.022l-0.0146 7e-3h-0.0585c-0.21216 0-0.38773-0.16825-0.38773-0.38041 0-0.21215 0.17557-0.38772 0.38773-0.38772 0.21215 0 0.38772 0.17557 0.38772 0.38772z"/><path d="m402.56 710.76v0.0365l-7e-3 0.0219v0.0146l-7e-3 0.0219-7e-3 0.0146v0.0219l-0.014 0.0292-0.0146 0.0219-0.014 0.0292-0.0146 0.0146-7e-3 0.0146-0.0438 0.0438-0.0146 7e-3 -0.0146 0.0146-0.0146 7e-3 -0.0146 0.0146-0.0292 0.014-0.022 7e-3 -0.0292 0.014-0.022 7e-3h-0.0219l-0.0146 7e-3h-0.0585c-0.21215 0-0.38773-0.16825-0.38773-0.38041 0-0.21215 0.17558-0.38772 0.38773-0.38772s0.38773 0.17557 0.38773 0.38772z"/><path d="m405.8 710.76v0.0365l-7e-3 0.0219v0.0146l-7e-3 0.0219v0.0146l-7e-3 0.0219-0.014 0.0292-0.0146 0.0219-0.014 0.0292-0.0146 0.0146-7e-3 0.0146-0.0654 0.0654-0.0146 7e-3 -0.022 0.0146-0.0438 0.021-0.022 7e-3 -0.0146 7e-3 -0.022 7e-3h-0.0146l-0.022 7e-3h-0.0585c-0.21215 0-0.38773-0.16825-0.38773-0.38041 0-0.21215 0.17558-0.38772 0.38773-0.38772s0.38772 0.17557 0.38772 0.38772z"/><path d="m418.36 712.48 9e-3 0.0182v0.0182l9e-3 0.0182 9e-3 0.0273v0.0182l9e-3 0.0273 9e-3 0.0363 9e-3 0.0453 9e-3 0.0455 0.0182 0.0453 9e-3 0.0453v0.0272l0.018 0.0364v0.0182l9e-3 0.0182v0.0363l9e-3 0.0182v0.0182l9e-3 9e-3v0.0632c0 0.15431-0.11797 0.23601-0.25416 0.23601-0.10895 0-0.27231-0.0726-0.33584-0.24509-0.0272-0.0635-0.45387-1.7882-0.51741-2.0242-0.0635-0.27231-0.0817-0.4357-0.0817-0.59909 0-0.0908 0-0.11797 9e-3 -0.15431-0.20878-0.48109-0.48109-0.76247-0.83509-0.76247-0.72618 0-0.72618 0.6717-0.72618 0.82601 0 0.28139 0.0453 0.6354 0.47201 1.7428 0.0998 0.27231 0.15431 0.39938 0.15431 0.58093 0 0.40847-0.29954 0.73525-0.74431 0.73525-0.85325 0-1.1891-1.3071-1.1891-1.3888 0-0.0908 0.0908-0.0908 0.10895-0.0908 0.0908 0 0.0998 0.0182 0.14523 0.16339 0.24509 0.84417 0.59002 1.1165 0.90771 1.1165 0.0816 0 0.22694-9e-3 0.22694-0.29048 0-0.22693-0.0998-0.48109-0.16339-0.63539-0.39032-1.062-0.49925-1.4796-0.49925-1.8245 0-0.81694 0.59909-1.1346 1.2799-1.1346 0.15431 0 0.58094 0 0.94402 0.6354 0.23601-0.58094 0.86233-0.6354 1.1256-0.6354 0.68078 0 1.0802 0.57186 1.3162 1.1074 0.30862 0.70802 0.59908 1.9153 0.59908 2.351 0 0.49016-0.24507 0.62632-0.39938 0.62632-0.22693 0-0.45385-0.22693-0.45385-0.42663 0-0.11796 0.0545-0.17246 0.13615-0.24507 0.0998-0.0998 0.32678-0.33586 0.32678-0.77156 0-0.29954-0.26324-1.1891-0.49925-1.652-0.236-0.47202-0.5537-0.7897-0.99848-0.7897-0.41754 0-0.66262 0.2723-0.66262 0.78062 0 0.25416 0.0635 0.53556 0.0998 0.66263z"/><path d="m425.15 711.88-9e-3 0.0908v0.0817l-9e-3 0.0816-0.0182 0.0816-9e-3 0.0816-0.0182 0.0726-0.0272 0.0726-0.0273 0.0726-0.0272 0.0726-0.0272 0.0635-0.0726 0.127-0.0453 0.0545-0.0363 0.0635-0.0453 0.0455-0.0998 0.0998-0.109 0.0906-0.109 0.0726-0.127 0.0726-0.0635 0.0273-0.0635 0.0181-0.0726 0.0272-0.0726 0.0182-0.0635 9e-3 -0.0726 0.0182-0.0726 9e-3h-0.1543c-1.2163 0-2.4236-1.2799-2.4236-2.5597 0-0.88956 0.60816-1.525 1.4432-1.525l9e-3 0.1997c-0.38124 0-0.78063 0.28138-0.78063 0.98032 0 0.44477 0.236 1.416 0.5174 1.879 0.45385 0.69894 0.97125 0.82602 1.2254 0.82602 0.52647 0 0.79878-0.42662 0.79878-0.97125 0-0.35401-0.18154-1.298-0.52647-1.879-0.3177-0.52647-0.81694-0.83509-1.2345-0.83509l-9e-3 -0.1997c1.2072 0 2.4327 1.2436 2.4327 2.5597z"/><path d="m426.08 709.96-9e-3 -0.0182v-0.0272l-9e-3 -9e-3v-0.0272l-9e-3 -0.0272-9e-3 -0.0363-9e-3 -0.0272v-0.0273l-9e-3 -0.0272v-0.0182l-9e-3 -9e-3v-0.0272l-9e-3 -9e-3v-0.0362l-9e-3 -9e-3v-0.054c0-0.16339 0.12711-0.24509 0.26324-0.24509 0.10895 0 0.26323 0.0726 0.32678 0.25415 0.0181 0.0363 0.32676 1.2617 0.36308 1.4251 0.0726 0.29955 0.236 0.92587 0.29046 1.171 0.0363 0.11796 0.29047 0.54463 0.50832 0.74432 0.0726 0.0635 0.33585 0.29047 0.72617 0.29047 0.236 0 0.37216-0.10895 0.38124-0.10895-0.27232-0.0363-0.47201-0.25417-0.47201-0.49017 0-0.14523 0.0998-0.31769 0.34492-0.31769 0.24508 0 0.49924 0.20877 0.49924 0.53555 0 0.31769-0.29046 0.58093-0.75339 0.58093-0.59002 0-0.9894-0.4357-1.1619-0.68986-0.0726 0.40847-0.3994 0.68986-0.82601 0.68986-0.40848 0-0.58094-0.34494-0.66264-0.50833-0.16339-0.30861-0.28139-0.85323-0.28139-0.88047 0-0.0908 0.0908-0.0908 0.10895-0.0908 0.0908 0 0.0998 9e-3 0.15431 0.20877 0.15432 0.64448 0.33586 1.0711 0.65356 1.0711 0.15431 0 0.28139-0.0635 0.28139-0.40848 0-0.19061-0.0272-0.29046-0.14524-0.76247z"/><path d="m434.29 715.59v0.018l-9e-3 9e-3v0.018h-9e-3v9e-3h-9e-3v9e-3l-9e-3 9e-3h-9e-3v9e-3h-9e-3l-9e-3 9e-3h-0.036v9e-3h-0.018c-0.13616 0-0.99848-0.0816-1.1528-0.0998-0.0726-9e-3 -0.12711-0.0545-0.12711-0.17245 0-0.10895 0.0817-0.10895 0.21786-0.10895 0.4357 0 0.45385-0.0635 0.45385-0.15431l-0.0272-0.18155-0.54463-2.1331c-0.16339 0.32676-0.42662 0.57185-0.83509 0.57185-1.0529 0-2.1785-1.3252-2.1785-2.6414 0-0.84417 0.49925-1.4433 1.2073-1.4433l0.0182 0.1997c-0.45385 0-0.58094 0.49924-0.58094 0.84416 0 0.45386 0.29048 1.5613 0.49925 1.9788 0.28138 0.53555 0.68078 0.86233 1.0439 0.86233 0.59001 0 0.71709-0.73526 0.71709-0.78971 0-0.0545-0.0182-0.10895-0.0272-0.15432l-0.45386-1.77c-0.0453-0.15431-0.0453-0.18153-0.18153-0.32677-0.39032-0.49924-0.76249-0.64447-1.0166-0.64447l-0.0182-0.1997c0.18155 0 0.6354 0.0363 1.171 0.68078 0.0726-0.38123 0.39032-0.68078 0.82602-0.68078 0.31769 0 0.52647 0.20877 0.6717 0.49924 0.15431 0.32678 0.27232 0.8714 0.27232 0.88956 0 0.0908-0.0816 0.0908-0.10895 0.0908-0.0908 0-0.0998-0.0363-0.12711-0.16339-0.15431-0.58093-0.3177-1.1165-0.68987-1.1165-0.24507 0-0.27231 0.236-0.27231 0.41754 0 0.21784 0.0182 0.28138 0.0545 0.42661z"/><path d="m443.02 714.55 0.0363 0.0635 0.0453 0.0635 0.0363 0.0545 0.0455 0.0545 0.0363 0.0545 0.0816 0.0816 0.0363 0.0453 0.0363 0.0363 0.0453 0.0273 0.0363 0.0363 0.0363 0.0273 0.0363 0.0272 0.0363 0.0182 0.0363 0.0272 0.1089 0.0546 0.0726 0.018 0.0273 9e-3 0.1089 0.027 0.0272 9e-3h0.0363l0.0635 9e-3h0.0635l0.0635 9e-3c0.10895 9e-3 0.11797 0.16339 0.11797 0.17246 0 0.0726-0.0455 0.10895-0.11797 0.10895-0.23601 0-0.49017-0.0272-0.73525-0.0272-0.29955 0-0.60817 0.0272-0.89863 0.0272-0.0545 0-0.17247 0-0.17247-0.17246 0-0.0998 0.0817-0.10895 0.14523-0.10895 0.24509-0.0182 0.41755-0.10895 0.41755-0.29955 0-0.13615-0.13615-0.33585-0.13615-0.34493l-2.7776-4.3933-0.61725 4.7655c0 0.15431 0.20879 0.27232 0.62633 0.27232 0.12711 0 0.22692 0 0.22692 0.18153 0 0.0816-0.0726 0.0998-0.12711 0.0998-0.36308 0-0.7534-0.0272-1.1256-0.0272-0.16338 0-0.33585 9e-3 -0.49923 9e-3 -0.16339 0-0.33586 0.0182-0.49017 0.0182-0.0635 0-0.17246 0-0.17246-0.17245 0-0.10895 0.0816-0.10895 0.22693-0.10895 0.50831 0 0.51739-0.0816 0.54461-0.30862l0.7171-5.5643c0.0272-0.18153 0.0635-0.20877 0.18154-0.20877 0.14524 0 0.18154 0.0453 0.25417 0.16339z"/></g>
   <g transform="matrix(.43834 0 0 -.43834 -55.492 474.43)" fill-rule="evenodd" stroke-miterlimit="10.433" stroke-width="0" ns0:preamble="default_packages.tex" ns0:text="$$doc 1$$\n$$doc 2$$\n$$\\cdots$$\n$$doc N$$\n" xml:space="preserve"><g fill-opacity=".86275"><path d="m299.84 701.34v0.012h-0.012v0.036l-0.012 0.012v0.012h-0.012v0.012h-0.012v0.012h-0.012l-0.012 0.012h-0.012l-0.012 0.012h-0.06c-0.1784 0-1.3082-0.10705-1.5104-0.11889-0.0952-0.012-0.1665-0.0714-0.1665-0.22598 0-0.14272 0.10704-0.14272 0.28544-0.14272 0.57086 0 0.59465-0.0832 0.59465-0.20218l-0.0357-0.23787-0.71359-2.8068c-0.21408 0.44004-0.55898 0.76116-1.0942 0.76116-1.3796 0-2.8544-1.7364-2.8544-3.4609 0-1.1061 0.65413-1.891 1.5818-1.891l0.012 0.26165c-0.58276 0-0.74927 0.65413-0.74927 1.1061 0 0.59466 0.38058 2.0456 0.65413 2.5927 0.35681 0.68982 0.892 1.1299 1.3677 1.1299 0.77306 0 0.93957-0.96336 0.93957-1.0347 0-0.0714-0.0238-0.14273-0.0357-0.20219l-0.59467-2.3192c-0.0595-0.21407-0.0595-0.23786-0.23786-0.44005-0.52331-0.64223-1.0109-0.83253-1.344-0.83253l-0.012-0.26165c0.22597 0 0.82064 0.0475 1.5342 0.89201 0.0952-0.49952 0.51141-0.89201 1.0823-0.89201 0.41626 0 0.68981 0.27355 0.8801 0.65414 0.20218 0.42815 0.3568 1.1417 0.3568 1.1655 0 0.1189-0.10705 0.1189-0.14273 0.1189-0.11889 0-0.13087-0.0475-0.1665-0.21408-0.20219-0.77306-0.41627-1.4629-0.90388-1.4629-0.32112 0-0.3568 0.30923-0.3568 0.5471 0 0.27354 0.0238 0.35679 0.0714 0.55898z"/><path d="m305.44 696.49v0.1189l-0.012 0.10704-0.012 0.10705-0.012 0.10704-0.0238 0.10705-0.0476 0.1904-0.0714 0.1904-0.1425 0.2496-0.0475 0.0714-0.1785 0.2142-0.0595 0.0595-0.1428 0.119-0.0714 0.0475-0.1664 0.095-0.0714 0.0357-0.0952 0.0357-0.0832 0.0357-0.0832 0.0238-0.1904 0.0476-0.2856 0.036h-0.0952c-1.5937 0-3.1874-1.6769-3.1874-3.3539 0-1.1655 0.79686-1.9981 1.9029-1.9981l0.012 0.26165c-0.51142 0-1.0347 0.36869-1.0347 1.2845 0 0.58277 0.30922 1.8554 0.68981 2.4619 0.59467 0.90388 1.2726 1.0823 1.6056 1.0823 0.67791 0 1.0347-0.57088 1.0347-1.2726 0-0.46383-0.23787-1.7007-0.67792-2.4738-0.41626-0.67791-1.0704-1.0823-1.6175-1.0823l-0.012-0.26165c1.5818 0 3.1755 1.6294 3.1755 3.3539z"/><path d="m310.31 697.76-0.0357-0.012h-0.1786l-0.0238-0.012h-0.024l-0.0238-0.012h-0.024l-0.0238-0.012h-0.012l-0.012-0.012-0.0238-0.012h-0.012l-0.012-0.012-0.0238-0.012-0.024-0.024-0.0238-0.012-0.024-0.024-0.0238-0.012c-0.19029-0.1784-0.20218-0.38059-0.20218-0.46384 0-0.28544 0.20218-0.41626 0.42816-0.41626 0.3449 0 0.66602 0.28544 0.66602 0.76117 0 0.57087-0.55899 1.0109-1.3915 1.0109-1.6056 0-3.1874-1.6888-3.1874-3.3539 0-1.0704 0.6898-1.9981 1.9267-1.9981 1.6888 0 2.6879 1.2488 2.6879 1.3915 0 0.0714-0.0714 0.1546-0.14273 0.1546-0.0595 0-0.0832-0.0238-0.1546-0.11889-0.93957-1.1655-2.224-1.1655-2.3668-1.1655-0.74927 0-1.0704 0.58277-1.0704 1.2845 0 0.48763 0.23787 1.6294 0.64223 2.3668 0.36869 0.66602 1.0228 1.1774 1.677 1.1774 0.40436 0 0.84442-0.15462 1.0109-0.46385z"/><path d="m316.87 700.74v0.156l-0.012 0.012v0.048l-0.012 0.012v0.012l-0.012 0.012v0.012h-0.012v0.012h-0.012l-0.012 0.012h-0.024v0.012h-0.096v0.012h-0.084c-0.73737-0.76116-1.784-0.76116-2.1646-0.76116v-0.3687c0.23787 0 0.93957 0 1.558 0.30922v-6.1131c0-0.42815-0.0357-0.57088-1.1061-0.57088h-0.38058v-0.36868c0.41626 0.0357 1.451 0.0357 1.9267 0.0357 0.47574 0 1.5104 0 1.9148-0.0357v0.36868h-0.38059c-1.0585 0-1.0942 0.13088-1.0942 0.57088z"/><path d="m298.89 683.62v0.0244h-0.0122v0.0366l-0.0122 0.0122v0.0122h-0.0122l-0.0122 0.0122v0.0122h-0.0244v0.0122h-0.0244l-0.0122 0.0122h-0.0488c-0.17989 0-1.3192-0.10792-1.5231-0.13191-0.096-0.0122-0.1679-0.072-0.1679-0.22787 0-0.14391 0.10791-0.14391 0.28782-0.14391 0.57565 0 0.59965-0.0839 0.59965-0.20388l-0.0359-0.22787-0.71957-2.8303c-0.21587 0.44373-0.56366 0.76753-1.1033 0.76753-1.3912 0-2.8783-1.751-2.8783-3.4899 0-1.1273 0.65961-1.9068 1.595-1.9068l0.0122 0.26383c-0.58765 0-0.75554 0.64762-0.75554 1.1153 0 0.59963 0.38377 2.0628 0.6596 2.6144 0.35978 0.69558 0.89946 1.1393 1.3792 1.1393 0.77953 0 0.94742-0.98342 0.94742-1.0554 0-0.072-0.024-0.13191-0.0359-0.19187l-0.59963-2.3386c-0.0599-0.21587-0.0599-0.23985-0.23987-0.44373-0.52768-0.64761-1.0194-0.8395-1.3552-0.8395l-0.0122-0.26383c0.22786 0 0.82751 0.048 1.5471 0.88746 0.096-0.4917 0.51569-0.88746 1.0914-0.88746 0.41974 0 0.69557 0.27583 0.88746 0.64761 0.20388 0.43173 0.35979 1.1633 0.35979 1.1873 0 0.11998-0.10792 0.11998-0.14393 0.11998-0.11998 0-0.13191-0.048-0.16789-0.21587-0.20388-0.77953-0.41975-1.4751-0.91145-1.4751-0.32381 0-0.35979 0.29982-0.35979 0.53968 0 0.28783 0.024 0.37177 0.072 0.57566z"/><path d="m304.54 678.73v0.10792l-0.0122 0.11998-0.0122 0.10791-0.0122 0.10792-0.024 0.096-0.024 0.10792-0.024 0.096-0.0359 0.096-0.0359 0.0839-0.048 0.096-0.048 0.0839-0.048 0.072-0.048 0.0839-0.1797 0.216-0.0599 0.0599-0.144 0.1198-0.072 0.048-0.1678 0.096-0.072 0.0359-0.096 0.0359-0.0839 0.0359-0.0839 0.024-0.192 0.048-0.288 0.0366h-0.096c-1.607 0-3.2141-1.691-3.2141-3.382 0-1.1873 0.80351-2.0148 1.9188-2.0148l0.0122 0.26384c-0.51568 0-1.0434 0.35979-1.0434 1.2952 0 0.58764 0.31181 1.8709 0.69557 2.4705 0.59964 0.92344 1.2832 1.1033 1.619 1.1033 0.68359 0 1.0434-0.57566 1.0434-1.2832 0-0.46772-0.23985-1.715-0.68358-2.4945-0.41974-0.69557-1.0794-1.0913-1.631-1.0913l-0.0122-0.26384c1.595 0 3.2021 1.643 3.2021 3.382z"/><path d="m309.46 680.01h-0.1676l-0.0359-0.0122h-0.0484l-0.0122-0.0122h-0.0362l-0.0122-0.0122h-0.024l-0.0122-0.0122h-0.0122l-0.024-0.0122-0.0122-0.0122h-0.0122l-0.024-0.0122-0.0244-0.0244-0.024-0.0122-0.0484-0.0484c-0.19188-0.17988-0.20387-0.38376-0.20387-0.46771 0-0.27584 0.20387-0.40776 0.43174-0.40776 0.34779 0 0.67159 0.28783 0.67159 0.75555 0 0.58764-0.56366 1.0314-1.4032 1.0314-1.619 0-3.2141-1.703-3.2141-3.382 0-1.0794 0.69558-2.0148 1.9428-2.0148 1.703 0 2.7104 1.2592 2.7104 1.4032 0 0.072-0.072 0.15591-0.14391 0.15591-0.0599 0-0.0839-0.024-0.1559-0.11998-0.94744-1.1753-2.2426-1.1753-2.3866-1.1753-0.75555 0-1.0794 0.57566-1.0794 1.2952 0 0.4917 0.23985 1.643 0.64761 2.3866 0.37177 0.67159 1.0314 1.1873 1.691 1.1873 0.40776 0 0.85149-0.15591 1.0194-0.47972z"/><path d="m313.61 676.03 1.2712 1.2352c1.8589 1.643 2.5784 2.2786 2.5784 3.4779 0 1.3552-1.0674 2.3026-2.5305 2.3026-1.3552 0-2.2307-1.0913-2.2307-2.1587 0-0.65961 0.58765-0.65961 0.62363-0.65961 0.20388 0 0.62362 0.14391 0.62362 0.63561 0 0.29983-0.21587 0.61164-0.63561 0.61164-0.096 0-0.11998 0-0.1559-0.0122 0.27583 0.77953 0.92344 1.2113 1.619 1.2113 1.0913 0 1.607-0.95943 1.607-1.9308 0-0.95942-0.59964-1.8948-1.2592-2.6384l-2.3026-2.5545c-0.11998-0.13191-0.11998-0.1559-0.11998-0.43174h4.4254l0.33579 2.0748h-0.29981c-0.0599-0.35979-0.14392-0.88747-0.26385-1.0674-0.0839-0.096-0.86348-0.096-1.1273-0.096z"/><path d="m300.9 661.79v0.02l-0.01 0.03v0.05l-0.01 0.03-0.01 0.02-0.01 0.03-0.02 0.04-0.01 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.06 0.06-0.02 0.01-0.02 0.02-0.02 0.01-0.02 0.02-0.04 0.02-0.03 0.01-0.02 0.01-0.03 0.01-0.02 0.01h-0.03l-0.02 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m305.33 661.79v0.05l-0.01 0.03v0.02l-0.01 0.03-0.01 0.02v0.03l-0.02 0.04-0.02 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.06 0.06-0.02 0.01-0.02 0.02-0.02 0.01-0.02 0.02-0.04 0.02-0.03 0.01-0.04 0.02-0.03 0.01h-0.03l-0.02 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m309.75 661.79v0.05l-0.01 0.03v0.02l-0.01 0.03v0.02l-0.01 0.03-0.02 0.04-0.02 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.09 0.09-0.02 0.01-0.03 0.02-0.06 0.03-0.03 0.01-0.02 0.01-0.03 0.01h-0.02l-0.03 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m297.54 647.1v0.0333l-0.0111 0.0111v0.0111l-0.0222 0.0222v0.0111h-0.0111l-0.0111 0.0111h-0.0111v0.0111h-0.0555v0.0111h-0.0222c-0.16622 0-1.219-0.0997-1.3963-0.1219-0.0997-0.0111-0.16622-0.0664-0.16622-0.21055 0-0.13298 0.0997-0.13298 0.26596-0.13298 0.53191 0 0.55409-0.0775 0.55409-0.18838l-0.0333-0.22164-0.65382-2.6042c-0.21055 0.39895-0.52085 0.69815-1.0195 0.69815-1.2966 0-2.6707-1.6179-2.6707-3.2248 0-1.0306 0.60949-1.762 1.4739-1.762l0.0222 0.2438c-0.55409 0-0.70923 0.60949-0.70923 1.0306 0 0.55409 0.35462 1.9061 0.60949 2.4158 0.34354 0.65383 0.84221 1.0528 1.2855 1.0528 0.72032 0 0.87547-0.89761 0.87547-0.96411 0-0.0664-0.0333-0.13298-0.0444-0.18839l-0.543-2.1609c-0.0555-0.18839-0.0555-0.22163-0.22164-0.39895-0.4876-0.60949-0.94194-0.78679-1.2522-0.78679l-0.0222-0.2438c0.22163 0 0.77571 0.0444 1.4406 0.83113 0.0886-0.46543 0.46543-0.83113 0.99736-0.83113 0.38786 0 0.64274 0.25488 0.82005 0.6095 0.18838 0.39893 0.33245 1.0638 0.33245 1.086 0 0.11081-0.0997 0.11081-0.13299 0.11081-0.1108 0-0.1219-0.0444-0.15514-0.19948-0.18839-0.70923-0.38785-1.363-0.84221-1.363-0.28813 0-0.32137 0.28812-0.32137 0.50975 0 0.26596 0.0111 0.34354 0.0666 0.52084z"/><path d="m302.77 642.58-0.0111 0.11081v0.0997l-0.0222 0.1994-0.0222 0.0997-0.0222 0.0886-0.0999 0.2658-0.0333 0.0775-0.0888 0.155-0.0555 0.0664-0.0444 0.0775-0.0555 0.0555-0.0555 0.0664-0.0664 0.0555-0.0664 0.0553-0.0664 0.0555-0.1328 0.0888-0.155 0.0888-0.0775 0.0333-0.0775 0.0222-0.0886 0.0333-0.0775 0.0222-0.0886 0.0111-0.0886 0.0222-0.0886 0.0111h-0.1883c-1.4739 0-2.9588-1.5625-2.9588-3.125 0-1.086 0.74247-1.8617 1.7731-1.8617v0.2438c-0.46543 0-0.95302 0.34353-0.95302 1.1968 0 0.54301 0.28811 1.7288 0.64273 2.2939 0.54301 0.85329 1.1747 1.0084 1.485 1.0084 0.64273 0 0.97518-0.52084 0.97518-1.1858 0-0.43219-0.22163-1.5847-0.64273-2.2939-0.38787-0.64274-0.99736-1.0195-1.5071-1.0195v-0.2438c1.4628 0 2.9588 1.5182 2.9588 3.125z"/><path d="m307.32 643.77h-0.0999l-0.0333-0.0111h-0.0888l-0.0111-0.0111h-0.0333l-0.0222-0.0111h-0.0111l-0.0111-0.0111h-0.0222l-0.0111-0.0111h-0.0111l-0.0222-0.0111-0.0333-0.0333-0.0222-0.0111-0.0222-0.0222-0.0222-0.0111-0.0111-0.0111c-0.1773-0.16623-0.19947-0.35462-0.19947-0.43218 0-0.26597 0.19947-0.38787 0.41003-0.38787 0.32137 0 0.62057 0.26597 0.62057 0.70923 0 0.54301-0.52084 0.94194-1.3076 0.94194-1.485 0-2.9588-1.5736-2.9588-3.125 0-0.98628 0.64273-1.8617 1.7842-1.8617 1.5847 0 2.5156 1.1747 2.5156 1.2966 0 0.0664-0.0664 0.14406-0.13299 0.14406-0.0555 0-0.0775-0.0222-0.14406-0.0997-0.87545-1.0971-2.0834-1.0971-2.2163-1.0971-0.68707 0-0.98627 0.543-0.98627 1.1968 0 0.45435 0.22163 1.5182 0.58733 2.2053 0.34353 0.63166 0.95303 1.0971 1.5625 1.0971 0.37677 0 0.79789-0.14406 0.95303-0.43218z"/><path d="m317.84 645.85 0.0222 0.0886 0.0111 0.0333 0.0111 0.0444 0.0111 0.0333 0.0222 0.0444 0.0111 0.0333 0.0444 0.0666 0.0111 0.0333 0.0444 0.0666 0.0333 0.0333 0.0222 0.0333 0.0333 0.0222 0.0333 0.0333 0.0333 0.0222 0.0333 0.0333 0.0333 0.0222 0.1332 0.0666 0.0555 0.0111 0.0555 0.0222 0.0555 0.0111 0.0553 0.0111 0.0664 0.0222 0.0664 0.0111h0.0664l0.0664 0.0111 0.0775 0.0111h0.0888c0.0553 0 0.18838 0.0111 0.18838 0.22164 0 0.0111 0 0.1219-0.14406 0.1219-0.36569 0-0.75355-0.0333-1.1192-0.0333-0.37679 0-0.76465 0.0333-1.1304 0.0333-0.0664 0-0.19947 0-0.19947-0.22164 0-0.1219 0.11081-0.1219 0.19947-0.1219 0.63166-0.0111 0.75356-0.24379 0.75356-0.48759 0-0.0333-0.0222-0.19946-0.0333-0.23272l-1.2412-4.8981-2.438 5.7514c-0.0886 0.19947-0.0997 0.21055-0.35462 0.21055h-1.4739c-0.22164 0-0.32138 0-0.32138-0.22164 0-0.1219 0.0997-0.1219 0.3103-0.1219 0.0444 0 0.74247 0 0.74247-0.0997l-1.4739-5.9176c-0.1108-0.43219-0.2992-0.78681-1.1968-0.82005-0.0666 0-0.18839-0.0111-0.18839-0.22163 0-0.0775 0.0555-0.1219 0.14407-0.1219 0.35461 0 0.74247 0.0333 1.1082 0.0333 0.37677 0 0.76464-0.0333 1.1303-0.0333 0.0555 0 0.19947 0 0.19947 0.22163 0 0.11081-0.0997 0.1219-0.22164 0.1219-0.63166 0.0222-0.72031 0.26596-0.72031 0.4876 0 0.0775 0.0111 0.13298 0.0444 0.24379l1.4517 5.8179c0.0444-0.0664 0.0444-0.0886 0.0997-0.19948l2.7594-6.4828c0.0775-0.18838 0.1108-0.21054 0.19947-0.21054 0.1219 0 0.1219 0.0333 0.1773 0.23271z"/></g><g fill-rule="evenodd" stroke-miterlimit="10.433"><path d="m349.99 661.93v0.02l-0.01 0.03v0.05l-0.01 0.03-0.01 0.02-0.01 0.03-0.02 0.04-0.01 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.06 0.06-0.02 0.01-0.02 0.02-0.02 0.01-0.02 0.02-0.04 0.02-0.03 0.01-0.02 0.01-0.03 0.01-0.02 0.01h-0.03l-0.02 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m354.42 661.93v0.05l-0.01 0.03v0.02l-0.01 0.03-0.01 0.02v0.03l-0.02 0.04-0.02 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.06 0.06-0.02 0.01-0.02 0.02-0.02 0.01-0.02 0.02-0.04 0.02-0.03 0.01-0.04 0.02-0.03 0.01h-0.03l-0.02 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m358.84 661.93v0.05l-0.01 0.03v0.02l-0.01 0.03v0.02l-0.01 0.03-0.02 0.04-0.02 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.09 0.09-0.02 0.01-0.03 0.02-0.06 0.03-0.03 0.01-0.02 0.01-0.03 0.01h-0.02l-0.03 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m380.93 661.78v0.02l-0.01 0.03v0.05l-0.01 0.03-0.01 0.02-0.01 0.03-0.02 0.04-0.01 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.06 0.06-0.02 0.01-0.02 0.02-0.02 0.01-0.02 0.02-0.04 0.02-0.03 0.01-0.02 0.01-0.03 0.01-0.02 0.01h-0.03l-0.02 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m385.36 661.78v0.05l-0.01 0.03v0.02l-0.01 0.03-0.01 0.02v0.03l-0.02 0.04-0.02 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.06 0.06-0.02 0.01-0.02 0.02-0.02 0.01-0.02 0.02-0.04 0.02-0.03 0.01-0.04 0.02-0.03 0.01h-0.03l-0.02 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m389.78 661.78v0.05l-0.01 0.03v0.02l-0.01 0.03v0.02l-0.01 0.03-0.02 0.04-0.02 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.09 0.09-0.02 0.01-0.03 0.02-0.06 0.03-0.03 0.01-0.02 0.01-0.03 0.01h-0.02l-0.03 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m415.38 661.09v0.02l-0.01 0.03v0.05l-0.01 0.03-0.01 0.02-0.01 0.03-0.02 0.04-0.01 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.06 0.06-0.02 0.01-0.02 0.02-0.02 0.01-0.02 0.02-0.04 0.02-0.03 0.01-0.02 0.01-0.03 0.01-0.02 0.01h-0.03l-0.02 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m419.81 661.09v0.05l-0.01 0.03v0.02l-0.01 0.03-0.01 0.02v0.03l-0.02 0.04-0.02 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.06 0.06-0.02 0.01-0.02 0.02-0.02 0.01-0.02 0.02-0.04 0.02-0.03 0.01-0.04 0.02-0.03 0.01h-0.03l-0.02 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m424.23 661.09v0.05l-0.01 0.03v0.02l-0.01 0.03v0.02l-0.01 0.03-0.02 0.04-0.02 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.09 0.09-0.02 0.01-0.03 0.02-0.06 0.03-0.03 0.01-0.02 0.01-0.03 0.01h-0.02l-0.03 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m415.53 643.18v0.02l-0.01 0.03v0.05l-0.01 0.03-0.01 0.02-0.01 0.03-0.02 0.04-0.01 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.06 0.06-0.02 0.01-0.02 0.02-0.02 0.01-0.02 0.02-0.04 0.02-0.03 0.01-0.02 0.01-0.03 0.01-0.02 0.01h-0.03l-0.02 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m419.96 643.18v0.05l-0.01 0.03v0.02l-0.01 0.03-0.01 0.02v0.03l-0.02 0.04-0.02 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.06 0.06-0.02 0.01-0.02 0.02-0.02 0.01-0.02 0.02-0.04 0.02-0.03 0.01-0.04 0.02-0.03 0.01h-0.03l-0.02 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m424.38 643.18v0.05l-0.01 0.03v0.02l-0.01 0.03v0.02l-0.01 0.03-0.02 0.04-0.02 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.09 0.09-0.02 0.01-0.03 0.02-0.06 0.03-0.03 0.01-0.02 0.01-0.03 0.01h-0.02l-0.03 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m415.45 697.31v0.02l-0.01 0.03v0.05l-0.01 0.03-0.01 0.02-0.01 0.03-0.02 0.04-0.01 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.06 0.06-0.02 0.01-0.02 0.02-0.02 0.01-0.02 0.02-0.04 0.02-0.03 0.01-0.02 0.01-0.03 0.01-0.02 0.01h-0.03l-0.02 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m419.88 697.31v0.05l-0.01 0.03v0.02l-0.01 0.03-0.01 0.02v0.03l-0.02 0.04-0.02 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.06 0.06-0.02 0.01-0.02 0.02-0.02 0.01-0.02 0.02-0.04 0.02-0.03 0.01-0.04 0.02-0.03 0.01h-0.03l-0.02 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m424.3 697.31v0.05l-0.01 0.03v0.02l-0.01 0.03v0.02l-0.01 0.03-0.02 0.04-0.02 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.09 0.09-0.02 0.01-0.03 0.02-0.06 0.03-0.03 0.01-0.02 0.01-0.03 0.01h-0.02l-0.03 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m415.23 678.39v0.02l-0.01 0.03v0.05l-0.01 0.03-0.01 0.02-0.01 0.03-0.02 0.04-0.01 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.06 0.06-0.02 0.01-0.02 0.02-0.02 0.01-0.02 0.02-0.04 0.02-0.03 0.01-0.02 0.01-0.03 0.01-0.02 0.01h-0.03l-0.02 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m419.66 678.39v0.05l-0.01 0.03v0.02l-0.01 0.03-0.01 0.02v0.03l-0.02 0.04-0.02 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.06 0.06-0.02 0.01-0.02 0.02-0.02 0.01-0.02 0.02-0.04 0.02-0.03 0.01-0.04 0.02-0.03 0.01h-0.03l-0.02 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m424.08 678.39v0.05l-0.01 0.03v0.02l-0.01 0.03v0.02l-0.01 0.03-0.02 0.04-0.02 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.09 0.09-0.02 0.01-0.03 0.02-0.06 0.03-0.03 0.01-0.02 0.01-0.03 0.01h-0.02l-0.03 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m448.76 661.32v0.02l-0.01 0.03v0.05l-0.01 0.03-0.01 0.02-0.01 0.03-0.02 0.04-0.01 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.06 0.06-0.02 0.01-0.02 0.02-0.02 0.01-0.02 0.02-0.04 0.02-0.03 0.01-0.02 0.01-0.03 0.01-0.02 0.01h-0.03l-0.02 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m453.19 661.32v0.05l-0.01 0.03v0.02l-0.01 0.03-0.01 0.02v0.03l-0.02 0.04-0.02 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.06 0.06-0.02 0.01-0.02 0.02-0.02 0.01-0.02 0.02-0.04 0.02-0.03 0.01-0.04 0.02-0.03 0.01h-0.03l-0.02 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m457.61 661.32v0.05l-0.01 0.03v0.02l-0.01 0.03v0.02l-0.01 0.03-0.02 0.04-0.02 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.09 0.09-0.02 0.01-0.03 0.02-0.06 0.03-0.03 0.01-0.02 0.01-0.03 0.01h-0.02l-0.03 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/></g></g>
  </g>
  <ellipse cx="208.26" cy="218.66" rx="39.121" ry="12.757" fill="#0065ff" fill-opacity=".19324" stroke-width="0"/>
  <text x="186.40762" y="225.86844" fill="#000000" font-family="Verdana" font-size="8.9055px" letter-spacing="0px" stroke-width=".22264" word-spacing="0px" style="line-height:1.25" xml:space="preserve"><tspan x="186.40762" y="225.86844" font-family="'Helvetica Neue LT'" font-size="8.9055px" stroke-width=".22264" style="font-feature-settings:normal;font-variant-caps:normal;font-variant-ligatures:normal;font-variant-numeric:normal">get_params</tspan></text>
  <rect x="167.39" y="270.33" width="84.143" height="54.413" fill="none" stroke="#000" stroke-linecap="square" stroke-linejoin="bevel" stroke-opacity=".48235" stroke-width=".7"/>
  <path d="m167.7 282.45h84.139" fill="none" stroke="#000" stroke-opacity=".48235" stroke-width=".7"/>
  <text x="193.70111" y="278.29968" fill="#000000" font-family="Verdana" font-size="7.0914px" letter-spacing="0px" stroke-width=".17729" word-spacing="0px" style="line-height:1.25" xml:space="preserve"><tspan x="193.70111" y="278.29968" font-family="'Helvetica Neue LT'" font-size="7.0914px" stroke-width=".17729" style="font-feature-settings:normal;font-variant-caps:normal;font-variant-ligatures:normal;font-variant-numeric:normal">params</tspan></text>
  <path d="m188.9 283.01v41.547" fill="none" stroke="#000" stroke-opacity=".19608" stroke-width=".9"/>
  <path d="m251.13 290.35h-81.902" fill="none" stroke="#000" stroke-opacity=".19608" stroke-width=".9"/>
  <g fill-rule="evenodd" stroke="#000" stroke-miterlimit="10.433" stroke-width="0">
   <g transform="matrix(.5149 0 0 -.5149 20.675 653.4)" fill-opacity=".86275" ns0:preamble="default_packages.tex" ns0:text="$$word \\, \\, 1$$\n$$word \\, \\, 2$$\n$$\\cdots$$\n$$word \\, \\, V$$\n" xml:space="preserve"><path d="m336.05 712.5v0.0177l9e-3 0.0177v0.0177l9e-3 0.0265v0.0176l9e-3 0.0177 0.018 0.0882 0.0177 0.0441 0.018 0.0882 9e-3 0.0177v0.0265l9e-3 0.0177v0.0177l9e-3 0.0177v0.0177l9e-3 0.0177v0.0267l9e-3 0.0177v0.0357l9e-3 9e-3v9e-3c0 0.15882-0.12356 0.23822-0.25588 0.23822-0.1059 0-0.2647-0.0706-0.32646-0.24705-0.0264-0.0617-0.44116-1.7294-0.50292-1.9588-0.0617-0.26469-0.0794-0.42351-0.0794-0.58233 0-0.097 0-0.11467 9e-3 -0.15882-0.20294-0.45881-0.46763-0.73233-0.81175-0.73233-0.70585 0-0.70585 0.6441-0.70585 0.7941 0 0.28234 0.053 0.62645 0.4588 1.7029 0.097 0.26469 0.15 0.38822 0.15 0.56467 0 0.38823-0.29116 0.7147-0.72351 0.7147-0.82937 0-1.1558-1.2706-1.1558-1.35 0-0.0883 0.0882-0.0883 0.10591-0.0883 0.0883 0 0.097 0.0177 0.14116 0.15882 0.23824 0.81174 0.58234 1.0852 0.89116 1.0852 0.0794 0 0.21176-9e-3 0.21176-0.29116 0-0.21176-0.0883-0.45881-0.15882-0.6088-0.37941-1.0323-0.48529-1.4382-0.48529-1.7735 0-0.7941 0.59117-1.1029 1.2441-1.1029 0.15 0 0.56469 0 0.91762 0.61762 0.2294-0.56468 0.83822-0.61762 1.1029-0.61762 0.66176 0 1.0412 0.55586 1.2706 1.0764 0.29999 0.68822 0.58233 1.8617 0.58233 2.2764 0 0.48527-0.23823 0.61762-0.38822 0.61762-0.22059 0-0.44117-0.22941-0.44117-0.42351 0-0.10591 0.053-0.15882 0.13236-0.22942 0.097-0.097 0.31763-0.32645 0.31763-0.74997 0-0.29999-0.25587-1.1558-0.48528-1.6058-0.2294-0.45881-0.53821-0.76763-0.96173-0.76763-0.4147 0-0.65292 0.2647-0.65292 0.7588 0 0.24706 0.0617 0.52058 0.097 0.64411z"/><path d="m342.64 711.92v0.0883l-0.027 0.2382-0.0177 0.0794-0.0177 0.0706-0.0792 0.2118-0.0265 0.0617-0.0706 0.1234-0.0353 0.053-0.1323 0.159-0.053 0.0441-0.053 0.0442-0.053 0.0441-0.106 0.0706-0.0617 0.0353-0.0617 0.0264-0.0617 0.0265-0.0617 0.0264-0.0706 0.0177-0.0617 0.0177-0.0706 0.0177-0.2118 0.027h-0.0794c-1.1735 0-2.3558-1.2441-2.3558-2.4882 0-0.86468 0.59116-1.4823 1.4117-1.4823l9e-3 0.19411c-0.37941 0-0.76762 0.27353-0.76762 0.95292 0 0.43233 0.22939 1.3764 0.51174 1.8264 0.44116 0.67057 0.93526 0.80291 1.1823 0.80291 0.51175 0 0.77644-0.42351 0.77644-0.94408 0-0.34412-0.17646-1.2617-0.51175-1.8352-0.30882-0.50294-0.79409-0.80293-1.1911-0.80293l-9e-3 -0.19411c1.1647 0 2.3558 1.2088 2.3558 2.4882z"/><path d="m343.55 710.04v-0.0177l-9e-3 -9e-3v-0.0267l-9e-3 -0.0264-9e-3 -0.0353-9e-3 -0.0264v-0.0265l-9e-3 -0.0264-9e-3 -0.0265v-0.0177l-9e-3 -9e-3v-0.0444l-9e-3 -9e-3v-0.0357l-9e-3 -9e-3v-0.036c0-0.1588 0.12356-0.23822 0.25588-0.23822 0.1059 0 0.26468 0.0706 0.32645 0.24705 0.0176 0.0353 0.31764 1.2264 0.34411 1.3852 0.0794 0.29118 0.2294 0.89998 0.28234 1.1382 0.0353 0.11467 0.28234 0.5294 0.4941 0.71468 0.0706 0.0617 0.32647 0.29117 0.70587 0.29117 0.2294 0 0.36175-0.10591 0.37057-0.10591-0.26469-0.0441-0.45881-0.25587-0.45881-0.47644 0-0.14119 0.097-0.30882 0.33529-0.30882 0.23822 0 0.48527 0.20294 0.48527 0.51175s-0.28234 0.57351-0.73232 0.57351c-0.57351 0-0.96174-0.43234-1.1294-0.67056-0.0706 0.38821-0.3794 0.67056-0.7941 0.67056-0.40586 0-0.57351-0.34411-0.65291-0.50293-0.15882-0.29117-0.27353-0.82056-0.27353-0.84703 0-0.0882 0.0883-0.0882 0.10591-0.0882 0.0883 0 0.097 9e-3 0.14999 0.20293 0.15 0.61763 0.32647 1.0411 0.64411 1.0411 0.14999 0 0.27351-0.0706 0.27351-0.39705 0-0.18528-0.0264-0.28233-0.14117-0.74114z"/><path d="m351.53 715.51v0.018l-9e-3 9e-3v0.018h-9e-3v9e-3l-0.018 0.018h-9e-3v9e-3h-0.018v9e-3h-0.054c-0.13236 0-0.97057-0.0794-1.1206-0.0883-0.0706-9e-3 -0.12355-0.053-0.12355-0.16765 0-0.1059 0.0794-0.1059 0.21176-0.1059 0.4235 0 0.44115-0.0617 0.44115-0.15001l-0.0264-0.17646-0.5294-2.0823c-0.15882 0.32645-0.40587 0.56469-0.80291 0.56469-1.0323 0-2.1264-1.2882-2.1264-2.5676 0-0.82056 0.48528-1.4029 1.1735-1.4029l0.0177 0.19411c-0.44117 0-0.56469 0.48528-0.56469 0.82056 0 0.44117 0.28234 1.5176 0.48528 1.9235 0.27352 0.51175 0.67056 0.8382 1.0235 0.8382 0.56469 0 0.68822-0.71468 0.68822-0.76762 0-0.053-0.0177-0.10591-0.0264-0.15l-0.44117-1.7205c-0.0353-0.15882-0.0353-0.17647-0.16765-0.32646-0.38822-0.47645-0.74997-0.61763-0.99702-0.61763l-0.0177-0.19411c0.17647 0 0.61763 0.0353 1.147 0.66174 0.0617-0.37058 0.37058-0.66174 0.79409-0.66174 0.30881 0 0.51174 0.20293 0.65293 0.48528 0.14999 0.31763 0.26468 0.84703 0.26468 0.86468 0 0.0883-0.0794 0.0883-0.1059 0.0883-0.0883 0-0.097-0.0353-0.12356-0.15882-0.14999-0.57352-0.30881-1.0853-0.67057-1.0853-0.23822 0-0.2647 0.22941-0.2647 0.40587 0 0.20294 0.0177 0.2647 0.053 0.4147z"/><path d="m357.08 715.14v0.126l-9e-3 9e-3v0.027l-9e-3 9e-3v0.018l-9e-3 9e-3h-9e-3v9e-3h-9e-3l-9e-3 9e-3h-0.018v9e-3h-0.072v9e-3h-0.063c-0.54704-0.56469-1.3235-0.56469-1.6058-0.56469v-0.27353c0.17647 0 0.69705 0 1.1559 0.22941v-4.5351c0-0.31764-0.0264-0.42352-0.82057-0.42352h-0.28235v-0.27353c0.30882 0.0265 1.0764 0.0265 1.4294 0.0265s1.1206 0 1.4294-0.0265v0.27353h-0.28233c-0.7941 0-0.82057 0.097-0.82057 0.42352z"/><path d="m368.62 713.14v0.0267l9e-3 0.0178v0.0178l9e-3 0.0178v0.0267l9e-3 0.0178 0.018 0.089 0.0178 0.0446 0.018 0.089 9e-3 0.0178v0.0178l9e-3 0.0268v0.0178l9e-3 0.0178v0.0178l9e-3 0.0178v0.0268l9e-3 0.0178v0.0358l9e-3 9e-3v9e-3c0 0.16034-0.12474 0.24051-0.25832 0.24051-0.10691 0-0.26721-0.0713-0.32957-0.24942-0.0267-0.0623-0.44537-1.7548-0.50772-1.9774-0.0623-0.26723-0.0801-0.42756-0.0801-0.5879 0-0.0979 0-0.11576 9e-3 -0.16033-0.20487-0.4721-0.4721-0.73931-0.81948-0.73931-0.71259 0-0.71259 0.65024-0.71259 0.80166 0 0.28503 0.0534 0.63243 0.46318 1.7191 0.0979 0.25831 0.15143 0.38301 0.15143 0.56116 0 0.40083-0.29396 0.73041-0.73041 0.73041-0.8373 0-1.1669-1.2827-1.1669-1.3628 0-0.0891 0.0891-0.0891 0.10692-0.0891 0.0891 0 0.0979 0.0178 0.14252 0.16033 0.2405 0.81948 0.58788 1.0956 0.89964 1.0956 0.0801 0 0.21378-9e-3 0.21378-0.29395 0-0.22268-0.0891-0.47209-0.16033-0.61461-0.38301-1.0511-0.4899-1.4519-0.4899-1.7904 0-0.81059 0.59679-1.1134 1.2559-1.1134 0.15142 0 0.57007 0 0.92637 0.61461 0.23159-0.56116 0.8462-0.61461 1.1134-0.61461 0.66805 0 1.0511 0.55227 1.2827 1.0867 0.30286 0.68587 0.5879 1.8795 0.5879 2.2981 0 0.4899-0.24051 0.62352-0.39193 0.62352-0.22268 0-0.44537-0.2316-0.44537-0.42756 0-0.11577 0.0535-0.16925 0.13361-0.2405 0.0979-0.0979 0.32066-0.32067 0.32066-0.74823 0-0.30285-0.25831-1.1669-0.4899-1.6211-0.2316-0.4721-0.54336-0.77495-0.97091-0.77495-0.41864 0-0.65915 0.25832-0.65915 0.76604 0 0.24941 0.0623 0.52554 0.0979 0.65024z"/><path d="m375.27 712.57v0.0801l-9e-3 0.0891-0.018 0.1602-0.0178 0.0713-0.0178 0.0801-0.0534 0.1426-0.0268 0.0623-0.0267 0.0713-0.0356 0.0623-0.0356 0.0534-0.0356 0.0623-0.0445 0.0535-0.0445 0.0534-0.0445 0.0535-0.0534 0.0445-0.107 0.089-0.0535 0.0356-0.0534 0.0356-0.0623 0.0356-0.0623 0.0267-0.0623 0.0268-0.0623 0.0267-0.0713 0.0178-0.0623 0.0178-0.0713 0.0178-0.2139 0.027h-0.0801c-1.1847 0-2.3783-1.256-2.3783-2.5119 0-0.88183 0.59679-1.4964 1.4252-1.4964l9e-3 0.19597c-0.38302 0-0.77495 0.26721-0.77495 0.96199 0 0.43646 0.2316 1.3896 0.51663 1.8349 0.44538 0.68587 0.94419 0.81949 1.1936 0.81949 0.51663 0 0.78385-0.42756 0.78385-0.9531 0-0.34739-0.17815-1.2738-0.51663-1.8527-0.31176-0.51663-0.80167-0.81057-1.2025-0.81057l-9e-3 -0.19597c1.1758 0 2.3783 1.2203 2.3783 2.5119z"/><path d="m376.19 710.67v-9e-3l-9e-3 -0.0178v-0.0268l-9e-3 -0.0267-9e-3 -0.0268-9e-3 -0.0267v-0.0267l-9e-3 -0.0356-9e-3 -0.0267v-9e-3l-9e-3 -0.0178v-0.0358l-9e-3 -9e-3v-0.0358l-9e-3 -9e-3v-0.036c0-0.16033 0.12474-0.24051 0.25832-0.24051 0.10691 0 0.26722 0.0713 0.32958 0.24941 0.0178 0.0356 0.32067 1.2381 0.34738 1.3896 0.0801 0.29394 0.2316 0.91746 0.28505 1.158 0.0356 0.11577 0.28503 0.52554 0.49881 0.7215 0.0713 0.0623 0.32957 0.29396 0.71259 0.29396 0.2316 0 0.3652-0.10692 0.37411-0.10692-0.26722-0.0446-0.46318-0.25831-0.46318-0.48991 0-0.13361 0.0979-0.30285 0.33847-0.30285 0.24051 0 0.48992 0.19596 0.48992 0.51663 0 0.31176-0.28504 0.57898-0.73932 0.57898-0.57899 0-0.97091-0.43647-1.1402-0.68587-0.0713 0.40083-0.38301 0.68587-0.80166 0.68587-0.40974 0-0.57899-0.3474-0.65916-0.50772-0.16032-0.30286-0.27612-0.82839-0.27612-0.85512 0-0.0891 0.0891-0.0891 0.10691-0.0891 0.0891 0 0.0979 9e-3 0.15143 0.20487 0.15142 0.62352 0.32957 1.0511 0.65023 1.0511 0.15143 0 0.27614-0.0713 0.27614-0.40976 0-0.18705-0.0267-0.28503-0.14252-0.7393z"/><path d="m384.24 716.19v0.027l-9e-3 9e-3v0.018h-9e-3v9e-3h-9e-3v9e-3h-9e-3v9e-3h-9e-3v9e-3h-0.027v9e-3h-0.045c-0.13361 0-0.97982-0.0801-1.1312-0.0979-0.0713-9e-3 -0.12473-0.0535-0.12473-0.16924 0-0.10692 0.0801-0.10692 0.21377-0.10692 0.42756 0 0.44538-0.0623 0.44538-0.15142l-0.0268-0.16925-0.53445-2.1021c-0.16032 0.32956-0.40974 0.57007-0.81057 0.57007-1.0422 0-2.1467-1.3005-2.1467-2.5921 0-0.8373 0.4899-1.4163 1.1847-1.4163l0.0178 0.19597c-0.44536 0-0.57007 0.48099-0.57007 0.82838 0 0.44538 0.28504 1.5321 0.48991 1.9418 0.27614 0.51663 0.67697 0.84621 1.0333 0.84621 0.57008 0 0.69479-0.73041 0.69479-0.78385 0-0.0535-0.0178-0.098-0.0267-0.14253l-0.44536-1.7369c-0.0356-0.16034-0.0356-0.17815-0.16925-0.32958-0.39192-0.48099-0.75712-0.62351-1.0065-0.62351l-0.0178-0.19597c0.17814 0 0.62352 0.0356 1.158 0.65915 0.0623-0.3652 0.37411-0.65915 0.80167-0.65915 0.31176 0 0.51663 0.20487 0.65914 0.481 0.15143 0.32067 0.26723 0.86402 0.26723 0.88184 0 0.0891-0.0801 0.0891-0.10692 0.0891-0.0891 0-0.0979-0.0356-0.12473-0.16033-0.15143-0.57897-0.31176-1.0956-0.67697-1.0956-0.24049 0-0.26721 0.22267-0.26721 0.40083 0 0.21378 0.0178 0.27612 0.0535 0.42754z"/><path d="m388.35 710.83 0.94419 0.91747c1.3896 1.2203 1.9151 1.6924 1.9151 2.5832 0 1.0065-0.79277 1.7102-1.8795 1.7102-1.0065 0-1.6568-0.81059-1.6568-1.6033 0-0.48991 0.43647-0.48991 0.46318-0.48991 0.15143 0 0.46319 0.10691 0.46319 0.47209 0 0.22269-0.16034 0.45428-0.4721 0.45428-0.0713 0-0.0891 0-0.11576-9e-3 0.20487 0.57899 0.68587 0.89966 1.2025 0.89966 0.81058 0 1.1936-0.71259 1.1936-1.4341 0-0.71261-0.44537-1.4074-0.93528-1.9596l-1.7102-1.8973c-0.0891-0.0979-0.0891-0.11576-0.0891-0.32067h3.2868l0.2494 1.541h-0.22269c-0.0356-0.26722-0.10691-0.65916-0.18705-0.79276-0.0623-0.0713-0.65025-0.0713-0.84621-0.0713z"/><path d="m400.63 710.56v0.0146l-7e-3 0.0219v0.0365l-7e-3 0.0219-7e-3 0.0146-7e-3 0.0219-0.014 0.0292-7e-3 0.0219-0.014 0.0292-0.0146 0.0146-7e-3 0.0146-0.0438 0.0438-0.0146 7e-3 -0.0146 0.0146-0.0146 7e-3 -0.0146 0.0146-0.0292 0.014-0.0219 7e-3 -0.0146 7e-3 -0.022 7e-3 -0.0146 7e-3h-0.022l-0.0146 7e-3h-0.0585c-0.21216 0-0.38773-0.16825-0.38773-0.38041 0-0.21215 0.17557-0.38772 0.38773-0.38772 0.21215 0 0.38772 0.17557 0.38772 0.38772z"/><path d="m403.87 710.56v0.0365l-7e-3 0.0219v0.0146l-7e-3 0.0219-7e-3 0.0146v0.0219l-0.014 0.0292-0.0146 0.0219-0.014 0.0292-0.0146 0.0146-7e-3 0.0146-0.0438 0.0438-0.0146 7e-3 -0.0146 0.0146-0.0146 7e-3 -0.0146 0.0146-0.0292 0.014-0.022 7e-3 -0.0292 0.014-0.022 7e-3h-0.0219l-0.0146 7e-3h-0.0585c-0.21215 0-0.38773-0.16825-0.38773-0.38041 0-0.21215 0.17558-0.38772 0.38773-0.38772s0.38773 0.17557 0.38773 0.38772z"/><path d="m407.1 710.56v0.0365l-7e-3 0.0219v0.0146l-7e-3 0.0219v0.0146l-7e-3 0.0219-0.014 0.0292-0.0146 0.0219-0.014 0.0292-0.0146 0.0146-7e-3 0.0146-0.0654 0.0654-0.0146 7e-3 -0.022 0.0146-0.0438 0.021-0.022 7e-3 -0.0146 7e-3 -0.022 7e-3h-0.0146l-0.022 7e-3h-0.0585c-0.21215 0-0.38773-0.16825-0.38773-0.38041 0-0.21215 0.17558-0.38772 0.38773-0.38772s0.38772 0.17557 0.38772 0.38772z"/><path d="m420.21 713.06 9e-3 0.0175v0.0175l9e-3 0.0175 9e-3 0.0263v0.0175l9e-3 0.0263 9e-3 0.035 9e-3 0.0437 9e-3 0.0439 0.0175 0.0437 9e-3 0.0437v0.0262l0.018 0.035v0.0175l9e-3 0.0175v0.035l9e-3 0.0175v0.0175l9e-3 9e-3v0.0625c0 0.14887-0.11381 0.2277-0.24521 0.2277-0.10511 0-0.26273-0.0701-0.32402-0.23646-0.0262-0.0613-0.43789-1.7252-0.49919-1.9529-0.0613-0.26273-0.0788-0.42037-0.0788-0.57801 0-0.0876 0-0.11381 9e-3 -0.14888-0.20142-0.46415-0.46415-0.73563-0.8057-0.73563-0.70061 0-0.70061 0.64806-0.70061 0.79694 0 0.27147 0.0437 0.61303 0.45539 1.6814 0.0963 0.26273 0.14888 0.38533 0.14888 0.56049 0 0.39409-0.289 0.70936-0.71811 0.70936-0.82321 0-1.1472-1.2611-1.1472-1.3399 0-0.0876 0.0876-0.0876 0.10512-0.0876 0.0876 0 0.0963 0.0175 0.14011 0.15764 0.23647 0.81445 0.56925 1.0772 0.87577 1.0772 0.0788 0 0.21894-9e-3 0.21894-0.28025 0-0.21894-0.0963-0.46415-0.15764-0.61303-0.37658-1.0246-0.48167-1.4275-0.48167-1.7603 0-0.78819 0.578-1.0947 1.2348-1.0947 0.14888 0 0.56049 0 0.91079 0.61303 0.2277-0.56048 0.83197-0.61303 1.0859-0.61303 0.65682 0 1.0422 0.55173 1.2698 1.0684 0.29775 0.68309 0.57799 1.8478 0.57799 2.2682 0 0.4729-0.23645 0.60427-0.38533 0.60427-0.21894 0-0.43787-0.21895-0.43787-0.41161 0-0.11381 0.0526-0.1664 0.13136-0.23645 0.0963-0.0963 0.31527-0.32404 0.31527-0.7444 0-0.289-0.25397-1.1472-0.48167-1.5939-0.22769-0.45541-0.53421-0.76191-0.96333-0.76191-0.40285 0-0.6393 0.26271-0.6393 0.75315 0 0.24521 0.0613 0.5167 0.0963 0.63929z"/><path d="m426.76 712.48-9e-3 0.0876v0.0788l-9e-3 0.0788-0.0175 0.0788-9e-3 0.0788-0.0175 0.0701-0.0262 0.0701-0.0263 0.0701-0.0262 0.0701-0.0262 0.0612-0.07 0.1226-0.0437 0.0525-0.035 0.0613-0.0437 0.0439-0.0525 0.0526-0.0437 0.0437-0.1052 0.0874-0.0526 0.035-0.0525 0.035-0.1226 0.07-0.0613 0.0263-0.0613 0.0175-0.0701 0.0262-0.0701 0.0175-0.0613 9e-3 -0.0701 0.0175-0.0701 9e-3h-0.1489c-1.1735 0-2.3383-1.2348-2.3383-2.4696 0-0.85824 0.58675-1.4713 1.3924-1.4713l9e-3 0.19268c-0.36783 0-0.75315 0.27147-0.75315 0.94581 0 0.42911 0.22769 1.3662 0.49918 1.8128 0.43787 0.67434 0.93706 0.79695 1.1823 0.79695 0.50794 0 0.77067-0.41161 0.77067-0.93707 0-0.34154-0.17516-1.2523-0.50794-1.8128-0.30651-0.50794-0.78819-0.80569-1.191-0.80569l-9e-3 -0.19268c1.1648 0 2.347 1.1998 2.347 2.4696z"/><path d="m427.66 710.62-9e-3 -0.0175v-0.0265l-9e-3 -9e-3v-0.0262l-9e-3 -0.0262-9e-3 -0.035-9e-3 -0.0262v-0.0263l-9e-3 -0.0262v-0.0175l-9e-3 -9e-3v-0.0265l-9e-3 -9e-3v-0.0355l-9e-3 -9e-3v-0.054c0-0.15763 0.12263-0.23646 0.25397-0.23646 0.10511 0 0.25397 0.0701 0.31527 0.24521 0.0175 0.035 0.31526 1.2173 0.3503 1.3749 0.0701 0.28901 0.22769 0.89328 0.28024 1.1297 0.035 0.11382 0.28025 0.52546 0.49043 0.71812 0.0701 0.0613 0.32402 0.28025 0.70061 0.28025 0.22769 0 0.35906-0.10511 0.36782-0.10511-0.26273-0.035-0.4554-0.24523-0.4554-0.47292 0-0.14011 0.0963-0.30651 0.33279-0.30651 0.23645 0 0.48166 0.20142 0.48166 0.5167 0 0.30651-0.28024 0.56048-0.72687 0.56048-0.56925 0-0.95458-0.42037-1.121-0.66558-0.0701 0.39409-0.38535 0.66558-0.79694 0.66558-0.3941 0-0.56049-0.33279-0.63931-0.49043-0.15764-0.29776-0.27149-0.8232-0.27149-0.84948 0-0.0876 0.0876-0.0876 0.10512-0.0876 0.0876 0 0.0963 9e-3 0.14887 0.20142 0.1489 0.6218 0.32404 1.0334 0.63056 1.0334 0.14887 0 0.27148-0.0612 0.27148-0.3941 0-0.1839-0.0262-0.28023-0.14013-0.73563z"/><path d="m435.58 716.05v0.018l-9e-3 9e-3v0.018h-9e-3v9e-3h-9e-3v9e-3l-9e-3 9e-3h-9e-3v9e-3h-9e-3l-9e-3 9e-3h-0.036v9e-3h-0.018c-0.13137 0-0.96333-0.0788-1.1122-0.0963-0.0701-9e-3 -0.12263-0.0526-0.12263-0.16639 0-0.10511 0.0788-0.10511 0.21018-0.10511 0.42037 0 0.43787-0.0613 0.43787-0.14888l-0.0262-0.17516-0.52546-2.058c-0.15764 0.31526-0.41161 0.55172-0.8057 0.55172-1.0159 0-2.1018-1.2786-2.1018-2.5484 0-0.81446 0.48167-1.3925 1.1648-1.3925l0.0175 0.19267c-0.43788 0-0.56049 0.48167-0.56049 0.81445 0 0.43788 0.28025 1.5063 0.48167 1.9092 0.27148 0.51669 0.65682 0.83197 1.0071 0.83197 0.56925 0 0.69185-0.70938 0.69185-0.76191 0-0.0526-0.0175-0.10512-0.0262-0.14889l-0.43789-1.7077c-0.0437-0.14888-0.0437-0.17515-0.17514-0.31528-0.37658-0.48166-0.73564-0.62178-0.98085-0.62178l-0.0175-0.19267c0.17515 0 0.61303 0.035 1.1297 0.65682 0.0701-0.36781 0.37658-0.65682 0.79695-0.65682 0.3065 0 0.50793 0.20142 0.64805 0.48167 0.14888 0.31528 0.26273 0.84072 0.26273 0.85824 0 0.0876-0.0788 0.0876-0.10511 0.0876-0.0876 0-0.0963-0.035-0.12264-0.15764-0.14888-0.56048-0.30652-1.0772-0.66558-1.0772-0.23645 0-0.26273 0.22769-0.26273 0.40285 0 0.21017 0.0175 0.27147 0.0526 0.41159z"/><path d="m444.01 715.05 0.035 0.0613 0.0437 0.0612 0.035 0.0526 0.0439 0.0526 0.035 0.0526 0.0787 0.0787 0.035 0.0437 0.035 0.035 0.0437 0.0263 0.035 0.035 0.035 0.0263 0.035 0.0262 0.035 0.0175 0.035 0.0262 0.105 0.0525 0.07 0.018 0.0263 9e-3 0.105 0.027 0.0262 9e-3h0.035l0.0613 9e-3h0.0613l0.0613 9e-3c0.10512 9e-3 0.11381 0.15764 0.11381 0.16638 0 0.0701-0.0439 0.10512-0.11381 0.10512-0.2277 0-0.47291-0.0262-0.70936-0.0262-0.28901 0-0.58677 0.0262-0.867 0.0262-0.0526 0-0.1664 0-0.1664-0.16638 0-0.0963 0.0788-0.10512 0.14012-0.10512 0.23646-0.0175 0.40284-0.10512 0.40284-0.28901 0-0.13136-0.13135-0.32402-0.13135-0.33278l-2.6798-4.2387-0.59552 4.5977c0 0.14888 0.20143 0.26273 0.60428 0.26273 0.12264 0 0.21893 0 0.21893 0.17514 0 0.0788-0.0701 0.0963-0.12263 0.0963-0.35031 0-0.72689-0.0262-1.086-0.0262-0.15764 0-0.32403 9e-3 -0.48166 9e-3 -0.15764 0-0.32403 0.0175-0.47291 0.0175-0.0613 0-0.1664 0-0.1664-0.16639 0-0.10511 0.0788-0.10511 0.21895-0.10511 0.49041 0 0.49917-0.0788 0.52544-0.29776l0.69186-5.3684c0.0262-0.17514 0.0612-0.20142 0.17514-0.20142 0.14013 0 0.17516 0.0437 0.24522 0.15763z"/></g>
   <g transform="matrix(.43834 0 0 -.43834 44.069 601.87)" ns0:preamble="default_packages.tex" ns0:text="$$doc 1$$\n$$doc 2$$\n$$\\cdots$$\n$$doc N$$\n" xml:space="preserve"><g fill-opacity=".86275"><path d="m318.13 701.72v0.13l-0.01 0.01v0.04l-0.01 0.01v0.01l-0.01 0.01v0.01h-0.01v0.01h-0.01l-0.01 0.01h-0.02v0.01h-0.08v0.01h-0.07c-0.59499-0.61419-1.4395-0.61419-1.7466-0.61419v-0.29751c0.19193 0 0.75814 0 1.2572 0.24953v-4.9327c0-0.34549-0.0288-0.46064-0.89249-0.46064h-0.3071v-0.2975c0.33589 0.0288 1.1708 0.0288 1.5547 0.0288 0.38389 0 1.2188 0 1.5451-0.0288v0.2975h-0.3071c-0.8541 0-0.88289 0.10562-0.88289 0.46064z"/><path d="m313.92 673.98 1.1137 1.0822c1.6285 1.4394 2.2589 1.9962 2.2589 3.0469 0 1.1872-0.93506 2.0172-2.2168 2.0172-1.1872 0-1.9542-0.95607-1.9542-1.8912 0-0.57786 0.51481-0.57786 0.54633-0.57786 0.17861 0 0.54634 0.1261 0.54634 0.55683 0 0.26267-0.18913 0.53584-0.55685 0.53584-0.084 0-0.10502 0-0.1366-0.0108 0.24163 0.68292 0.80899 1.0612 1.4184 1.0612 0.9561 0 1.4079-0.84052 1.4079-1.6915 0-0.84051-0.52532-1.66-1.1032-2.3114l-2.0172-2.2379c-0.10503-0.11563-0.10503-0.13661-0.10503-0.37824h3.8769l0.29418 1.8176h-0.26267c-0.0526-0.31518-0.12609-0.77748-0.23114-0.93507-0.0735-0.084-0.75645-0.084-0.98759-0.084z"/><path d="m300.9 661.79v0.02l-0.01 0.03v0.05l-0.01 0.03-0.01 0.02-0.01 0.03-0.02 0.04-0.01 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.06 0.06-0.02 0.01-0.02 0.02-0.02 0.01-0.02 0.02-0.04 0.02-0.03 0.01-0.02 0.01-0.03 0.01-0.02 0.01h-0.03l-0.02 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m305.33 661.79v0.05l-0.01 0.03v0.02l-0.01 0.03-0.01 0.02v0.03l-0.02 0.04-0.02 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.06 0.06-0.02 0.01-0.02 0.02-0.02 0.01-0.02 0.02-0.04 0.02-0.03 0.01-0.04 0.02-0.03 0.01h-0.03l-0.02 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m309.75 661.79v0.05l-0.01 0.03v0.02l-0.01 0.03v0.02l-0.01 0.03-0.02 0.04-0.02 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.09 0.09-0.02 0.01-0.03 0.02-0.06 0.03-0.03 0.01-0.02 0.01-0.03 0.01h-0.02l-0.03 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m318.93 647.47 0.0206 0.0823 0.0108 0.0308 0.0108 0.0413 0.0108 0.0308 0.0206 0.0413 0.0108 0.0309 0.0206 0.0309 0.0206 0.0308 0.0108 0.0309 0.0206 0.0308 0.0206 0.0309 0.0308 0.0309 0.0206 0.0308 0.0309 0.0206 0.0309 0.0309 0.0309 0.0206 0.0309 0.0308 0.0308 0.0206 0.1239 0.0618 0.0513 0.0108 0.0513 0.0206 0.1026 0.0216 0.0617 0.0206 0.0617 0.0108h0.0617l0.0617 0.0108 0.0719 0.0108h0.0823c0.0513 0 0.17479 0.0108 0.17479 0.20565 0 0.0108 0 0.11313-0.13366 0.11313-0.3393 0-0.69916-0.0309-1.0384-0.0309-0.34957 0-0.70944 0.0309-1.0487 0.0309-0.0617 0-0.18509 0-0.18509-0.20565 0-0.11313 0.10285-0.11313 0.18509-0.11313 0.58604-0.0108 0.69915-0.22619 0.69915-0.45239 0-0.0308-0.0206-0.18506-0.0308-0.21591l-1.1515-4.5445-2.262 5.3362c-0.0823 0.18507-0.0926 0.19534-0.329 0.19534h-1.3675c-0.20562 0-0.29817 0-0.29817-0.20562 0-0.11313 0.0926-0.11313 0.28791-0.11313 0.0412 0 0.68884 0 0.68884-0.0926l-1.3674-5.4904c-0.10273-0.40101-0.2776-0.73002-1.1104-0.76087-0.0617 0-0.17479-0.0108-0.17479-0.20563 0-0.0719 0.0513-0.11313 0.13365-0.11313 0.32902 0 0.68887 0.0309 1.0282 0.0309 0.34958 0 0.70943-0.0309 1.0487-0.0309 0.0513 0 0.18507 0 0.18507 0.20562 0 0.10273-0.0926 0.11314-0.20562 0.11314-0.58606 0.0206-0.66832 0.24677-0.66832 0.45239 0 0.0719 0.0108 0.12353 0.0411 0.22621l1.3469 5.3979c0.0413-0.0617 0.0413-0.0823 0.0926-0.18508l2.5602-6.0148c0.0719-0.17479 0.10273-0.19535 0.18505-0.19535 0.11313 0 0.11313 0.0308 0.16452 0.21592z"/></g><g fill-rule="evenodd" stroke-miterlimit="10.433"><path d="m349.99 661.93v0.02l-0.01 0.03v0.05l-0.01 0.03-0.01 0.02-0.01 0.03-0.02 0.04-0.01 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.06 0.06-0.02 0.01-0.02 0.02-0.02 0.01-0.02 0.02-0.04 0.02-0.03 0.01-0.02 0.01-0.03 0.01-0.02 0.01h-0.03l-0.02 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m354.42 661.93v0.05l-0.01 0.03v0.02l-0.01 0.03-0.01 0.02v0.03l-0.02 0.04-0.02 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.06 0.06-0.02 0.01-0.02 0.02-0.02 0.01-0.02 0.02-0.04 0.02-0.03 0.01-0.04 0.02-0.03 0.01h-0.03l-0.02 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m358.84 661.93v0.05l-0.01 0.03v0.02l-0.01 0.03v0.02l-0.01 0.03-0.02 0.04-0.02 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.09 0.09-0.02 0.01-0.03 0.02-0.06 0.03-0.03 0.01-0.02 0.01-0.03 0.01h-0.02l-0.03 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m380.93 661.78v0.02l-0.01 0.03v0.05l-0.01 0.03-0.01 0.02-0.01 0.03-0.02 0.04-0.01 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.06 0.06-0.02 0.01-0.02 0.02-0.02 0.01-0.02 0.02-0.04 0.02-0.03 0.01-0.02 0.01-0.03 0.01-0.02 0.01h-0.03l-0.02 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m385.36 661.78v0.05l-0.01 0.03v0.02l-0.01 0.03-0.01 0.02v0.03l-0.02 0.04-0.02 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.06 0.06-0.02 0.01-0.02 0.02-0.02 0.01-0.02 0.02-0.04 0.02-0.03 0.01-0.04 0.02-0.03 0.01h-0.03l-0.02 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m389.78 661.78v0.05l-0.01 0.03v0.02l-0.01 0.03v0.02l-0.01 0.03-0.02 0.04-0.02 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.09 0.09-0.02 0.01-0.03 0.02-0.06 0.03-0.03 0.01-0.02 0.01-0.03 0.01h-0.02l-0.03 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m415.38 661.09v0.02l-0.01 0.03v0.05l-0.01 0.03-0.01 0.02-0.01 0.03-0.02 0.04-0.01 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.06 0.06-0.02 0.01-0.02 0.02-0.02 0.01-0.02 0.02-0.04 0.02-0.03 0.01-0.02 0.01-0.03 0.01-0.02 0.01h-0.03l-0.02 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m419.81 661.09v0.05l-0.01 0.03v0.02l-0.01 0.03-0.01 0.02v0.03l-0.02 0.04-0.02 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.06 0.06-0.02 0.01-0.02 0.02-0.02 0.01-0.02 0.02-0.04 0.02-0.03 0.01-0.04 0.02-0.03 0.01h-0.03l-0.02 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m424.23 661.09v0.05l-0.01 0.03v0.02l-0.01 0.03v0.02l-0.01 0.03-0.02 0.04-0.02 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.09 0.09-0.02 0.01-0.03 0.02-0.06 0.03-0.03 0.01-0.02 0.01-0.03 0.01h-0.02l-0.03 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m415.53 643.18v0.02l-0.01 0.03v0.05l-0.01 0.03-0.01 0.02-0.01 0.03-0.02 0.04-0.01 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.06 0.06-0.02 0.01-0.02 0.02-0.02 0.01-0.02 0.02-0.04 0.02-0.03 0.01-0.02 0.01-0.03 0.01-0.02 0.01h-0.03l-0.02 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m419.96 643.18v0.05l-0.01 0.03v0.02l-0.01 0.03-0.01 0.02v0.03l-0.02 0.04-0.02 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.06 0.06-0.02 0.01-0.02 0.02-0.02 0.01-0.02 0.02-0.04 0.02-0.03 0.01-0.04 0.02-0.03 0.01h-0.03l-0.02 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m424.38 643.18v0.05l-0.01 0.03v0.02l-0.01 0.03v0.02l-0.01 0.03-0.02 0.04-0.02 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.09 0.09-0.02 0.01-0.03 0.02-0.06 0.03-0.03 0.01-0.02 0.01-0.03 0.01h-0.02l-0.03 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m415.45 697.31v0.02l-0.01 0.03v0.05l-0.01 0.03-0.01 0.02-0.01 0.03-0.02 0.04-0.01 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.06 0.06-0.02 0.01-0.02 0.02-0.02 0.01-0.02 0.02-0.04 0.02-0.03 0.01-0.02 0.01-0.03 0.01-0.02 0.01h-0.03l-0.02 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m419.88 697.31v0.05l-0.01 0.03v0.02l-0.01 0.03-0.01 0.02v0.03l-0.02 0.04-0.02 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.06 0.06-0.02 0.01-0.02 0.02-0.02 0.01-0.02 0.02-0.04 0.02-0.03 0.01-0.04 0.02-0.03 0.01h-0.03l-0.02 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m424.3 697.31v0.05l-0.01 0.03v0.02l-0.01 0.03v0.02l-0.01 0.03-0.02 0.04-0.02 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.09 0.09-0.02 0.01-0.03 0.02-0.06 0.03-0.03 0.01-0.02 0.01-0.03 0.01h-0.02l-0.03 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m415.23 678.39v0.02l-0.01 0.03v0.05l-0.01 0.03-0.01 0.02-0.01 0.03-0.02 0.04-0.01 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.06 0.06-0.02 0.01-0.02 0.02-0.02 0.01-0.02 0.02-0.04 0.02-0.03 0.01-0.02 0.01-0.03 0.01-0.02 0.01h-0.03l-0.02 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m419.66 678.39v0.05l-0.01 0.03v0.02l-0.01 0.03-0.01 0.02v0.03l-0.02 0.04-0.02 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.06 0.06-0.02 0.01-0.02 0.02-0.02 0.01-0.02 0.02-0.04 0.02-0.03 0.01-0.04 0.02-0.03 0.01h-0.03l-0.02 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m424.08 678.39v0.05l-0.01 0.03v0.02l-0.01 0.03v0.02l-0.01 0.03-0.02 0.04-0.02 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.09 0.09-0.02 0.01-0.03 0.02-0.06 0.03-0.03 0.01-0.02 0.01-0.03 0.01h-0.02l-0.03 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m448.76 661.32v0.02l-0.01 0.03v0.05l-0.01 0.03-0.01 0.02-0.01 0.03-0.02 0.04-0.01 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.06 0.06-0.02 0.01-0.02 0.02-0.02 0.01-0.02 0.02-0.04 0.02-0.03 0.01-0.02 0.01-0.03 0.01-0.02 0.01h-0.03l-0.02 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m453.19 661.32v0.05l-0.01 0.03v0.02l-0.01 0.03-0.01 0.02v0.03l-0.02 0.04-0.02 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.06 0.06-0.02 0.01-0.02 0.02-0.02 0.01-0.02 0.02-0.04 0.02-0.03 0.01-0.04 0.02-0.03 0.01h-0.03l-0.02 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m457.61 661.32v0.05l-0.01 0.03v0.02l-0.01 0.03v0.02l-0.01 0.03-0.02 0.04-0.02 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.09 0.09-0.02 0.01-0.03 0.02-0.06 0.03-0.03 0.01-0.02 0.01-0.03 0.01h-0.02l-0.03 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><g transform="matrix(1.9 0 0 1.9 -340.03 -580.38)" stroke="#000" ns0:preamble="default_packages.tex" ns0:text="$$p_{ki}$$\n$$tag$$\n\n" xml:space="preserve"><path d="m391.38 693.66-0.0324-0.0975v-0.0325l-0.0108-0.0217-0.0108-0.0325v-0.0217l-0.0324-0.0651v-0.0108l-0.0108-0.0217-0.0108-0.0108-0.0108-0.0217-0.0217-0.0108-0.0216-0.0216-0.0217-0.0108-0.0108-0.0108-0.0434-0.0216h-0.0217l-0.0217-0.0108h-0.0217l-0.0325-0.0108h-0.0758l-0.0217-0.0108h-0.1518c-0.13006 0-0.24928 0-0.24928-0.20593 0-0.0867 0.0542-0.13006 0.1409-0.13006 0.29263 0 0.60695 0.0325 0.89958 0.0325 0.35767 0 0.72617-0.0325 1.073-0.0325 0.0542 0 0.19509 0 0.19509 0.21677 0 0.11922-0.10838 0.11922-0.26012 0.11922-0.54192 0-0.54192 0.0759-0.54192 0.17341 0 0.13006 0.45521 1.8859 0.53108 2.1568 0.1409-0.31432 0.44437-0.69366 0.99713-0.69366l-0.0108 0.23845c-0.71533 0-0.85623 0.88874-0.85623 0.95377 0 0.0217 0 0.0434 0.0325 0.17341l0.53108 2.1135c0.0759 0.29263 0.3685 0.59611 0.56359 0.75868 0.37934 0.32515 0.69366 0.40102 0.87791 0.40102 0.43353 0 0.68282-0.37934 0.68282-1.008 0-0.63946-0.34683-1.875-0.54192-2.2869-0.36851-0.74785-0.88875-1.1055-1.2898-1.1055l0.0108-0.23845c1.2572 0 2.6012 1.5716 2.6012 3.154 0 1.0188-0.60695 1.7233-1.4307 1.7233-0.54191 0-1.0622-0.39018-1.4198-0.80203-0.10838 0.57443-0.57443 0.80203-0.97545 0.80203-0.49857 0-0.7045-0.42269-0.7912-0.61778-0.19509-0.35767-0.33599-1.008-0.33599-1.0405 0-0.10838 0.10838-0.10838 0.13006-0.10838 0.10838 0 0.11922 0.0108 0.18425 0.24928 0.17341 0.75869 0.39018 1.2789 0.78036 1.2789 0.18425 0 0.33599-0.0867 0.33599-0.48773 0-0.24928-0.0325-0.3685-0.0759-0.55275z"/><path d="m398.7 698.36v0.0216l0.0108 0.0108v0.0324l0.0108 0.0108v0.0432h0.0108c0 0.0434-0.0325 0.10839-0.11922 0.10839-0.16257 0-0.7912-0.0542-0.97545-0.0759-0.065-0.0108-0.16258-0.0108-0.16258-0.17341 0-0.10839 0.0975-0.10839 0.19509-0.10839 0.35767 0 0.35767-0.0434 0.35767-0.10838 0-0.0542-0.0108-0.0976-0.0217-0.16258l-1.073-4.2703c-0.0434-0.13006-0.0434-0.15174-0.0434-0.16258 0-0.11922 0.0976-0.23844 0.26012-0.23844 0.20593 0 0.30348 0.1409 0.34683 0.31431 0.0217 0.0325 0.34683 1.3548 0.37934 1.4632 0.53108-0.0542 0.96462-0.22761 0.96462-0.61779 0-0.0434 0-0.0759-0.0108-0.15173-0.0325-0.11923-0.0325-0.15174-0.0325-0.23845 0-0.53108 0.43354-0.76952 0.80204-0.76952 0.73701 0 0.96462 1.138 0.96462 1.1489 0 0.0975-0.0976 0.0975-0.11923 0.0975-0.10838 0-0.11922-0.0325-0.15173-0.18425-0.0976-0.32515-0.30348-0.85623-0.67198-0.85623-0.20593 0-0.26012 0.18425-0.26012 0.39018 0 0.13006 0 0.1409 0.0433 0.34683 0.0108 0.0217 0.0325 0.14089 0.0325 0.2276 0 0.67198-0.91043 0.78036-1.2247 0.80204 0.21677 0.13006 0.49857 0.37934 0.62863 0.49857 0.39018 0.35766 0.76952 0.71533 1.1922 0.71533 0.0867 0 0.18425-0.0217 0.24928-0.0975-0.32515-0.0542-0.39018-0.31431-0.39018-0.42269 0-0.17342 0.11922-0.2818 0.30347-0.2818 0.19509 0 0.4227 0.16257 0.4227 0.48772 0 0.23845-0.17342 0.52025-0.57443 0.52025-0.43354 0-0.83456-0.30348-1.2139-0.66114-0.32515-0.30348-0.57443-0.53108-0.87791-0.66114z"/><path d="m403.55 698.06v0.0324l-0.0108 0.0217v0.0324l-0.0108 0.0108v0.0108l-0.0108 0.0108v0.0108l-0.0216 0.0216v0.0217l-0.0324 0.0324h-0.0108l-0.0432 0.0432h-0.0108l-0.0217 0.0108-0.0108 0.0108h-0.0108l-0.0217 0.0108h-0.0542l-0.0108 0.0108h-0.0217c-0.20593 0-0.4227-0.20593-0.4227-0.42269 0-0.11923 0.0975-0.2818 0.30348-0.2818 0.21677 0 0.42269 0.20593 0.42269 0.41186z"/><path d="m402 694.24v-0.0217l-0.0108-0.0108-0.0108-0.0433-0.0108-0.0108v-0.0217l-0.0108-0.0217v-0.0325l-0.0108-0.0217v-0.0217l-0.0108-0.0217v-0.0864c0-0.35766 0.30347-0.63946 0.71533-0.63946 0.75869 0 1.0947 1.0405 1.0947 1.1489 0 0.0975-0.0976 0.0975-0.11923 0.0975-0.10838 0-0.11922-0.0433-0.1409-0.13006-0.18425-0.59611-0.5094-0.91042-0.81287-0.91042-0.15174 0-0.18425 0.0975-0.18425 0.26012 0 0.17341 0.0542 0.32515 0.11922 0.48772 0.0759 0.20593 0.16257 0.40102 0.24928 0.60695 0.065 0.17342 0.33599 0.86707 0.3685 0.95378 0.0217 0.0759 0.0434 0.16257 0.0434 0.23844 0 0.35767-0.30348 0.63947-0.71533 0.63947-0.75869 0-1.1055-1.0188-1.1055-1.1489 0-0.0976 0.10839-0.0976 0.13006-0.0976 0.10839 0 0.11922 0.0325 0.1409 0.11923 0.19509 0.6503 0.52024 0.92126 0.80204 0.92126 0.13006 0 0.19509-0.065 0.19509-0.27096 0-0.16258-0.0434-0.2818-0.23844-0.74785z"/><g fill-opacity=".86275"><path d="m334.96 674.13h0.62893c0.13374 0 0.20072 0 0.20072 0.13387 0 0.0735-0.0669 0.0735-0.18735 0.0735h-0.58878c0.24088 0.94338 0.27433 1.0772 0.27433 1.1173 0 0.11372-0.0802 0.18064-0.19403 0.18064-0.0202 0-0.20741-7e-3 -0.26764-0.24084l-0.26093-1.0571h-0.62223c-0.13388 0-0.20072 0-0.20072-0.12716 0-0.0802 0.0535-0.0802 0.18733-0.0802h0.5821c-0.48173-1.8868-0.5018-2.0005-0.5018-2.1209 0-0.35462 0.24755-0.60886 0.60885-0.60886 0.68245 0 1.0638 0.97016 1.0638 1.0237 0 0.0669-0.0535 0.0669-0.0803 0.0669-0.0602 0-0.0669-0.0202-0.10027-0.0937-0.2877-0.68915-0.64231-0.84972-0.86979-0.84972-0.14049 0-0.20742 0.0802-0.20742 0.30107 0 0.16059 0.0137 0.20742 0.0401 0.32116z"/><path d="m338.48 673.98-7e-3 0.0267-0.0411 0.0606-0.0137 0.0267-0.0411 0.0606-0.0339 0.0339-0.0137 0.0202-0.0202 0.0137-0.0137 0.0202-0.02 0.0137-0.0339 0.0339-0.0404 0.0274-0.02 0.0137-0.0202 7e-3 -0.0267 0.0137-0.02 7e-3 -0.0202 0.0137-0.0267 7e-3 -0.0202 7e-3 -0.0267 7e-3 -0.0202 7e-3 -0.0267 7e-3h-0.0534l-0.0267 7e-3h-0.0267c-0.78282 0-1.6124-0.97684-1.6124-1.947 0-0.62893 0.36797-1.0638 0.88985-1.0638l0.0137 0.14718c-0.33454 0-0.42821 0.36131-0.42821 0.62223 0 0.33455 0.21409 1.1508 0.36798 1.4586 0.20742 0.38806 0.50851 0.63561 0.77612 0.63561 0.4349 0 0.52188-0.54863 0.52188-0.58878 0-0.0402-0.0137-0.0735-0.0202-0.10699l-0.32784-1.3047c-0.0335-0.12044-0.0335-0.13374-0.13388-0.24755-0.29439-0.3613-0.56869-0.46834-0.75604-0.46834l-0.0137-0.1472c0.13387 0 0.46834 0.0267 0.86978 0.49511 0.0535-0.27432 0.281-0.49511 0.60216-0.49511 0.23416 0 0.38807 0.15389 0.49512 0.3613 0.11371 0.24085 0.2007 0.64898 0.2007 0.66237 0 0.0669-0.0602 0.0669-0.0802 0.0669-0.0669 0-0.0737-0.0267-0.0937-0.12043-0.11371-0.4349-0.23416-0.82297-0.50849-0.82297-0.18064 0-0.2007 0.16726-0.2007 0.30109 0 0.14719 0.0137 0.20072 0.0868 0.49511 0.0735 0.28101 0.0868 0.34123 0.14719 0.59547l0.24087 0.93c0.0469 0.18734 0.0469 0.20072 0.0469 0.22749 0 0.11371-0.0802 0.18063-0.19403 0.18063-0.16057 0-0.26095-0.14719-0.28102-0.29438z"/><path d="m342.66 673.97v0.021l7e-3 0.0137v0.0207l7e-3 7e-3v0.028l7e-3 7e-3v0.0277c0 0.11372-0.0802 0.18065-0.19403 0.18065-0.0669 0-0.24755-0.0468-0.2743-0.28771-0.12044 0.24756-0.35461 0.42151-0.62225 0.42151l7e-3 -0.1472c0.44157 0 0.53524-0.54193 0.53524-0.57539 0-0.0267-0.0137-0.0669-0.0202-0.0937l-0.32115-1.2712c-0.04-0.17394-0.19404-0.34121-0.34123-0.46834-0.14051-0.12043-0.34791-0.23417-0.54195-0.23417-0.32784 0-0.42819 0.34122-0.42819 0.60885 0 0.32116 0.19403 1.104 0.36797 1.4452 0.18066 0.32115 0.46836 0.58877 0.74936 0.58877l-7e-3 0.1472c-0.76273 0-1.5857-0.93-1.5857-1.8868 0-0.66235 0.40814-1.0504 0.88319-1.0504 0.39473 0 0.70919 0.30778 0.77609 0.38137l7e-3 -7e-3c-0.1405-0.58877-0.2208-0.86308-0.2208-0.87647-0.0267-0.0602-0.25425-0.71591-0.95678-0.71591-0.12715 0-0.34792 7e-3 -0.53524 0.0669 0.20071 0.0602 0.27432 0.23417 0.27432 0.34121 0 0.107-0.0737 0.23419-0.25425 0.23419-0.14719 0-0.3613-0.12043-0.3613-0.38137 0-0.27434 0.24756-0.40814 0.88986-0.40814 0.82965 0 1.3114 0.51519 1.4117 0.91663z"/><g fill-rule="evenodd" stroke-miterlimit="10.433"><path d="m334.57 662.85h0.57166c0.12166 0 0.18246 0 0.18246 0.12167 0 0.0669-0.0608 0.0669-0.17029 0.0669h-0.53518c0.21894 0.8575 0.24935 0.97913 0.24935 1.0156 0 0.10346-0.0729 0.16421-0.17637 0.16421-0.0183 0-0.18852-6e-3 -0.24325-0.21894l-0.23718-0.96088h-0.56558c-0.12167 0-0.18245 0-0.18245-0.11552 0-0.0729 0.0487-0.0729 0.17028-0.0729h0.52909c-0.43787-1.715-0.45611-1.8184-0.45611-1.9278 0-0.32232 0.22502-0.55342 0.55342-0.55342 0.62031 0 0.96695 0.88183 0.96695 0.93047 0 0.0608-0.0486 0.0608-0.0729 0.0608-0.0547 0-0.0608-0.0183-0.0913-0.0851-0.2615-0.62641-0.58382-0.77234-0.79059-0.77234-0.12771 0-0.18854 0.0729-0.18854 0.27365 0 0.14596 0.0123 0.18853 0.0364 0.29191z"/><path d="m337.76 662.72-6e-3 0.0244-0.0123 0.0182-0.0123 0.0183-0.0123 0.0182-0.0123 0.0244-0.0123 0.0182-0.0246 0.0366-0.0123 0.0123-0.0183 0.0182-0.0123 0.0183-0.0183 0.0123-0.0123 0.0183-0.0182 0.0123-0.0306 0.0306-0.0366 0.0246-0.0182 0.0123-0.0183 6e-3 -0.0244 0.0123-0.0182 6e-3 -0.0183 0.0123-0.0244 6e-3 -0.0183 6e-3 -0.0244 6e-3 -0.0183 6e-3 -0.0244 6e-3h-0.0488l-0.0243 6e-3h-0.0244c-0.71155 0-1.4656-0.8879-1.4656-1.7697 0-0.57165 0.33447-0.96696 0.80883-0.96696l0.0123 0.13379c-0.30409 0-0.38923 0.32841-0.38923 0.56559 0 0.30407 0.19461 1.046 0.33449 1.3258 0.18853 0.35272 0.46219 0.57774 0.70546 0.57774 0.3953 0 0.47435-0.49868 0.47435-0.53518 0-0.0364-0.0123-0.0669-0.0182-0.0973l-0.29799-1.1859c-0.0305-0.10949-0.0305-0.12167-0.12166-0.22501-0.26758-0.3284-0.51692-0.42571-0.68721-0.42571l-0.0123-0.13379c0.12167 0 0.42571 0.0244 0.79059 0.45004 0.0487-0.24935 0.25543-0.45004 0.54734-0.45004 0.21285 0 0.35273 0.13987 0.45002 0.32841 0.10334 0.21892 0.18246 0.5899 0.18246 0.60206 0 0.0608-0.0547 0.0608-0.0729 0.0608-0.0608 0-0.0669-0.0243-0.0851-0.10948-0.10333-0.3953-0.21284-0.74803-0.46219-0.74803-0.16419 0-0.18245 0.15203-0.18245 0.27367 0 0.13379 0.0123 0.18245 0.079 0.45004 0.0669 0.25542 0.079 0.31015 0.13378 0.54124l0.21894 0.84533c0.0426 0.17029 0.0426 0.18245 0.0426 0.20677 0 0.10334-0.073 0.16421-0.17637 0.16421-0.14595 0-0.23718-0.1338-0.25541-0.26759z"/><path d="m341.57 662.71v0.018l6e-3 0.0123v0.0183l6e-3 6e-3v0.024l6e-3 6e-3v0.0243c0 0.10333-0.0729 0.1642-0.17636 0.1642-0.0608 0-0.22501-0.0426-0.24933-0.26151-0.10949 0.22501-0.32233 0.38313-0.56559 0.38313l6e-3 -0.1338c0.40137 0 0.48651-0.49259 0.48651-0.523 0-0.0244-0.0123-0.0608-0.0183-0.0851l-0.29191-1.1555c-0.0364-0.15811-0.17637-0.31014-0.31016-0.4257-0.12771-0.10949-0.31623-0.21286-0.49261-0.21286-0.29798 0-0.3892 0.31017-0.3892 0.55342 0 0.29193 0.17636 1.0034 0.33447 1.3136 0.16421 0.29192 0.42571 0.53518 0.68113 0.53518l-6e-3 0.13379c-0.69328 0-1.4413-0.84533-1.4413-1.715 0-0.60206 0.37097-0.95479 0.80277-0.95479 0.3588 0 0.64463 0.27974 0.70545 0.34664l6e-3 -6e-3c-0.12771-0.53517-0.20069-0.78452-0.20069-0.79668-0.0244-0.0548-0.23111-0.65072-0.86966-0.65072-0.11551 0-0.31624 6e-3 -0.48652 0.0608 0.18245 0.0547 0.24934 0.21286 0.24934 0.31015 0 0.0973-0.0669 0.21286-0.2311 0.21286-0.13379 0-0.3284-0.10948-0.3284-0.34664 0-0.24935 0.22502-0.37097 0.80885-0.37097 0.7541 0 1.192 0.46827 1.2832 0.83316z"/><path d="m334.9 645.67h0.56345c0.11986 0 0.17982 0 0.17982 0.11986 0 0.066-0.06 0.066-0.16784 0.066h-0.52748c0.2158 0.84517 0.24576 0.96504 0.24576 1.001 0 0.10195-0.0719 0.16184-0.17382 0.16184-0.018 0-0.18582-5e-3 -0.23976-0.21577l-0.23377-0.94707h-0.55743c-0.11986 0-0.17983 0-0.17983-0.11385 0-0.0719 0.048-0.0719 0.16783-0.0719h0.52148c-0.43156-1.6903-0.44956-1.7922-0.44956-1.9001 0-0.31768 0.22178-0.54546 0.54546-0.54546 0.61139 0 0.95304 0.86914 0.95304 0.91708 0 0.06-0.0479 0.06-0.0719 0.06-0.054 0-0.06-0.018-0.0898-0.0839-0.25773-0.61739-0.57541-0.76124-0.77921-0.76124-0.12586 0-0.18583 0.0719-0.18583 0.26973 0 0.14386 0.0123 0.18582 0.036 0.28772z"/><path d="m338.06 645.54-5e-3 0.024-0.0246 0.036-0.0123 0.0179-0.0123 0.024-0.0123 0.0179-0.0246 0.036-0.0123 0.0123-0.018 0.0179-0.0123 0.018-0.018 0.0123-0.0123 0.018-0.0179 0.0123-0.0303 0.0303-0.036 0.0246-0.0179 0.0123-0.018 5e-3 -0.024 0.0123-0.0179 5e-3 -0.018 0.0123-0.0241 5e-3 -0.018 5e-3 -0.0241 5e-3 -0.018 5e-3 -0.0241 5e-3h-0.048l-0.024 5e-3h-0.024c-0.70129 0-1.4446-0.87511-1.4446-1.7442 0-0.56344 0.32967-0.95305 0.79721-0.95305l0.0123 0.13187c-0.2997 0-0.38362 0.32367-0.38362 0.55743 0 0.2997 0.19181 1.031 0.32967 1.3067 0.18583 0.34764 0.45555 0.56942 0.6953 0.56942 0.38963 0 0.46754-0.4915 0.46754-0.52749 0-0.0359-0.0123-0.066-0.018-0.0959l-0.29371-1.1688c-0.0301-0.10796-0.0301-0.11986-0.11986-0.22178-0.26375-0.32367-0.50949-0.41957-0.67732-0.41957l-0.0123-0.13187c0.11986 0 0.41958 0.024 0.77923 0.44355 0.048-0.24576 0.25174-0.44355 0.53945-0.44355 0.20979 0 0.34766 0.13785 0.44357 0.32368 0.10195 0.21576 0.17982 0.58141 0.17982 0.5934 0 0.06-0.054 0.06-0.0719 0.06-0.06 0-0.066-0.024-0.0839-0.10784-0.10195-0.38961-0.20978-0.73726-0.45553-0.73726-0.16184 0-0.17982 0.14985-0.17982 0.26972 0 0.13188 0.0123 0.17983 0.0779 0.44357 0.066 0.25175 0.0779 0.30569 0.13188 0.53347l0.21576 0.83317c0.042 0.16782 0.042 0.17982 0.042 0.20379 0 0.10195-0.0719 0.16183-0.17384 0.16183-0.14385 0-0.23377-0.13187-0.25174-0.26372z"/><path d="m341.8 645.53v0.015l5e-3 0.0123v0.0183l5e-3 5e-3v0.02l5e-3 5e-3v0.0223c0 0.10195-0.0719 0.16184-0.17382 0.16184-0.06 0-0.22179-0.042-0.24575-0.25775-0.10797 0.22177-0.31768 0.3776-0.55746 0.3776l5e-3 -0.13185c0.39561 0 0.47951-0.48551 0.47951-0.51548 0-0.0241-0.0123-0.06-0.018-0.0839l-0.28771-1.1389c-0.036-0.15585-0.17383-0.30568-0.30569-0.41957-0.12588-0.10796-0.31169-0.2098-0.48553-0.2098-0.2937 0-0.38361 0.3057-0.38361 0.54545 0 0.28772 0.17383 0.98902 0.32966 1.2947 0.16186 0.28772 0.41959 0.52747 0.67134 0.52747l-5e-3 0.13188c-0.68332 0-1.4206-0.83318-1.4206-1.6903 0-0.5934 0.36562-0.94106 0.79121-0.94106 0.35364 0 0.63536 0.27573 0.69529 0.34167l6e-3 -5e-3c-0.12587-0.52747-0.1978-0.77323-0.1978-0.78522-0.024-0.054-0.22778-0.64135-0.85715-0.64135-0.11386 0-0.3117 5e-3 -0.47952 0.06 0.17982 0.054 0.24575 0.20978 0.24575 0.3057 0 0.0959-0.066 0.20979-0.22776 0.20979-0.13188 0-0.32368-0.10796-0.32368-0.34167 0-0.24576 0.22177-0.36563 0.7972-0.36563 0.74326 0 1.1748 0.46154 1.2647 0.82118z"/></g></g><g transform="matrix(.64658 0 0 .64658 81.355 252.81)" fill-rule="evenodd" stroke="#000" stroke-miterlimit="10.433" ns0:preamble="default_packages.tex" ns0:text="$$13 \\%$$\n$$7 \\%$$\n$$1 \\%$$\n$$2.2 \\%$$\n" xml:space="preserve"><path d="m434.48 651.79v0.144l-9e-3 9e-3v0.018l-9e-3 9e-3v9e-3l-0.018 0.018h-9e-3v9e-3h-0.018l-9e-3 9e-3h-0.063l-9e-3 9e-3h-0.063c-0.53359-0.55983-1.3034-0.55983-1.5833-0.55983v-0.27117c0.17495 0 0.69104 0 1.1372 0.22744v-4.4961c0-0.31491-0.0263-0.41987-0.80476-0.41987h-0.27992v-0.27117c0.30616 0.0262 1.0584 0.0262 1.4083 0.0262 0.34989 0 1.1109 0 1.4171-0.0262v0.27117h-0.27992c-0.78726 0-0.8135 0.0962-0.8135 0.41987z"/><path d="m438.8 649.28 0.07 0.0175 0.0612 0.0263 0.07 0.0262 0.0612 0.035 0.0612 0.0262 0.0612 0.035 0.0525 0.035 0.0612 0.035 0.0525 0.0437 0.0525 0.035 0.105 0.0874 0.0437 0.0437 0.0437 0.0525 0.0437 0.0437 0.0437 0.0525 0.035 0.0525 0.0437 0.0437 0.035 0.0612 0.0262 0.0525 0.035 0.0525 0.0262 0.0612 0.0262 0.0525 0.0262 0.0612 0.035 0.1224 0.0175 0.0525 9e-3 0.07 0.0175 0.0612v0.0612l9e-3 0.0612v0.0612c0 0.71728-0.76977 1.2071-1.6095 1.2071-0.87474 0-1.5395-0.52484-1.5395-1.1896 0-0.27991 0.19244-0.44611 0.44612-0.44611 0.27117 0 0.44611 0.19244 0.44611 0.43737 0 0.43736-0.41112 0.43736-0.54233 0.43736 0.27116 0.42862 0.84849 0.54234 1.1634 0.54234 0.3499 0 0.831-0.19244 0.831-0.9797 0-0.0962-0.0175-0.60357-0.24492-0.98846-0.25368-0.41112-0.55983-0.43736-0.76977-0.44611-0.07-9e-3 -0.27992-0.0262-0.34115-0.0262-0.07-9e-3 -0.13121-0.0175-0.13121-0.10497 0-0.0962 0.0612-0.0962 0.20994-0.0962h0.38488c0.70854 0 1.0322-0.59482 1.0322-1.4433 0-1.1809-0.60357-1.4346-0.9797-1.4346-0.37614 0-1.0322 0.14871-1.3384 0.65605 0.30616-0.0437 0.57733 0.14871 0.57733 0.48111 0 0.3149-0.23618 0.48985-0.48985 0.48985-0.20994 0-0.48986-0.12246-0.48986-0.50735 0-0.78726 0.81351-1.3646 1.7582-1.3646 1.0672 0 1.8632 0.78726 1.8632 1.6795 0 0.70853-0.55109 1.3908-1.4608 1.5745z"/><path d="m446.6 652.41 9e-3 9e-3v9e-3l0.018 0.018v9e-3h9e-3v9e-3l9e-3 9e-3v9e-3h9e-3v0.018h9e-3v0.018l9e-3 9e-3v0.045c0 0.0962-0.0787 0.17494-0.1662 0.17494s-0.13121-0.0612-0.19244-0.12246c-0.42862-0.58607-1.0147-0.82225-1.6532-0.82225-0.61231 0-1.1546 0.21868-1.627 0.6648-0.1662 0.1487-0.3499 0.27991-0.62981 0.27991l9e-3 -0.19244c0.46361 0 0.90098-0.63856 0.90098-1.557 0-0.93596-0.44611-1.557-0.90098-1.557-0.17494 0-0.75227 0.11372-0.75227 1.5483 0 1.4521 0.56858 1.5658 0.75227 1.5658l-9e-3 0.19244c-0.6648 0-1.2859-0.72603-1.2859-1.7495 0-1.0497 0.62981-1.7495 1.2859-1.7495 0.63855 0 1.1284 0.76977 1.1284 1.7407 0 0.12247 0 0.53359-0.17495 0.98846 0.55983-0.32366 0.9972-0.36739 1.3121-0.36739 0.6648 0 1.1109 0.27116 1.1634 0.30615l9e-3 -9e-3 -3.9188-5.817c-0.0787-0.11372-0.0787-0.15745-0.0787-0.17495 0-0.0962 0.0875-0.17494 0.17494-0.17494 0.0875 0 0.10497 0.035 0.17495 0.13121z"/><path d="m447.4 647.48v0.1837l-0.018 0.175-0.035 0.175-0.0175 0.0787-0.0175 0.0875-0.0262 0.0787-0.0262 0.07-0.0263 0.0787-0.0262 0.07-0.035 0.07-0.105 0.1836-0.0437 0.0612-0.035 0.0525-0.0437 0.0525-0.0437 0.0437-0.0525 0.0437-0.0437 0.0437-0.0525 0.035-0.0437 0.035-0.0525 0.035-0.0525 0.0263-0.0525 0.0262-0.0612 0.0175-0.0525 9e-3 -0.0525 0.0175h-0.0612l-0.0525 9e-3c-0.6648 0-1.2859-0.73477-1.2859-1.7495 0-1.0497 0.62981-1.7582 1.2859-1.7582v0.19244c-0.1662 0-0.74353 0.11371-0.74353 1.557 0 1.4521 0.56858 1.5658 0.74353 1.5658 0.4636 0 0.90972-0.6473 0.90972-1.557 0-0.93596-0.45486-1.5658-0.90972-1.5658v-0.19244c0.6298 0 1.1284 0.77851 1.1284 1.7495z"/><path d="m464.11 651.92 9e-3 9e-3v9e-3l0.018 0.018v9e-3l9e-3 9e-3h9e-3v0.018l9e-3 9e-3v9e-3h9e-3v0.027l9e-3 9e-3v0.036l9e-3 0.018v0.126h-2.1845c-1.0878 0-1.1058 0.11686-1.1417 0.28767h-0.22474l-0.29667-1.8339h0.22475c0.027 0.14384 0.10787 0.70121 0.22474 0.80909 0.0629 0.0539 0.76414 0.0539 0.88101 0.0539h1.8519c-0.0989-0.14384-0.80909-1.1148-1.0069-1.4114-0.8001-1.2046-1.0968-2.4452-1.0968-3.3532 0-0.0899 0-0.49444 0.41354-0.49444 0.40454 0 0.40454 0.40454 0.40454 0.49444v0.44949c0 0.49445 0.027 0.9799 0.0989 1.4654 0.036 0.20677 0.16182 0.97091 0.55737 1.5283z"/><path d="m470.46 652.83 9e-3 9e-3v9e-3h9e-3v9e-3l0.018 0.018v9e-3h9e-3v9e-3l9e-3 9e-3v0.018h9e-3v0.018h9e-3v0.054c0 0.0989-0.0809 0.1798-0.17979 0.1798-0.0809 0-0.13485-0.0629-0.18879-0.12586-0.4405-0.61131-1.0428-0.85403-1.6991-0.85403-0.62929 0-1.1867 0.22474-1.6721 0.69222-0.17081 0.15282-0.35959 0.28767-0.64727 0.28767l9e-3 -0.19778c0.47646 0 0.93495-0.66525 0.93495-1.6002 0-0.96192-0.46748-1.6092-0.93495-1.6092-0.1798 0-0.77313 0.11687-0.77313 1.6002 0 1.4923 0.58434 1.6092 0.77313 1.6092l-9e-3 0.19778c-0.68323 0-1.3215-0.75515-1.3215-1.798 0-1.0788 0.64727-1.807 1.3215-1.807 0.65626 0 1.1597 0.80009 1.1597 1.798 0 0.12586 0 0.54838-0.17979 1.0069 0.57535-0.32363 1.0248-0.36858 1.3485-0.36858 0.68323 0 1.1417 0.2697 1.1957 0.30565l9e-3 -9e-3 -4.0275-5.9693c-0.0809-0.11687-0.0809-0.16182-0.0809-0.1798 0-0.0989 0.0899-0.1798 0.1798-0.1798s0.10787 0.036 0.17979 0.13485z"/><path d="m471.28 647.77v0.1888l-0.018 0.1798-0.036 0.1798-0.036 0.1618-0.054 0.1618-0.054 0.1438-0.072 0.1438-0.072 0.1258-0.045 0.0539-0.036 0.0629-0.0449 0.0539-0.045 0.045-0.0539 0.0449-0.0449 0.0449-0.0539 0.036-0.0449 0.036-0.0539 0.036-0.0539 0.027-0.0539 0.018-0.0629 0.018-0.0539 0.018-0.0539 9e-3 -0.0629 9e-3h-0.0629c-0.67424 0-1.3125-0.74616-1.3125-1.789 0-1.0878 0.64727-1.807 1.3125-1.807l0.018 0.19778c-0.1798 0-0.77313 0.11687-0.77313 1.6002 0 1.4923 0.58434 1.6092 0.77313 1.6092 0.46747 0 0.92595-0.66525 0.92595-1.6002 0-0.9709-0.46747-1.6092-0.92595-1.6092l-0.018-0.19778c0.65626 0 1.1687 0.8001 1.1687 1.798z"/><path d="m461.55 637.21v0.153l-9e-3 9e-3v0.018l-9e-3 9e-3v9e-3l-9e-3 9e-3h-9e-3v9e-3h-0.018l-9e-3 9e-3h-0.027l-9e-3 9e-3h-0.108c-0.55882-0.5863-1.365-0.5863-1.6581-0.5863v-0.27483c0.18322 0 0.72371 0 1.2001 0.22902v-4.7087c0-0.32979-0.0275-0.43972-0.85197-0.43972h-0.29314v-0.28399c0.32063 0.0275 1.1176 0.0275 1.4841 0.0275 0.35728 0 1.1543 0 1.4749-0.0275v0.28399h-0.29315c-0.82448 0-0.85196 0.10077-0.85196 0.43972z"/><path d="m469.69 637.87 0.018 0.018v9e-3h9e-3v9e-3l0.018 0.018v9e-3l9e-3 9e-3v9e-3l9e-3 9e-3v0.018h9e-3v0.054c0 0.0916-0.0824 0.17406-0.18321 0.17406-0.0824 0-0.13742-0.0641-0.19238-0.12825-0.44889-0.61378-1.0627-0.86113-1.7314-0.86113-0.64126 0-1.2092 0.22903-1.7039 0.70539-0.17405 0.14658-0.36643 0.28399-0.65958 0.28399l9e-3 -0.19238c0.48553 0 0.95273-0.67791 0.95273-1.6306 0-0.98938-0.47636-1.6398-0.95273-1.6398-0.18322 0-0.78784 0.11909-0.78784 1.6306 0 1.5207 0.59546 1.6398 0.78784 1.6398l-9e-3 0.19238c-0.69623 0-1.3467-0.76035-1.3467-1.823 0-1.0993 0.65959-1.8413 1.3467-1.8413 0.66875 0 1.1818 0.81532 1.1818 1.8322 0 0.12825 0 0.54965-0.18322 1.026 0.5863-0.33895 1.0443-0.38476 1.3741-0.38476 0.69622 0 1.1634 0.28399 1.2184 0.32063l9e-3 -9e-3 -4.1041-6.0828c-0.0824-0.11909-0.0824-0.1649-0.0824-0.18322 0-0.10077 0.0916-0.18322 0.18322-0.18322 0.0916 0 0.10993 0.0366 0.18322 0.13741z"/><path d="m470.52 632.71v0.20154l-0.018 0.1832-0.0183 0.0916-0.0183 0.0824-0.0183 0.0916-0.0183 0.0824-0.055 0.1648-0.055 0.1466-0.0366 0.0733-0.0367 0.0733-0.0732 0.1282-0.0458 0.055-0.0366 0.055-0.0916 0.11-0.055 0.0458-0.0458 0.0458-0.055 0.0366-0.0458 0.0366-0.165 0.0825-0.0641 0.0183-0.055 0.0183-0.055 9e-3 -0.0641 9e-3h-0.0641l0.0183-0.20154c0.47637 0 0.94358-0.66875 0.94358-1.6306 0-0.98021-0.47637-1.6306-0.94358-1.6306-0.18321 0-0.78784 0.10993-0.78784 1.6215 0 1.5207 0.59546 1.6398 0.78784 1.6398l-0.0183 0.20154c-0.68707 0-1.3375-0.76036-1.3375-1.8322 0-1.0993 0.65958-1.8322 1.3375-1.8322 0.66875 0 1.1909 0.80616 1.1909 1.823z"/><path d="m431.4 630.68 0.94823 0.91245c1.3955 1.2255 1.9233 1.7086 1.9233 2.5942 0 1.0108-0.79616 1.7265-1.8875 1.7265-1.0019 0-1.6639-0.82299-1.6639-1.6102 0-0.50095 0.44728-0.50095 0.47411-0.50095 0.14313 0 0.45623 0.10734 0.45623 0.47411 0 0.23259-0.16102 0.46517-0.46517 0.46517-0.0716 0-0.0984 0-0.12524-9e-3 0.21469 0.57251 0.68881 0.9035 1.2076 0.9035 0.81405 0 1.1987-0.71565 1.1987-1.4492 0-0.7067-0.44728-1.4134-0.93929-1.968l-1.7086-1.8965c-0.0984-0.0984-0.0984-0.11629-0.0984-0.33098h3.3099l0.24153 1.5476h-0.21469c-0.0447-0.26837-0.10735-0.66197-0.19681-0.78721-0.0626-0.0716-0.65302-0.0716-0.84983-0.0716z"/><path d="m436.43 630.46v0.0715l-9e-3 0.0179v0.0268l-9e-3 0.0179-9e-3 0.0268-9e-3 0.0179-9e-3 0.0268-0.018 0.0358-0.0179 0.0179-9e-3 0.0179-0.0179 0.0179-9e-3 0.0179-0.0358 0.0358-0.0179 9e-3 -0.0179 0.0179-0.0179 9e-3 -0.0179 0.0179-0.0179 9e-3 -0.0268 9e-3 -0.0358 0.018-0.0268 9e-3 -0.0179 9e-3h-0.0268l-0.0268 9e-3h-0.0626c-0.25942 0-0.47412-0.20575-0.47412-0.46517s0.2147-0.47411 0.47412-0.47411c0.25047 0 0.46517 0.21469 0.46517 0.47411z"/><path d="m438.33 630.68 0.94823 0.91245c1.3866 1.2255 1.9233 1.7086 1.9233 2.5942 0 1.0108-0.79615 1.7265-1.8875 1.7265-1.0108 0-1.6639-0.82299-1.6639-1.6102 0-0.50095 0.43834-0.50095 0.46517-0.50095 0.15208 0 0.46517 0.10734 0.46517 0.47411 0 0.23259-0.16102 0.46517-0.47411 0.46517-0.0716 0-0.0895 0-0.1163-9e-3 0.20575 0.57251 0.68881 0.9035 1.2077 0.9035 0.81404 0 1.1987-0.71565 1.1987-1.4492 0-0.7067-0.44728-1.4134-0.93928-1.968l-1.7176-1.8965c-0.0895-0.0984-0.0895-0.11629-0.0895-0.33098h3.3009l0.25047 1.5476h-0.22364c-0.0447-0.26837-0.10734-0.66197-0.1968-0.78721-0.0537-0.0716-0.64408-0.0716-0.84088-0.0716z"/><path d="m447.77 636.32 9e-3 9e-3v9e-3l0.018 0.018v9e-3h9e-3v9e-3l9e-3 9e-3v9e-3l9e-3 9e-3v0.018h9e-3v0.027h9e-3v0.027c0 0.0984-0.0805 0.17891-0.17891 0.17891-0.0895 0-0.13418-0.0626-0.18786-0.12524-0.44728-0.59935-1.0466-0.84088-1.6996-0.84088-0.61725 0-1.1719 0.22364-1.6639 0.68881-0.16997 0.14313-0.35782 0.27731-0.64408 0.27731l9e-3 -0.1968c0.47411 0 0.93033-0.65303 0.93033-1.5834 0-0.96612-0.46517-1.6012-0.93033-1.6012-0.17892 0-0.76932 0.11629-0.76932 1.5923s0.58146 1.5923 0.76932 1.5923l-9e-3 0.1968c-0.67986 0-1.306-0.74248-1.306-1.7802 0-1.0824 0.63513-1.798 1.306-1.798 0.65303 0 1.1629 0.78721 1.1629 1.7891 0 0.12524 0 0.53673-0.18785 1.0019 0.58146-0.33098 1.0287-0.37571 1.3418-0.37571 0.67986 0 1.1361 0.27731 1.1898 0.31309l9e-3 -9e-3 -4.0076-5.9488c-0.0805-0.1163-0.0805-0.16102-0.0805-0.17891 0-0.0895 0.0895-0.16997 0.17892-0.16997 0.0894 0 0.10734 0.0268 0.17891 0.13418z"/><path d="m448.58 631.28-9e-3 0.0984v0.0895l-9e-3 0.0984-9e-3 0.0895-0.0179 0.0895-9e-3 0.0805-0.0179 0.0894-0.0268 0.0805-0.0179 0.0805-0.0268 0.0716-0.0268 0.0805-0.0358 0.0716-0.0268 0.0716-0.0358 0.0626-0.0358 0.0716-0.0447 0.0537-0.0358 0.0626-0.1341 0.1611-0.0447 0.0447-0.0537 0.0358-0.0447 0.0447-0.0537 0.0358-0.1611 0.0804-0.0537 0.0179-0.0626 0.0179-0.1074 0.018h-0.0626l9e-3 -0.19681c0.47411 0 0.93033-0.66197 0.93033-1.5923 0-0.95717-0.46517-1.6012-0.93033-1.6012-0.17892 0-0.76932 0.11629-0.76932 1.5923 0 1.485 0.58146 1.6012 0.76932 1.6012l-9e-3 0.19681c-0.67986 0-1.315-0.75143-1.315-1.7891 0-1.0735 0.64408-1.7891 1.315-1.7891 0.65303 0 1.1629 0.78721 1.1629 1.7802z"/></g></g><path d="m448.5 647.63v0.187l-0.011 0.011v0.022l-0.0111 0.011v0.011l-0.0111 0.011h-0.011v0.011h-0.0221l-0.0111 0.011h-0.0332l-0.0111 0.011h-0.1328c-0.68651-0.7203-1.6769-0.7203-2.037-0.7203v-0.3376c0.22509 0 0.88908 0 1.4743 0.2813v-5.7846c0-0.4051-0.0338-0.5402-1.0466-0.5402h-0.36012v-0.3489c0.39389 0.034 1.373 0.034 1.8232 0.034 0.43892 0 1.418 0 1.8119-0.034v0.3489h-0.36013c-1.0129 0-1.0466 0.1238-1.0466 0.5402z"/><path d="m458.49 648.44 0.0222 0.022v0.011h0.0111v0.011l0.0111 0.011 0.011 0.011v0.011l0.0111 0.011v0.011l0.0111 0.011v0.022h0.0111v0.066c-1e-5 0.1126-0.10124 0.2139-0.22509 0.2139-0.10122 0-0.16882-0.079-0.23633-0.1576-0.55146-0.754-1.3055-1.0579-2.127-1.0579-0.78779 0-1.4856 0.2814-2.0933 0.8666-0.21382 0.18-0.45016 0.3489-0.81029 0.3489l0.0111-0.2364c0.59647 0 1.1704-0.8328 1.1704-2.0032 0-1.2155-0.58521-2.0145-1.1704-2.0145-0.22508 0-0.96786 0.1463-0.96786 2.0032 0 1.8682 0.73152 2.0145 0.96786 2.0145l-0.0111 0.2364c-0.85532 0-1.6544-0.9341-1.6544-2.2396 0-1.3505 0.8103-2.2621 1.6544-2.2621 0.82155 0 1.4518 1.0016 1.4518 2.2508 0 0.1576 0 0.6753-0.22508 1.2605 0.72026-0.4164 1.283-0.4727 1.6881-0.4727 0.8553 0 1.4293 0.3489 1.4968 0.3939l0.0111-0.011-5.0419-7.4728c-0.10122-0.1463-0.10122-0.2026-0.10122-0.2251 0-0.1238 0.11253-0.2251 0.22508-0.2251 0.11253 0 0.13505 0.045 0.22508 0.1689z"/><path d="m459.52 642.09v0.2476l-0.0222 0.225-0.0225 0.1126-0.0225 0.1012-0.0225 0.1125-0.0225 0.1012-0.0338 0.1013-0.0338 0.1012-0.0676 0.18-0.045 0.09-0.0451 0.09-0.09 0.158-0.0563 0.067-0.045 0.068-0.1126 0.136-0.0676 0.056-0.0563 0.056-0.0676 0.045-0.0563 0.045-0.2028 0.102-0.0787 0.022-0.0676 0.023-0.0676 0.011-0.0787 0.011h-0.0787l0.0225-0.2476c0.58522 0 1.1592-0.8215 1.1592-2.0032 0-1.2042-0.58522-2.0032-1.1592-2.0032-0.22507 0-0.96785 0.135-0.96785 1.9919 0 1.8682 0.73152 2.0145 0.96785 2.0145l-0.0225 0.2476c-0.84406 0-1.6431-0.9341-1.6431-2.2508 0-1.3505 0.81029-2.2508 1.6431-2.2508 0.82155 0 1.463 0.9903 1.463 2.2395z"/></g></g>
   <g transform="matrix(.47238 0 0 -.47238 117.79 625.1)" ns0:preamble="default_packages.tex" ns0:text="$$5 \\%$$\n$$11 \\%$$\n$$9.3 \\%$$\n$$0.7 \\%$$\n" xml:space="preserve"><path d="m261.44 694.38v0.11764l-0.0107 0.11764-0.0107 0.10695-0.0214 0.11764-0.0214 0.10694-0.0214 0.11764-0.0321 0.10695-0.0321 0.10694-0.0428 0.0963-0.0428 0.10695-0.0535 0.0963-0.0535 0.0855-0.0535 0.0963-0.0535 0.0856-0.1284 0.1712-0.1498 0.1498-0.0749 0.0642-0.0749 0.0749-0.0855 0.0535-0.0856 0.0642-0.0855 0.0428-0.0856 0.0535-0.0962 0.0428-0.2886 0.0963-0.10695 0.0214-0.10694 0.0107-0.0962 0.0107h-0.10695c-0.51333 0-0.9732-0.17111-1.3582-0.53473v2.064c0.21389-0.0642 0.56681-0.13902 0.90903-0.13902 1.3047 0 2.0534 0.9732 2.0534 1.1122 0 0.0642-0.0321 0.11764-0.10695 0.11764-0.0107 0-0.0321 0-0.0856-0.0321-0.21389-0.0962-0.73792-0.31014-1.4438-0.31014-0.42778 0-0.91973 0.0749-1.4224 0.29945-0.0856 0.0321-0.10695 0.0321-0.12834 0.0321-0.10694 0-0.10694-0.0856-0.10694-0.25667v-3.1442c0-0.1925 0-0.27806 0.14972-0.27806 0.0749 0 0.0962 0.0321 0.13903 0.0963 0.11764 0.17111 0.51334 0.73792 1.3796 0.73792 0.54542 0 0.81279-0.48125 0.89834-0.67375 0.17111-0.3957 0.1925-0.81279 0.1925-1.3368 0-0.3743 0-1.016-0.25667-1.4544-0.25666-0.41708-0.65236-0.69514-1.1336-0.69514-0.7807 0-1.3903 0.56681-1.5721 1.1871 0.0321-0.0107 0.0642-0.0214 0.18181-0.0214 0.35292 0 0.53473 0.26736 0.53473 0.52403s-0.18181 0.52403-0.53473 0.52403c-0.14972 0-0.52403-0.0749-0.52403-0.56681 0-0.91973 0.73792-1.9464 1.9357-1.9464 1.2299 0 2.3207 1.0267 2.3207 2.3742z"/><path d="m269.28 699.8 0.0107 0.0107v0.0107l0.0214 0.0214v0.0107h0.0107v0.0107l0.0107 0.0107v0.0107h0.0107v0.0214h0.0107v0.0214l0.0107 0.0107v0.0535c0 0.11764-0.0962 0.21389-0.21389 0.21389-0.0962 0-0.16042-0.0749-0.22458-0.14973-0.52403-0.71653-1.2406-1.0053-2.0213-1.0053-0.74862 0-1.4117 0.26736-1.9892 0.81278-0.2032 0.18181-0.42778 0.34223-0.77001 0.34223l0.0107-0.23528c0.56681 0 1.1122-0.7807 1.1122-1.9036 0-1.1443-0.55612-1.9036-1.1122-1.9036-0.21389 0-0.91973 0.13903-0.91973 1.8929 0 1.7753 0.69514 1.9143 0.91973 1.9143l-0.0107 0.23528c-0.81278 0-1.5721-0.88765-1.5721-2.1389 0-1.2833 0.77001-2.1389 1.5721-2.1389 0.7807 0 1.3796 0.94111 1.3796 2.1282 0 0.14973 0 0.65237-0.21389 1.2085 0.68445-0.3957 1.2192-0.44917 1.6042-0.44917 0.81278 0 1.3582 0.33153 1.4224 0.37431l0.0107-0.0107-4.7911-7.1118c-0.0963-0.13903-0.0963-0.1925-0.0963-0.21389 0-0.11764 0.10695-0.21389 0.21389-0.21389 0.10695 0 0.12834 0.0428 0.21389 0.16042z"/><path d="m270.26 693.78v0.22458l-0.0107 0.10695-0.0107 0.10694-0.0214 0.10695-0.0214 0.10694-0.0214 0.0963-0.0214 0.10694-0.0321 0.0963-0.0321 0.0856-0.0321 0.0962-0.0321 0.0856-0.0428 0.0856-0.1284 0.2247-0.0535 0.0749-0.0428 0.0642-0.0535 0.0642-0.0535 0.0535-0.0642 0.0535-0.0535 0.0535-0.0642 0.0428-0.0535 0.0428-0.0642 0.0428-0.1284 0.0642-0.0749 0.0214-0.0642 0.0107-0.0642 0.0214h-0.0749l-0.0749 0.0107c-0.80209 0-1.5614-0.89834-1.5614-2.1389 0-1.2833 0.77001-2.1496 1.5614-2.1496l0.0214 0.23528c-0.21389 0-0.91973 0.13903-0.91973 1.9036 0 1.7753 0.69514 1.9143 0.91973 1.9143 0.55611 0 1.1015-0.7914 1.1015-1.9036 0-1.1443-0.55611-1.9143-1.1015-1.9143l-0.0214-0.23528c0.7807 0 1.3903 0.95181 1.3903 2.1389z"/><path d="m196.59 651.76v0.17l-0.01 0.01v0.02l-0.01 0.01v0.01h-0.01v0.01h-0.01l-0.02 0.02h-0.02l-0.01 0.01h-0.14c-0.61-0.63-1.49-0.63-1.81-0.63v-0.31c0.2 0 0.79 0 1.3 0.26v-5.15c0-0.36-0.03-0.48-0.92-0.48h-0.32v-0.3c0.35 0.03 1.21 0.03 1.61 0.03s1.27 0 1.62-0.03v0.3h-0.32c-0.9 0-0.93 0.11-0.93 0.48z"/><path d="m201.57 651.76v0.17l-0.01 0.01v0.02l-0.01 0.01v0.01l-0.01 0.01h-0.01v0.01h-0.01l-0.01 0.01h-0.02l-0.01 0.01h-0.14c-0.61-0.63-1.49-0.63-1.81-0.63v-0.31c0.2 0 0.79 0 1.31 0.26v-5.15c0-0.36-0.03-0.48-0.93-0.48h-0.32v-0.3c0.35 0.03 1.22 0.03 1.62 0.03 0.39 0 1.26 0 1.61-0.03v0.3h-0.32c-0.9 0-0.93 0.11-0.93 0.48z"/><path d="m210.45 652.48 0.01 0.01v0.01h0.01v0.01l0.02 0.02v0.01h0.01v0.01l0.01 0.01v0.02h0.01v0.02h0.01v0.06c0 0.11-0.09 0.2-0.19 0.2s-0.15-0.07-0.22-0.14c-0.49-0.68-1.16-0.95-1.89-0.95-0.7 0-1.32 0.25-1.86 0.77-0.19 0.17-0.4 0.32-0.72 0.32l0.01-0.22c0.53 0 1.03-0.74 1.03-1.78 0-1.07-0.51-1.79-1.03-1.79-0.2 0-0.86 0.13-0.86 1.78 0 1.66 0.65 1.79 0.86 1.79l-0.01 0.22c-0.76 0-1.47-0.84-1.47-2 0-1.2 0.72-2.01 1.47-2.01 0.73 0 1.29 0.89 1.29 2 0 0.14 0 0.61-0.2 1.12 0.64-0.36 1.14-0.41 1.5-0.41 0.76 0 1.27 0.3 1.33 0.34l0.01-0.01-4.48-6.64c-0.09-0.13-0.09-0.18-0.09-0.2 0-0.11 0.1-0.2 0.2-0.2s0.12 0.04 0.2 0.15z"/><path d="m211.36 646.85v0.21l-0.02 0.2-0.04 0.2-0.04 0.18-0.06 0.18-0.06 0.16-0.08 0.16-0.08 0.14-0.05 0.06-0.04 0.07-0.05 0.06-0.05 0.05-0.06 0.05-0.05 0.05-0.06 0.04-0.05 0.04-0.06 0.04-0.06 0.03-0.06 0.02-0.07 0.02-0.06 0.02-0.06 0.01-0.07 0.01h-0.06c-0.76 0-1.47-0.83-1.47-1.99 0-1.21 0.72-2.01 1.47-2.01v0.22c-0.19 0-0.85 0.13-0.85 1.78 0 1.66 0.65 1.79 0.85 1.79 0.53 0 1.04-0.74 1.04-1.78 0-1.08-0.52-1.79-1.04-1.79v-0.22c0.72 0 1.29 0.89 1.29 2z"/><path d="m256.19 678.87v-0.31c0-2.33-1.04-2.79-1.62-2.79-0.17 0-0.71 0.02-0.98 0.36 0.44 0 0.52 0.29 0.52 0.46 0 0.31-0.24 0.46-0.46 0.46-0.15 0-0.45-0.09-0.45-0.48 0-0.67 0.54-1.07 1.38-1.07 1.29 0 2.51 1.35 2.51 3.48 0 2.67-1.15 3.35-2.04 3.35-0.55 0-1.04-0.18-1.46-0.63-0.41-0.44-0.64-0.86-0.64-1.61 0-1.24 0.87-2.21 1.99-2.21l0.01 0.23c-0.16 0-0.62 0-0.93 0.62-0.18 0.37-0.18 0.87-0.18 1.35 0 0.54 0 1.01 0.21 1.38 0.27 0.49 0.65 0.62 1 0.62 0.46 0 0.79-0.34 0.96-0.78 0.12-0.32 0.16-0.95 0.16-1.4 0-0.83-0.34-1.79-1.22-1.79l-0.01-0.23c0.61 0 1.02 0.42 1.25 0.99z"/><path d="m259.42 676.24v0.08l-0.01 0.03v0.02l-0.01 0.03-0.02 0.04-0.01 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.02 0.02-0.01 0.02-0.06 0.06-0.02 0.01-0.02 0.02-0.04 0.02-0.03 0.01-0.02 0.02h-0.02l-0.03 0.01-0.02 0.01-0.03 0.01h-0.05l-0.03 0.01h-0.02c-0.29 0-0.53-0.24-0.53-0.53s0.24-0.53 0.53-0.53c0.28 0 0.52 0.24 0.52 0.53z"/><path d="m263.17 679.21 0.07 0.03 0.08 0.03 0.14 0.06 0.07 0.04 0.07 0.03 0.07 0.04 0.06 0.05 0.06 0.04 0.12 0.1 0.06 0.04 0.05 0.06 0.06 0.05 0.05 0.05 0.04 0.06 0.05 0.06 0.12 0.18 0.03 0.07 0.03 0.06 0.03 0.07 0.03 0.06 0.02 0.07 0.03 0.07 0.01 0.07 0.02 0.07 0.02 0.14v0.15c0 0.81-0.87 1.37-1.83 1.37-1.01 0-1.77-0.6-1.77-1.35 0-0.33 0.22-0.52 0.51-0.52 0.31 0 0.51 0.22 0.51 0.51 0 0.5-0.47 0.5-0.62 0.5 0.31 0.48 0.97 0.61 1.33 0.61 0.41 0 0.96-0.22 0.96-1.11 0-0.12-0.02-0.7-0.28-1.13-0.3-0.48-0.64-0.51-0.89-0.52-0.08-0.01-0.32-0.03-0.39-0.03-0.08-0.01-0.15-0.02-0.15-0.12 0-0.11 0.07-0.11 0.24-0.11h0.44c0.82 0 1.19-0.67 1.19-1.65 0-1.35-0.69-1.64-1.13-1.64-0.43 0-1.18 0.17-1.53 0.76 0.35-0.05 0.66 0.17 0.66 0.54 0 0.36-0.27 0.56-0.56 0.56-0.24 0-0.56-0.14-0.56-0.57 0-0.91 0.93-1.56 2.02-1.56 1.22 0 2.12 0.9 2.12 1.91 0 0.82-0.62 1.59-1.66 1.8z"/><path d="m272.09 682.79v0.01h0.01v0.01l0.02 0.02v0.01h0.01v0.02h0.01v0.01l0.01 0.01v0.02h0.01v0.03h0.01v0.04c0 0.1-0.09 0.19-0.2 0.19-0.1 0-0.15-0.07-0.21-0.14-0.5-0.67-1.17-0.94-1.9-0.94-0.69 0-1.31 0.25-1.86 0.77-0.19 0.16-0.4 0.31-0.72 0.31l0.01-0.21c0.53 0 1.04-0.74 1.04-1.78 0-1.08-0.52-1.79-1.04-1.79-0.2 0-0.86 0.13-0.86 1.78 0 1.66 0.65 1.79 0.86 1.79l-0.01 0.21c-0.76 0-1.46-0.83-1.46-1.99 0-1.2 0.71-2.01 1.46-2.01 0.73 0 1.3 0.89 1.3 2 0 0.14 0 0.6-0.21 1.12 0.65-0.37 1.15-0.42 1.5-0.42 0.76 0 1.27 0.31 1.33 0.35l0.01-0.01-4.48-6.64c-0.09-0.13-0.09-0.18-0.09-0.2 0-0.11 0.1-0.2 0.2-0.2s0.12 0.04 0.2 0.15z"/><path d="m273 677.15-0.01 0.11v0.11l-0.02 0.2-0.02 0.1-0.01 0.09-0.02 0.1-0.03 0.09-0.02 0.09-0.03 0.09-0.03 0.08-0.04 0.08-0.03 0.08-0.04 0.08-0.04 0.07-0.05 0.07-0.04 0.06-0.15 0.18-0.05 0.05-0.06 0.05-0.05 0.04-0.06 0.04-0.18 0.09-0.06 0.02-0.07 0.02-0.12 0.02h-0.07l0.01-0.22c0.53 0 1.04-0.73 1.04-1.78 0-1.07-0.52-1.78-1.04-1.78-0.2 0-0.86 0.12-0.86 1.77 0 1.66 0.65 1.79 0.86 1.79l-0.01 0.22c-0.76 0-1.47-0.83-1.47-2 0-1.2 0.72-2 1.47-2 0.73 0 1.3 0.88 1.3 1.99z"/><path d="m167.45 648.23-0.01 0.15v0.3l-0.01 0.15-0.01 0.14-0.02 0.3-0.02 0.15-0.03 0.14-0.02 0.15-0.04 0.14-0.03 0.14-0.05 0.15-0.1 0.28-0.06 0.14c-0.46 0.95-1.28 1.11-1.7 1.11-0.6 0-1.33-0.26-1.73-1.18-0.32-0.69-0.37-1.46-0.37-2.26 0-0.74 0.04-1.64 0.45-2.39 0.42-0.81 1.15-1.01 1.64-1.01v0.22c-0.39 0-0.98 0.25-1.16 1.2-0.1 0.6-0.1 1.51-0.1 2.1 0 0.64 0 1.29 0.07 1.83 0.19 1.18 0.94 1.27 1.19 1.27 0.33 0 0.99-0.18 1.18-1.16 0.1-0.56 0.1-1.31 0.1-1.94 0-0.75 0-1.42-0.11-2.06-0.15-0.94-0.72-1.24-1.17-1.24v-0.22c0.54 0 1.3 0.21 1.74 1.16 0.31 0.68 0.37 1.46 0.37 2.24z"/><path d="m169.75 645.58v0.08l-0.01 0.02v0.03l-0.01 0.02-0.01 0.03-0.01 0.02-0.01 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.02 0.02-0.01 0.02-0.04 0.04-0.02 0.01-0.02 0.02-0.02 0.01-0.02 0.02-0.02 0.01-0.03 0.01-0.04 0.02-0.03 0.01-0.02 0.01h-0.03l-0.03 0.01h-0.07c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53c0.28 0 0.52 0.24 0.52 0.53z"/><path d="m175.35 651.1 0.03 0.03v0.01l0.01 0.01v0.01l0.01 0.01v0.01l0.02 0.02v0.02l0.01 0.01v0.03l0.01 0.01v0.18h-2.42c-1.22 0-1.24 0.13-1.28 0.32h-0.25l-0.33-2.05h0.25c0.03 0.16 0.12 0.79 0.25 0.91 0.07 0.06 0.85 0.06 0.98 0.06h2.06c-0.1-0.16-0.89-1.25-1.11-1.58-0.9-1.34-1.23-2.72-1.23-3.73 0-0.1 0-0.55 0.46-0.55s0.46 0.45 0.46 0.55v0.51c0 0.54 0.03 1.09 0.11 1.63 0.04 0.22 0.18 1.08 0.62 1.69z"/><path d="m182.42 652.13 0.01 0.01v0.01l0.02 0.02v0.01h0.01v0.01l0.01 0.01v0.01l0.01 0.01v0.02h0.01v0.03h0.01v0.03c0 0.11-0.09 0.2-0.2 0.2-0.1 0-0.15-0.07-0.21-0.14-0.5-0.67-1.17-0.94-1.9-0.94-0.69 0-1.31 0.25-1.86 0.77-0.19 0.16-0.4 0.31-0.72 0.31l0.01-0.22c0.53 0 1.04-0.73 1.04-1.77 0-1.08-0.52-1.79-1.04-1.79-0.2 0-0.86 0.13-0.86 1.78s0.65 1.78 0.86 1.78l-0.01 0.22c-0.76 0-1.46-0.83-1.46-1.99 0-1.21 0.71-2.01 1.46-2.01 0.73 0 1.3 0.88 1.3 2 0 0.14 0 0.6-0.21 1.12 0.65-0.37 1.15-0.42 1.5-0.42 0.76 0 1.27 0.31 1.33 0.35l0.01-0.01-4.48-6.65c-0.09-0.13-0.09-0.18-0.09-0.2 0-0.1 0.1-0.19 0.2-0.19s0.12 0.03 0.2 0.15z"/><path d="m183.33 646.49-0.01 0.11v0.1l-0.01 0.11-0.01 0.1-0.02 0.1-0.01 0.09-0.02 0.1-0.03 0.09-0.02 0.09-0.03 0.08-0.03 0.09-0.04 0.08-0.03 0.08-0.04 0.07-0.04 0.08-0.05 0.06-0.04 0.07-0.15 0.18-0.05 0.05-0.06 0.04-0.05 0.05-0.06 0.04-0.18 0.09-0.06 0.02-0.07 0.02-0.12 0.02h-0.07l0.01-0.22c0.53 0 1.04-0.74 1.04-1.78 0-1.07-0.52-1.79-1.04-1.79-0.2 0-0.86 0.13-0.86 1.78 0 1.66 0.65 1.79 0.86 1.79l-0.01 0.22c-0.76 0-1.47-0.84-1.47-2 0-1.2 0.72-2 1.47-2 0.73 0 1.3 0.88 1.3 1.99z"/></g>
  </g>
  <ellipse cx="16.305" cy="326.87" rx="41.718" ry="13.604" fill="#0065ff" fill-opacity=".19324" stroke-width="0"/>
  <text x="-15.478102" y="333.83505" fill="#000000" font-family="Verdana" font-size="7.0914px" letter-spacing="0px" stroke-width=".17729" word-spacing="0px" style="line-height:1.25" xml:space="preserve"><tspan x="-15.478102" y="333.83505" font-family="'Helvetica Neue LT'" font-size="7.0914px" stroke-width=".17729" style="font-feature-settings:normal;font-variant-caps:normal;font-variant-ligatures:normal;font-variant-numeric:normal">calculate_likelihoods</tspan></text>
  <g>
   <path d="m15.924 191.93-0.10335 119.53" fill="none" marker-end="url(#k)" stroke="#000" stroke-opacity=".57971" stroke-width=".9"/>
   <path d="m167.09 290.38c4.6174-1.5284-137.6 20.338-148.5 22.407" fill="none" marker-end="url(#j)" stroke="#000" stroke-opacity=".57971" stroke-width=".9"/>
   <ellipse cx="269.67" cy="110.45" rx="41.718" ry="13.604" fill="#0065ff" fill-opacity=".19324" stroke-width="0"/>
  </g>
  <text x="242.16074" y="100.73705" fill="#000000" font-family="'Helvetica Neue LT'" font-size="8.0044px" letter-spacing="0px" stroke-width=".20011" word-spacing="0px" style="line-height:1.25" xml:space="preserve"><tspan x="242.16074" y="108.03972" style="font-feature-settings:normal;font-variant-caps:normal;font-variant-ligatures:normal;font-variant-numeric:normal"/><tspan x="242.16074" y="118.04523" style="font-feature-settings:normal;font-variant-caps:normal;font-variant-ligatures:normal;font-variant-numeric:normal">calculate_priors</tspan></text>
  <g fill="none" stroke="#000">
   <path d="m208.09 73.49 61.369 22.796" marker-end="url(#p)" stroke-opacity=".57971" stroke-width=".9"/>
   <rect x="254.01" y="146.95" width="42.562" height="49.402" stroke-linecap="square" stroke-linejoin="bevel" stroke-opacity=".48235" stroke-width=".7"/>
   <path d="m254.02 159.85h42.507" stroke-opacity=".48235" stroke-width=".7"/>
  </g>
  <text x="257.62863" y="155.97679" fill="#000000" font-family="Verdana" font-size="7.0914px" letter-spacing="0px" stroke-width=".17729" word-spacing="0px" style="line-height:1.25" xml:space="preserve"><tspan x="257.62863" y="155.97679" font-family="'Helvetica Neue LT'" font-size="7.0914px" stroke-width=".17729" style="font-feature-settings:normal;font-variant-caps:normal;font-variant-ligatures:normal;font-variant-numeric:normal">tags priors</tspan></text>
  <g stroke="#000">
   <path d="m279.68 160.58v35.195" fill="none" stroke-opacity=".19608" stroke-width=".9"/>
   <g>
    <g transform="matrix(.52265 0 0 -.52265 108.6 528.89)" fill-opacity=".87923" fill-rule="evenodd" stroke-miterlimit="10.433" stroke-width="0" ns0:preamble="default_packages.tex" ns0:text="$$tag \\, \\, 1$$\n$$tag \\, \\, 2$$\n$$\\cdots$$\n$$tag \\, \\, N$$\n\n" xml:space="preserve"><path d="m294.69 698.23h1.1042c0.23494 0 0.35242 0 0.35242 0.23495 0 0.12923-0.11743 0.12923-0.32892 0.12923h-1.0337c0.42289 1.6563 0.48163 1.8913 0.48163 1.9618 0 0.19971-0.14103 0.31717-0.34066 0.31717-0.0353 0-0.36417-0.0117-0.46989-0.42289l-0.45813-1.856h-1.1042c-0.22319 0-0.35241 0-0.35241-0.22318 0-0.14104 0.10578-0.14104 0.32892-0.14104h1.0337c-0.84578-3.3127-0.89277-3.5124-0.89277-3.7238 0-0.62259 0.44638-1.069 1.0807-1.069 1.1982 0 1.8678 1.7033 1.8678 1.7973 0 0.11744-0.094 0.11744-0.1409 0.11744-0.10578 0-0.11743-0.0353-0.17621-0.16446-0.50511-1.2099-1.1277-1.4919-1.5271-1.4919-0.24668 0-0.36415 0.15271-0.36415 0.54037 0 0.27018 0.0235 0.35242 0.0705 0.55212z"/><path d="m300.87 697.97-0.0117 0.0469-0.0235 0.0353-0.0235 0.0471-0.0705 0.1059-0.0234 0.0353-0.0235 0.0353-0.0353 0.0353-0.0235 0.0353-0.1176 0.1176-0.0352 0.0235-0.0235 0.0235-0.0706 0.047-0.0353 0.0234-0.0353 0.0235-0.0469 0.0117-0.0353 0.0235-0.0353 0.0117-0.0471 0.0117-0.0353 0.0117-0.0471 0.0117-0.0469 0.0117-0.0353 0.0116h-0.0469l-0.0469 0.0117h-0.094c-1.3744 0-2.831-1.7151-2.831-3.4184 0-1.0925 0.64609-1.8678 1.5624-1.8678l0.0235 0.25843c-0.58735 0-0.7518 0.64608-0.7518 1.0925 0 0.58735 0.37589 2.0205 0.64608 2.5608 0.36415 0.68133 0.89277 1.116 1.3627 1.116 0.7518 0 0.91626-0.95152 0.91626-1.022 0-0.0705-0.0235-0.14103-0.0353-0.19969l-0.58735-2.2907c-0.0471-0.21144-0.0471-0.23494-0.23495-0.43464-0.50513-0.63434-0.98674-0.82229-1.3157-0.82229l-0.0234-0.25843c0.23494 0 0.82229 0.0469 1.5271 0.88102 0.0822-0.49337 0.49338-0.88102 1.0572-0.88102 0.41115 0 0.68132 0.27019 0.86928 0.64608 0.1997 0.4229 0.35241 1.1277 0.35241 1.1512 0 0.11743-0.10578 0.11743-0.1409 0.11743-0.11743 0-0.12923-0.0469-0.16445-0.21144-0.1997-0.76356-0.41114-1.4449-0.89277-1.4449-0.31718 0-0.35241 0.30543-0.35241 0.54036 0 0.25845 0.0235 0.34066 0.15271 0.85754 0.12924 0.49337 0.15271 0.61084 0.25843 1.0455l0.4229 1.6446c0.0822 0.31716 0.0822 0.34066 0.0822 0.38765 0 0.1997-0.1409 0.31717-0.34067 0.31717-0.28192 0-0.45813-0.25843-0.49337-0.51687z"/><path d="m308.21 697.96v0.0469l0.0117 0.0117v0.0352l0.0116 0.0116v0.0703l0.0117 0.0117v0.0234c0 0.1997-0.14104 0.31717-0.34067 0.31717-0.11743 0-0.43464-0.0822-0.48163-0.50512-0.21144 0.43464-0.62259 0.74006-1.0925 0.74006l0.0117-0.25843c0.77531 0 0.93976-0.93976 0.93976-0.99851 0-0.0587-0.0235-0.12924-0.0353-0.17619l-0.56386-2.232c-0.0705-0.30541-0.34067-0.59909-0.5991-0.81053-0.24669-0.21146-0.61085-0.4229-0.95152-0.4229-0.58735 0-0.7518 0.61084-0.7518 1.069 0 0.56385 0.34066 1.9382 0.64608 2.5374 0.31718 0.57559 0.8223 1.0337 1.3157 1.0337l-0.0116 0.25843c-1.3392 0-2.784-1.6328-2.784-3.3127 0-1.1512 0.71657-1.8443 1.5506-1.8443 0.69307 0 1.2452 0.55211 1.3627 0.68133l0.0117-0.0117c-0.24668-1.0455-0.38765-1.5154-0.38765-1.5506-0.0469-0.094-0.44638-1.2569-1.6798-1.2569-0.22319 0-0.61084 0.0116-0.93976 0.11743 0.35241 0.10578 0.48163 0.41116 0.48163 0.61086 0 0.18795-0.12924 0.41114-0.44639 0.41114-0.25843 0-0.63434-0.21145-0.63434-0.68133 0-0.48163 0.43465-0.71656 1.5624-0.71656 1.4566 0 2.3024 0.91626 2.4786 1.6093z"/><path d="m316.04 701.04v0.1753l-0.0116 0.0117v0.035l-0.0117 0.0117v0.0117l-0.0117 0.0117-0.0116 0.0117h-0.0117v0.0117h-0.0234l-0.0117 0.0117h-0.0817l-0.0117 0.0117h-0.0817c-0.72832-0.75182-1.7503-0.75182-2.1262-0.75182v-0.36415c0.23495 0 0.91627 0 1.5271 0.30542v-6.038c0-0.4229-0.0353-0.56386-1.0807-0.56386h-0.3759v-0.36416c0.41114 0.0353 1.4214 0.0353 1.8913 0.0353 0.46989 0 1.4919 0 1.903-0.0353v0.36416h-0.37589c-1.0572 0-1.0925 0.12923-1.0925 0.56386z"/><path d="m294.66 679.98h1.0534c0.22414 0 0.3362 0 0.3362 0.22412 0 0.12322-0.11204 0.12322-0.31378 0.12322h-0.98618c0.40343 1.5801 0.45947 1.8043 0.45947 1.8715 0 0.19051-0.13454 0.30256-0.32499 0.30256-0.0337 0-0.34741-0.0109-0.44826-0.40343l-0.43707-1.7706h-1.0534c-0.21292 0-0.33619 0-0.33619-0.21292 0-0.13453 0.10086-0.13453 0.31378-0.13453h0.98618c-0.80687-3.1603-0.8517-3.3508-0.8517-3.5525 0-0.59394 0.42585-1.0198 1.031-1.0198 1.1431 0 1.7818 1.6249 1.7818 1.7146 0 0.11204-0.0897 0.11204-0.13453 0.11204-0.10087 0-0.11204-0.0337-0.16811-0.15688-0.48188-1.1543-1.0758-1.4232-1.4569-1.4232-0.23534 0-0.34739 0.13453-0.34739 0.5043 0 0.26895 0.0223 0.34739 0.0672 0.5379z"/><path d="m300.56 679.73-0.0109 0.0448-0.0672 0.1011-0.0223 0.0448-0.0672 0.1011-0.0561 0.0561-0.0224 0.0337-0.0337 0.0224-0.0224 0.0337-0.0337 0.0224-0.0337 0.0337-0.0224 0.0223-0.0337 0.0224-0.0337 0.0225-0.0337 0.0223-0.0337 0.0109-0.0448 0.0225-0.0337 0.0109-0.0337 0.0223-0.0448 0.0109-0.0337 0.0109-0.0896 0.0218-0.0337 0.0109h-0.0896l-0.0448 0.0109h-0.0448c-1.3112 0-2.7008-1.6362-2.7008-3.2611 0-1.0534 0.61637-1.7818 1.4905-1.7818l0.0224 0.24654c-0.56032 0-0.71722 0.60515-0.71722 1.0422 0 0.56034 0.35861 1.9275 0.61636 2.443 0.34741 0.64999 0.8517 1.0646 1.3 1.0646 0.71723 0 0.87411-0.91894 0.87411-0.98618 0-0.0672-0.0224-0.12321-0.0337-0.1793l-0.56033-2.1853c-0.0448-0.20171-0.0448-0.22414-0.22414-0.41465-0.48188-0.60514-0.94135-0.78445-1.2551-0.78445l-0.0224-0.24654c0.22412 0 0.78445 0.0448 1.4568 0.82928 0.0784-0.45947 0.47069-0.82928 1.0086-0.82928 0.39224 0 0.64999 0.25775 0.8293 0.60514 0.19051 0.40344 0.33619 1.087 0.33619 1.1094 0 0.11204-0.10087 0.11204-0.13453 0.11204-0.11204 0-0.12322-0.0448-0.15689-0.20171-0.19052-0.72843-0.39224-1.3784-0.8517-1.3784-0.30258 0-0.33621 0.28016-0.33621 0.5043 0 0.24654 0.0224 0.33619 0.1457 0.82928 0.12322 0.47068 0.14568 0.57153 0.24654 0.99738l0.40343 1.5577c0.0785 0.31378 0.0785 0.33619 0.0785 0.38102 0 0.19051-0.13453 0.30258-0.325 0.30258-0.26895 0-0.43705-0.24655-0.47067-0.49309z"/><path d="m307.56 679.72v0.0327l0.0109 0.0223v0.0334l0.0109 0.0109v0.0545l0.0109 0.0224v0.0218c0 0.19051-0.13453 0.30258-0.325 0.30258-0.11204 0-0.41464-0.0785-0.45947-0.48188-0.20171 0.41464-0.59395 0.70601-1.0422 0.70601l0.0109-0.24655c0.73963 0 0.89653-0.90772 0.89653-0.96376 0-0.0448-0.0225-0.11204-0.0337-0.15689l-0.53792-2.1292c-0.0672-0.29137-0.32499-0.57154-0.57153-0.78446-0.23534-0.20172-0.58274-0.39223-0.90774-0.39223-0.56033 0-0.71721 0.57153-0.71721 1.0198 0 0.53792 0.32499 1.8491 0.61636 2.4206 0.30258 0.53792 0.78447 0.98618 1.2551 0.98618l-0.0109 0.24655c-1.2776 0-2.656-1.5577-2.656-3.1603 0-1.1094 0.6836-1.7594 1.4793-1.7594 0.66119 0 1.1879 0.51549 1.3 0.63877l0.0109-0.0109c-0.23534-0.98618-0.36982-1.4456-0.36982-1.4681-0.0448-0.10087-0.42585-1.1991-1.6025-1.1991-0.21292 0-0.58274 0.0109-0.89652 0.11204 0.33619 0.10086 0.45946 0.39224 0.45946 0.57155 0 0.1793-0.12322 0.39222-0.42585 0.39222-0.24654 0-0.60516-0.20171-0.60516-0.63877 0-0.45948 0.41465-0.6836 1.4905-0.6836 1.3896 0 2.1965 0.86291 2.3646 1.5353z"/><path d="m314.06 675.39 1.1767 1.1543c1.7482 1.5353 2.4206 2.1293 2.4206 3.2499 0 1.2663-1.0086 2.1517-2.3758 2.1517-1.2551 0-2.0844-1.0198-2.0844-2.0172 0-0.61636 0.56033-0.61636 0.59394-0.61636 0.19051 0 0.58275 0.13453 0.58275 0.59395 0 0.28016-0.20171 0.57153-0.59395 0.57153-0.0897 0-0.11204 0-0.14568-0.0109 0.25775 0.72841 0.8517 1.1319 1.5017 1.1319 1.0198 0 1.5017-0.89654 1.5017-1.8043 0-0.89653-0.56032-1.7706-1.1767-2.4654l-2.1404-2.387c-0.12322-0.12322-0.12322-0.14568-0.12322-0.40343h4.1464l0.31378 1.9387h-0.28017c-0.056-0.33619-0.13453-0.82928-0.24654-0.99738-0.0784-0.0897-0.81808-0.0897-1.0646-0.0897z"/><path d="m300.9 661.79v0.02l-0.01 0.03v0.05l-0.01 0.03-0.01 0.02-0.01 0.03-0.02 0.04-0.01 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.06 0.06-0.02 0.01-0.02 0.02-0.02 0.01-0.02 0.02-0.04 0.02-0.03 0.01-0.02 0.01-0.03 0.01-0.02 0.01h-0.03l-0.02 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m305.33 661.79v0.05l-0.01 0.03v0.02l-0.01 0.03-0.01 0.02v0.03l-0.02 0.04-0.02 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.06 0.06-0.02 0.01-0.02 0.02-0.02 0.01-0.02 0.02-0.04 0.02-0.03 0.01-0.04 0.02-0.03 0.01h-0.03l-0.02 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m309.75 661.79v0.05l-0.01 0.03v0.02l-0.01 0.03v0.02l-0.01 0.03-0.02 0.04-0.02 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.09 0.09-0.02 0.01-0.03 0.02-0.06 0.03-0.03 0.01-0.02 0.01-0.03 0.01h-0.02l-0.03 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m294.49 649.33h1.0209c0.21721 0 0.3258 0 0.3258 0.20636 0 0.11942-0.1085 0.11942-0.30409 0.11942h-0.94486c0.38012 1.5313 0.43441 1.7486 0.43441 1.8137 0 0.18462-0.13035 0.29325-0.31494 0.29325-0.0327 0-0.32583-0.0116-0.42355-0.39099l-0.42357-1.716h-1.0209c-0.21722 0-0.32581 0-0.32581-0.19548 0-0.13035 0.0868-0.13035 0.3041-0.13035h0.95572c-0.78195-3.0735-0.82538-3.2582-0.82538-3.4428 0-0.58648 0.41268-0.9992 0.99916-0.9992 1.0969 0 1.716 1.5748 1.716 1.6617 0 0.10865-0.0868 0.10865-0.13035 0.10865-0.0977 0-0.10851-0.0327-0.16295-0.15209-0.467-1.1186-1.0318-1.3793-1.401-1.3793-0.22808 0-0.33667 0.14113-0.33667 0.49957 0 0.26067 0.0217 0.3367 0.0652 0.51046z"/><path d="m300.22 649.09-0.0217 0.0326-0.0217 0.0327-0.0217 0.0436-0.0116 0.0326-0.0434 0.0654-0.0327 0.0326-0.0651 0.0981-0.0327 0.0217-0.0217 0.0327-0.0327 0.0217-0.0217 0.0327-0.1962 0.1302-0.0327 0.0116-0.0327 0.0217-0.0436 0.0116-0.0327 0.0217-0.0436 0.0116-0.0327 0.0116h-0.0436l-0.0872 0.0232h-0.1308c-1.2707 0-2.6066-1.5856-2.6066-3.1604 0-1.01 0.59733-1.7268 1.4336-1.7268l0.0217 0.23895c-0.54303 0-0.68422 0.59731-0.68422 1.01 0 0.54303 0.33667 1.868 0.58647 2.3676 0.33668 0.64078 0.82539 1.0318 1.2598 1.0318 0.70595 0 0.85798-0.87972 0.85798-0.94488 0-0.0651-0.0217-0.13035-0.0327-0.18463l-0.54303-2.1178c-0.0543-0.18462-0.0543-0.2172-0.2172-0.39097-0.47786-0.59733-0.92315-0.77109-1.2272-0.77109l-0.0217-0.23895c0.21721 0 0.76025 0.0436 1.4119 0.81454 0.0868-0.45614 0.46701-0.81454 0.9883-0.81454 0.38012 0 0.62994 0.24981 0.80369 0.59733 0.17378 0.391 0.31496 1.0426 0.31496 1.0643 0 0.10865-0.0977 0.10865-0.11943 0.10865-0.11943 0-0.11943-0.0436-0.16296-0.19549-0.17377-0.69508-0.36926-1.3358-0.81454-1.3358-0.29323 0-0.32583 0.28237-0.32583 0.49956 0 0.23894 0.0217 0.32585 0.14114 0.79284 0.11943 0.45614 0.14112 0.56474 0.23892 0.97744l0.391 1.5096c0.0759 0.3041 0.0759 0.32581 0.0759 0.36926 0 0.18462-0.13035 0.28236-0.31497 0.28236-0.26064 0-0.42354-0.22806-0.45614-0.467z"/><path d="m306.99 649.08v0.0116l0.0116 0.0116v0.0232l0.0116 0.0217v0.0464l0.0116 0.0217v0.0696c0 0.18462-0.13035 0.28236-0.31496 0.28236-0.10865 0-0.40185-0.076-0.44528-0.45614-0.19548 0.40185-0.56476 0.67336-0.99917 0.67336l0.0116-0.23893c0.70593 0 0.85797-0.86884 0.85797-0.92315 0-0.0543-0.0217-0.11943-0.0327-0.16295l-0.51044-2.0635c-0.0651-0.27151-0.31497-0.54303-0.55391-0.74938-0.22807-0.19548-0.56473-0.39099-0.87969-0.39099-0.54303 0-0.70595 0.56476-0.70595 0.99918 0 0.51044 0.31498 1.7811 0.60819 2.335 0.29325 0.53219 0.76026 0.95574 1.2164 0.95574l-0.0116 0.23892c-1.2381 0-2.5848-1.5096-2.5848-3.0627 0-1.0644 0.66249-1.7051 1.4444-1.7051 0.64079 0 1.1512 0.51045 1.2598 0.62991l0.0116-0.0116c-0.22807-0.9666-0.3584-1.401-0.3584-1.4227-0.0436-0.0977-0.41269-1.1729-1.5639-1.1729-0.20635 0-0.56476 0.0116-0.86885 0.10866 0.32581 0.0977 0.44528 0.3801 0.44528 0.56476 0 0.17375-0.11943 0.3801-0.41269 0.3801-0.23894 0-0.58647-0.19549-0.58647-0.62992 0-0.44528 0.40184-0.65162 1.4444-0.65162 1.3576 0 2.1287 0.83625 2.3024 1.4879z"/><path d="m319.23 651.24 0.0217 0.0868 0.0116 0.0327 0.0116 0.0436 0.0217 0.0327 0.0116 0.0436 0.0116 0.0327 0.0651 0.0981 0.0217 0.0326 0.0651 0.0981 0.0327 0.0217 0.0327 0.0327 0.0327 0.0217 0.0327 0.0327 0.0436 0.0217 0.0327 0.0217 0.0436 0.0217 0.0543 0.0217 0.0436 0.0116 0.0543 0.0217 0.1086 0.0232 0.0651 0.0217 0.0652 0.0116h0.0652l0.076 0.0116 0.0759 0.0116h0.0763c0.0543 0 0.18463 0.0116 0.18463 0.21721 0 0.0116 0 0.11943-0.14114 0.11943-0.35839 0-0.73849-0.0327-1.0969-0.0327-0.36925 0-0.74938 0.0327-1.1078 0.0327-0.0543 0-0.19551 0-0.19551-0.21722 0-0.11943 0.10851-0.11943 0.19551-0.11943 0.61906-0.0116 0.73853-0.23894 0.73853-0.47788 0-0.0327-0.0217-0.19547-0.0327-0.22806l-1.2055-4.8004-2.4002 5.6366c-0.0868 0.1955-0.0977 0.20637-0.34754 0.20637h-1.4444c-0.21721 0-0.31496 0-0.31496-0.21722 0-0.11943 0.0977-0.11943 0.3041-0.11943 0.0541 0 0.73851 0 0.73851-0.0977l-1.4553-5.7995c-0.10864-0.42357-0.29323-0.77111-1.173-0.80369-0.0651 0-0.18462-0.0116-0.18462-0.2172 0-0.0759 0.0543-0.11943 0.14128-0.11943 0.34753 0 0.72766 0.0327 1.086 0.0327 0.36925 0 0.76023-0.0327 1.1186-0.0327 0.0543 0 0.19548 0 0.19548 0.21722 0 0.10851-0.0977 0.11943-0.21719 0.11943-0.62992 0.0217-0.71682 0.26066-0.71682 0.47788 0 0.0759 0.0116 0.13035 0.0436 0.23891l1.4227 5.7018c0.0436-0.0652 0.0436-0.0868 0.0977-0.1955l2.7043-6.3534c0.076-0.18466 0.10866-0.20637 0.20637-0.20637 0.11943 0 0.11943 0.0327 0.17376 0.22809z"/></g>
    <g transform="matrix(.49906 0 0 -.49906 135.45 513.4)" fill-rule="evenodd" stroke-miterlimit="10.433" stroke-width="0" ns0:preamble="default_packages.tex" ns0:text="$$0.2$$\n$$0.1$$\n$$\\cdots$$\n$$0.3$$\n" xml:space="preserve"><path d="m304.11 699.3v0.18532l-0.0123 0.18533v0.18532l-0.0124 0.17297-0.0124 0.18533-0.0124 0.18532-0.0123 0.18533-0.0247 0.17297-0.0371 0.18532-0.0247 0.17297-0.0494 0.18533-0.0371 0.17297-0.1854 0.51891-0.0742 0.173c-0.56833 1.1861-1.5814 1.3838-2.1004 1.3838-0.7413 0-1.6309-0.32123-2.1374-1.4702-0.39536-0.84014-0.45714-1.8038-0.45714-2.7799 0-0.92663 0.0494-2.0262 0.55598-2.9528 0.51891-1.0008 1.4208-1.2478 2.0262-1.2478v0.27181c-0.48185 0-1.1984 0.30887-1.4332 1.4826-0.12355 0.7413-0.12355 1.8656-0.12355 2.5946 0 0.77836 0 1.5938 0.0865 2.261 0.23475 1.4579 1.1614 1.5691 1.4702 1.5691 0.40771 0 1.2231-0.22239 1.4579-1.4332 0.12355-0.69188 0.12355-1.6309 0.12355-2.3969 0-0.92663 0-1.7544-0.13591-2.5451-0.18532-1.1614-0.88956-1.532-1.4455-1.532v-0.27181c0.66717 0 1.6062 0.25945 2.1498 1.4208 0.39536 0.8525 0.45713 1.8038 0.45713 2.7799z"/><path d="m306.95 696.02v0.0989l-0.0123 0.0247v0.0371l-0.0124 0.0247-0.0124 0.0371-0.0124 0.0247-0.0123 0.0247-0.0124 0.0371-0.0124 0.0247-0.0247 0.0247-0.0124 0.0247-0.0247 0.0247-0.0123 0.0247-0.0494 0.0494-0.0247 0.0123-0.0247 0.0247-0.0247 0.0124-0.0247 0.0247-0.0247 0.0123-0.0371 0.0124-0.0247 0.0123-0.0247 0.0124-0.0371 0.0123-0.0247 0.0124h-0.0371l-0.0247 0.0124h-0.0989c-0.35829 0-0.65481-0.29652-0.65481-0.64246 0-0.35829 0.29652-0.65481 0.65481-0.65481 0.3583 0 0.64246 0.29652 0.64246 0.65481z"/><path d="m309.57 696.31 1.3096 1.2726c1.9274 1.6926 2.6687 2.3474 2.6687 3.583 0 1.3961-1.112 2.3845-2.6193 2.3845-1.3961 0-2.298-1.1367-2.298-2.2239 0-0.69188 0.61775-0.69188 0.65482-0.69188 0.19768 0 0.6301 0.14826 0.6301 0.65481 0 0.32123-0.22239 0.63011-0.65481 0.63011h-0.16062c0.29652 0.79072 0.95134 1.2478 1.6679 1.2478 1.1243 0 1.6556-1.0008 1.6556-2.0015 0-0.9884-0.61775-1.9521-1.2973-2.7181l-2.3598-2.6193c-0.13591-0.1359-0.13591-0.16061-0.13591-0.45713h4.5714l0.34594 2.1374h-0.30887c-0.0618-0.37065-0.14826-0.91427-0.27181-1.0996-0.0865-0.0988-0.90192-0.0988-1.1737-0.0988z"/><path d="m305.02 679.61v0.19187l-0.0128 0.17909v0.19187l-0.0128 0.19188-0.0128 0.19187-0.0128 0.19188-0.0128 0.17908-0.0256 0.19188-0.0384 0.19187-0.0256 0.17909-0.0512 0.19187-0.0384 0.17909-0.064 0.17908-0.064 0.17909-0.064 0.17908-0.0768 0.179c-0.58842 1.2152-1.6373 1.4199-2.1746 1.4199-0.7675 0-1.6885-0.33259-2.213-1.5094-0.40933-0.88263-0.47329-1.8676-0.47329-2.8781 0-0.95938 0.0512-2.0978 0.57563-3.07 0.53725-1.0233 1.471-1.2792 2.0978-1.2792v0.28142c-0.49888 0-1.2408 0.307-1.4838 1.535-0.12792 0.75471-0.12792 1.9316-0.12792 2.6735 0 0.81867 0 1.6629 0.0895 2.3409 0.24305 1.5094 1.2024 1.6246 1.5222 1.6246 0.42213 0 1.2664-0.21746 1.5094-1.4838 0.12792-0.70355 0.12792-1.6757 0.12792-2.4816 0-0.94659 0-1.8164-0.14071-2.6223-0.19188-1.2152-0.921-1.5862-1.4966-1.5862v-0.28142c0.69075 0 1.6629 0.26863 2.2258 1.471 0.40933 0.88263 0.47329 1.8676 0.47329 2.8781z"/><path d="m307.96 676.21v0.1024l-0.0128 0.0384v0.0256l-0.0128 0.0384-0.0128 0.0256-0.0128 0.0384-0.0256 0.0512-0.0128 0.0384-0.0256 0.0256-0.0128 0.0256-0.0256 0.0256-0.0128 0.0256-0.0256 0.0128-0.0512 0.0512-0.0256 0.0128-0.0256 0.0256-0.0256 0.0128-0.0256 0.0256-0.0384 0.0128-0.0512 0.0256-0.0384 0.0128h-0.0256l-0.0384 0.0128h-0.0256l-0.0384 0.0128h-0.064c-0.37096 0-0.67796-0.307-0.67796-0.67796s0.307-0.66517 0.67796-0.66517 0.66517 0.29421 0.66517 0.66517z"/><path d="m312.81 683.66v0.192l-0.0128 0.0128v0.0384l-0.0128 0.0128v0.0256h-0.0128v0.0128h-0.0128l-0.0128 0.0128h-0.0128l-0.0128 0.0128h-0.0256v0.0128h-0.1792c-0.79309-0.80588-1.9188-0.80588-2.3281-0.80588v-0.39655c0.26862 0 1.0105 0 1.6757 0.33259v-6.5877c0-0.4605-0.0384-0.614-1.1896-0.614h-0.40933v-0.38375c0.44771 0.0384 1.5606 0.0384 2.0722 0.0384 0.51167 0 1.6246 0 2.0723-0.0384v0.38375h-0.40933c-1.1513 0-1.1896 0.14071-1.1896 0.614z"/><path d="m302.95 661.64v0.02l-0.01 0.03v0.05l-0.01 0.03-0.01 0.02-0.01 0.03-0.02 0.04-0.01 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.06 0.06-0.02 0.01-0.02 0.02-0.02 0.01-0.02 0.02-0.04 0.02-0.03 0.01-0.02 0.01-0.03 0.01-0.02 0.01h-0.03l-0.02 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m307.38 661.64v0.05l-0.01 0.03v0.02l-0.01 0.03-0.01 0.02v0.03l-0.02 0.04-0.02 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.06 0.06-0.02 0.01-0.02 0.02-0.02 0.01-0.02 0.02-0.04 0.02-0.03 0.01-0.04 0.02-0.03 0.01h-0.03l-0.02 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m311.8 661.64v0.05l-0.01 0.03v0.02l-0.01 0.03v0.02l-0.01 0.03-0.02 0.04-0.02 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.09 0.09-0.02 0.01-0.03 0.02-0.06 0.03-0.03 0.01-0.02 0.01-0.03 0.01h-0.02l-0.03 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m306.26 645.73v0.19422l-0.0129 0.19423v0.19423l-0.013 0.19422-0.0129 0.18128-0.0129 0.19423-0.0129 0.19422-0.0259 0.19423-0.0389 0.18128-0.0259 0.19422-0.0518 0.18128-0.0388 0.18128-0.0647 0.19423-0.0648 0.18127-0.0647 0.18128-0.0776 0.1812c-0.59563 1.2301-1.6574 1.4373-2.2012 1.4373-0.77691 0-1.7092-0.33666-2.2401-1.5279-0.41435-0.89344-0.47909-1.8905-0.47909-2.9263 0-0.95818 0.0518-2.1235 0.58268-3.0947 0.54383-1.0488 1.4891-1.3078 2.1235-1.3078v0.28487c-0.50499 0-1.256 0.32371-1.502 1.5538-0.12948 0.77691-0.12948 1.9552-0.12948 2.7192 0 0.8287 0 1.6704 0.0906 2.3696 0.24602 1.5279 1.2172 1.6444 1.5409 1.6444 0.4273 0 1.2819-0.23307 1.5279-1.502 0.12949-0.72511 0.12949-1.6962 0.12949-2.512 0-0.97113 0-1.8387-0.14243-2.6674-0.19423-1.2172-0.93229-1.6056-1.515-1.6056v-0.28487c0.69922 0 1.6833 0.27192 2.253 1.502 0.41435 0.88049 0.47909 1.8905 0.47909 2.9004z"/><path d="m309.24 642.3v0.1035l-0.0129 0.0259v0.0389l-0.013 0.0259-0.0129 0.0389-0.0129 0.0259-0.0129 0.0389-0.0258 0.0518-0.0259 0.0259-0.013 0.0259-0.0259 0.0259-0.0129 0.0259-0.0518 0.0518-0.0259 0.013-0.0259 0.0259-0.0259 0.013-0.0259 0.0259-0.0259 0.0129-0.0388 0.013-0.0259 0.0129-0.0259 0.013-0.0389 0.013-0.0259 0.0129h-0.0388l-0.0259 0.0129h-0.1035c-0.37551 0-0.68627-0.29781-0.68627-0.67331 0-0.37551 0.31076-0.68627 0.68627-0.68627 0.3755 0 0.67331 0.31076 0.67331 0.68627z"/><path d="m314.09 646.15 0.10359 0.0388 0.0906 0.0259 0.0906 0.0518 0.0906 0.0389 0.0906 0.0388 0.1812 0.1036 0.0777 0.0518 0.0906 0.0647 0.0777 0.0647 0.0648 0.0518 0.0777 0.0648 0.0777 0.0647 0.0647 0.0777 0.0647 0.0647 0.0518 0.0777 0.0648 0.0777 0.1554 0.2331 0.0388 0.0777 0.0518 0.0906 0.0388 0.0777 0.0259 0.0906 0.0388 0.0906 0.0259 0.0906 0.0129 0.0906 0.0259 0.0906 0.0129 0.0906 0.013 0.0906v0.0906l0.0129 0.10359c0 1.0488-1.1395 1.7739-2.3825 1.7739-1.3078 0-2.2789-0.77691-2.2789-1.748 0-0.4273 0.28487-0.67332 0.64742-0.67332 0.4014 0 0.66037 0.28487 0.66037 0.66037 0 0.63447-0.60858 0.63447-0.78985 0.63447 0.38845 0.63448 1.243 0.8028 1.7092 0.8028 0.53088 0 1.243-0.28486 1.243-1.4373 0-0.15538-0.0259-0.90639-0.36256-1.4761-0.38845-0.60858-0.8287-0.64742-1.1524-0.66037-0.10359-0.013-0.41435-0.0388-0.50499-0.0388-0.10358-0.0129-0.19422-0.0259-0.19422-0.15539 0-0.14243 0.0906-0.14243 0.31076-0.14243h0.56973c1.0618 0 1.5409-0.88049 1.5409-2.1365 0-1.748-0.89344-2.1235-1.4632-2.1235-0.55678 0-1.5279 0.22012-1.9682 0.98408 0.44025-0.0647 0.84165 0.20718 0.84165 0.69922 0 0.46614-0.34961 0.72511-0.72511 0.72511-0.29782 0-0.71217-0.18128-0.71217-0.75101 0-1.1654 1.1913-2.02 2.6026-2.02 1.5797 0 2.758 1.1783 2.758 2.4861 0 1.0618-0.81575 2.0588-2.1624 2.3307z"/></g>
    <rect x="-4.8293" y="365.43" width="48.225" height="49.391" fill="none" stroke-linecap="square" stroke-linejoin="bevel" stroke-opacity=".48235" stroke-width=".7"/>
   </g>
   <path d="m-4.8208 378.37h48.233" fill="none" stroke-opacity=".48235" stroke-width=".7"/>
  </g>
  <text x="-2.9689298" y="374.34888" fill="#000000" font-family="Verdana" font-size="7.0914px" letter-spacing="0px" stroke-width=".17729" word-spacing="0px" style="line-height:1.25" xml:space="preserve"><tspan x="-2.9689298" y="374.34888" font-family="'Helvetica Neue LT'" font-size="7.0914px" stroke-width=".17729" style="font-feature-settings:normal;font-variant-caps:normal;font-variant-ligatures:normal;font-variant-numeric:normal">tags likelihood</tspan></text>
  <path d="m20.842 379.6v33.73" fill="none" stroke="#000" stroke-opacity=".19608" stroke-width=".9"/>
  <g transform="matrix(.52265 0 0 -.52265 -150.24 747.36)" fill-opacity=".7451" fill-rule="evenodd" stroke="#000" stroke-miterlimit="10.433" stroke-width="0" ns0:preamble="default_packages.tex" ns0:text="$$tag \\, \\, 1$$\n$$tag \\, \\, 2$$\n$$\\cdots$$\n$$tag \\, \\, N$$\n\n" xml:space="preserve"><path d="m294.19 698.68h1.1689c0.24869 0 0.37304 0 0.37304 0.2487 0 0.13678-0.12435 0.13678-0.34817 0.13678h-1.0943c0.44765 1.7533 0.50983 2.002 0.50983 2.0766 0 0.21138-0.14923 0.33574-0.36061 0.33574-0.0372 0-0.38548-0.0124-0.49739-0.44765l-0.48496-1.9647h-1.1689c-0.23626 0-0.37304 0-0.37304-0.23626 0-0.14922 0.11191-0.14922 0.34818-0.14922h1.0942c-0.8953-3.5066-0.94504-3.718-0.94504-3.9418 0-0.65904 0.47252-1.1316 1.144-1.1316 1.2684 0 1.9771 1.803 1.9771 1.9025 0 0.12434-0.0995 0.12434-0.14921 0.12434-0.11191 0-0.12435-0.0373-0.18653-0.17408-0.53469-1.2808-1.1937-1.5792-1.6165-1.5792-0.26113 0-0.38547 0.16165-0.38547 0.572 0 0.28599 0.0249 0.37304 0.0746 0.58443z"/><path d="m300.73 698.41-0.0125 0.0497-0.0249 0.0374-0.0249 0.0497-0.0249 0.0372-0.0249 0.0374-0.0249 0.0372-0.0498 0.0748-0.0372 0.0373-0.0249 0.0374-0.0249 0.0249-0.0372 0.0374-0.0621 0.0621-0.0373 0.0249-0.0249 0.0249-0.0374 0.0249-0.0373 0.0249-0.0372 0.0249-0.0373 0.0249-0.0497 0.0125-0.0372 0.0249-0.0373 0.0125-0.0497 0.0124-0.0372 0.0125-0.0994 0.025-0.0373 0.0125h-0.0497l-0.0497 0.0125h-0.0994c-1.4549 0-2.9968-1.8155-2.9968-3.6185 0-1.1564 0.68392-1.9771 1.6538-1.9771l0.0249 0.27357c-0.62175 0-0.79583 0.68391-0.79583 1.1564 0 0.62173 0.39791 2.1388 0.68391 2.7108 0.38547 0.72122 0.94503 1.1813 1.4424 1.1813 0.79583 0 0.96991-1.0072 0.96991-1.0818s-0.0249-0.14923-0.0372-0.2114l-0.62173-2.4248c-0.0497-0.22382-0.0497-0.24869-0.2487-0.46009-0.53469-0.67147-1.0445-0.87042-1.3927-0.87042l-0.0249-0.27357c0.24869 0 0.87042 0.0497 1.6165 0.93261 0.087-0.52226 0.52226-0.93261 1.1191-0.93261 0.43522 0 0.72121 0.28601 0.92018 0.68392 0.21138 0.44764 0.37303 1.1937 0.37303 1.2186 0 0.12434-0.11191 0.12434-0.14921 0.12434-0.12435 0-0.13678-0.0497-0.17409-0.22382-0.21139-0.80826-0.43521-1.5295-0.94504-1.5295-0.33574 0-0.37304 0.3233-0.37304 0.572 0 0.27356 0.0249 0.36061 0.16166 0.90773 0.13678 0.52226 0.16165 0.64661 0.27356 1.1067l0.44765 1.7409c0.0871 0.33574 0.0871 0.36061 0.0871 0.41034 0 0.21139-0.14921 0.33574-0.36061 0.33574-0.29842 0-0.48495-0.27356-0.52225-0.54712z"/><path d="m308.5 698.4v0.0498l0.0125 0.0125v0.0374l0.0124 0.0124v0.0749l0.0125 0.0125v0.025c0 0.21139-0.14922 0.33575-0.36061 0.33575-0.12435 0-0.46009-0.0871-0.50983-0.5347-0.22382 0.46009-0.65903 0.78338-1.1564 0.78338l0.0124-0.27355c0.82068 0 0.99477-0.99478 0.99477-1.057 0-0.0622-0.0249-0.13679-0.0374-0.18653l-0.59687-2.3626c-0.0746-0.3233-0.36061-0.63418-0.63418-0.858-0.26112-0.22383-0.6466-0.44765-1.0072-0.44765-0.62173 0-0.79581 0.6466-0.79581 1.1316 0 0.59686 0.36059 2.0517 0.6839 2.6859 0.33574 0.60931 0.87043 1.0943 1.3927 1.0943l-0.0124 0.27355c-1.4176 0-2.947-1.7284-2.947-3.5066 0-1.2186 0.75852-1.9522 1.6414-1.9522 0.73366 0 1.3181 0.58444 1.4424 0.72122l0.0125-0.0125c-0.26112-1.1067-0.41035-1.6041-0.41035-1.6414-0.0497-0.0995-0.47252-1.3305-1.7782-1.3305-0.23627 0-0.64661 0.0125-0.99478 0.12434 0.37304 0.11191 0.50982 0.43522 0.50982 0.6466 0 0.19896-0.13678 0.43522-0.47252 0.43522-0.27357 0-0.67147-0.22382-0.67147-0.72121 0-0.50982 0.46008-0.75852 1.6538-0.75852 1.5419 0 2.4372 0.96991 2.6237 1.7036z"/><path d="m316.78 701.65v0.1873l-0.0125 0.0125v0.0374l-0.0124 0.0125v0.0125l-0.025 0.025h-0.0124v0.0125h-0.025l-0.0124 0.0124h-0.0873l-0.0125 0.0125h-0.0873c-0.77096-0.79582-1.8528-0.79582-2.2507-0.79582v-0.38548c0.2487 0 0.96991 0 1.6165 0.3233v-6.3914c0-0.44765-0.0373-0.59687-1.144-0.59687h-0.39791v-0.38547c0.43522 0.0373 1.5046 0.0373 2.002 0.0373s1.5792 0 2.0144-0.0373v0.38547h-0.39791c-1.1191 0-1.1564 0.13678-1.1564 0.59687z"/><path d="m294.03 680.45h1.1596c0.24671 0 0.37007 0 0.37007 0.24671 0 0.13569-0.12336 0.13569-0.3454 0.13569h-1.0856c0.44409 1.7394 0.50577 1.9861 0.50577 2.0601 0 0.20972-0.14804 0.33308-0.35774 0.33308-0.037 0-0.38241-0.0123-0.49344-0.4441l-0.4811-1.9491h-1.1596c-0.23439 0-0.37008 0-0.37008-0.23438 0-0.14802 0.11103-0.14802 0.34541-0.14802h1.0856c-0.88818-3.4787-0.93754-3.6884-0.93754-3.9105 0-0.6538 0.46877-1.1226 1.1349-1.1226 1.2582 0 1.9614 1.7887 1.9614 1.8874 0 0.12337-0.0987 0.12337-0.14803 0.12337-0.11102 0-0.12337-0.037-0.18504-0.17271-0.53044-1.2706-1.1842-1.5667-1.6037-1.5667-0.25905 0-0.38241 0.14803-0.38241 0.55512 0 0.29606 0.0247 0.38242 0.074 0.59212z"/><path d="m300.52 680.18-0.0123 0.0493-0.0741 0.111-0.0247 0.0493-0.0741 0.111-0.0617 0.0617-0.0247 0.037-0.037 0.0247-0.0247 0.037-0.037 0.0247-0.0617 0.0617-0.111 0.0741-0.037 0.0123-0.0493 0.0247-0.037 0.0123-0.037 0.0247-0.0493 0.0123-0.037 0.0123-0.0986 0.0246-0.037 0.0123h-0.0986l-0.0493 0.0123h-0.0493c-1.4433 0-2.973-1.801-2.973-3.5898 0-1.1596 0.67848-1.9614 1.6407-1.9614l0.0247 0.27139c-0.61679 0-0.78949 0.66614-0.78949 1.1472 0 0.6168 0.39474 2.1218 0.67848 2.6892 0.38241 0.71548 0.93752 1.1719 1.431 1.1719 0.7895 0 0.9622-1.0115 0.9622-1.0856 0-0.074-0.0247-0.1357-0.037-0.19737l-0.61679-2.4055c-0.0493-0.22205-0.0493-0.24671-0.24672-0.45642-0.53045-0.66614-1.0362-0.86352-1.3816-0.86352l-0.0247-0.27139c0.24672 0 0.86352 0.0493 1.6037 0.91286 0.0863-0.50577 0.51811-0.91286 1.1102-0.91286 0.43175 0 0.71549 0.28372 0.91285 0.66614 0.20972 0.44409 0.37009 1.1966 0.37009 1.2212 0 0.12336-0.11103 0.12336-0.14804 0.12336-0.12335 0-0.13569-0.0493-0.17269-0.22204-0.20971-0.80183-0.43176-1.5173-0.93754-1.5173-0.33307 0-0.37008 0.3084-0.37008 0.55512 0 0.27139 0.0247 0.37007 0.16037 0.91286 0.1357 0.51811 0.16037 0.62912 0.27139 1.0979l0.4441 1.7147c0.0863 0.3454 0.0863 0.37007 0.0863 0.41941 0 0.20971-0.14804 0.33307-0.35775 0.33307-0.29605 0-0.4811-0.27138-0.51811-0.54278z"/><path d="m308.23 680.17v0.0369l0.0123 0.0247v0.037l0.0123 0.0123v0.0615l0.0123 0.0247v0.0246c0 0.20972-0.14804 0.33308-0.35774 0.33308-0.12336 0-0.45643-0.0863-0.50577-0.53045-0.22205 0.45643-0.65381 0.77717-1.1472 0.77717l0.0123-0.2714c0.81417 0 0.98687-0.99921 0.98687-1.0609 0-0.0493-0.0247-0.12336-0.037-0.17271l-0.59213-2.3438c-0.074-0.32073-0.35774-0.62914-0.62913-0.86351-0.25905-0.22205-0.64147-0.43176-0.99921-0.43176-0.6168 0-0.7895 0.62913-0.7895 1.1226 0 0.59212 0.35775 2.0354 0.67848 2.6646 0.33307 0.59213 0.86351 1.0856 1.3816 1.0856l-0.0123 0.2714c-1.4063 0-2.9236-1.7147-2.9236-3.4787 0-1.2213 0.75249-1.9367 1.6284-1.9367 0.72781 0 1.3076 0.56745 1.431 0.70315l0.0123-0.0123c-0.25905-1.0856-0.40708-1.5913-0.40708-1.616-0.0493-0.11102-0.46876-1.3199-1.764-1.3199-0.23438 0-0.64147 0.0123-0.98688 0.12336 0.37008 0.11102 0.50578 0.43175 0.50578 0.62913 0 0.19737-0.1357 0.43175-0.46877 0.43175-0.27139 0-0.66614-0.22205-0.66614-0.70315 0-0.50576 0.45643-0.75248 1.6407-0.75248 1.5297 0 2.4178 0.94986 2.6029 1.69z"/><path d="m313.98 675.79 1.2953 1.2706c1.9244 1.69 2.6646 2.3438 2.6646 3.5774 0 1.394-1.1102 2.3685-2.6152 2.3685-1.3816 0-2.2945-1.1226-2.2945-2.2205 0-0.67848 0.61679-0.67848 0.6538-0.67848 0.20972 0 0.64147 0.14802 0.64147 0.6538 0 0.3084-0.22204 0.62912-0.6538 0.62912-0.0987 0-0.12336 0-0.16037-0.0123 0.28373 0.80184 0.93753 1.2459 1.653 1.2459 1.1226 0 1.653-0.98688 1.653-1.9861 0-0.98687-0.61679-1.9491-1.2953-2.7139l-2.3562-2.6276c-0.13569-0.13569-0.13569-0.16037-0.13569-0.44409h4.5643l0.3454 2.1341h-0.3084c-0.0617-0.37007-0.14802-0.91285-0.27138-1.0979-0.0863-0.0987-0.90053-0.0987-1.1719-0.0987z"/><path d="m300.9 661.79v0.02l-0.01 0.03v0.05l-0.01 0.03-0.01 0.02-0.01 0.03-0.02 0.04-0.01 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.06 0.06-0.02 0.01-0.02 0.02-0.02 0.01-0.02 0.02-0.04 0.02-0.03 0.01-0.02 0.01-0.03 0.01-0.02 0.01h-0.03l-0.02 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m305.33 661.79v0.05l-0.01 0.03v0.02l-0.01 0.03-0.01 0.02v0.03l-0.02 0.04-0.02 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.06 0.06-0.02 0.01-0.02 0.02-0.02 0.01-0.02 0.02-0.04 0.02-0.03 0.01-0.04 0.02-0.03 0.01h-0.03l-0.02 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m309.75 661.79v0.05l-0.01 0.03v0.02l-0.01 0.03v0.02l-0.01 0.03-0.02 0.04-0.02 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.09 0.09-0.02 0.01-0.03 0.02-0.06 0.03-0.03 0.01-0.02 0.01-0.03 0.01h-0.02l-0.03 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m291.81 647.31h1.1225c0.23882 0 0.35824 0 0.35824 0.22688 0 0.13135-0.11942 0.13135-0.33436 0.13135h-1.0389c0.41794 1.6837 0.47764 1.9225 0.47764 1.9942 0 0.20301-0.14329 0.32241-0.34629 0.32241-0.0358 0-0.35824-0.0119-0.46571-0.42988l-0.4657-1.8867h-1.1225c-0.23882 0-0.35824 0-0.35824-0.21494 0-0.14329 0.0956-0.14329 0.33436-0.14329h1.0508c-0.85976-3.3794-0.90752-3.5823-0.90752-3.7853 0-0.64482 0.45376-1.0986 1.0986-1.0986 1.2061 0 1.8867 1.7315 1.8867 1.827 0 0.11942-0.0956 0.11942-0.1433 0.11942-0.10745 0-0.1194-0.0358-0.17912-0.16717-0.51346-1.23-1.1344-1.5165-1.5404-1.5165-0.25076 0-0.37018 0.15523-0.37018 0.54929 0 0.28659 0.0239 0.37018 0.0717 0.56123z"/><path d="m298.11 647.05-0.0478 0.0716-0.0239 0.0478-0.0119 0.0358-0.0478 0.0716-0.0358 0.0358-0.0717 0.1074-0.0358 0.0239-0.0239 0.0358-0.0358 0.0239-0.0239 0.0358-0.2148 0.1434-0.0358 0.012-0.0358 0.0239-0.0478 0.0119-0.0358 0.0239-0.0478 0.0119-0.0358 0.0119h-0.0478l-0.0478 0.0119-0.0478 0.012h-0.1434c-1.3971 0-2.8659-1.7434-2.8659-3.4749 0-1.1105 0.65676-1.8986 1.5762-1.8986l0.0239 0.2627c-0.59707 0-0.7523 0.65676-0.7523 1.1105 0 0.59706 0.37017 2.0539 0.64483 2.6032 0.37016 0.70453 0.90752 1.1344 1.3852 1.1344 0.77618 0 0.94335-0.96723 0.94335-1.0389 0-0.0717-0.0239-0.14329-0.0358-0.203l-0.59705-2.3285c-0.0597-0.203-0.0597-0.23882-0.23883-0.42988-0.52541-0.65676-1.015-0.84782-1.3493-0.84782l-0.0239-0.2627c0.23882 0 0.83589 0.0478 1.5523 0.89558 0.0956-0.50153 0.51348-0.89558 1.0866-0.89558 0.41794 0 0.69259 0.27464 0.88365 0.65675 0.19105 0.42988 0.34629 1.1464 0.34629 1.1702 0 0.11942-0.10745 0.11942-0.13135 0.11942-0.13136 0-0.13136-0.0478-0.17912-0.21494-0.19106-0.76423-0.406-1.4688-0.89559-1.4688-0.32241 0-0.35823 0.31047-0.35823 0.54929 0 0.26271 0.0239 0.35824 0.15524 0.8717 0.13135 0.50153 0.15523 0.62094 0.2627 1.0747l0.42988 1.6598c0.0836 0.33436 0.0836 0.35824 0.0836 0.406 0 0.203-0.1433 0.31047-0.34629 0.31047-0.28659 0-0.46571-0.25076-0.50153-0.51347z"/><path d="m305.56 647.04v0.0119l0.012 0.0119v0.0238l0.0119 0.0239v0.0476l0.012 0.0239v0.0715c0 0.203-0.1433 0.31047-0.34629 0.31047-0.11942 0-0.44183-0.0836-0.48959-0.50153-0.21494 0.44183-0.62094 0.74035-1.0986 0.74035l0.0119-0.2627c0.77618 0 0.94336-0.95529 0.94336-1.015 0-0.0597-0.0239-0.13135-0.0358-0.17912l-0.56124-2.2688c-0.0717-0.29853-0.3463-0.59706-0.609-0.82393-0.25076-0.21494-0.62094-0.42989-0.96723-0.42989-0.59706 0-0.77617 0.62094-0.77617 1.0986 0 0.56123 0.34629 1.9583 0.6687 2.5673 0.32241 0.58512 0.83588 1.0508 1.3374 1.0508l-0.0119 0.2627c-1.3613 0-2.842-1.6598-2.842-3.3674 0-1.1702 0.72842-1.8748 1.5882-1.8748 0.70453 0 1.2658 0.56123 1.3852 0.69259l0.0119-0.012c-0.25076-1.0628-0.39406-1.5404-0.39406-1.5643-0.0478-0.10745-0.45376-1.2896-1.7195-1.2896-0.22687 0-0.62093 0.0119-0.95529 0.11941 0.35824 0.10745 0.4896 0.41793 0.4896 0.62094 0 0.19105-0.13136 0.41794-0.45377 0.41794-0.2627 0-0.64481-0.21494-0.64481-0.69258 0-0.4896 0.44181-0.71647 1.5882-0.71647 1.4926 0 2.3405 0.91946 2.5315 1.6359z"/><path d="m319.01 649.42 0.0239 0.0956 0.0119 0.0358 0.0119 0.0478 0.0239 0.0358 0.0119 0.0478 0.0119 0.0358 0.1673 0.2506 0.0358 0.0239 0.0358 0.0358 0.0358 0.0239 0.0358 0.0358 0.0478 0.0239 0.0358 0.0239 0.0478 0.0239 0.0597 0.0239 0.0478 0.0119 0.0597 0.0239 0.1194 0.0238 0.0717 0.0239 0.0717 0.0119h0.0717l0.1672 0.0238h0.0836c0.0597 0 0.203 0.0119 0.203 0.23882 0 0.0119 0 0.13135-0.15523 0.13135-0.39406 0-0.812-0.0358-1.2061-0.0358-0.40599 0-0.82394 0.0358-1.218 0.0358-0.0597 0-0.21494 0-0.21494-0.23882 0-0.13135 0.11941-0.13135 0.21494-0.13135 0.68064-0.0119 0.812-0.26271 0.812-0.52542 0-0.0358-0.0239-0.21494-0.0358-0.25075l-1.3255-5.278-2.639 6.1974c-0.0956 0.21493-0.10744 0.22687-0.38212 0.22687h-1.5882c-0.23882 0-0.34629 0-0.34629-0.23882 0-0.13135 0.10745-0.13135 0.33436-0.13135 0.0597 0 0.81199 0 0.81199-0.10745l-1.6001-6.3766c-0.11942-0.46571-0.32241-0.84782-1.2896-0.88364-0.0717 0-0.20299-0.0119-0.20299-0.23882 0-0.0836 0.0597-0.13136 0.15524-0.13136 0.38211 0 0.80005 0.0358 1.1941 0.0358 0.406 0 0.83587-0.0358 1.2299-0.0358 0.0597 0 0.21494 0 0.21494 0.23882 0 0.11941-0.10745 0.13136-0.23882 0.13136-0.69259 0.0239-0.78812 0.28659-0.78812 0.5254 0 0.0836 0.0119 0.1433 0.0478 0.26271l1.5643 6.2691c0.0478-0.0717 0.0478-0.0956 0.10745-0.21494l2.9733-6.9856c0.0836-0.20299 0.11941-0.22688 0.22688-0.22688 0.13135 0 0.13135 0.0358 0.19106 0.25076z"/></g>
  <ellipse cx="259.93" cy="389.6" rx="41.718" ry="13.604" fill="#0065ff" fill-opacity=".19324" stroke-width="0"/>
  <text x="226.82999" y="380.68896" fill="#000000" font-family="'Helvetica Neue LT'" font-size="7.4469px" letter-spacing="0px" stroke-width=".18617" word-spacing="0px" style="line-height:1.25" xml:space="preserve"><tspan x="226.82999" y="387.483" style="font-feature-settings:normal;font-variant-caps:normal;font-variant-ligatures:normal;font-variant-numeric:normal"/><tspan x="226.82999" y="396.79166" style="font-feature-settings:normal;font-variant-caps:normal;font-variant-ligatures:normal;font-variant-numeric:normal">calculate posteriors</tspan></text>
  <rect x="239.59" y="421.22" width="48.225" height="49.391" fill="none" stroke="#000" stroke-linecap="square" stroke-linejoin="bevel" stroke-opacity=".48235" stroke-width=".7"/>
  <path d="m239.6 434.16h48.233" fill="none" stroke="#000" stroke-opacity=".48235" stroke-width=".7"/>
  <text x="241.45494" y="430.13858" fill="#000000" font-family="Verdana" font-size="7.0914px" letter-spacing="0px" stroke-width=".17729" word-spacing="0px" style="line-height:1.25" xml:space="preserve"><tspan x="241.45494" y="430.13858" font-family="'Helvetica Neue LT'" font-size="7.0914px" stroke-width=".17729" style="font-feature-settings:normal;font-variant-caps:normal;font-variant-ligatures:normal;font-variant-numeric:normal">tags posterior</tspan></text>
  <g stroke="#000">
   <path d="m265.27 434v36.669" fill="none" stroke-opacity=".42029" stroke-width=".4"/>
   <g transform="matrix(.52265 0 0 -.52265 94.186 803.15)" fill-opacity=".76471" fill-rule="evenodd" stroke-miterlimit="10.433" stroke-width="0" ns0:preamble="default_packages.tex" ns0:text="$$tag \\, \\, 1$$\n$$tag \\, \\, 2$$\n$$\\cdots$$\n$$tag \\, \\, N$$\n\n" xml:space="preserve"><path d="m294.41 698.42h1.1674c0.24839 0 0.37258 0 0.37258 0.24839 0 0.13661-0.12419 0.13661-0.34774 0.13661h-1.0929c0.4471 1.7512 0.5092 1.9996 0.5092 2.0741 0 0.21114-0.14903 0.33533-0.36017 0.33533-0.0373 0-0.385-0.0124-0.49678-0.4471l-0.48436-1.9623h-1.1674c-0.23598 0-0.37259 0-0.37259-0.23597 0-0.14903 0.11178-0.14903 0.34775-0.14903h1.0929c-0.8942-3.5023-0.94388-3.7134-0.94388-3.937 0-0.65824 0.47194-1.1302 1.1426-1.1302 1.2668 0 1.9747 1.8008 1.9747 1.9002 0 0.12419-0.0993 0.12419-0.14903 0.12419-0.11178 0-0.1242-0.0372-0.1863-0.17387-0.53403-1.2792-1.1923-1.5773-1.6145-1.5773-0.26081 0-0.38501 0.16146-0.38501 0.5713 0 0.28565 0.0248 0.37259 0.0745 0.58372z"/><path d="m300.94 698.15-0.0124 0.0497-0.0248 0.0373-0.0248 0.0497-0.124 0.1865-0.0373 0.0372-0.0248 0.0373-0.1242 0.1242-0.0373 0.0248-0.0248 0.0248-0.1119 0.0744-0.0372 0.0248-0.0497 0.0124-0.0373 0.0248-0.0373 0.0124-0.0497 0.0124-0.0373 0.0124-0.0994 0.0248-0.0373 0.0124h-0.0497l-0.0497 0.0124h-0.0994c-1.4531 0-2.9931-1.8132-2.9931-3.6141 0-1.155 0.68307-1.9747 1.6518-1.9747l0.0248 0.27323c-0.62098 0-0.79485 0.68308-0.79485 1.155 0 0.62097 0.39742 2.1362 0.68307 2.7074 0.38501 0.72034 0.94388 1.1799 1.4407 1.1799 0.79485 0 0.96872-1.006 0.96872-1.0805 0-0.0745-0.0248-0.14903-0.0373-0.21113l-0.62098-2.4218c-0.0497-0.22355-0.0497-0.24839-0.24839-0.45952-0.53404-0.67066-1.0432-0.86937-1.391-0.86937l-0.0248-0.27323c0.24839 0 0.86936 0.0497 1.6145 0.93147 0.0869-0.52162 0.52162-0.93147 1.1178-0.93147 0.43469 0 0.72034 0.28565 0.91905 0.68308 0.21113 0.4471 0.37258 1.1923 0.37258 1.2171 0 0.12419-0.11177 0.12419-0.14903 0.12419-0.1242 0-0.13662-0.0497-0.17387-0.22355-0.21114-0.80727-0.43469-1.5276-0.94389-1.5276-0.33533 0-0.37258 0.32291-0.37258 0.5713 0 0.27323 0.0248 0.36017 0.16145 0.90662 0.13661 0.52162 0.16145 0.64582 0.27323 1.1053l0.4471 1.7387c0.0869 0.33532 0.0869 0.36016 0.0869 0.40984 0 0.21113-0.14904 0.33533-0.36017 0.33533-0.29807 0-0.48436-0.27323-0.52162-0.54646z"/><path d="m308.71 698.13v0.0496l0.0124 0.0124v0.0372l0.0124 0.0124v0.0744l0.0124 0.0124v0.0248c0 0.21113-0.14903 0.33533-0.36016 0.33533-0.1242 0-0.45952-0.0869-0.5092-0.53404-0.22355 0.45952-0.65824 0.78243-1.155 0.78243l0.0124-0.27323c0.81969 0 0.99356-0.99356 0.99356-1.0557s-0.0248-0.13662-0.0373-0.18629l-0.59613-2.3597c-0.0745-0.32291-0.36017-0.6334-0.6334-0.85695-0.26081-0.22355-0.64581-0.4471-1.006-0.4471-0.62098 0-0.79485 0.64581-0.79485 1.1302 0 0.59614 0.36017 2.0492 0.68307 2.6826 0.33533 0.60856 0.86937 1.0929 1.391 1.0929l-0.0124 0.27323c-1.4158 0-2.9434-1.7263-2.9434-3.5023 0-1.2171 0.7576-1.9499 1.6394-1.9499 0.73275 0 1.3165 0.58371 1.4407 0.72033l0.0124-0.0124c-0.26081-1.1053-0.40985-1.6021-0.40985-1.6394-0.0497-0.0994-0.47194-1.3289-1.776-1.3289-0.23597 0-0.64582 0.0124-0.99356 0.1242 0.37258 0.11177 0.5092 0.43468 0.5092 0.64581 0 0.19872-0.13662 0.43469-0.47194 0.43469-0.27323 0-0.67066-0.22355-0.67066-0.72034 0-0.5092 0.45952-0.75759 1.6518-0.75759 1.54 0 2.4342 0.96873 2.6205 1.7015z"/><path d="m316.98 701.39v0.186l-0.0124 0.0124v0.0372l-0.0124 0.0124v0.0124l-0.0248 0.0248h-0.0124v0.0124h-0.0248l-0.0124 0.0124h-0.0868l-0.0124 0.0124h-0.0868c-0.77001-0.79485-1.8505-0.79485-2.2479-0.79485v-0.385c0.24839 0 0.96872 0 1.6145 0.3229v-6.3836c0-0.4471-0.0372-0.59614-1.1426-0.59614h-0.39743v-0.385c0.43469 0.0373 1.5028 0.0373 1.9996 0.0373s1.5773 0 2.012-0.0373v0.385h-0.39743c-1.1178 0-1.155 0.13662-1.155 0.59614z"/><path d="m294.96 679.14h1.0953c0.23304 0 0.34955 0 0.34955 0.23303 0 0.12817-0.11651 0.12817-0.32625 0.12817h-1.0254c0.41946 1.6429 0.47772 1.8759 0.47772 1.9458 0 0.19808-0.13982 0.3146-0.3379 0.3146-0.035 0-0.3612-0.0117-0.46607-0.41946l-0.45441-1.841h-1.0953c-0.22138 0-0.34955 0-0.34955-0.22138 0-0.13982 0.10487-0.13982 0.32625-0.13982h1.0254c-0.83892-3.2858-0.88553-3.4839-0.88553-3.6936 0-0.61754 0.44277-1.0603 1.072-1.0603 1.1885 0 1.8526 1.6895 1.8526 1.7827 0 0.11651-0.0932 0.11651-0.13983 0.11651-0.10486 0-0.11651-0.035-0.17477-0.16312-0.50103-1.2001-1.1186-1.4798-1.5147-1.4798-0.24468 0-0.3612 0.13982-0.3612 0.52433 0 0.27964 0.0233 0.3612 0.0699 0.55928z"/><path d="m301.09 678.88-0.0116 0.0466-0.0233 0.0349-0.0233 0.035-0.0233 0.0349-0.0233 0.0466-0.0233 0.035-0.0233 0.0349-0.0233 0.035-0.0583 0.0583-0.0233 0.0349-0.0349 0.0233-0.0233 0.035-0.0349 0.0233-0.0583 0.0583-0.0349 0.0233-0.07 0.0466-0.0349 0.0117-0.0466 0.0233-0.035 0.0116-0.035 0.0233-0.0466 0.0116-0.035 0.0117-0.0932 0.0234-0.035 0.0117h-0.0932l-0.0466 0.0116h-0.0466c-1.3632 0-2.8081-1.7012-2.8081-3.3906 0-1.0953 0.64085-1.8526 1.5497-1.8526l0.0233 0.25634c-0.58259 0-0.74571 0.62919-0.74571 1.0836 0 0.58259 0.37285 2.0041 0.64084 2.5401 0.36121 0.6758 0.88554 1.1069 1.3516 1.1069 0.74571 0 0.90883-0.95544 0.90883-1.0254 0-0.0699-0.0233-0.12817-0.0349-0.18643l-0.58259-2.2721c-0.0466-0.20973-0.0466-0.23303-0.23304-0.43111-0.50102-0.62919-0.97874-0.81562-1.305-0.81562l-0.0233-0.25634c0.23303 0 0.81562 0.0466 1.5147 0.86223 0.0816-0.47772 0.48937-0.86223 1.0487-0.86223 0.40781 0 0.6758 0.26799 0.86223 0.62919 0.19808 0.41947 0.34955 1.1302 0.34955 1.1535 0 0.11651-0.10487 0.11651-0.13982 0.11651-0.11652 0-0.12817-0.0466-0.16313-0.20973-0.19808-0.75736-0.40781-1.4332-0.88553-1.4332-0.3146 0-0.34955 0.29129-0.34955 0.52433 0 0.25634 0.0233 0.34955 0.15147 0.86223 0.12817 0.48937 0.15147 0.59423 0.25634 1.037l0.41946 1.6196c0.0816 0.32625 0.0816 0.34955 0.0816 0.39616 0 0.19808-0.13982 0.3146-0.3379 0.3146-0.27964 0-0.45441-0.25634-0.48937-0.51268z"/><path d="m308.37 678.87v0.035l0.0116 0.0233v0.035l0.0117 0.0117v0.0583l0.0116 0.0233v0.0234c0 0.19808-0.13982 0.3146-0.3379 0.3146-0.11652 0-0.43111-0.0816-0.47772-0.50103-0.20973 0.43112-0.61754 0.73406-1.0836 0.73406l0.0116-0.25634c0.76902 0 0.93214-0.94379 0.93214-1.002 0-0.0466-0.0233-0.11651-0.035-0.16312l-0.55928-2.2138c-0.0699-0.30294-0.3379-0.59424-0.59424-0.81562-0.24468-0.20973-0.60589-0.40781-0.94379-0.40781-0.58259 0-0.74571 0.59424-0.74571 1.0603 0 0.55928 0.3379 1.9225 0.64085 2.5168 0.31459 0.55928 0.81562 1.0254 1.305 1.0254l-0.0116 0.25634c-1.3283 0-2.7615-1.6196-2.7615-3.2858 0-1.1535 0.71075-1.8293 1.538-1.8293 0.68745 0 1.2351 0.53598 1.3516 0.66415l0.0116-0.0116c-0.24469-1.0254-0.38451-1.5031-0.38451-1.5264-0.0466-0.10487-0.44276-1.2467-1.6662-1.2467-0.22138 0-0.60589 0.0117-0.93213 0.11652 0.34955 0.10487 0.47772 0.40781 0.47772 0.59424s-0.12817 0.40781-0.44277 0.40781c-0.25634 0-0.62919-0.20973-0.62919-0.66415 0-0.47772 0.43111-0.71075 1.5497-0.71075 1.4448 0 2.2837 0.89718 2.4585 1.5963z"/><path d="m314.2 675.4 1.2234 1.2001c1.8177 1.5963 2.5168 2.2138 2.5168 3.379 0 1.3166-1.0486 2.2371-2.4702 2.2371-1.305 0-2.1672-1.0603-2.1672-2.0973 0-0.64085 0.58259-0.64085 0.61754-0.64085 0.19808 0 0.60589 0.13982 0.60589 0.61754 0 0.2913-0.20973 0.59424-0.61754 0.59424-0.0932 0-0.11652 0-0.15147-0.0117 0.26799 0.75736 0.88553 1.1768 1.5613 1.1768 1.0603 0 1.5613-0.93213 1.5613-1.8759 0-0.93213-0.58258-1.841-1.2234-2.5634l-2.2255-2.4818c-0.12817-0.12817-0.12817-0.15147-0.12817-0.41946h4.3111l0.32625 2.0158h-0.29129c-0.0583-0.34955-0.13982-0.86223-0.25634-1.037-0.0816-0.0932-0.85058-0.0932-1.1069-0.0932z"/><path d="m300.9 661.79v0.02l-0.01 0.03v0.05l-0.01 0.03-0.01 0.02-0.01 0.03-0.02 0.04-0.01 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.06 0.06-0.02 0.01-0.02 0.02-0.02 0.01-0.02 0.02-0.04 0.02-0.03 0.01-0.02 0.01-0.03 0.01-0.02 0.01h-0.03l-0.02 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m305.33 661.79v0.05l-0.01 0.03v0.02l-0.01 0.03-0.01 0.02v0.03l-0.02 0.04-0.02 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.06 0.06-0.02 0.01-0.02 0.02-0.02 0.01-0.02 0.02-0.04 0.02-0.03 0.01-0.04 0.02-0.03 0.01h-0.03l-0.02 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m309.75 661.79v0.05l-0.01 0.03v0.02l-0.01 0.03v0.02l-0.01 0.03-0.02 0.04-0.02 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.09 0.09-0.02 0.01-0.03 0.02-0.06 0.03-0.03 0.01-0.02 0.01-0.03 0.01h-0.02l-0.03 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m293.72 646.41h1.0586c0.22524 0 0.33786 0 0.33786 0.21398 0 0.12388-0.11262 0.12388-0.31534 0.12388h-0.97981c0.39417 1.588 0.45049 1.8132 0.45049 1.8808 0 0.19146-0.13515 0.30408-0.32661 0.30408-0.0338 0-0.33787-0.0113-0.43923-0.40544l-0.43922-1.7794h-1.0586c-0.22525 0-0.33787 0-0.33787-0.20272 0-0.13514 0.0901-0.13514 0.31534-0.13514h0.99108c-0.81088-3.1872-0.85593-3.3787-0.85593-3.5701 0-0.60816 0.42796-1.0361 1.0361-1.0361 1.1375 0 1.7794 1.633 1.7794 1.7231 0 0.11262-0.0901 0.11262-0.13515 0.11262-0.10136 0-0.11262-0.0338-0.16893-0.15767-0.48428-1.16-1.0699-1.4303-1.4528-1.4303-0.23651 0-0.34913 0.14641-0.34913 0.51807 0 0.27029 0.0225 0.34913 0.0676 0.52932z"/><path d="m299.66 646.16-0.045 0.0676-0.0225 0.045-0.0113 0.0338-0.045 0.0676-0.0338 0.0338-0.0675 0.1014-0.0338 0.0225-0.0225 0.0338-0.0338 0.0225-0.0225 0.0338-0.2028 0.135-0.0338 0.0113-0.0338 0.0225-0.0451 0.0113-0.0338 0.0225-0.0451 0.0113-0.0338 0.0113h-0.0451l-0.0902 0.0226h-0.1352c-1.3177 0-2.7029-1.6443-2.7029-3.2773 0-1.0474 0.61942-1.7907 1.4866-1.7907l0.0225 0.24776c-0.56311 0-0.70952 0.61943-0.70952 1.0474 0 0.56311 0.34913 1.9371 0.60816 2.4552 0.34913 0.66447 0.85593 1.0699 1.3064 1.0699 0.73204 0 0.88972-0.91224 0.88972-0.97981 0-0.0676-0.0225-0.13515-0.0338-0.19146l-0.56311-2.1961c-0.0563-0.19145-0.0563-0.22524-0.22525-0.40544-0.49554-0.61942-0.95729-0.79962-1.2726-0.79962l-0.0225-0.24776c0.22524 0 0.78835 0.045 1.4641 0.84466 0.0901-0.47301 0.48427-0.84466 1.0249-0.84466 0.39418 0 0.65321 0.25903 0.8334 0.61942 0.1802 0.40544 0.32661 1.0812 0.32661 1.1037 0 0.11262-0.10136 0.11262-0.12389 0.11262-0.12388 0-0.12388-0.045-0.16893-0.20272-0.18019-0.72078-0.38292-1.3853-0.84467-1.3853-0.30408 0-0.33786 0.29282-0.33786 0.51807 0 0.24777 0.0225 0.33786 0.14641 0.82214 0.12388 0.47301 0.1464 0.58564 0.24776 1.0136l0.40544 1.5654c0.0788 0.31534 0.0788 0.33787 0.0788 0.38292 0 0.19145-0.13515 0.29281-0.3266 0.29281-0.2703 0-0.43923-0.2365-0.47302-0.48427z"/><path d="m306.68 646.15v0.0113l0.0113 0.0113v0.0226l0.0113 0.0225v0.0452l0.0113 0.0225v0.0678c0 0.19145-0.13515 0.29281-0.3266 0.29281-0.11263 0-0.41671-0.0788-0.46175-0.47301-0.20273 0.4167-0.58564 0.69826-1.0361 0.69826l0.0113-0.24777c0.73205 0 0.88972-0.90098 0.88972-0.95729 0-0.0563-0.0225-0.12388-0.0338-0.16893l-0.52932-2.1398c-0.0676-0.28155-0.32661-0.56311-0.57438-0.77709-0.2365-0.20272-0.58563-0.40544-0.91224-0.40544-0.56311 0-0.73204 0.58563-0.73204 1.0361 0 0.52933 0.3266 1.847 0.63068 2.4214 0.30408 0.55185 0.78836 0.99108 1.2614 0.99108l-0.0113 0.24777c-1.2839 0-2.6804-1.5654-2.6804-3.176 0-1.1037 0.68699-1.7682 1.4979-1.7682 0.66447 0 1.1938 0.52932 1.3064 0.65321l0.0113-0.0113c-0.23651-1.0023-0.37166-1.4528-0.37166-1.4754-0.0451-0.10136-0.42796-1.2163-1.6218-1.2163-0.21398 0-0.58563 0.0113-0.90098 0.11263 0.33787 0.10136 0.46175 0.39417 0.46175 0.58563 0 0.1802-0.12388 0.39418-0.42796 0.39418-0.24777 0-0.60816-0.20272-0.60816-0.65321 0-0.46175 0.4167-0.67573 1.4979-0.67573 1.4078 0 2.2074 0.86719 2.3876 1.5429z"/><path d="m319.38 648.39 0.0225 0.0901 0.0113 0.0338 0.0113 0.045 0.0225 0.0338 0.0113 0.0451 0.0113 0.0338 0.1575 0.2366 0.0338 0.0225 0.0338 0.0338 0.0338 0.0225 0.0338 0.0338 0.045 0.0225 0.0338 0.0225 0.0451 0.0225 0.0563 0.0225 0.0451 0.0113 0.0563 0.0225 0.1126 0.0226 0.0676 0.0225 0.0676 0.0113h0.0676l0.1576 0.0226h0.0789c0.0563 0 0.19146 0.0113 0.19146 0.22524 0 0.0113 0 0.12388-0.14641 0.12388-0.37165 0-0.76583-0.0338-1.1375-0.0338-0.38291 0-0.77709 0.0338-1.1487 0.0338-0.0563 0-0.20272 0-0.20272-0.22524 0-0.12388 0.11262-0.12388 0.20272-0.12388 0.64194-0.0113 0.76583-0.24777 0.76583-0.49554 0-0.0338-0.0225-0.20272-0.0338-0.23651l-1.2501-4.9779-2.489 5.8451c-0.0901 0.20272-0.10136 0.21398-0.36039 0.21398h-1.4979c-0.22524 0-0.3266 0-0.3266-0.22524 0-0.12388 0.10136-0.12388 0.31534-0.12388 0.0563 0 0.76583 0 0.76583-0.10136l-1.5091-6.014c-0.11262-0.43922-0.30408-0.79961-1.2163-0.8334-0.0676 0-0.19146-0.0113-0.19146-0.22525 0-0.0788 0.0563-0.12388 0.14641-0.12388 0.36039 0 0.75457 0.0338 1.1262 0.0338 0.38291 0 0.78835-0.0338 1.16-0.0338 0.0563 0 0.20272 0 0.20272 0.22524 0 0.11263-0.10136 0.12389-0.22525 0.12389-0.65321 0.0225-0.74331 0.27029-0.74331 0.49554 0 0.0788 0.0113 0.13514 0.0451 0.24777l1.4754 5.9127c0.045-0.0676 0.045-0.0901 0.10136-0.20272l2.8043-6.5884c0.0788-0.19146 0.11262-0.21398 0.21398-0.21398 0.12389 0 0.12389 0.0338 0.1802 0.23651z"/></g>
   <path d="m276.84 196.3-0.0548 179" fill="none" marker-end="url(#e)" stroke-opacity=".57971" stroke-width=".9"/>
   <path d="m276.53 401.79 0.0258 17.302" fill="none" marker-end="url(#g)" stroke-opacity=".57971" stroke-width=".9"/>
  </g>
  <ellipse cx="160.18" cy="500.33" rx="41.718" ry="13.604" fill="#0065ff" fill-opacity=".19324"/>
  <text x="137.8044" y="490.51987" fill="#000000" font-family="'Helvetica Neue LT'" font-size="8.3339px" letter-spacing="0px" stroke-width=".20835" word-spacing="0px" style="line-height:1.25" xml:space="preserve"><tspan x="137.8044" y="498.12314" style="font-feature-settings:normal;font-variant-caps:normal;font-variant-ligatures:normal;font-variant-numeric:normal"/><tspan x="137.8044" y="508.5405" style="font-feature-settings:normal;font-variant-caps:normal;font-variant-ligatures:normal;font-variant-numeric:normal">do_decision</tspan></text>
  <g fill="none" stroke="#000">
   <path d="m265.27 470.67-104.02 15.754" marker-end="url(#f)" stroke-opacity=".57971" stroke-width=".9"/>
   <rect x="146.59" y="533.31" width="24.036" height="21.495" stroke-linecap="square" stroke-linejoin="bevel" stroke-opacity=".48235" stroke-width=".7"/>
   <path d="m146.67 544.82h24.12" stroke-opacity=".48235" stroke-width=".7"/>
  </g>
  <text x="152.95433" y="541.11334" fill="#000000" font-family="Verdana" font-size="7.538px" letter-spacing="0px" stroke-width=".18845" word-spacing="0px" style="line-height:1.25" xml:space="preserve"><tspan x="152.95433" y="541.11334" font-family="'Helvetica Neue LT'" font-size="7.538px" stroke-width=".18845" style="font-feature-settings:normal;font-variant-caps:normal;font-variant-ligatures:normal;font-variant-numeric:normal">tag</tspan></text>
  <path d="m159.77 513.95v17.234" fill="none" marker-end="url(#d)" stroke="#000" stroke-opacity=".57971" stroke-width=".9"/>
  <g transform="matrix(.51439 0 0 -.51439 120.48 798.59)" fill-rule="evenodd" stroke="#000" stroke-miterlimit="10.433" stroke-width="0" ns0:preamble="default_packages.tex" ns0:text="$$0.2$$\n$$0.3$$\n$$\\cdots$$\n$$0.6$$\n" xml:space="preserve"><path d="m300.9 661.79v0.02l-0.01 0.03v0.05l-0.01 0.03-0.01 0.02-0.01 0.03-0.02 0.04-0.01 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.06 0.06-0.02 0.01-0.02 0.02-0.02 0.01-0.02 0.02-0.04 0.02-0.03 0.01-0.02 0.01-0.03 0.01-0.02 0.01h-0.03l-0.02 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m305.33 661.79v0.05l-0.01 0.03v0.02l-0.01 0.03-0.01 0.02v0.03l-0.02 0.04-0.02 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.06 0.06-0.02 0.01-0.02 0.02-0.02 0.01-0.02 0.02-0.04 0.02-0.03 0.01-0.04 0.02-0.03 0.01h-0.03l-0.02 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m309.75 661.79v0.05l-0.01 0.03v0.02l-0.01 0.03v0.02l-0.01 0.03-0.02 0.04-0.02 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.09 0.09-0.02 0.01-0.03 0.02-0.06 0.03-0.03 0.01-0.02 0.01-0.03 0.01h-0.02l-0.03 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/></g>
  <g fill="none" stroke="#000">
   <path d="m274.48 124.03 0.10962 20.808" marker-end="url(#o)" stroke-opacity=".57971" stroke-width=".9"/>
   <path d="m16.924 340.51c0.20671 0.82683 0 24.495 0 24.495" marker-end="url(#i)" stroke-opacity=".57971" stroke-width=".9"/>
   <g stroke-opacity=".57971" stroke-width=".9">
    <path d="m208.09 73.49v130.72" marker-end="url(#m)"/>
    <path d="m151.96 173.82 53.934 31.271" marker-end="url(#n)"/>
    <path d="m43.664 381.7 172.78 8.4477" marker-end="url(#h)"/>
   </g>
  </g>
  <g transform="matrix(.57563 0 0 -.57563 -108.81 806.71)" fill-rule="evenodd" stroke="#000" stroke-miterlimit="10.433" stroke-width="0" ns0:preamble="default_packages.tex" ns0:text="$$0.2$$\n$$0.1$$\n$$\\cdots$$\n$$0.3$$\n" xml:space="preserve"><path d="m244.98 737.82v0.15l-0.01 0.14v0.15l-0.03 0.45-0.01 0.14-0.02 0.15-0.03 0.15-0.02 0.14-0.04 0.15-0.03 0.14-0.15 0.42-0.06 0.14c-0.46 0.95-1.28 1.11-1.7 1.11-0.6 0-1.32-0.26-1.73-1.18-0.32-0.69-0.37-1.46-0.37-2.25 0-0.75 0.04-1.64 0.45-2.4 0.42-0.8 1.15-1 1.64-1v0.22c-0.39 0-0.97 0.24-1.16 1.2-0.1 0.59-0.1 1.51-0.1 2.09 0 0.64 0 1.3 0.07 1.83 0.19 1.18 0.94 1.27 1.19 1.27 0.33 0 0.99-0.17 1.18-1.16 0.1-0.55 0.1-1.31 0.1-1.94 0-0.74 0-1.42-0.11-2.05-0.15-0.95-0.72-1.24-1.17-1.24v-0.22c0.54 0 1.3 0.21 1.74 1.15 0.32 0.69 0.37 1.46 0.37 2.25z"/><path d="m247.28 735.16v0.08l-0.01 0.03v0.02l-0.01 0.03-0.01 0.02-0.01 0.03-0.02 0.04-0.01 0.03-0.02 0.02-0.01 0.02-0.02 0.02-0.01 0.02-0.02 0.01-0.04 0.04-0.02 0.01-0.02 0.02-0.02 0.01-0.02 0.02-0.03 0.01-0.04 0.02-0.03 0.01h-0.02l-0.03 0.01h-0.02l-0.03 0.01h-0.05c-0.29 0-0.53-0.24-0.53-0.53s0.24-0.52 0.53-0.52 0.52 0.23 0.52 0.52z"/><path d="m251.07 740.99v0.15l-0.01 0.01v0.03l-0.01 0.01v0.02h-0.01v0.01h-0.01l-0.01 0.01h-0.01l-0.01 0.01h-0.02v0.01h-0.14c-0.62-0.63-1.5-0.63-1.82-0.63v-0.31c0.21 0 0.79 0 1.31 0.26v-5.15c0-0.36-0.03-0.48-0.93-0.48h-0.32v-0.3c0.35 0.03 1.22 0.03 1.62 0.03s1.27 0 1.62-0.03v0.3h-0.32c-0.9 0-0.93 0.11-0.93 0.48z"/><path d="m242.71 704.17v0.02l-0.01 0.03v0.05l-0.01 0.03-0.01 0.02-0.01 0.03-0.02 0.04-0.01 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.06 0.06-0.02 0.01-0.02 0.02-0.02 0.01-0.02 0.02-0.04 0.02-0.03 0.01-0.02 0.01-0.03 0.01-0.02 0.01h-0.03l-0.02 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m247.14 704.17v0.05l-0.01 0.03v0.02l-0.01 0.03-0.01 0.02v0.03l-0.02 0.04-0.02 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.06 0.06-0.02 0.01-0.02 0.02-0.02 0.01-0.02 0.02-0.04 0.02-0.03 0.01-0.04 0.02-0.03 0.01h-0.03l-0.02 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m251.56 704.17v0.05l-0.01 0.03v0.02l-0.01 0.03v0.02l-0.01 0.03-0.02 0.04-0.02 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.09 0.09-0.02 0.01-0.03 0.02-0.06 0.03-0.03 0.01-0.02 0.01-0.03 0.01h-0.02l-0.03 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.53 0.24 0.53 0.53z"/><path d="m244.98 721.28v0.15l-0.01 0.15v0.15l-0.01 0.15-0.01 0.14-0.02 0.3-0.02 0.15-0.03 0.14-0.02 0.15-0.04 0.14-0.03 0.14-0.05 0.15-0.1 0.28-0.06 0.14c-0.46 0.95-1.28 1.11-1.7 1.11-0.6 0-1.32-0.26-1.73-1.18-0.32-0.69-0.37-1.46-0.37-2.26 0-0.74 0.04-1.64 0.45-2.39 0.42-0.81 1.15-1.01 1.64-1.01v0.22c-0.39 0-0.97 0.25-1.16 1.2-0.1 0.6-0.1 1.51-0.1 2.1 0 0.64 0 1.29 0.07 1.83 0.19 1.18 0.94 1.27 1.19 1.27 0.33 0 0.99-0.18 1.18-1.16 0.1-0.56 0.1-1.31 0.1-1.94 0-0.75 0-1.42-0.11-2.06-0.15-0.94-0.72-1.24-1.17-1.24v-0.22c0.54 0 1.3 0.21 1.74 1.16 0.32 0.68 0.37 1.46 0.37 2.24z"/><path d="m247.28 718.63v0.08l-0.01 0.02v0.03l-0.01 0.02-0.01 0.03-0.01 0.02-0.01 0.03-0.02 0.04-0.02 0.02-0.01 0.02-0.02 0.02-0.01 0.02-0.04 0.04-0.02 0.01-0.02 0.02-0.02 0.01-0.02 0.02-0.02 0.01-0.03 0.01-0.04 0.02-0.03 0.01-0.02 0.01h-0.03l-0.02 0.01h-0.08c-0.29 0-0.53-0.23-0.53-0.52s0.24-0.53 0.53-0.53 0.52 0.24 0.52 0.53z"/><path d="m251.03 721.6 0.08 0.03 0.07 0.02 0.07 0.04 0.14 0.06 0.14 0.08 0.06 0.04 0.07 0.05 0.06 0.05 0.05 0.04 0.12 0.1 0.05 0.06 0.05 0.05 0.04 0.06 0.05 0.06 0.12 0.18 0.03 0.06 0.04 0.07 0.03 0.06 0.02 0.07 0.03 0.07 0.02 0.07 0.01 0.07 0.02 0.07 0.02 0.14v0.07l0.01 0.08c0 0.81-0.88 1.37-1.84 1.37-1.01 0-1.76-0.6-1.76-1.35 0-0.33 0.22-0.52 0.5-0.52 0.31 0 0.51 0.22 0.51 0.51 0 0.49-0.47 0.49-0.61 0.49 0.3 0.49 0.96 0.62 1.32 0.62 0.41 0 0.96-0.22 0.96-1.11 0-0.12-0.02-0.7-0.28-1.14-0.3-0.47-0.64-0.5-0.89-0.51-0.08-0.01-0.32-0.03-0.39-0.03-0.08-0.01-0.15-0.02-0.15-0.12 0-0.11 0.07-0.11 0.24-0.11h0.44c0.82 0 1.19-0.68 1.19-1.65 0-1.35-0.69-1.64-1.13-1.64-0.43 0-1.18 0.17-1.52 0.76 0.34-0.05 0.65 0.16 0.65 0.54 0 0.36-0.27 0.56-0.56 0.56-0.23 0-0.55-0.14-0.55-0.58 0-0.9 0.92-1.56 2.01-1.56 1.22 0 2.13 0.91 2.13 1.92 0 0.82-0.63 1.59-1.67 1.8z"/><g fill-rule="evenodd" stroke-miterlimit="10.433"><g transform="matrix(1.8819 0 0 1.8819 -7.0566 -660.79)" stroke="#000" ns0:preamble="default_packages.tex" ns0:text="$$0.02$$\n$$0.03$$\n$$0.003$$\n" xml:space="preserve"><path d="m357.33 690.63-6e-3 0.0869v0.1739l-6e-3 0.0811-0.018 0.2607-0.0115 0.0812-0.0174 0.0869-0.0115 0.0812-0.0232 0.0869-0.0174 0.0811-0.0289 0.0811-0.0289 0.0812-0.0289 0.0811-0.0348 0.0812c-0.26655 0.55627-0.7417 0.64898-0.98507 0.64898-0.34767 0-0.77066-0.15065-1.0024-0.68954-0.18543-0.39404-0.2144-0.846-0.2144-1.3038 0-0.43458 0.0231-0.95029 0.26075-1.3849 0.24338-0.46935 0.66637-0.58525 0.9503-0.58525v0.12749c-0.22598 0-0.56786 0.14487-0.67216 0.69534-0.0637 0.34767-0.0637 0.87498-0.0637 1.2168 0 0.36506 0 0.74749 0.0463 1.0604 0.11009 0.68375 0.54469 0.7359 0.68955 0.7359 0.19123 0 0.57366-0.10429 0.68375-0.67216 0.058-0.3245 0.058-0.76488 0.058-1.1241 0-0.4346 0-0.82282-0.0637-1.1937-0.0869-0.54468-0.41721-0.71851-0.67796-0.71851v-0.12749c0.31291 0 0.75329 0.12169 1.0082 0.66637 0.17963 0.39982 0.2144 0.846 0.2144 1.3038z"/><path d="m358.67 689.09-6e-3 0.0115v0.0348l-6e-3 0.0115v0.0174l-6e-3 0.0115-6e-3 0.0174-0.012 0.023-6e-3 0.0174-6e-3 0.0115-0.0115 0.0115-6e-3 0.0115-0.0115 0.0115-6e-3 0.0115-0.023 0.023-0.0115 6e-3 -0.0115 0.0115-0.0115 6e-3 -0.0115 0.0115-0.0115 6e-3 -0.0174 6e-3 -0.023 0.012-0.0174 6e-3 -0.0115 6e-3h-0.0174l-0.0115 6e-3h-0.0463c-0.16804 0-0.30711-0.13907-0.30711-0.30132 0-0.16804 0.13907-0.30711 0.30711-0.30711 0.16224 0 0.30712 0.13907 0.30712 0.30711z"/><path d="m361.82 690.63v0.2608l-6e-3 0.0811-0.012 0.1738-0.0115 0.0869-6e-3 0.0812-0.0174 0.0869-0.0174 0.0812-0.0174 0.0869-0.0464 0.1622-0.0289 0.0812-0.0289 0.0811-0.0174 0.0406-0.0232 0.0406c-0.26075 0.55627-0.73591 0.64898-0.97927 0.64898-0.34767 0-0.77067-0.15065-1.0082-0.68954-0.17963-0.39404-0.20859-0.846-0.20859-1.3038 0-0.43458 0.0232-0.95029 0.26075-1.3849 0.24336-0.46935 0.66637-0.58525 0.95029-0.58525v0.12749c-0.22598 0-0.56786 0.14487-0.67216 0.69534-0.0637 0.34767-0.0637 0.87498-0.0637 1.2168 0 0.36506 0 0.74749 0.0463 1.0604 0.11009 0.68375 0.54469 0.7359 0.68954 0.7359 0.19122 0 0.57366-0.10429 0.68376-0.67216 0.058-0.3245 0.058-0.76488 0.058-1.1241 0-0.4346 0-0.82282-0.0637-1.1937-0.087-0.54468-0.4172-0.71851-0.67796-0.71851v-0.12749c0.31291 0 0.75328 0.12169 1.0024 0.66637 0.18543 0.39982 0.21439 0.846 0.21439 1.3038z"/><path d="m362.79 689.23 0.60842 0.59683c0.90394 0.79385 1.2516 1.101 1.2516 1.6804 0 0.65478-0.52151 1.1183-1.2284 1.1183-0.65477 0-1.0778-0.53309-1.0778-1.043 0-0.32448 0.28972-0.32448 0.30131-0.32448 0.10431 0 0.30131 0.0696 0.30131 0.30711 0 0.15066-0.1043 0.29551-0.30131 0.29551h-0.0753c0.12747 0.37085 0.44038 0.58525 0.77645 0.58525 0.5273 0 0.77647-0.46935 0.77647-0.93871 0-0.46356-0.28973-0.91553-0.60842-1.2748l-1.1068-1.2284c-0.0637-0.0637-0.0637-0.0753-0.0637-0.21439h2.144l0.16224 1.0024h-0.14485c-0.0289-0.17384-0.0696-0.4288-0.12749-0.51571-0.0405-0.0463-0.423-0.0463-0.55047-0.0463z"/><path d="m357.63 682.8-6e-3 0.0873v0.1687l-0.018 0.2619-6e-3 0.0814-0.0116 0.0873-0.0175 0.0873-0.0116 0.0814-0.0233 0.0873-0.0174 0.0814-0.0873 0.2442-0.0175 0.0408-0.0174 0.0408c-0.26756 0.55258-0.74453 0.64564-0.98883 0.64564-0.34899 0-0.77361-0.15123-1.0063-0.68636-0.18614-0.40135-0.21522-0.84922-0.21522-1.3087 0-0.43625 0.0233-0.95392 0.26174-1.396 0.24431-0.46533 0.66892-0.58166 0.95393-0.58166v0.12796c-0.22684 0-0.57002 0.13961-0.67473 0.698-0.064 0.34318-0.064 0.87832-0.064 1.2157 0 0.37226 0 0.75616 0.0465 1.0644 0.11051 0.68636 0.54676 0.73871 0.69217 0.73871 0.19196 0 0.57585-0.0989 0.68636-0.67472 0.0581-0.31992 0.0581-0.76199 0.0581-1.1284 0-0.43043 0-0.82597-0.064-1.1924-0.0873-0.55258-0.4188-0.72127-0.68055-0.72127v-0.12796c0.3141 0 0.75616 0.12215 1.0121 0.66891 0.18031 0.40134 0.21521 0.84923 0.21521 1.3087z"/><path d="m358.97 681.25-6e-3 0.0174v0.0291l-6e-3 0.0174v0.0116l-6e-3 0.0175-6e-3 0.0116-6e-3 0.0175-0.012 0.0232-6e-3 0.0175-0.0116 0.0116-6e-3 0.0116-0.0116 0.0116-6e-3 0.0116-0.0116 6e-3 -0.0232 0.0232-0.0116 6e-3 -0.0116 0.0116-0.0116 6e-3 -0.0116 0.0116-0.0175 6e-3 -0.0232 0.012-0.0175 6e-3h-0.0116l-0.0174 6e-3h-0.0116l-0.0174 6e-3h-0.0291c-0.16868 0-0.30828-0.1396-0.30828-0.30828 0-0.16869 0.1396-0.30247 0.30828-0.30247 0.16287 0 0.30828 0.13378 0.30828 0.30247z"/><path d="m362.13 682.8v0.256l-0.018 0.2619-0.0116 0.0814-6e-3 0.0873-0.0175 0.0873-0.0174 0.0814-0.0175 0.0873-0.0233 0.0814-0.0232 0.0814-0.0582 0.1628-0.0174 0.0408-0.0233 0.0408c-0.26174 0.55258-0.73871 0.64564-0.98301 0.64564-0.349 0-0.7736-0.15123-1.0121-0.68636-0.18031-0.40135-0.2094-0.84922-0.2094-1.3087 0-0.43625 0.0233-0.95392 0.26175-1.396 0.2443-0.46533 0.66892-0.58166 0.95393-0.58166v0.12796c-0.22685 0-0.57004 0.13961-0.67473 0.698-0.064 0.34318-0.064 0.87832-0.064 1.2157 0 0.37226 0 0.75616 0.0465 1.0644 0.11052 0.68636 0.54676 0.73871 0.69218 0.73871 0.19195 0 0.57585-0.0989 0.68636-0.67472 0.0581-0.31992 0.0581-0.76199 0.0581-1.1284 0-0.43043 0-0.82597-0.064-1.1924-0.0873-0.55258-0.41879-0.72127-0.68054-0.72127v-0.12796c0.3141 0 0.75616 0.12215 1.0063 0.66891 0.18613 0.40134 0.21522 0.84923 0.21522 1.3087z"/><path d="m364.04 682.98 0.0465 0.0175 0.1224 0.0525 0.0407 0.0233 0.0816 0.0466 0.0349 0.0232 0.0407 0.0233 0.0349 0.0291 0.0291 0.0291 0.0698 0.0582 0.0291 0.0291 0.0291 0.0349 0.0232 0.0291 0.0291 0.0349 0.0699 0.1047 0.0174 0.0408 0.0233 0.0349 0.0175 0.0408 0.0116 0.0408 0.0175 0.0349 0.0116 0.0408 6e-3 0.0407 0.0116 0.0408 6e-3 0.0408 6e-3 0.0465v0.0408l6e-3 0.0408c0 0.47696-0.51187 0.79688-1.0703 0.79688-0.58748 0-1.0237-0.34318-1.0237-0.78524 0-0.19195 0.12796-0.30247 0.29665-0.30247 0.17449 0 0.29083 0.12797 0.29083 0.29665 0 0.29083-0.27339 0.29083-0.36063 0.29083 0.18031 0.2792 0.56421 0.35481 0.7736 0.35481 0.23849 0 0.55841-0.12215 0.55841-0.64564 0-0.0698-0.0116-0.40134-0.16287-0.65727-0.1745-0.2792-0.37227-0.29666-0.51768-0.30247-0.0465-6e-3 -0.18614-0.0175-0.22685-0.0175-0.0465-6e-3 -0.0873-0.0116-0.0873-0.0698s0.0407-0.0582 0.1396-0.0582h0.25593c0.47697 0 0.69219-0.39553 0.69219-0.95975 0-0.78525-0.40135-0.95392-0.65729-0.95392-0.25011 0-0.68054 0.0989-0.88413 0.43624 0.20359-0.0291 0.37808 0.0989 0.37808 0.31992 0 0.2094-0.15123 0.31991-0.31991 0.31991-0.1396 0-0.32573-0.0756-0.32573-0.33155 0-0.52931 0.54094-0.90739 1.1691-0.90739 0.70963 0 1.2331 0.5235 1.2331 1.1168 0 0.47115-0.36063 0.92485-0.96556 1.0412z"/><path d="m356.88 666.03v0.2568l-0.01 0.1712-5e-3 0.0799-5e-3 0.0856-0.0114 0.0856-0.0171 0.0799-0.0114 0.0856-0.0228 0.0799-0.0171 0.0856-0.0855 0.2397-0.0342 0.08c-0.26271 0.54258-0.73104 0.63396-0.97092 0.63396-0.34267 0-0.75388-0.1485-0.98804-0.67393-0.18277-0.39408-0.21132-0.83385-0.21132-1.2908 0-0.42263 0.0228-0.93665 0.25701-1.365 0.23987-0.46262 0.65679-0.57113 0.93664-0.57113v0.11993c-0.22273 0-0.5597 0.14279-0.6625 0.69107-0.0628 0.33697-0.0628 0.86241-0.0628 1.1937 0 0.36551 0 0.73675 0.0457 1.0452 0.10852 0.67393 0.53686 0.72533 0.67964 0.72533 0.18848 0 0.56542-0.1028 0.67394-0.66251 0.0571-0.31983 0.0571-0.74817 0.0571-1.108 0-0.42264 0-0.811-0.0628-1.1708-0.0856-0.54256-0.41122-0.71391-0.66823-0.71391v-0.11993c0.30842 0 0.74247 0.11422 0.99377 0.6568 0.18275 0.38836 0.20561 0.83384 0.20561 1.2793z"/><path d="m358.2 664.52v0.0456l-5e-3 0.0171v0.0114l-5e-3 0.0171-0.01 0.0228-5e-3 0.0171-0.01 0.0228-0.0114 0.0114-5e-3 0.0114-0.0114 0.0114-5e-3 0.0114-0.0342 0.0342-0.0114 5e-3 -0.0114 0.0114-0.0228 0.01-0.0171 5e-3 -0.0114 0.0114h-0.0114l-0.0171 5e-3 -0.0114 5e-3 -0.0171 5e-3h-0.0285l-0.0171 5e-3h-0.0171c-0.15991 0-0.29699-0.13707-0.29699-0.3027s0.13708-0.3027 0.29699-0.3027c0.16563 0 0.3027 0.13707 0.3027 0.3027z"/><path d="m361.3 666.03v0.2568l-0.01 0.1712-5e-3 0.0799-0.0228 0.1712-0.0114 0.0799-0.0171 0.0856-0.0171 0.0799-0.0228 0.0856-0.0228 0.0799-0.057 0.1598-0.0171 0.04-0.0228 0.04c-0.257 0.54258-0.72533 0.63396-0.9652 0.63396-0.34268 0-0.7596-0.1485-0.99376-0.67393-0.18277-0.39408-0.20561-0.83385-0.20561-1.2908 0-0.42263 0.0171-0.93665 0.2513-1.365 0.24558-0.46262 0.6625-0.57113 0.94235-0.57113v0.11993c-0.22273 0-0.5597 0.14279-0.6625 0.69107-0.0628 0.33697-0.0628 0.86241-0.0628 1.1937 0 0.36551 0 0.73675 0.0457 1.0452 0.10851 0.67393 0.53687 0.72533 0.67964 0.72533 0.18848 0 0.56542-0.1028 0.67393-0.66251 0.0571-0.31983 0.0571-0.74817 0.0571-1.108 0-0.42264 0-0.811-0.0628-1.1708-0.0856-0.54256-0.41122-0.71391-0.66823-0.71391v-0.11993c0.30842 0 0.73676 0.11422 0.98806 0.6568 0.18275 0.38836 0.21131 0.83384 0.21131 1.2793z"/><path d="m364.15 666.03v0.2568l-0.01 0.1712-5e-3 0.0799-0.0114 0.0856-5e-3 0.0856-0.0171 0.0799-0.0114 0.0856-0.0228 0.0799-0.0171 0.0856-0.0855 0.2397-0.0342 0.08c-0.26272 0.54258-0.73104 0.63396-0.97092 0.63396-0.34268 0-0.7596-0.1485-0.98805-0.67393-0.18276-0.39408-0.21132-0.83385-0.21132-1.2908 0-0.42263 0.0228-0.93665 0.25701-1.365 0.23987-0.46262 0.6568-0.57113 0.93665-0.57113v0.11993c-0.22274 0-0.5597 0.14279-0.66251 0.69107-0.0628 0.33697-0.0628 0.86241-0.0628 1.1937 0 0.36551 0 0.73675 0.0457 1.0452 0.10852 0.67393 0.53686 0.72533 0.67965 0.72533 0.18847 0 0.56542-0.1028 0.67393-0.66251 0.0571-0.31983 0.0571-0.74817 0.0571-1.108 0-0.42264 0-0.811-0.0628-1.1708-0.0856-0.54256-0.41121-0.71391-0.66822-0.71391v-0.11993c0.30841 0 0.74247 0.11422 0.99377 0.6568 0.18275 0.38836 0.2056 0.83384 0.2056 1.2793z"/><path d="m366.03 666.22 0.0457 0.0171 0.08 0.0342 0.0457 0.0171 0.04 0.0228 0.0342 0.0171 0.04 0.0228 0.0343 0.0285 0.04 0.0228 0.0343 0.0285 0.0342 0.0285 0.0285 0.0228 0.0912 0.0912 0.0285 0.0343 0.0228 0.0343 0.0228 0.0342 0.0456 0.0686 0.0228 0.04 0.0171 0.0343 0.0171 0.04 0.0114 0.0343 0.0171 0.04 0.0228 0.08 0.015 0.12 5e-3 0.0457v0.04c0 0.46262-0.50258 0.78245-1.0509 0.78245-0.57683 0-1.0052-0.34268-1.0052-0.77102 0-0.18847 0.12565-0.29699 0.28556-0.29699 0.17705 0 0.29128 0.12565 0.29128 0.29128 0 0.28556-0.26272 0.28556-0.34839 0.28556 0.17705 0.27414 0.54828 0.34839 0.75389 0.34839 0.23416 0 0.54828-0.12564 0.54828-0.63395 0-0.0685-0.0114-0.39979-0.15992-0.64538-0.17133-0.27413-0.36552-0.29127-0.5083-0.29698-0.0457-5e-3 -0.18276-0.0171-0.22274-0.0171-0.0457-5e-3 -0.0856-0.0114-0.0856-0.0685 0-0.0628 0.04-0.0628 0.13707-0.0628h0.25129c0.46833 0 0.67965-0.38265 0.67965-0.94235 0-0.77102-0.39409-0.93665-0.64537-0.93665-0.24559 0-0.67394 0.0971-0.86812 0.43406 0.19418-0.0285 0.37123 0.0971 0.37123 0.30841 0 0.2056-0.14849 0.31983-0.31412 0.31983-0.13707 0-0.31983-0.0799-0.31983-0.32554 0-0.51973 0.52544-0.89096 1.148-0.89096 0.69678 0 1.2165 0.51401 1.2165 1.0908 0 0.46832-0.35981 0.90809-0.95378 1.028z"/></g><path d="m244.87 690.21v0.15l-0.01 0.14v0.15l-0.03 0.45-0.01 0.14-0.02 0.15-0.03 0.15-0.02 0.14-0.04 0.15-0.03 0.14-0.15 0.42-0.06 0.14c-0.46 0.95-1.28 1.11-1.7 1.11-0.6 0-1.32-0.26-1.73-1.18-0.32-0.69-0.37-1.46-0.37-2.25 0-0.75 0.04-1.64 0.45-2.4 0.42-0.8 1.15-1 1.64-1v0.22c-0.39 0-0.97 0.24-1.16 1.2-0.1 0.59-0.1 1.51-0.1 2.09 0 0.64 0 1.3 0.07 1.83 0.19 1.18 0.94 1.27 1.19 1.27 0.33 0 0.99-0.17 1.18-1.16 0.1-0.55 0.1-1.31 0.1-1.94 0-0.74 0-1.42-0.11-2.05-0.15-0.95-0.72-1.24-1.17-1.24v-0.22c0.54 0 1.3 0.21 1.74 1.15 0.32 0.69 0.37 1.46 0.37 2.25z"/><path d="m247.17 687.55v0.08l-0.01 0.03v0.02l-0.01 0.03-0.01 0.02-0.01 0.03-0.02 0.04-0.01 0.03-0.02 0.02-0.01 0.02-0.02 0.02-0.01 0.02-0.02 0.01-0.04 0.04-0.02 0.01-0.02 0.02-0.02 0.01-0.02 0.02-0.03 0.01-0.04 0.02-0.03 0.01h-0.02l-0.03 0.01h-0.02l-0.03 0.01h-0.05c-0.29 0-0.53-0.24-0.53-0.53s0.24-0.52 0.53-0.52 0.52 0.23 0.52 0.52z"/><path d="m256.88 693.56v0.15l-0.01 0.01v0.03l-0.01 0.01v0.02h-0.01v0.01h-0.01l-0.01 0.01h-0.01l-0.01 0.01h-0.02v0.01h-0.14c-0.61996-0.63-1.5-0.63-1.82-0.63v-0.31c0.21 0 0.79 0 1.31 0.26v-5.15c0-0.36-0.03-0.48-0.93-0.48h-0.32v-0.3c0.35 0.03 1.22 0.03 1.62 0.03s1.27 0 1.62-0.03v0.3h-0.32c-0.9 0-0.93 0.11-0.93 0.48z"/><path d="m253.28 690.22v0.15l-0.01 0.14v0.15l-0.03 0.45-0.01 0.14-0.02 0.15-0.03 0.15-0.02 0.14-0.04 0.15-0.03 0.14-0.15 0.42-0.06 0.14c-0.46 0.95-1.28 1.11-1.7 1.11-0.6 0-1.32-0.26-1.73-1.18-0.32-0.69-0.37-1.46-0.37-2.25 0-0.75 0.04-1.64 0.45-2.4 0.42-0.8 1.15-1 1.64-1v0.22c-0.39 0-0.97 0.24-1.16 1.2-0.1 0.59-0.1 1.51-0.1 2.09 0 0.64 0 1.3 0.07 1.83 0.19 1.18 0.94 1.27 1.19 1.27 0.33 0 0.99-0.17 1.18-1.16 0.1-0.55 0.1-1.31 0.1-1.94 0-0.74 0-1.42-0.11-2.05-0.15-0.95-0.72-1.24-1.17-1.24v-0.22c0.54 0 1.3 0.21 1.74 1.15 0.32 0.69 0.37 1.46 0.37 2.25z"/></g></g>
  <g fill="none" stroke="#000">
   <g stroke-opacity=".57971" stroke-width=".9">
    <path d="m17.096 71.916c22.445 15.767 46.049 42.232-1.0976 69.524" marker-end="url(#c)"/>
    <path d="m108.77 72.78 0.0258 20.985" marker-end="url(#b)"/>
    <path d="m109.94 120.55 0.10335 20.199" marker-end="url(#a)"/>
   </g>
   <rect x="247.45" y="24.076" width="44.058" height="49.395" stroke-linecap="square" stroke-linejoin="bevel" stroke-opacity=".48235" stroke-width=".7"/>
   <path d="m247.46 36.973h43.916" stroke-opacity=".48235" stroke-width=".7"/>
  </g>
  <g>
   <text x="249.16391" y="32.654339" fill="#000000" font-family="Verdana" font-size="8.7182px" letter-spacing="0px" stroke-width=".21796" word-spacing="0px" style="line-height:1.25" xml:space="preserve"><tspan x="249.16391" y="32.654339" font-family="'Helvetica Neue LT'" font-size="8.7182px" stroke-width=".21796" style="font-feature-settings:normal;font-variant-caps:normal;font-variant-ligatures:normal;font-variant-numeric:normal">categories</tspan></text>
   <g transform="matrix(.46792 0 0 -.46792 123.03 367.99)" fill-rule="evenodd" stroke="#000" stroke-miterlimit="10.433" stroke-width="0" ns0:preamble="default_packages.tex" ns0:text='$$0="Physics"$$\n$$1="Lyrics"$$\n$$\\cdots$$\n$$N="Religion"$$\n' xml:space="preserve"><path d="m291.34 702.4v0.2l0.0125 0.075 0.0125 0.0625v0.0625l0.025 0.125 0.0125 0.05 0.025 0.0625 0.0125 0.05 0.025 0.05 0.0125 0.05 0.075 0.15 0.025 0.0375 0.0375 0.0375 0.025 0.0375 0.0375 0.0375 0.025 0.0375 0.15 0.1 0.05 0.025 0.075 0.025 0.1 0.025h0.1c0.43768 0 0.66277-0.32513 0.66277-0.65027 0-0.32513-0.22509-0.66277-0.66277-0.66277-0.2501 0-0.42518 0.12505-0.50021 0.20008-0.0125 0.025-0.025 0.0375-0.0375 0.0375-0.025 0-0.025-0.17508-0.025-0.2251 0-0.40016 0.0875-1.288 0.87536-2.0634 0.13755-0.15006 0.13755-0.17507 0.13755-0.21258 0-0.0875-0.05-0.13756-0.12505-0.13756-0.12505 0-1.163 0.95039-1.163 2.4135z"/><path d="m288.91 702.4v0.075l0.0125 0.0625v0.1375l0.05 0.25 0.0125 0.05 0.0125 0.0625 0.025 0.05 0.0125 0.05 0.1 0.2 0.05 0.075 0.0375 0.0375 0.025 0.0375 0.0375 0.0375 0.1875 0.125 0.0375 0.0125 0.1 0.025 0.0375 0.0125h0.1c0.43768 0 0.66277-0.32513 0.66277-0.65027 0-0.32513-0.21258-0.66277-0.66277-0.66277-0.2501 0-0.42517 0.12505-0.5002 0.20008-0.0125 0.025-0.025 0.0375-0.0375 0.0375-0.025 0-0.025-0.17508-0.025-0.2251 0-0.40016 0.0875-1.288 0.87536-2.0634 0.15006-0.15006 0.15006-0.17507 0.15006-0.21258 0-0.0875-0.0625-0.13756-0.12505-0.13756-0.13756 0-1.1755 0.95039-1.1755 2.4135z"/><path d="m296.62 698.5 0.0375 0.32513 0.85035 3.3764c0.11255 0.43768 0.13756 0.4752 0.67528 0.4752h1.2005c1.0379 0 1.7007-0.33764 1.7007-1.2005 0-0.47519-0.25011-1.5506-0.73781-2.0008-0.62525-0.55023-1.3756-0.65027-1.9258-0.65027h-1.7632l-0.0375-0.32513h2.1134c1.7757 0 3.5014 1.288 3.5014 2.6886 0 0.95039-0.81283 1.8758-2.451 1.8758h-4.0392c-0.2376 0-0.37516 0-0.37516-0.2376 0-0.15006 0.11255-0.15006 0.36265-0.15006 0.16257 0 0.38766-0.0125 0.53773-0.025 0.20008-0.025 0.27511-0.0625 0.27511-0.20009 0-0.05-0.0125-0.0875-0.05-0.23759l-1.6757-6.6652c-0.12505-0.4877-0.15006-0.58774-1.138-0.58774-0.21259 0-0.35014 0-0.35014-0.2376 0-0.15006 0.15006-0.15006 0.18757-0.15006 0.35015 0 1.238 0.0375 1.5882 0.0375 0.2626 0 0.53772-0.0125 0.80033-0.0125 0.27511 0 0.55022-0.025 0.80033-0.025 0.10004 0 0.2626 0 0.2626 0.25011 0 0.13755-0.11254 0.13755-0.35014 0.13755-0.46269 0-0.81284 0-0.81284 0.22509 0 0.075 0.025 0.13756 0.0375 0.21259z"/><path d="m306.15 703.07v0.0125l-0.0125 0.0125v0.025l-0.0125 0.0125v0.0125h-0.0125v0.0125h-0.0125v0.0125h-0.0125v0.0125h-0.025l-0.0125 0.0125h-0.0625c-0.28761 0-1.2005-0.10004-1.5256-0.11255-0.10005-0.0125-0.2376-0.025-0.2376-0.2501 0-0.15006 0.11254-0.15006 0.30012-0.15006 0.60025 0 0.62526-0.0875 0.62526-0.21259l-0.0375-0.2501-1.8132-7.153c-0.05-0.17507-0.05-0.20008-0.05-0.27511 0-0.28762 0.2501-0.35014 0.36265-0.35014 0.20008 0 0.40016 0.15006 0.46269 0.32513l0.23759 0.93789 0.27512 1.1255c0.075 0.27511 0.15006 0.55023 0.21259 0.82534 0.025 0.075 0.12505 0.4877 0.13755 0.56273 0.0375 0.11255 0.42518 0.81283 0.85035 1.138 0.27511 0.20008 0.66277 0.43768 1.188 0.43768 0.53772 0 0.67528-0.42518 0.67528-0.86286 0-0.67527-0.4752-2.0258-0.76282-2.7886-0.11254-0.28762-0.16256-0.43768-0.16256-0.67528 0-0.58774 0.43768-1.0254 1.0129-1.0254 1.1755 0 1.6382 1.8132 1.6382 1.9133 0 0.12505-0.11255 0.12505-0.15006 0.12505-0.12505 0-0.12505-0.0375-0.18758-0.22509-0.18758-0.66277-0.58774-1.5381-1.2755-1.5381-0.21258 0-0.30012 0.12505-0.30012 0.41267 0 0.30012 0.11255 0.60025 0.22509 0.87536 0.20009 0.53772 0.76282 2.0133 0.76282 2.7386 0 0.80033-0.50021 1.3255-1.4381 1.3255-0.77532 0-1.3756-0.38766-1.8383-0.95039z"/><path d="m315.81 699.32 0.0125 0.0125v0.025l0.0125 0.0125v0.0375l0.0125 0.0125v0.05l0.0125 0.0125v0.1125c0 0.22509-0.17507 0.33764-0.36264 0.33764-0.12506 0-0.32514-0.075-0.43768-0.26261-0.025-0.0625-0.12506-0.43768-0.17508-0.66278-0.0875-0.32513-0.17507-0.66277-0.2501-1.0004l-0.55023-2.2384c-0.05-0.18758-0.58774-1.0504-1.4131-1.0504-0.63776 0-0.77532 0.55023-0.77532 1.0004 0 0.57524 0.21259 1.3506 0.63777 2.4385 0.20008 0.51272 0.2501 0.65027 0.2501 0.90038 0 0.55022-0.40017 1.0129-1.0254 1.0129-1.188 0-1.6507-1.8007-1.6507-1.9133 0-0.12505 0.12505-0.12505 0.15006-0.12505 0.12505 0 0.13756 0.025 0.20008 0.2251 0.33764 1.163 0.83785 1.5381 1.263 1.5381 0.10004 0 0.31263 0 0.31263-0.40017 0-0.30012-0.12505-0.62525-0.21259-0.86285-0.5002-1.313-0.7253-2.0258-0.7253-2.6136 0-1.1004 0.78783-1.4756 1.5256-1.4756 0.4877 0 0.91288 0.21258 1.263 0.56273-0.16257-0.65027-0.31263-1.263-0.81283-1.9133-0.32514-0.42517-0.80033-0.78782-1.3756-0.78782-0.17507 0-0.7378 0.0375-0.95039 0.52521 0.20008 0 0.36265 0 0.53772 0.15006 0.12505 0.11255 0.2501 0.27512 0.2501 0.51271 0 0.37516-0.33764 0.42518-0.46269 0.42518-0.28761 0-0.70028-0.20008-0.70028-0.80033 0-0.62526 0.55022-1.088 1.3255-1.088 1.288 0 2.5636 1.138 2.9137 2.5386z"/><path d="m321.18 699.23-0.0375-0.0125h-0.0625l-0.025-0.0125h-0.0375l-0.075-0.0375-0.0375-0.0125-0.075-0.0375-0.0375-0.0375-0.025-0.0125-0.025-0.025-0.0125-0.025-0.025-0.0125-0.025-0.05-0.025-0.0125-0.0375-0.075v-0.025l-0.025-0.05v-0.025l-0.0125-0.025v-0.075c0-0.17508 0.11255-0.36265 0.38766-0.36265s0.57524 0.21258 0.57524 0.70028c0 0.55023-0.53773 1.0629-1.4881 1.0629-1.6507 0-2.1009-1.263-2.1009-1.8132 0-0.9629 0.91287-1.1505 1.2755-1.2255 0.65027-0.12505 1.3005-0.26261 1.3005-0.95039 0-0.32513-0.28762-1.3631-1.7882-1.3631-0.17507 0-1.1255 0-1.4131 0.66277 0.4752-0.0625 0.77532 0.30013 0.77532 0.65027 0 0.28762-0.18758 0.43768-0.46269 0.43768-0.31263 0-0.68778-0.26261-0.68778-0.82534 0-0.70029 0.70029-1.2005 1.7757-1.2005 2.0258 0 2.5135 1.5006 2.5135 2.0634 0 0.45018-0.2376 0.76281-0.38766 0.90037-0.33764 0.35014-0.70029 0.41267-1.2505 0.52521-0.45019 0.10004-0.95039 0.18758-0.95039 0.75031 0 0.36265 0.30012 1.113 1.4006 1.113 0.31263 0 0.93789-0.0875 1.1255-0.56273z"/><path d="m325.69 702.34v0.0625l-0.0125 0.025v0.05l-0.0125 0.0125v0.025l-0.0125 0.0125-0.0125 0.025-0.0125 0.0125-0.0125 0.025v0.0125l-0.1125 0.1125h-0.025l-0.0125 0.0125-0.025 0.0125-0.0125 0.0125h-0.025l-0.025 0.0125h-0.025l-0.025 0.0125h-0.05l-0.025 0.0125h-0.025c-0.33764 0-0.67528-0.32513-0.67528-0.66277 0-0.2376 0.17507-0.45018 0.4752-0.45018 0.28762 0 0.66277 0.28761 0.66277 0.66277z"/><path d="m324.74 697.67 0.0125 0.0375 0.0125 0.025 0.0125 0.0375 0.05 0.1v0.025l0.0125 0.0125 0.0125 0.025v0.025l0.0125 0.0125v0.025l0.0125 0.0125 0.0375 0.1125 0.0125 0.0125v0.025l0.0125 0.0125v0.025l0.0125 0.0125 0.0125 0.025v0.025l0.0375 0.075v0.0375l0.0125 0.025c0.10004 0.25011 0.16257 0.42518 0.16257 0.66278 0 0.55022-0.40017 1.0129-1.0254 1.0129-1.163 0-1.6382-1.8007-1.6382-1.9133 0-0.12505 0.12505-0.12505 0.15006-0.12505 0.12506 0 0.13756 0.025 0.20009 0.2251 0.33764 1.163 0.83784 1.5381 1.263 1.5381 0.0875 0 0.30012 0 0.30012-0.40017 0-0.2501-0.0875-0.51271-0.13756-0.63776-0.10004-0.32513-0.65026-1.7632-0.85035-2.301-0.12505-0.32513-0.28762-0.7378-0.28762-0.9879 0-0.58774 0.42518-1.0254 1.0129-1.0254 1.1755 0 1.6382 1.8007 1.6382 1.9133 0 0.12505-0.11255 0.12505-0.15007 0.12505-0.12505 0-0.12505-0.0375-0.18757-0.22509-0.2251-0.77532-0.63777-1.5381-1.2755-1.5381-0.21259 0-0.28762 0.12505-0.28762 0.41267 0 0.30012 0.075 0.47519 0.35015 1.238z"/><path d="m331.38 699.3-0.0375-0.0125h-0.2l-0.0125-0.0125h-0.0375l-0.0125-0.0125h-0.0375l-0.0125-0.0125h-0.0125l-0.025-0.0125-0.0125-0.0125h-0.0125l-0.025-0.0125-0.025-0.025-0.025-0.0125-0.025-0.025-0.025-0.0125-0.0125-0.0125c-0.20008-0.18758-0.22509-0.40017-0.22509-0.4877 0-0.30013 0.22509-0.43768 0.46269-0.43768 0.36265 0 0.70029 0.30012 0.70029 0.80032 0 0.60025-0.58774 1.0629-1.4756 1.0629-1.6757 0-3.3389-1.7757-3.3389-3.5264 0-1.1255 0.7253-2.1009 2.0258-2.1009 1.7757 0 2.8262 1.313 2.8262 1.4631 0 0.075-0.075 0.16257-0.15007 0.16257-0.0625 0-0.0875-0.025-0.16256-0.12506-0.98791-1.2255-2.351-1.2255-2.501-1.2255-0.77532 0-1.113 0.61275-1.113 1.3506 0 0.51271 0.2501 1.7132 0.67528 2.4885 0.38766 0.70029 1.0629 1.238 1.7507 1.238 0.42517 0 0.90037-0.16257 1.0754-0.4877z"/><path d="m336.7 699.23-0.025-0.0125h-0.0625l-0.0375-0.0125h-0.025l-0.0375-0.0125-0.15-0.075-0.025-0.025-0.05-0.025-0.0125-0.025-0.0625-0.0625-0.0125-0.025-0.0125-0.0125-0.0625-0.125v-0.025l-0.0125-0.025v-0.05l-0.0125-0.025v-0.025c0-0.17508 0.11254-0.36265 0.38766-0.36265 0.27511 0 0.57523 0.21258 0.57523 0.70028 0 0.55023-0.53772 1.0629-1.4881 1.0629-1.6382 0-2.1009-1.263-2.1009-1.8132 0-0.9629 0.92538-1.1505 1.288-1.2255 0.65026-0.12505 1.288-0.26261 1.288-0.95039 0-0.32513-0.28762-1.3631-1.7757-1.3631-0.17508 0-1.138 0-1.4256 0.66277 0.47519-0.0625 0.78782 0.30013 0.78782 0.65027 0 0.28762-0.20008 0.43768-0.46269 0.43768-0.32513 0-0.70029-0.26261-0.70029-0.82534 0-0.70029 0.7128-1.2005 1.7882-1.2005 2.0133 0 2.501 1.5006 2.501 2.0634 0 0.45018-0.2376 0.76281-0.38766 0.90037-0.33764 0.35014-0.70029 0.41267-1.2505 0.52521-0.43768 0.10004-0.93789 0.18758-0.93789 0.75031 0 0.36265 0.30012 1.113 1.3881 1.113 0.31263 0 0.93789-0.0875 1.1255-0.56273z"/><path d="m339.57 701.89v0.2l-0.0125 0.075v0.0625l-0.0375 0.1875-0.0125 0.05-0.0125 0.0625-0.025 0.05-0.0125 0.05-0.1 0.2-0.05 0.075-0.0375 0.0375-0.025 0.0375-0.0375 0.0375-0.1875 0.125-0.05 0.0125-0.0375 0.0125-0.05 0.0125-0.0375 0.0125h-0.1c-0.43768 0-0.66277-0.32514-0.66277-0.65027s0.21259-0.66277 0.66277-0.66277c0.25011 0 0.42518 0.12505 0.4877 0.20008 0.025 0.025 0.025 0.0375 0.0375 0.0375 0.025 0 0.025-0.17508 0.025-0.2251 0-0.40016-0.075-1.288-0.86286-2.0634-0.15006-0.15006-0.15006-0.17507-0.15006-0.21258 0-0.0875 0.0625-0.13756 0.12505-0.13756 0.13756 0 1.163 0.95039 1.163 2.4135z"/><path d="m341.99 701.89v0.1375l-0.0125 0.0625v0.075l-0.025 0.125v0.0625l-0.025 0.0625-0.0125 0.05-0.0125 0.0625-0.0125 0.05-0.05 0.1-0.0125 0.05-0.05 0.1-0.0375 0.0375-0.05 0.075-0.1 0.1-0.075 0.05-0.05 0.025-0.0375 0.025-0.0375 0.0125-0.05 0.0125-0.0375 0.0125-0.05 0.0125h-0.1c-0.43768 0-0.66278-0.32514-0.66278-0.65027s0.21259-0.66277 0.66278-0.66277c0.2501 0 0.42517 0.12505 0.5002 0.20008 0.0125 0.025 0.025 0.0375 0.0375 0.0375 0.025 0 0.025-0.17508 0.025-0.2251 0-0.40016-0.0875-1.288-0.87536-2.0634-0.15006-0.15006-0.15006-0.17507-0.15006-0.21258 0-0.0875 0.0625-0.13756 0.12505-0.13756 0.13755 0 1.1755 0.95039 1.1755 2.4135z"/><path d="m294.79 682.27 0.0468 0.1396v0.0349l0.0117 0.0234 0.0117 0.0349 0.0117 0.0233 0.0233 0.0233 0.0234 0.0466 0.0699 0.0699 0.0232 0.0117 0.0233 0.0117 0.0349 0.0233 0.0698 0.0234 0.0466 0.0117 0.0234 0.0117h0.0466l0.0232 0.0117h0.0233l0.0233 0.0117h0.0931l0.0233 0.0117h0.1047l0.0349 0.0117h0.4426c0.34956 0 0.44278 0 0.44278 0.22139 0 0.13983-0.12818 0.13983-0.18644 0.13983-0.38453 0-1.34-0.0349-1.7245-0.0349-0.34958 0-1.1885 0.0349-1.5381 0.0349-0.0815 0-0.22139 0-0.22139-0.23305 0-0.12817 0.10487-0.12817 0.32626-0.12817 0.0233 0 0.24471 0 0.44279-0.0233 0.20974-0.0233 0.31462-0.0349 0.31462-0.18644 0-0.0466-0.0234-0.0699-0.0583-0.20973l-1.5498-6.2224c-0.11653-0.45445-0.13983-0.54766-1.0604-0.54766-0.19809 0-0.3146 0-0.3146-0.2214 0-0.12818 0.10487-0.12818 0.3146-0.12818h5.3834c0.27965 0 0.2913 0 0.36121 0.1981l0.90889 2.4936c0.0583 0.12817 0.0583 0.15148 0.0583 0.16312 0 0.0466-0.0349 0.12819-0.13982 0.12819-0.10488 0-0.11653-0.0583-0.1981-0.2447-0.39617-1.0604-0.90887-2.3887-2.9247-2.3887h-1.0837c-0.16314 0-0.18644 0-0.25635 0.0117-0.11653 0.0233-0.15149 0.0349-0.15149 0.11653 0 0.0349 0 0.0583 0.0583 0.268z"/><path d="m303.99 679.67v0.0117l0.0117 0.0117v0.0351l0.0117 0.0117v0.0351h0.0117v0.0585h0.0117v0.1052c0 0.20975-0.16314 0.31463-0.33792 0.31463-0.11652 0-0.30296-0.0699-0.40783-0.24471-0.0233-0.0583-0.11653-0.41949-0.16313-0.62922-0.0815-0.29132-0.16313-0.60593-0.23305-0.92054l-0.52436-2.0858c-0.0466-0.17479-0.54766-0.9788-1.3051-0.9788-0.59426 0-0.72245 0.50105-0.72245 0.93219 0 0.53601 0.19809 1.2468 0.59428 2.2722 0.18643 0.47774 0.23304 0.59426 0.23304 0.82731 0 0.52435-0.37287 0.95549-0.95549 0.95549-1.107 0-1.5381-1.6779-1.5381-1.7828 0-0.11653 0.11652-0.11653 0.13983-0.11653 0.11652 0 0.12817 0.0233 0.18643 0.20974 0.31462 1.0837 0.78071 1.4332 1.1769 1.4332 0.0932 0 0.29131 0 0.29131-0.37287 0-0.29131-0.11653-0.58261-0.1981-0.80401-0.4661-1.2352-0.67583-1.8877-0.67583-2.4354 0-1.0254 0.7341-1.375 1.4216-1.375 0.45444 0 0.83896 0.19808 1.1652 0.52435-0.13982-0.60592-0.2913-1.1769-0.74574-1.7945-0.30296-0.38451-0.74575-0.72243-1.2818-0.72243-0.16312 0-0.68748 0.0349-0.88558 0.48939 0.18645 0 0.33793 0 0.50106 0.13983 0.11653 0.0932 0.23305 0.2447 0.23305 0.46609 0 0.36123-0.31462 0.40784-0.43114 0.40784-0.26801 0-0.65253-0.18644-0.65253-0.74575 0-0.58262 0.5127-1.0138 1.2351-1.0138 1.2002 0 2.3887 1.0487 2.715 2.3654z"/><path d="m305.47 675.94v-0.035l-0.0117-0.0117v-0.0233l-0.0117-0.0349v-0.0349l-0.0234-0.0698v-0.0466l-0.0117-0.0349-0.0117-0.0117v-0.0582l-0.0117-0.0117v-0.035l-0.0117-0.0117v-0.0702h-0.0117c0-0.20974 0.16314-0.31461 0.33792-0.31461 0.13983 0 0.34957 0.0932 0.43113 0.32626 0.0233 0.0466 0.41949 1.6197 0.46609 1.8178 0.0932 0.38453 0.30297 1.2002 0.37289 1.5148 0.0466 0.15149 0.37287 0.68749 0.65253 0.94385 0.0932 0.0815 0.43114 0.38452 0.93219 0.38452 0.30295 0 0.47774-0.13982 0.48939-0.13982-0.34956-0.0583-0.60592-0.33793-0.60592-0.64089 0-0.17478 0.12818-0.39617 0.44279-0.39617 0.30296 0 0.62923 0.25634 0.62923 0.67583 0 0.40784-0.36123 0.7574-0.95549 0.7574-0.7574 0-1.2701-0.57096-1.4915-0.89723-0.0932 0.52435-0.51271 0.89723-1.0604 0.89723-0.53601 0-0.7574-0.45444-0.86228-0.66418-0.19808-0.39618-0.34956-1.0837-0.34956-1.1186 0-0.11652 0.11651-0.11652 0.13982-0.11652 0.10488 0 0.12818 0.0117 0.1981 0.268 0.18644 0.81567 0.41948 1.375 0.83897 1.375 0.19808 0 0.36122-0.0932 0.36122-0.53601 0-0.2447-0.0349-0.37288-0.18644-0.96714z"/><path d="m313.31 682.5v0.0816l-0.0117 0.0233v0.0117l-0.0117 0.0233v0.0232l-0.0117 0.0117-0.0117 0.0233v0.0117l-0.0117 0.0233-0.0234 0.0234-0.0117 0.0233-0.0233 0.0117-0.0351 0.0351-0.0233 0.0117-0.0117 0.0117-0.0466 0.0234-0.0117 0.0117h-0.0233l-0.0233 0.0117h-0.0233l-0.0117 0.0117h-0.1048c-0.31461 0-0.61758-0.30296-0.61758-0.60592 0-0.22141 0.16314-0.41949 0.43114-0.41949 0.26801 0 0.61758 0.268 0.61758 0.61757z"/><path d="m312.43 678.14 0.0233 0.0349 0.0117 0.0233 0.0117 0.0232v0.0349l0.0351 0.0699 0.0117 0.0117v0.0233l0.0117 0.0233 0.0117 0.0117v0.0233l0.0117 0.0117v0.0233l0.0117 0.0349 0.0117 0.0233 0.0117 0.0349 0.0117 0.0233v0.0117l0.0117 0.0233 0.0117 0.0117v0.0233l0.0117 0.0233v0.0233l0.0468 0.0932v0.0349c0.0932 0.22139 0.15148 0.38453 0.15148 0.60592 0 0.52435-0.37287 0.95549-0.94384 0.95549-1.0953 0-1.5381-1.6779-1.5381-1.7828 0-0.11653 0.11652-0.11653 0.13983-0.11653 0.11652 0 0.12817 0.0233 0.18644 0.20974 0.31461 1.0837 0.7807 1.4332 1.1769 1.4332 0.0932 0 0.2913 0 0.2913-0.37287 0-0.2447-0.0815-0.4894-0.12818-0.59427-0.0932-0.30297-0.61757-1.6546-0.80401-2.144-0.11651-0.30297-0.268-0.68749-0.268-0.93219 0-0.54766 0.39618-0.94384 0.9555-0.94384 1.0837 0 1.5148 1.6779 1.5148 1.7828 0 0.11652-0.10486 0.11652-0.13983 0.11652-0.11651 0-0.11651-0.0349-0.17478-0.20974-0.20973-0.7341-0.58262-1.4332-1.1769-1.4332-0.1981 0-0.27966 0.11652-0.27966 0.37287 0 0.29131 0.0699 0.45445 0.33792 1.1652z"/><path d="m318.61 679.66h-0.163l-0.0349-0.0117h-0.0467l-0.0234-0.0117h-0.0234l-0.0233-0.0117h-0.0117l-0.0117-0.0117h-0.0233l-0.0234-0.0234h-0.0233l-0.0351-0.0351-0.0233-0.0117-0.0467-0.0467c-0.18643-0.17478-0.20974-0.37287-0.20974-0.45443 0-0.26801 0.20974-0.39619 0.43114-0.39619 0.33793 0 0.65254 0.27966 0.65254 0.73411 0 0.57096-0.54766 1.0021-1.375 1.0021-1.5614 0-3.1112-1.6546-3.1112-3.286 0-1.0487 0.67584-1.9576 1.8877-1.9576 1.6546 0 2.6334 1.2235 2.6334 1.3633 0 0.0699-0.0699 0.15148-0.13983 0.15148-0.0583 0-0.0815-0.0233-0.15149-0.11652-0.92053-1.1419-2.179-1.1419-2.3188-1.1419-0.7341 0-1.0487 0.55932-1.0487 1.2584 0 0.47775 0.23304 1.5964 0.62922 2.3188 0.36122 0.65253 1.0021 1.1536 1.6313 1.1536 0.39618 0 0.83897-0.15149 1.0021-0.4661z"/><path d="m323.58 679.59h-0.0582l-0.0233-0.0117h-0.0349l-0.0233-0.0117h-0.0233l-0.0349-0.0117-0.0932-0.0468-0.0233-0.0233-0.0466-0.0234-0.0117-0.0233-0.0233-0.0117-0.0233-0.0233-0.0117-0.0233-0.035-0.035-0.0351-0.0699-0.0117-0.0117-0.0117-0.0233v-0.0233l-0.0234-0.0466v-0.0699l-0.0117-0.0233c0-0.16314 0.10488-0.33792 0.36123-0.33792 0.25636 0 0.536 0.1981 0.536 0.64088 0 0.52436-0.50104 1.0021-1.375 1.0021-1.5381 0-1.9692-1.1885-1.9692-1.6896 0-0.90888 0.86228-1.0837 1.2002-1.1536 0.60592-0.11651 1.2118-0.24469 1.2118-0.87391 0-0.30297-0.268-1.2701-1.6663-1.2701-0.16312 0-1.0604 0-1.3284 0.60592 0.44279-0.0583 0.7341 0.29131 0.7341 0.61758 0 0.26801-0.18645 0.40783-0.43114 0.40783-0.30296 0-0.65253-0.24469-0.65253-0.76905 0-0.65253 0.66418-1.1186 1.6663-1.1186 1.8877 0 2.3305 1.3983 2.3305 1.9226 0 0.40783-0.2214 0.69913-0.36123 0.83896-0.30296 0.32627-0.64088 0.38454-1.1536 0.4894-0.41948 0.0932-0.88557 0.17478-0.88557 0.69914 0 0.32627 0.27966 1.0371 1.3051 1.0371 0.29131 0 0.86227-0.0815 1.0371-0.53601z"/><path d="m326.26 682.08v0.0583l-0.0117 0.0699v0.0583l-0.0117 0.0583v0.0699l-0.0117 0.0583-0.0117 0.0466-0.0234 0.1166-0.0117 0.0466-0.0233 0.0583-0.0117 0.0466-0.0233 0.0466-0.0234 0.0466-0.0233 0.0466-0.0233 0.0349-0.0232 0.0466-0.0233 0.0349-0.0349 0.0349-0.0233 0.0349-0.0349 0.0233-0.0349 0.0349-0.1396 0.0932-0.0466 0.0117-0.0349 0.0117-0.0466 0.0117-0.0349 0.0117h-0.0932c-0.40783 0-0.61757-0.31461-0.61757-0.61757 0-0.29131 0.19809-0.60593 0.61757-0.60593 0.23305 0 0.39618 0.11653 0.46609 0.18644 0.0117 0.0233 0.0233 0.0349 0.0349 0.0349 0.0233 0 0.0233-0.16313 0.0233-0.20974 0-0.37288-0.0815-1.2118-0.81566-1.9226-0.13983-0.13983-0.13983-0.16313-0.13983-0.19808 0-0.0815 0.0583-0.12818 0.11651-0.12818 0.12819 0 1.0953 0.88558 1.0953 2.2489z"/><path d="m328.51 682.08v0.1282l-0.0117 0.0583v0.0583l-0.0117 0.0699v0.0583l-0.0117 0.0466-0.0117 0.0583-0.0233 0.0583-0.0117 0.0466-0.0117 0.0583-0.0233 0.0466-0.0234 0.0466-0.0117 0.0466-0.0233 0.0466-0.0233 0.0349-0.0349 0.0466-0.0233 0.0349-0.0234 0.0349-0.0931 0.0931-0.1047 0.0699-0.0466 0.0233-0.0349 0.0117-0.0466 0.0117-0.0349 0.0117-0.0466 0.0117h-0.0932c-0.40782 0-0.61757-0.31461-0.61757-0.61757 0-0.29131 0.1981-0.60593 0.61757-0.60593 0.23306 0 0.39619 0.11653 0.4661 0.18644 0.0117 0.0233 0.0233 0.0349 0.0349 0.0349 0.0234 0 0.0234-0.16313 0.0234-0.20974 0-0.37288-0.0815-1.2118-0.81567-1.9226-0.13983-0.13983-0.13983-0.16313-0.13983-0.19808 0-0.0815 0.0582-0.12818 0.11652-0.12818 0.12818 0 1.0953 0.88558 1.0953 2.2489z"/><path d="m294.51 659.91v0.0261l-0.0131 0.0392v0.0654l-0.0131 0.0392-0.0131 0.0261-0.0131 0.0392-0.0262 0.0522-0.0131 0.0392-0.0262 0.0522-0.0261 0.0261-0.0131 0.0261-0.0783 0.0783-0.0261 0.0131-0.0261 0.0261-0.0261 0.0131-0.0261 0.0261-0.0522 0.0262-0.0392 0.0131-0.0261 0.0131-0.0392 0.0131-0.0261 0.0131h-0.0392l-0.0261 0.0131h-0.1045c-0.37941 0-0.6934-0.30091-0.6934-0.68032 0-0.3794 0.31399-0.6934 0.6934-0.6934s0.6934 0.314 0.6934 0.6934z"/><path d="m300.31 659.91v0.0653l-0.0131 0.0392v0.0262l-0.0131 0.0392-0.0131 0.0261v0.0392l-0.0262 0.0522-0.0262 0.0392-0.0262 0.0522-0.0261 0.0261-0.0131 0.0261-0.0783 0.0783-0.0261 0.0131-0.0261 0.0261-0.0261 0.0131-0.0261 0.0261-0.0522 0.0262-0.0392 0.0131-0.0522 0.0262-0.0392 0.0131h-0.0392l-0.0261 0.0131h-0.1045c-0.3794 0-0.6934-0.30091-0.6934-0.68032 0-0.3794 0.314-0.6934 0.6934-0.6934 0.37942 0 0.69341 0.314 0.69341 0.6934z"/><path d="m306.09 659.91v0.0653l-0.0131 0.0392v0.0262l-0.0131 0.0392v0.0261l-0.0131 0.0392-0.0262 0.0522-0.0261 0.0392-0.0262 0.0522-0.0261 0.0261-0.0131 0.0261-0.0261 0.0261-0.0262 0.0261-0.0653 0.0653-0.0261 0.0131-0.0392 0.0261-0.0783 0.0393-0.0392 0.0131-0.0261 0.0131-0.0392 0.0131h-0.0261l-0.0392 0.0131h-0.1045c-0.37941 0-0.6934-0.30091-0.6934-0.68032 0-0.3794 0.31399-0.6934 0.6934-0.6934s0.6934 0.314 0.6934 0.6934z"/><path d="m298.11 639.85 0.11146 0.0223 0.10028 0.0334 0.11146 0.0223 0.10027 0.0333 0.11146 0.0445 0.11144 0.0334 0.10028 0.0445 0.11145 0.0445 0.10028 0.0445 0.11146 0.0557 0.10028 0.0557 0.10027 0.0557 0.10028 0.0557 0.0892 0.0669 0.0892 0.0557 0.10027 0.0669 0.078 0.078 0.0892 0.0669 0.078 0.0781 0.078 0.078 0.1338 0.156 0.0557 0.0892 0.0557 0.078 0.0557 0.0892 0.0445 0.10028 0.0334 0.0893 0.0335 0.10028 0.0223 0.0892 0.0223 0.10028 0.0112 0.10027v0.11146c0 0.94739-0.99196 1.6607-2.452 1.6607h-3.1654c-0.22291 0-0.32323 0-0.32323-0.22292 0-0.1226 0.10027-0.1226 0.31208-0.1226 0.0223 0 0.23406 0 0.42354-0.0223 0.20061-0.0223 0.30093-0.0334 0.30093-0.17833 0-0.0445-0.0112-0.078-0.0445-0.21177l-1.4935-5.9406c-0.11146-0.43468-0.13375-0.52385-1.0143-0.52385-0.20063 0-0.30094 0-0.30094-0.22291 0-0.12261 0.13375-0.12261 0.15604-0.12261 0.31209 0 1.0923 0.0335 1.4044 0.0335 0.31207 0 1.1034-0.0335 1.4155-0.0335 0.0892 0 0.22292 0 0.22292 0.22292 0 0.1226-0.10027 0.1226-0.31208 0.1226-0.41239 0-0.72447 0-0.72447 0.20062 0 0.0669 0.0223 0.12261 0.0335 0.18949l0.73561 2.9313h1.3152l-0.0112 0.2452h-1.2372l0.71332 2.8867c0.0669 0.2675 0.10027 0.37896 0.31208 0.4124 0.10027 0.0112 0.45696 0.0112 0.67989 0.0112 0.79134 0 2.0285 0 2.0285-1.1034 0-0.36781-0.17832-1.1369-0.61301-1.5604-0.28979-0.28979-0.88051-0.64645-1.8836-0.64645l0.0112-0.2452c1.0142 0 1.2149-0.62417 1.2149-1.0031 0-0.16719-0.0892-0.5127-0.15605-0.76906-0.078-0.31207-0.17833-0.72446-0.17833-0.93623 0-1.2037 1.3375-1.2037 1.4824-1.2037 0.94738 0 1.3263 1.1257 1.3263 1.2818 0 0.12258-0.12259 0.12258-0.13374 0.12258-0.0892 0-0.1226-0.078-0.13375-0.14489-0.27865-0.82478-0.7579-1.0143-1.0143-1.0143-0.3678 0-0.44582 0.24522-0.44582 0.6799 0 0.3455 0.0669 0.9028 0.11144 1.2595 0.0223 0.15604 0.0445 0.36781 0.0445 0.52384 0 0.84709-0.74676 1.1926-1.0477 1.304z"/><path d="m302.93 638.64h0.3229l0.0557 0.0112h0.1559l0.0557 0.0112h0.0557l0.1226 0.0112 0.1226 0.0223 0.11146 0.0112 0.12261 0.0223 0.12259 0.0223 0.1226 0.0334 0.0557 0.0112 0.0557 0.0223 0.0557 0.0112 0.1671 0.0669c0.78019 0.32323 0.83593 0.98082 0.83593 1.1369 0 0.47927-0.42354 0.93624-1.1926 0.93624v-0.2452c0.52385 0 0.82478-0.3121 0.82478-0.69104 0-1.148-1.7833-1.148-2.2403-1.148h-0.45697c0.43467 1.683 1.5827 1.839 1.8725 1.839v0.2452c-1.2372 0-2.909-1.07-2.909-3.0093 0-1.1257 0.6576-2.0062 1.7387-2.0062 1.5938 0 2.5301 1.1814 2.5301 1.304 0 0.0669-0.0669 0.14491-0.13375 0.14491-0.0557 0-0.078-0.0223-0.14489-0.10028-0.88051-1.1034-2.0954-1.1034-2.2291-1.1034-0.85821 0-0.95853 0.93625-0.95853 1.2818 0 0.13375 0.0112 0.47926 0.17834 1.1592z"/><path d="m308.89 643.65v0.0336l-0.0112 0.0112v0.0112l-0.0224 0.0224v0.0112h-0.0112l-0.0112 0.0112h-0.0112v0.0112h-0.056v0.0112h-0.0224c-0.25635 0-1.07-0.0892-1.3598-0.11145-0.0781-0.0112-0.20063-0.0223-0.20063-0.23407 0-0.1226 0.10029-0.1226 0.2675-0.1226 0.535 0 0.54615-0.10028 0.54615-0.18947l-0.0335-0.22292-1.3932-5.5394c-0.0334-0.11146-0.0557-0.18948-0.0557-0.3678 0-0.63531 0.49041-1.0254 1.0031-1.0254 0.3678 0 0.64645 0.22292 0.83592 0.62415 0.20063 0.42354 0.33438 1.0588 0.33438 1.0811 0 0.11146-0.10028 0.11146-0.13375 0.11146-0.11146 0-0.1226-0.0445-0.15604-0.20063-0.18948-0.71331-0.40124-1.3709-0.84708-1.3709-0.33437 0-0.33437 0.35667-0.33437 0.51271 0 0.2675 0.0223 0.32322 0.0669 0.52384z"/><path d="m312.71 643v0.0781l-0.0112 0.0223v0.0223l-0.0112 0.0112v0.0223l-0.0112 0.0223-0.0112 0.0112-0.0112 0.0223v0.0112l-0.0112 0.0112-0.0112 0.0223-0.0112 0.0112-0.0223 0.0112-0.0112 0.0223-0.0224 0.0224-0.0223 0.0112-0.0112 0.0112-0.0223 0.0112h-0.0223l-0.0112 0.0112-0.0223 0.0112h-0.0223l-0.0223 0.0112h-0.0335l-0.0223 0.0112h-0.0557c-0.30094 0-0.59073-0.28979-0.59073-0.59074 0-0.21177 0.1449-0.40123 0.4124-0.40123 0.25635 0 0.59072 0.25634 0.59072 0.59073z"/><path d="m311.86 638.83 0.0112 0.0335 0.0223 0.0334v0.0223l0.0448 0.0892v0.0223l0.0112 0.0223 0.0112 0.0112v0.0223l0.0112 0.0112 0.0112 0.0223v0.0112l0.0112 0.0334 0.0112 0.0335 0.0112 0.0334 0.0112 0.0112v0.0223l0.0112 0.0112v0.0223l0.0112 0.0223 0.0112 0.0112v0.0223l0.0336 0.0669v0.0334l0.0112 0.0223c0.0892 0.22291 0.14488 0.37895 0.14488 0.59072 0 0.50155-0.35665 0.90279-0.91394 0.90279-1.0366 0-1.4601-1.605-1.4601-1.7053 0-0.11147 0.11147-0.11147 0.13375-0.11147 0.11147 0 0.12261 0.0223 0.17834 0.20063 0.30093 1.0477 0.74676 1.3709 1.1257 1.3709 0.0893 0 0.2675 0 0.2675-0.34552 0-0.23406-0.078-0.46811-0.11146-0.57958-0.10027-0.28978-0.59072-1.5715-0.76905-2.0508-0.11146-0.28979-0.25635-0.64645-0.25635-0.88051 0-0.52384 0.37895-0.91395 0.91395-0.91395 1.0366 0 1.4489 1.605 1.4489 1.7053 0 0.11145-0.10028 0.11145-0.13375 0.11145-0.11147 0-0.11147-0.0334-0.16718-0.20062-0.20062-0.69104-0.56844-1.3709-1.1369-1.3709-0.17834 0-0.25636 0.11146-0.25636 0.36781 0 0.27864 0.0669 0.43468 0.32324 1.1034z"/><path d="m318.61 640.26v0.0224l0.0112 0.0112v0.0335l0.0112 0.0112v0.0559l0.0112 0.0112v0.056c0 0.18948-0.13375 0.28979-0.32323 0.28979-0.11146 0-0.41238-0.0781-0.45698-0.46812-0.20062 0.41239-0.59071 0.69103-1.0365 0.69103l0.0112-0.2452c0.73561 0 0.89166-0.89166 0.89166-0.94739 0-0.0557-0.0223-0.1226-0.0333-0.16718l-0.53499-2.1177c-0.0669-0.27864-0.32323-0.55729-0.56843-0.76906-0.22292-0.20061-0.56844-0.40124-0.89166-0.40124-0.55728 0-0.72447 0.57958-0.72447 1.0254 0 0.52385 0.32322 1.8279 0.62416 2.3963 0.30094 0.54613 0.7802 0.98082 1.2372 0.98082l-0.0112 0.2452c-1.2595 0-2.6415-1.5492-2.6415-3.1431 0-1.0923 0.67988-1.7499 1.4824-1.7499 0.64644 0 1.1703 0.52384 1.2818 0.64644l0.0112-0.0112c-0.23405-0.99198-0.3678-1.4378-0.3678-1.4601-0.0445-0.10028-0.42355-1.2037-1.5938-1.2037-0.21177 0-0.57957 0.0112-0.89165 0.11146 0.33437 0.10027 0.45697 0.3901 0.45697 0.57958 0 0.17833-0.1226 0.3901-0.42353 0.3901-0.24521 0-0.60186-0.20063-0.60186-0.64646 0-0.45696 0.41238-0.66874 1.4824-0.66874 1.3932 0 2.1846 0.85823 2.3518 1.527z"/><path d="m322.24 643-0.0112 0.0223v0.0558l-0.0112 0.0223v0.0335l-0.0224 0.0446v0.0112l-0.0112 0.0223-0.0224 0.0224-0.0112 0.0223-0.0671 0.0671-0.0223 0.0112-0.0112 0.0112-0.0223 0.0112h-0.0112l-0.0446 0.0224h-0.0112l-0.0223 0.0112h-0.0446l-0.0223 0.0112h-0.0446c-0.30093 0-0.60187-0.28979-0.60187-0.59074 0-0.21177 0.15604-0.40123 0.42353-0.40123 0.25636 0 0.59073 0.25634 0.59073 0.59073z"/><path d="m321.39 638.83 0.0112 0.0335 0.0112 0.0334 0.0336 0.0669v0.0223l0.0224 0.0446v0.0223l0.0112 0.0112 0.0112 0.0223v0.0112l0.0112 0.0223v0.0112l0.0112 0.0334 0.0112 0.0335 0.0223 0.0334v0.0112l0.0112 0.0223v0.0112l0.0112 0.0223v0.0223l0.0112 0.0112 0.0112 0.0223v0.0223l0.0224 0.0446 0.0112 0.0334 0.0112 0.0223c0.0893 0.22291 0.14489 0.37895 0.14489 0.59072 0 0.50155-0.35666 0.90279-0.91394 0.90279-1.0477 0-1.4712-1.605-1.4712-1.7053 0-0.11147 0.11146-0.11147 0.13376-0.11147 0.11145 0 0.12259 0.0223 0.17833 0.20063 0.30092 1.0477 0.74676 1.3709 1.1257 1.3709 0.0892 0 0.27865 0 0.27865-0.34552 0-0.23406-0.0781-0.46811-0.12262-0.57958-0.0892-0.28978-0.59072-1.5715-0.76904-2.0508-0.11147-0.28979-0.25637-0.64645-0.25637-0.88051 0-0.52384 0.37897-0.91395 0.91396-0.91395 1.0477 0 1.4601 1.605 1.4601 1.7053 0 0.11145-0.10027 0.11145-0.1449 0.11145-0.10027 0-0.10027-0.0334-0.16718-0.20062-0.18948-0.69104-0.55729-1.3709-1.1257-1.3709-0.18948 0-0.26749 0.11146-0.26749 0.36781 0 0.27864 0.0669 0.43468 0.32322 1.1034z"/><path d="m328.11 639.1v0.11146l-0.0112 0.10027-0.0112 0.10028-0.0112 0.10027-0.0223 0.10027-0.0446 0.1784-0.0334 0.0892-0.0335 0.0892-0.089 0.156-0.0445 0.0781-0.0445 0.0669-0.0557 0.078-0.0557 0.0557-0.0557 0.0669-0.0557 0.0557-0.1338 0.1114-0.0669 0.0445-0.156 0.089-0.0669 0.0445-0.0892 0.0334-0.078 0.0223-0.078 0.0334-0.0892 0.0223-0.0891 0.0112-0.0892 0.0223-0.0892 0.0112h-0.1785c-1.4935 0-2.987-1.5715-2.987-3.1431 0-1.0923 0.74676-1.8725 1.7833-1.8725l0.0112 0.24521c-0.47927 0-0.96968 0.34551-0.96968 1.2037 0 0.54615 0.28979 1.7387 0.64645 2.3072 0.55729 0.85823 1.1926 1.0143 1.5047 1.0143 0.64646 0 0.96968-0.52384 0.96968-1.1926 0-0.43467-0.22292-1.5938-0.6353-2.3072-0.39011-0.64645-1.0031-1.0254-1.5158-1.0254l-0.0112-0.24521c1.4824 0 2.9759 1.527 2.9759 3.1431z"/><path d="m329.26 636.74v-0.0335l-0.0112-0.0223v-0.0112l-0.0112-0.0335-0.0112-0.0334v-0.0445l-0.0112-0.0335-0.0112-0.0334v-0.0334l-0.0112-0.0223v-0.0335l-0.0112-0.0112v-0.0335l-0.0112-0.0112v-0.0448l-0.0112-0.0112v-0.0336c0-0.20062 0.15604-0.30092 0.32323-0.30092 0.13375 0 0.33438 0.0892 0.41239 0.31207 0.0112 0.0223 0.1449 0.54614 0.21177 0.82478l0.2452 0.99198c0.0669 0.24519 0.13375 0.4904 0.18948 0.74675 0.0445 0.18948 0.13375 0.50155 0.14489 0.54614 0.16719 0.34552 0.75791 1.3486 1.8056 1.3486 0.50154 0 0.60185-0.40124 0.60185-0.76906 0-0.69104-0.54614-2.1065-0.72446-2.5858-0.10027-0.24519-0.11147-0.37894-0.11147-0.50156 0-0.52384 0.3901-0.91393 0.91396-0.91393 1.0477 0 1.4601 1.6161 1.4601 1.7053 0 0.11146-0.10027 0.11146-0.13375 0.11146-0.11144 0-0.11144-0.0333-0.16717-0.20062-0.22291-0.74676-0.59073-1.3709-1.1369-1.3709-0.18947 0-0.2675 0.11147-0.2675 0.36781 0 0.27865 0.10028 0.54614 0.20062 0.7802 0.21177 0.59072 0.6799 1.8056 0.6799 2.4409 0 0.73563-0.47927 1.1814-1.2818 1.1814-0.99196 0-1.5381-0.70218-1.7276-0.95852-0.0557 0.62415-0.51271 0.95852-1.0254 0.95852s-0.72447-0.42353-0.83592-0.62416c-0.17834-0.37894-0.33437-1.0365-0.33437-1.0811 0-0.11144 0.11146-0.11144 0.13375-0.11144 0.11145 0 0.12261 0.0112 0.18948 0.25634 0.18946 0.79135 0.41239 1.3152 0.81363 1.3152 0.22292 0 0.34553-0.1449 0.34553-0.50157 0-0.23405-0.0335-0.35665-0.17834-0.93622z"/><path d="m336.65 642.6v0.1895l-0.0112 0.0557v0.0557l-0.0336 0.1671-0.0112 0.0445-0.0223 0.0557-0.0112 0.0445-0.0223 0.0445-0.0112 0.0557-0.0223 0.0334-0.0446 0.089-0.0223 0.0334-0.0334 0.0334-0.0223 0.0335-0.0335 0.0334-0.0223 0.0334-0.0334 0.0223-0.0335 0.0334-0.0334 0.0223-0.0335 0.0112-0.0334 0.0223-0.0445 0.0112-0.0334 0.0223h-0.0445l-0.0445 0.0112h-0.0445l-0.0334 0.0112c-0.3901 0-0.59072-0.30093-0.59072-0.59072s0.18947-0.59072 0.59072-0.59072c0.21176 0 0.36781 0.11146 0.43468 0.17833 0.0112 0.0223 0.0223 0.0334 0.0334 0.0334 0.0223 0 0.0223-0.15604 0.0223-0.20064 0-0.34549-0.078-1.148-0.76904-1.839-0.13375-0.12259-0.13375-0.15602-0.13375-0.17832 0-0.0781 0.0557-0.1226 0.11145-0.1226 0.1226 0 1.0366 0.83593 1.0366 2.14z"/><path d="m338.81 642.6v0.0669l-0.0112 0.0557v0.1226l-0.0448 0.2228-0.0112 0.0445-0.0112 0.0557-0.0223 0.0445-0.0112 0.0445-0.0223 0.0557-0.0223 0.0334-0.0446 0.089-0.0446 0.0668-0.0335 0.0335-0.0223 0.0334-0.0335 0.0334-0.0334 0.0223-0.0334 0.0334-0.0334 0.0223-0.0335 0.0112-0.0335 0.0223-0.0334 0.0112-0.0445 0.0223h-0.0445l-0.0334 0.0112h-0.0445l-0.0445 0.0112c-0.39009 0-0.59072-0.30093-0.59072-0.59072s0.18948-0.59072 0.59072-0.59072c0.22291 0 0.37896 0.11146 0.44583 0.17833 0.0112 0.0223 0.0223 0.0334 0.0335 0.0334 0.0223 0 0.0223-0.15604 0.0223-0.20064 0-0.34549-0.078-1.148-0.78018-1.839-0.13376-0.12259-0.13376-0.15602-0.13376-0.17832 0-0.0781 0.0557-0.1226 0.11145-0.1226 0.12262 0 1.0477 0.83593 1.0477 2.14z"/><g fill-rule="evenodd" stroke-miterlimit="10.433"><path d="m289.6 682.63v0.1865l0.0117 0.0699 0.0117 0.0583v0.0583l0.0234 0.1166 0.0117 0.0466 0.0233 0.0583 0.0117 0.0466 0.0233 0.0466 0.0117 0.0466 0.0699 0.1398 0.0233 0.0349 0.0349 0.0349 0.0232 0.0349 0.0349 0.0349 0.0233 0.0349 0.0349 0.0232 0.1047 0.0699 0.0466 0.0233 0.0698 0.0234 0.0932 0.0234h0.0932c0.40784 0 0.61758-0.30296 0.61758-0.60592s-0.20974-0.61758-0.61758-0.61758c-0.23304 0-0.39617 0.11652-0.46609 0.18644-0.0117 0.0233-0.0233 0.0349-0.0349 0.0349-0.0233 0-0.0233-0.16313-0.0233-0.20974 0-0.37288 0.0815-1.2002 0.81567-1.9226 0.12817-0.13983 0.12817-0.16312 0.12817-0.19807 0-0.0815-0.0466-0.12819-0.11653-0.12819-0.11651 0-1.0837 0.88559-1.0837 2.2489z"/><path d="m287.34 682.63v0.0699l0.0117 0.0583v0.1282l0.0468 0.2332 0.0117 0.0466 0.0117 0.0583 0.0233 0.0466 0.0117 0.0466 0.0932 0.1864 0.0466 0.0698 0.0349 0.0349 0.0233 0.0349 0.0349 0.0349 0.0349 0.0232 0.1396 0.0932 0.0349 0.0117 0.0932 0.0234 0.0349 0.0117h0.0932c0.40785 0 0.61759-0.30296 0.61759-0.60592s-0.1981-0.61758-0.61759-0.61758c-0.23304 0-0.39617 0.11652-0.46609 0.18644-0.0117 0.0233-0.0233 0.0349-0.0349 0.0349-0.0233 0-0.0233-0.16313-0.0233-0.20974 0-0.37288 0.0815-1.2002 0.81567-1.9226 0.13983-0.13983 0.13983-0.16312 0.13983-0.19807 0-0.0815-0.0583-0.12819-0.11652-0.12819-0.12818 0-1.0953 0.88559-1.0953 2.2489z"/><path d="m291.85 644v0.1783l0.0112 0.0669 0.0112 0.0557v0.0557l0.0224 0.1114 0.0112 0.0445 0.0223 0.0557 0.0112 0.0445 0.0223 0.0445 0.0112 0.0445 0.0669 0.1335 0.0223 0.0335 0.0335 0.0334 0.0223 0.0334 0.0333 0.0335 0.0223 0.0334 0.0335 0.0223 0.0334 0.0223 0.0335 0.0223 0.0333 0.0223 0.0445 0.0223 0.0335 0.0112 0.0334 0.0112 0.089 0.0224h0.089c0.3901 0 0.59071-0.28979 0.59071-0.57958 0-0.28978-0.20061-0.59071-0.59071-0.59071-0.22292 0-0.37896 0.11145-0.44583 0.17832-0.0112 0.0223-0.0223 0.0335-0.0334 0.0335-0.0223 0-0.0223-0.15603-0.0223-0.20063 0-0.35665 0.078-1.148 0.78021-1.839 0.1226-0.13375 0.1226-0.15603 0.1226-0.18948 0-0.078-0.0445-0.1226-0.11146-0.1226-0.11146 0-1.0366 0.84708-1.0366 2.1511z"/><path d="m289.69 644v0.0669l0.0112 0.0557v0.1226l0.0448 0.2228 0.0112 0.0445 0.0112 0.0557 0.0223 0.0445 0.0112 0.0445 0.0892 0.178 0.0223 0.0335 0.0223 0.0334 0.0335 0.0334 0.0223 0.0335 0.0335 0.0334 0.0335 0.0223 0.1336 0.0892 0.0334 0.0112 0.089 0.0224 0.0334 0.0112h0.089c0.39009 0 0.59071-0.28979 0.59071-0.57958 0-0.28978-0.18948-0.59071-0.59071-0.59071-0.22293 0-0.37897 0.11145-0.44584 0.17832-0.0112 0.0223-0.0223 0.0335-0.0334 0.0335-0.0223 0-0.0223-0.15603-0.0223-0.20063 0-0.35665 0.078-1.148 0.78019-1.839 0.13375-0.13375 0.13375-0.15603 0.13375-0.18948 0-0.078-0.0557-0.1226-0.11146-0.1226-0.12259 0-1.0477 0.84708-1.0477 2.1511z"/><path d="m132.45-486.24 0.0394 0.1176 0.0197 0.0589 0.0197 0.0588v0.0589l0.0197 0.0395 0.0197 0.0589 0.0197 0.0393 0.0393 0.0393 0.0394 0.0786 0.1179 0.1179 0.0391 0.0197 0.0393 0.0196 0.0589 0.0393 0.0589 0.0196 0.0589 0.0197 0.0785 0.0196 0.0395 0.0196h0.0786l0.0391 0.0197h0.0393l0.0393 0.0196h0.1571l0.0393 0.0196h0.1767l0.0589 0.0196h0.7459c0.58909 0 0.74618 0 0.74618 0.37309 0 0.23565-0.21601 0.23565-0.3142 0.23565-0.64801 0-2.2582-0.0588-2.9062-0.0588-0.58912 0-2.003 0.0588-2.5921 0.0588-0.13735 0-0.37309 0-0.37309-0.39274 0-0.216 0.17673-0.216 0.54982-0.216 0.0393 0 0.41239 0 0.7462-0.0393 0.35346-0.0393 0.5302-0.0589 0.5302-0.31419 0-0.0785-0.0395-0.11779-0.0982-0.35343l-2.6117-10.486c-0.19638-0.76586-0.23565-0.92294-1.787-0.92294-0.33383 0-0.53017 0-0.53017-0.37311 0-0.21601 0.17672-0.21601 0.53017-0.21601h9.0722c0.47128 0 0.49091 0 0.60873 0.33385l1.5317 4.2023c0.0982 0.21599 0.0982 0.25529 0.0982 0.2749 0 0.0785-0.0589 0.21603-0.23563 0.21603-0.17674 0-0.19637-0.0983-0.33384-0.41238-0.66763-1.787-1.5316-4.0256-4.9288-4.0256h-1.8263c-0.27494 0-0.3142 0-0.43201 0.0196-0.19638 0.0393-0.2553 0.0589-0.2553 0.19638 0 0.0589 0 0.0983 0.0983 0.45164z"/><path d="m147.96-490.62v0.0197l0.0196 0.0196v0.0589l0.0196 0.0197v0.0589h0.0196v0.0981h0.0196v0.1766c0 0.35349-0.27492 0.53024-0.56946 0.53024-0.19636 0-0.51056-0.1178-0.68729-0.4124-0.0393-0.0983-0.19638-0.70694-0.2749-1.0604-0.13736-0.49094-0.27492-1.0211-0.39275-1.5513l-0.88366-3.515c-0.0785-0.29456-0.92294-1.6495-2.1993-1.6495-1.0015 0-1.2175 0.84439-1.2175 1.571 0 0.90329 0.33383 2.1011 1.0015 3.8292 0.31418 0.80511 0.39272 1.0015 0.39272 1.3942 0 0.88365-0.62836 1.6102-1.6102 1.6102-1.8655 0-2.5921-2.8277-2.5921-3.0044 0-0.19639 0.19637-0.19639 0.23565-0.19639 0.19636 0 0.21599 0.0393 0.31417 0.35347 0.53022 1.8262 1.3157 2.4153 1.9834 2.4153 0.15706 0 0.49092 0 0.49092-0.62837 0-0.49093-0.19638-0.98183-0.33385-1.355-0.78548-2.0815-1.1389-3.1812-1.1389-4.1041 0-1.728 1.2371-2.3171 2.3957-2.3171 0.76584 0 1.4138 0.33382 1.9637 0.88365-0.23562-1.0211-0.49091-1.9833-1.2567-3.0241-0.51056-0.64799-1.2568-1.2174-2.1601-1.2174-0.27488 0-1.1586 0.0588-1.4924 0.82472 0.3142 0 0.56949 0 0.8444 0.23565 0.19638 0.15706 0.39274 0.41236 0.39274 0.78546 0 0.60876-0.53021 0.6873-0.72657 0.6873-0.45165 0-1.0997-0.31419-1.0997-1.2568 0-0.98185 0.86401-1.7084 2.0815-1.7084 2.0226 0 4.0256 1.7673 4.5754 3.9863z"/><path d="m150.45-496.9v-0.059l-0.0196-0.0197v-0.0393l-0.0196-0.0588v-0.0589l-0.0196-0.0588-0.0196-0.0589v-0.0785l-0.0196-0.0589-0.0196-0.0196v-0.0979l-0.0196-0.0196v-0.059l-0.0196-0.0197v-0.1178h-0.0196c0-0.35346 0.27493-0.5302 0.56948-0.5302 0.23564 0 0.58909 0.15707 0.72654 0.54982 0.0393 0.0786 0.70693 2.7295 0.78547 3.0633 0.15707 0.64803 0.51057 2.0226 0.6284 2.5528 0.0785 0.2553 0.62838 1.1586 1.0997 1.5906 0.15706 0.13735 0.72656 0.64801 1.571 0.64801 0.51054 0 0.8051-0.23564 0.82473-0.23564-0.58908-0.0982-1.0211-0.56948-1.0211-1.08 0-0.29454 0.21601-0.66764 0.74619-0.66764 0.51055 0 1.0604 0.43199 1.0604 1.1389 0 0.6873-0.60876 1.2764-1.6102 1.2764-1.2764 0-2.1404-0.96221-2.5135-1.512-0.15707 0.88365-0.86405 1.512-1.787 1.512-0.9033 0-1.2764-0.76584-1.4531-1.1193-0.3338-0.66765-0.58909-1.8262-0.58909-1.8851 0-0.19637 0.19635-0.19637 0.23564-0.19637 0.17674 0 0.21601 0.0197 0.33384 0.45165 0.31419 1.3746 0.70691 2.3172 1.4138 2.3172 0.33382 0 0.60875-0.15706 0.60875-0.9033 0-0.41237-0.0589-0.62839-0.3142-1.6298z"/><path d="m163.67-485.85v0.1375l-0.0196 0.0393v0.0196l-0.0196 0.0393v0.039l-0.0196 0.0196-0.0196 0.0393v0.0196l-0.0196 0.0393-0.0392 0.0392-0.0196 0.0393-0.0393 0.0196-0.0196 0.0197-0.0392 0.0392-0.0393 0.0196-0.0196 0.0196-0.0786 0.0392-0.0196 0.0196h-0.0393l-0.0393 0.0197h-0.0393l-0.0196 0.0196h-0.1767c-0.53018 0-1.0408-0.51055-1.0408-1.0211 0-0.37312 0.27493-0.70694 0.72657-0.70694 0.45166 0 1.0408 0.45165 1.0408 1.0408z"/><path d="m162.18-493.19 0.0393 0.0589 0.0196 0.0393 0.0196 0.039v0.0588l0.0392 0.0786 0.0197 0.0393 0.0196 0.0196v0.0393l0.0196 0.0393 0.0196 0.0196v0.0393l0.0196 0.0196v0.0393l0.0196 0.0589 0.0196 0.0393 0.0196 0.0588 0.0196 0.0393v0.0196l0.0196 0.0393 0.0196 0.0197v0.0393l0.0196 0.0393v0.0393l0.0784 0.1572v0.0589c0.15707 0.37309 0.25527 0.64802 0.25527 1.0211 0 0.88365-0.62837 1.6102-1.5906 1.6102-1.8459 0-2.592-2.8277-2.592-3.0044 0-0.19638 0.19635-0.19638 0.23564-0.19638 0.19636 0 0.21599 0.0393 0.31419 0.35346 0.53019 1.8262 1.3157 2.4153 1.9833 2.4153 0.15706 0 0.4909 0 0.4909-0.62836 0-0.41238-0.13734-0.82476-0.21602-1.0015-0.15705-0.51058-1.0407-2.7884-1.3549-3.6132-0.19634-0.51058-0.45163-1.1586-0.45163-1.571 0-0.92293 0.66764-1.5906 1.6102-1.5906 1.8262 0 2.5528 2.8277 2.5528 3.0044 0 0.19634-0.17671 0.19634-0.23565 0.19634-0.19634 0-0.19634-0.0588-0.29454-0.35344-0.35345-1.2371-0.98185-2.4153-1.9833-2.4153-0.33384 0-0.4713 0.19635-0.4713 0.62837 0 0.49092 0.11781 0.76585 0.56948 1.9637z"/><path d="m172.6-490.64h-0.2749l-0.0588-0.0197h-0.0785l-0.0395-0.0196h-0.0392l-0.0393-0.0196h-0.0196l-0.0196-0.0197h-0.0393l-0.0196-0.0196-0.0196-0.0197h-0.0393l-0.0392-0.0392-0.0196-0.0197-0.0393-0.0196-0.0196-0.0197-0.0589-0.0589c-0.31419-0.29453-0.35346-0.62837-0.35346-0.76582 0-0.45165 0.35346-0.66766 0.72656-0.66766 0.56949 0 1.0997 0.47129 1.0997 1.2371 0 0.9622-0.92292 1.6888-2.3172 1.6888-2.6313 0-5.243-2.7884-5.243-5.5376 0-1.7673 1.139-3.299 3.1812-3.299 2.7884 0 4.4379 2.0618 4.4379 2.2975 0 0.1178-0.11779 0.25528-0.23564 0.25528-0.0983 0-0.13735-0.0393-0.2553-0.19636-1.5513-1.9244-3.6721-1.9244-3.9077-1.9244-1.2371 0-1.7673 0.94259-1.7673 2.1208 0 0.80511 0.39274 2.6902 1.0604 3.9078 0.60874 1.0997 1.6888 1.944 2.7492 1.944 0.66766 0 1.4139-0.25529 1.6888-0.78548z"/><path d="m180.97-490.76h-0.0981l-0.0393-0.0196h-0.0588l-0.0393-0.0196h-0.0393l-0.0589-0.0196-0.1572-0.0784-0.0393-0.0393-0.0393-0.0197-0.0393-0.0196-0.0196-0.0393-0.0393-0.0196-0.0393-0.0393-0.0196-0.0393-0.0589-0.0589-0.0588-0.1179-0.0196-0.0197-0.0196-0.0393v-0.0393l-0.0392-0.0786v-0.1179l-0.0196-0.0393c0-0.27492 0.17674-0.56947 0.60876-0.56947s0.90328 0.33386 0.90328 1.08c0 0.88366-0.84436 1.6888-2.3171 1.6888-2.592 0-3.3186-2.003-3.3186-2.8473 0-1.5317 1.4531-1.8262 2.0226-1.9441 1.0211-0.19635 2.0422-0.41235 2.0422-1.4727 0-0.51058-0.45164-2.1404-2.8081-2.1404-0.27489 0-1.7869 0-2.2386 1.0211 0.7462-0.0983 1.2371 0.49092 1.2371 1.0408 0 0.45165-0.31421 0.68727-0.72657 0.68727-0.51056 0-1.0997-0.41236-1.0997-1.296 0-1.0997 1.1193-1.8852 2.8081-1.8852 3.1812 0 3.9274 2.3564 3.9274 3.2401 0 0.68729-0.3731 1.1782-0.60876 1.4138-0.51054 0.54984-1.08 0.64805-1.944 0.82476-0.70693 0.15705-1.4924 0.29454-1.4924 1.1782 0 0.54985 0.47129 1.7477 2.1993 1.7477 0.49092 0 1.4531-0.13734 1.7477-0.9033z"/><path d="m185.48-486.56v0.0982l-0.0196 0.11779v0.0983l-0.0196 0.0983v0.11779l-0.0196 0.0983-0.0197 0.0786-0.0196 0.0983-0.0196 0.0982-0.0196 0.0785-0.0393 0.0983-0.0196 0.0785-0.0393 0.0785-0.0395 0.0786-0.0393 0.0785-0.0393 0.0589-0.039 0.0785-0.0393 0.0589-0.0588 0.0589-0.0393 0.0589-0.0589 0.0393-0.0589 0.0589-0.0588 0.0393-0.0589 0.0393-0.0588 0.0393-0.0589 0.0393-0.0786 0.0196-0.0589 0.0197-0.0786 0.0196-0.0588 0.0196h-0.1571c-0.68729 0-1.0408-0.53018-1.0408-1.0407 0-0.49093 0.33384-1.0211 1.0408-1.0211 0.39274 0 0.66765 0.19637 0.78546 0.31419 0.0196 0.0393 0.0393 0.0588 0.0588 0.0588 0.0393 0 0.0393-0.27491 0.0393-0.35345 0-0.62839-0.13736-2.0422-1.3746-3.2401-0.23564-0.23564-0.23564-0.2749-0.23564-0.33381 0-0.13734 0.0982-0.21601 0.19634-0.21601 0.21603 0 1.8459 1.4924 1.8459 3.7899z"/><path d="m189.27-486.56v0.21599l-0.0197 0.0983v0.0983l-0.0196 0.11779v0.0983l-0.0197 0.0786-0.0196 0.0983-0.0393 0.0982-0.0196 0.0785-0.0197 0.0983-0.0393 0.0785-0.0395 0.0785-0.0196 0.0786-0.0393 0.0785-0.0393 0.0589-0.0588 0.0785-0.0393 0.0589-0.0394 0.0589-0.1571 0.1571-0.0589 0.0393-0.0588 0.0393-0.0589 0.0393-0.0785 0.0393-0.0588 0.0196-0.0785 0.0197-0.0589 0.0196-0.0786 0.0196h-0.157c-0.68727 0-1.0408-0.53018-1.0408-1.0407 0-0.49093 0.33385-1.0211 1.0408-1.0211 0.39276 0 0.66767 0.19637 0.78548 0.31419 0.0196 0.0393 0.0393 0.0588 0.0589 0.0588 0.0395 0 0.0395-0.27491 0.0395-0.35345 0-0.62839-0.13733-2.0422-1.3746-3.2401-0.23564-0.23564-0.23564-0.2749-0.23564-0.33381 0-0.13734 0.0981-0.21601 0.19636-0.21601 0.21601 0 1.8459 1.4924 1.8459 3.7899z"/><path d="m123.7-485.63v0.3143l0.0197 0.11779 0.0197 0.0983v0.0982l0.0394 0.1966 0.0197 0.0785 0.0393 0.0983 0.0197 0.0785 0.0393 0.0786 0.0197 0.0785 0.0393 0.0786 0.0393 0.0785 0.0393 0.0786 0.0393 0.0588 0.0589 0.0589 0.0391 0.0589 0.0589 0.0589 0.0393 0.0588 0.0589 0.039 0.1767 0.1179 0.0785 0.0393 0.1178 0.0392 0.0785 0.0196 0.0785 0.0197h0.157c0.68731 0 1.0408-0.51056 1.0408-1.0211 0-0.51055-0.35346-1.0408-1.0408-1.0408-0.39272 0-0.66763 0.19636-0.78546 0.3142-0.0197 0.0393-0.0393 0.0589-0.0589 0.0589-0.0393 0-0.0393-0.27491-0.0393-0.35347 0-0.62838 0.13734-2.0226 1.3746-3.2401 0.21599-0.23563 0.21599-0.27489 0.21599-0.33379 0-0.13734-0.0785-0.21603-0.19637-0.21603-0.19635 0-1.8262 1.4924-1.8262 3.7899z"/><path d="m119.89-485.63v0.1178l0.0197 0.0983v0.21599l0.0197 0.0983 0.0197 0.0982 0.0394 0.1966 0.0197 0.0785 0.0197 0.0983 0.0393 0.0785 0.0197 0.0786 0.0393 0.0785 0.0393 0.0786 0.0393 0.0785 0.0393 0.0786 0.0393 0.0588 0.0393 0.0589 0.0588 0.0589 0.0393 0.0589 0.0588 0.0588 0.0588 0.039 0.2352 0.1572 0.0588 0.0196 0.157 0.0392 0.0588 0.0197h0.157c0.68732 0 1.0408-0.51056 1.0408-1.0211 0-0.51055-0.33384-1.0408-1.0408-1.0408-0.39272 0-0.66763 0.19636-0.78546 0.3142-0.0197 0.0393-0.0393 0.0589-0.0588 0.0589-0.0393 0-0.0393-0.27491-0.0393-0.35347 0-0.62838 0.13734-2.0226 1.3746-3.2401 0.23564-0.23563 0.23564-0.27489 0.23564-0.33379 0-0.13734-0.0982-0.21603-0.19636-0.21603-0.21602 0-1.8459 1.4924-1.8459 3.7899z"/></g></g>
   <g transform="matrix(.4625 0 0 -.4625 56.705 365.3)" fill-rule="evenodd" stroke="#000" stroke-miterlimit="10.433" stroke-width="0" ns0:preamble="default_packages.tex" ns0:text="$$doc \\, \\, 1$$\n$$doc \\, \\, 2$$\n$$\\cdots$$\n$$doc \\, \\, N$$\n" xml:space="preserve"><g fill-opacity=".76471"><path d="m298.01 706.46v0.0156h-0.0156v0.0468l-0.0156 0.0156v0.0156h-0.0156v0.0156h-0.0156v0.0156h-0.0156l-0.0156 0.0156h-0.0156l-0.0156 0.0156h-0.078c-0.23406 0-1.7164-0.14044-1.9817-0.15604-0.12483-0.0156-0.21845-0.0936-0.21845-0.29648 0-0.18724 0.14043-0.18724 0.37449-0.18724 0.74899 0 0.7802-0.10923 0.7802-0.26527l-0.0468-0.31208-0.93623-3.6825c-0.28087 0.57735-0.73338 0.99865-1.4356 0.99865-1.81 0-3.7449-2.2782-3.7449-4.5407 0-1.4512 0.85821-2.481 2.0753-2.481l0.0156 0.34329c-0.76459 0-0.98304 0.85821-0.98304 1.4512 0 0.7802 0.49932 2.6839 0.84261 3.4016 0.48372 0.90502 1.1859 1.4824 1.81 1.4824 1.0142 0 1.2327-1.2639 1.2327-1.3575 0-0.0936-0.0312-0.18725-0.0468-0.26527l-0.78019-3.0428c-0.078-0.28087-0.078-0.31208-0.31208-0.57735-0.68657-0.84261-1.3263-1.0923-1.7632-1.0923l-0.0156-0.34329c0.29647 0 1.0767 0.0624 2.0129 1.1703 0.12483-0.65537 0.67097-1.1703 1.42-1.1703 0.54614 0 0.90503 0.35889 1.1547 0.85822 0.26527 0.56174 0.46812 1.498 0.46812 1.5292 0 0.15604-0.14044 0.15604-0.18725 0.15604-0.15604 0-0.17164-0.0624-0.21845-0.28087-0.26527-1.0143-0.54614-1.9193-1.1859-1.9193-0.4213 0-0.46812 0.4057-0.46812 0.71778 0 0.35889 0.0312 0.46811 0.0936 0.73338z"/><path d="m305.36 700.09v0.15604l-0.0156 0.14043-0.0156 0.14044-0.0156 0.14043-0.0312 0.14044-0.0624 0.24966-0.0936 0.24966-0.1248 0.21846-0.0624 0.10922-0.0624 0.0936-0.234 0.2808-0.078 0.078-0.1872 0.156-0.0936 0.0624-0.10923 0.0624-0.10922 0.0624-0.0936 0.0468-0.12483 0.0468-0.10923 0.0468-0.10922 0.0312-0.12483 0.0312-0.12484 0.0312-0.37449 0.0468h-0.12483c-2.0909 0-4.1818-2.2002-4.1818-4.4003 0-1.5292 1.0455-2.6215 2.4966-2.6215l0.0156 0.34329c-0.67096 0-1.3575 0.48372-1.3575 1.6852 0 0.76459 0.4057 2.4342 0.90502 3.23 0.7802 1.1859 1.6696 1.42 2.1065 1.42 0.88942 0 1.3575-0.74899 1.3575-1.6696 0-0.60855-0.31207-2.2314-0.88942-3.2456-0.54613-0.88942-1.4044-1.42-2.1221-1.42l-0.0156-0.34329c2.0753 0 4.1662 2.1377 4.1662 4.4003z"/><path d="m311.76 701.76-0.0468-0.0156h-0.234l-0.0312-0.0156h-0.0312l-0.0312-0.0156h-0.0312l-0.0312-0.0156h-0.0156l-0.0156-0.0156-0.0312-0.0156h-0.0156l-0.0156-0.0156-0.0312-0.0156-0.0312-0.0312-0.0312-0.0156-0.0312-0.0312-0.0312-0.0156c-0.24966-0.23406-0.28087-0.49932-0.28087-0.60855 0-0.37449 0.28087-0.54614 0.57734-0.54614 0.45252 0 0.87382 0.3745 0.87382 0.99865 0 0.74899-0.73338 1.3263-1.8256 1.3263-2.1065 0-4.1818-2.2158-4.1818-4.4003 0-1.4044 0.90502-2.6215 2.5278-2.6215 2.2158 0 3.5265 1.6384 3.5265 1.8257 0 0.0936-0.0936 0.20285-0.18724 0.20285-0.078 0-0.10923-0.0312-0.20285-0.15604-1.2171-1.5292-2.9179-1.5292-3.1052-1.5292-0.98304 0-1.4044 0.76459-1.4044 1.6852 0 0.63976 0.31208 2.1377 0.84261 3.1052 0.48372 0.87382 1.3419 1.5448 2.2002 1.5448 0.53053 0 1.1079-0.20285 1.3263-0.60855z"/><path d="m322.09 705.79v0.2184l-0.0156 0.0156v0.0468l-0.0156 0.0156v0.0312l-0.0156 0.0156h-0.0156v0.0156h-0.0156l-0.0156 0.0156h-0.0312v0.0156h-0.1248v0.0156h-0.1092c-0.96744-0.99865-2.3406-0.99865-2.8399-0.99865v-0.48372c0.31208 0 1.2327 0 2.0441 0.4057v-8.0204c0-0.56174-0.0468-0.74898-1.4512-0.74898h-0.49933v-0.48372c0.54614 0.0468 1.9037 0.0468 2.5278 0.0468 0.62416 0 1.9817 0 2.5278-0.0468v0.48372h-0.49932c-1.4044 0-1.4512 0.17164-1.4512 0.74898z"/><path d="m295.95 685.37v0.0326h-0.0163v0.0488l-0.0163 0.0163v0.0162h-0.0163l-0.0163 0.0163v0.0163h-0.0326v0.0162h-0.0326l-0.0163 0.0163h-0.065c-0.24385 0-1.7882-0.14631-2.0646-0.17882-0.13005-0.0163-0.22759-0.0975-0.22759-0.30888 0-0.19508 0.14631-0.19508 0.39016-0.19508 0.78031 0 0.81283-0.11379 0.81283-0.27636l-0.0488-0.30887-0.9754-3.8366c-0.29262 0.60149-0.76406 1.0404-1.4956 1.0404-1.8858 0-3.9016-2.3735-3.9016-4.7307 0-1.5281 0.89411-2.5848 2.1621-2.5848l0.0163 0.35764c-0.79657 0-1.0242 0.87786-1.0242 1.5119 0 0.81283 0.52021 2.7961 0.87785 3.5439 0.50396 0.94288 1.2355 1.5444 1.8858 1.5444 1.0567 0 1.2843-1.333 1.2843-1.4306 0-0.0975-0.0325-0.17882-0.0488-0.2601l-0.81282-3.17c-0.0813-0.29262-0.0813-0.32513-0.32514-0.6015-0.71529-0.87785-1.3818-1.138-1.837-1.138l-0.0163-0.35764c0.30888 0 1.1217 0.065 2.0971 1.203 0.13006-0.66652 0.69904-1.203 1.4794-1.203 0.56898 0 0.94288 0.3739 1.203 0.87785 0.27636 0.58524 0.48769 1.5769 0.48769 1.6094 0 0.16257-0.1463 0.16257-0.19507 0.16257-0.16257 0-0.17883-0.065-0.2276-0.29262-0.27636-1.0567-0.56898-1.9996-1.2355-1.9996-0.43892 0-0.48769 0.40642-0.48769 0.73155 0 0.39016 0.0325 0.50395 0.0975 0.78031z"/><path d="m303.61 678.76v0.14631l-0.0163 0.16257-0.0162 0.14631-0.0163 0.14631-0.0325 0.13005-0.0325 0.14631-0.0325 0.13005-0.0488 0.13005-0.0488 0.1138-0.065 0.13005-0.065 0.1138-0.065 0.0975-0.065 0.11379-0.2439 0.2925-0.0813 0.0813-0.195 0.1626-0.0975 0.065-0.2276 0.13-0.0975 0.0488-0.13005 0.0488-0.11379 0.0488-0.1138 0.0325-0.2601 0.065-0.39016 0.0489h-0.13006c-2.1784 0-4.3568-2.2922-4.3568-4.5844 0-1.6094 1.0892-2.7311 2.601-2.7311l0.0163 0.35764c-0.69903 0-1.4143 0.4877-1.4143 1.7557 0 0.79658 0.42267 2.536 0.94288 3.3489 0.81283 1.2518 1.7394 1.4956 2.1946 1.4956 0.92662 0 1.4143-0.78031 1.4143-1.7394 0-0.63401-0.32513-2.3247-0.92662-3.3814-0.56898-0.94288-1.4631-1.4794-2.2109-1.4794l-0.0163-0.35764c2.1621 0 4.3405 2.2272 4.3405 4.5844z"/><path d="m310.28 680.48h-0.2277l-0.0488-0.0163h-0.0651l-0.0163-0.0162h-0.0488l-0.0163-0.0163h-0.0325l-0.0162-0.0163h-0.0163l-0.0325-0.0163-0.0163-0.0163h-0.0163l-0.0325-0.0163-0.0326-0.0326-0.0325-0.0163-0.0651-0.0651c-0.26011-0.24385-0.29262-0.52021-0.29262-0.63401 0-0.3739 0.29262-0.55272 0.60149-0.55272 0.47144 0 0.91037 0.39016 0.91037 1.0242 0 0.79657-0.76406 1.3981-1.902 1.3981-2.1946 0-4.3568-2.3084-4.3568-4.5844 0-1.4631 0.94288-2.7311 2.6336-2.7311 2.3084 0 3.674 1.7069 3.674 1.902 0 0.0975-0.0975 0.21133-0.19508 0.21133-0.0813 0-0.1138-0.0325-0.21134-0.16256-1.268-1.5932-3.04-1.5932-3.235-1.5932-1.0242 0-1.4631 0.78032-1.4631 1.7557 0 0.66652 0.32514 2.2272 0.87786 3.2351 0.50395 0.91037 1.3981 1.6094 2.2922 1.6094 0.55272 0 1.1542-0.21133 1.3818-0.65026z"/><path d="m318.32 675.59 1.7232 1.6744c2.536 2.2272 3.4952 3.0888 3.4952 4.7144 0 1.837-1.4468 3.1213-3.4301 3.1213-1.837 0-3.0237-1.4794-3.0237-2.9262 0-0.89411 0.79657-0.89411 0.84534-0.89411 0.27636 0 0.84534 0.19508 0.84534 0.8616 0 0.40641-0.29261 0.82909-0.86159 0.82909-0.13006 0-0.16257 0-0.21134-0.0163 0.3739 1.0567 1.2518 1.6419 2.1946 1.6419 1.4794 0 2.1784-1.3005 2.1784-2.6173 0-1.3005-0.81283-2.5685-1.7069-3.5764l-3.105-3.4626c-0.17882-0.17882-0.17882-0.21134-0.17882-0.58524h5.9987l0.45518 2.8124h-0.40641c-0.065-0.4877-0.17883-1.203-0.34139-1.4468-0.1138-0.13005-1.1867-0.13005-1.5444-0.13005z"/><path d="m301.52 661.15v0.0257l-0.0129 0.0386v0.0643l-0.0129 0.0386-0.0129 0.0257-0.0129 0.0386-0.0258 0.0514-0.0129 0.0386-0.0258 0.0514-0.0257 0.0257-0.0129 0.0257-0.0771 0.0771-0.0257 0.0129-0.0257 0.0257-0.0257 0.0129-0.0257 0.0257-0.0514 0.0258-0.0386 0.0129-0.0257 0.0129-0.0386 0.0129-0.0257 0.0129h-0.0386l-0.0257 0.0129h-0.1029c-0.37324 0-0.68212-0.29601-0.68212-0.66924 0-0.37324 0.30888-0.68212 0.68212-0.68212 0.37323 0 0.68211 0.30888 0.68211 0.68212z"/><path d="m307.22 661.15v0.0643l-0.0129 0.0386v0.0257l-0.0129 0.0386-0.0129 0.0257v0.0386l-0.0258 0.0514-0.0257 0.0386-0.0258 0.0514-0.0257 0.0257-0.0129 0.0257-0.0771 0.0771-0.0257 0.0129-0.0257 0.0257-0.0257 0.0129-0.0257 0.0257-0.0514 0.0258-0.0386 0.0129-0.0514 0.0258-0.0386 0.0129h-0.0386l-0.0257 0.0129h-0.1029c-0.37323 0-0.68212-0.29601-0.68212-0.66924 0-0.37324 0.30889-0.68212 0.68212-0.68212s0.68211 0.30888 0.68211 0.68212z"/><path d="m312.91 661.15v0.0643l-0.0129 0.0386v0.0257l-0.0129 0.0386v0.0257l-0.0129 0.0386-0.0258 0.0514-0.0257 0.0386-0.0258 0.0514-0.0257 0.0257-0.0129 0.0257-0.1157 0.1157-0.0257 0.0129-0.0386 0.0257-0.0514 0.0258-0.0258 0.0129-0.0386 0.0129-0.0257 0.0129-0.0386 0.0129h-0.0257l-0.0386 0.0129h-0.1029c-0.37323 0-0.68211-0.29601-0.68211-0.66924 0-0.37324 0.30888-0.68212 0.68211-0.68212 0.37324 0 0.68212 0.30888 0.68212 0.68212z"/></g><path d="m295.29 651.41v0.0498l-0.0166 0.0167v0.0166l-0.0167 0.0166-0.0167 0.0167v0.0167h-0.0167l-0.0166 0.0166h-0.0166v0.0166h-0.0832v0.0167h-0.0333c-0.25017 0-1.8345-0.1501-2.1014-0.18345-0.15009-0.0167-0.25016-0.10004-0.25016-0.31688 0-0.20012 0.1501-0.20012 0.40026-0.20012 0.80052 0 0.83387-0.11674 0.83387-0.28352l-0.0501-0.33356-0.98396-3.9192c-0.30021 0.60038-0.78385 1.0507-1.5343 1.0507-1.9513 0-4.0193-2.4349-4.0193-4.8532 0-1.551 0.91727-2.6517 2.2181-2.6517l0.0333 0.3669c-0.83388 0-1.0674 0.91727-1.0674 1.551 0 0.83388 0.53369 2.8685 0.91727 3.6357 0.517 0.98398 1.2675 1.5844 1.9346 1.5844 1.0674 0 1.3175-1.3509 1.3175-1.451 0-0.10005-0.0501-0.20013-0.0501-0.28351l-0.83387-3.2521c-0.0834-0.28352-0.0834-0.33355-0.33356-0.6004-0.7338-0.91726-1.4176-1.1841-1.8846-1.1841l-0.0333-0.3669c0.33355 0 1.1674 0.0667 2.1681 1.2508 0.13343-0.70046 0.70047-1.2508 1.501-1.2508 0.58371 0 0.9673 0.38357 1.2341 0.91725 0.28351 0.6004 0.50032 1.601 0.50032 1.6344 0 0.16677-0.15009 0.16677-0.20012 0.16677-0.16678 0-0.18345-0.0667-0.23349-0.3002-0.28352-1.0674-0.58371-2.0513-1.2675-2.0513-0.4503 0-0.50034 0.43361-0.50034 0.76716 0 0.40027 0.0333 0.51701 0.10005 0.78385z"/><path d="m303.16 644.61-0.0167 0.16677v0.1501l-0.0166 0.1501-0.0167 0.15009-0.0333 0.15011-0.0333 0.13341-0.0501 0.13342-0.0501 0.13343-0.0501 0.13342-0.0501 0.11674-0.1334 0.23348-0.0834 0.10005-0.0667 0.11674-0.0834 0.0834-0.0834 0.10005-0.10005 0.0834-0.10004 0.0834-0.10005 0.0834-0.10004 0.0667-0.10005 0.0667-0.23348 0.1334-0.11675 0.0501-0.11674 0.0333-0.13342 0.0501-0.11674 0.0334-0.13343 0.0166-0.13341 0.0333-0.13342 0.0167h-0.28352c-2.2181 0-4.4529-2.3515-4.4529-4.7031 0-1.6344 1.1174-2.8018 2.6684-2.8018v0.3669c-0.70046 0-1.4343 0.51701-1.4343 1.8012 0 0.8172 0.43362 2.6017 0.9673 3.4522 0.8172 1.2842 1.7678 1.5176 2.2348 1.5176 0.96731 0 1.4676-0.78384 1.4676-1.7845 0-0.65042-0.33356-2.3849-0.9673-3.4523-0.58372-0.96729-1.501-1.5343-2.2682-1.5343v-0.3669c2.2014 0 4.4529 2.2848 4.4529 4.7031z" fill-opacity=".76471"/><path d="m310 646.39h-0.1503l-0.0501-0.0166h-0.1334l-0.0333-0.0167h-0.0333l-0.0333-0.0167h-0.0167l-0.0167-0.0166h-0.0334l-0.0167-0.0166h-0.0166l-0.0333-0.0166-0.0333-0.0333-0.0333-0.0166-0.0333-0.0333-0.0166-0.0167-0.0333-0.0166-0.0166-0.0167c-0.26685-0.25016-0.30021-0.53368-0.30021-0.65043 0-0.40026 0.30021-0.58371 0.61707-0.58371 0.48365 0 0.93395 0.40026 0.93395 1.0674 0 0.8172-0.78384 1.4176-1.968 1.4176-2.2348 0-4.4529-2.3682-4.4529-4.7031 0-1.4843 0.9673-2.8018 2.6851-2.8018 2.3849 0 3.7858 1.7678 3.7858 1.9513 0 0.10004-0.10005 0.21681-0.20013 0.21681-0.0834 0-0.11674-0.0334-0.21681-0.1501-1.3175-1.6511-3.1354-1.6511-3.3355-1.6511-1.0507 0-1.4843 0.8172-1.4843 1.8012 0 0.68378 0.33355 2.2848 0.88391 3.3188 0.517 0.95062 1.4343 1.6511 2.3515 1.6511 0.56703 0 1.2008-0.2168 1.4343-0.65042z"/><path d="m328.71 649.66 0.0333 0.13343 0.0166 0.0501 0.0166 0.0667 0.0167 0.0501 0.0334 0.0667 0.0166 0.0501 0.0333 0.0501 0.0166 0.0501 0.0666 0.1002 0.0334 0.0501 0.0333 0.0501 0.0501 0.0501 0.0501 0.0333 0.0333 0.0501 0.0501 0.0333 0.0667 0.0501 0.0501 0.0334 0.1334 0.0666 0.0667 0.0334 0.0834 0.0166 0.0667 0.0334 0.0834 0.0166 0.10005 0.0166 0.0834 0.0333 0.10004 0.0166h0.11674l0.10005 0.0166 0.11674 0.0167h0.1334c0.0834 0 0.26684 0.0166 0.26684 0.33354 0 0.0167 0 0.18346-0.21681 0.18346-0.53368 0-1.1174-0.0501-1.6678-0.0501-0.56703 0-1.1508 0.0501-1.7011 0.0501-0.10005 0-0.3002 0-0.3002-0.33355 0-0.18345 0.16678-0.18345 0.3002-0.18345 0.95061-0.0167 1.1341-0.36692 1.1341-0.73382 0-0.0501-0.0333-0.30019-0.0501-0.35023l-1.8679-7.3715-3.6691 8.6556c-0.13342 0.30019-0.15009 0.31688-0.53368 0.31688h-2.2348c-0.33355 0-0.48365 0-0.48365-0.33355 0-0.18345 0.1501-0.18345 0.46698-0.18345 0.0834 0 1.1341 0 1.1341-0.15011l-2.2348-8.9058c-0.16678-0.65042-0.43362-1.1841-1.7845-1.2341-0.10005 0-0.28352-0.0167-0.28352-0.33355 0-0.11675 0.0834-0.18346 0.21681-0.18346 0.53368 0 1.1007 0.0501 1.6511 0.0501 0.56704 0 1.1674-0.0501 1.7178-0.0501 0.0834 0 0.30019 0 0.30019 0.33356 0 0.16676-0.1501 0.18345-0.33355 0.18345-0.96731 0.0334-1.1007 0.40026-1.1007 0.7338 0 0.11675 0.0166 0.20014 0.0667 0.36692l2.2014 8.7557c0.0667-0.10004 0.0667-0.13341 0.1501-0.30019l4.136-9.7564c0.11675-0.28352 0.16678-0.31687 0.31687-0.31687 0.18347 0 0.18347 0.0501 0.26685 0.35023z" fill-opacity=".76471"/><g fill-rule="evenodd" stroke-miterlimit="10.433"><path d="m349.01 660.93v0.0257l-0.0129 0.0386v0.0643l-0.0129 0.0386-0.0129 0.0257-0.0129 0.0386-0.0258 0.0514-0.0129 0.0386-0.0258 0.0514-0.0257 0.0257-0.0129 0.0257-0.0771 0.0771-0.0257 0.0129-0.0257 0.0257-0.0257 0.0129-0.0257 0.0257-0.0514 0.0258-0.0386 0.0129-0.0257 0.0129-0.0386 0.0129-0.0257 0.0129h-0.0386l-0.0257 0.0129h-0.1029c-0.37324 0-0.68212-0.29601-0.68212-0.66924 0-0.37324 0.30888-0.68212 0.68212-0.68212 0.37323 0 0.68211 0.30888 0.68211 0.68212z"/><path d="m354.71 660.93v0.0643l-0.0129 0.0386v0.0257l-0.0129 0.0386-0.0129 0.0257v0.0386l-0.0258 0.0514-0.0257 0.0386-0.0258 0.0514-0.0257 0.0257-0.0129 0.0257-0.0771 0.0771-0.0257 0.0129-0.0257 0.0257-0.0257 0.0129-0.0257 0.0257-0.0514 0.0258-0.0386 0.0129-0.0514 0.0258-0.0386 0.0129h-0.0386l-0.0257 0.0129h-0.1029c-0.37324 0-0.68213-0.29601-0.68213-0.66924 0-0.37324 0.30889-0.68212 0.68212-0.68212s0.68211 0.30888 0.68211 0.68212z"/><path d="m360.4 660.93v0.0643l-0.0129 0.0386v0.0257l-0.0129 0.0386v0.0257l-0.0129 0.0386-0.0258 0.0514-0.0257 0.0386-0.0258 0.0514-0.0257 0.0257-0.0129 0.0257-0.1157 0.1157-0.0257 0.0129-0.0386 0.0257-0.0514 0.0258-0.0258 0.0129-0.0386 0.0129-0.0257 0.0129-0.0386 0.0129h-0.0257l-0.0386 0.0129h-0.1029c-0.37324 0-0.68212-0.29601-0.68212-0.66924 0-0.37324 0.30888-0.68212 0.68211-0.68212 0.37324 0 0.68212 0.30888 0.68212 0.68212z"/></g></g>
  </g>
  <path d="m213.66 37.746v35.085" fill="none" stroke="#000" stroke-opacity=".19608" stroke-width=".9"/>
  <g transform="matrix(1.4536 0 0 -1.4536 -295.48 1584.6)" fill-rule="evenodd" stroke="#000" stroke-miterlimit="10.433" ns0:preamble="default_packages.tex" ns0:text="$$Answer \\, = \\text{categories}[1]= $$\n" xml:space="preserve"><rect transform="scale(1,-1)" x="277.83" y="-692.55" width="77.225" height="21.528" fill="#ff004c" fill-opacity=".0048309" stroke="#f00" stroke-linecap="square" stroke-linejoin="bevel" stroke-miterlimit="4" stroke-opacity=".44928" stroke-width=".71273"/><g stroke-width="0"><path d="m286.6 685.8-0.0192-0.0384-0.0256-0.0384-0.0256-0.0319-0.0256-0.032-0.0191-0.0319-0.0256-0.032-0.0256-0.0256-0.0192-0.0256-0.0512-0.0512-0.0256-0.0191-0.0192-0.0256-0.0256-0.0192-0.0256-0.0128-0.0256-0.0192-0.0192-0.0128-0.0256-0.0192-0.0256-0.0128-0.0256-6e-3 -0.0512-0.0256-0.0192-6e-3 -0.128-0.03h-0.0256l-0.1024-0.012c-0.0767-6e-3 -0.13426-6e-3 -0.13426-0.12788 0-0.0384 0.0319-0.0704 0.0831-0.0704 0.17263 0 0.37084 0.0192 0.54346 0.0192 0.211 0 0.43478-0.0192 0.63937-0.0192 0.0384 0 0.12149 0 0.12149 0.12148 0 0.0703-0.0575 0.0767-0.1023 0.0767-0.14706 0.0128-0.30051 0.064-0.30051 0.22378 0 0.0767 0.0384 0.14066 0.0895 0.23017l0.48592 0.8184h1.5984l-0.0192 0.19182h-1.4578l1.2532 2.0971 0.2046-2.0971 0.0192-0.19182c0.0128-0.13427 0.10231-0.99742 0.10231-1.0614 0-0.19181-0.32609-0.21099-0.45396-0.21099-0.0895 0-0.15345 0-0.15345-0.12788 0-0.0703 0.0767-0.0703 0.0895-0.0703 0.26214 0 0.53067 0.0192 0.79281 0.0192 0.15984 0 0.56265-0.0192 0.72249-0.0192 0.0384 0 0.11509 0 0.11509 0.12788 0 0.0703-0.064 0.0703-0.14705 0.0703-0.39641 0-0.39641 0.0448-0.4156 0.23017l-0.39002 3.9641c-0.0128 0.12787-0.0128 0.15344-0.12147 0.15344-0.10231 0-0.12788-0.0448-0.16624-0.10869z"/><path d="m290.8 685.45v-0.0128l-6e-3 -6e-3v-0.0188l-6e-3 -0.0192-6e-3 -0.0256-6e-3 -0.0192v-0.0192l-0.012-0.0384v-0.0188l-6e-3 -0.0128v-0.0188l-6e-3 -6e-3v-0.0248l-6e-3 -6e-3v-0.024c0-0.11509 0.0895-0.17263 0.18542-0.17263 0.0767 0 0.19182 0.0511 0.23657 0.17901 6e-3 0.0128 0.0831 0.3133 0.12148 0.46675l0.14066 0.57543c0.0384 0.14066 0.0767 0.28133 0.1087 0.42199 0.0256 0.10869 0.0703 0.29411 0.0831 0.31969 0.0959 0.1982 0.42838 0.77363 1.0358 0.77363 0.28772 0 0.34526-0.23657 0.34526-0.44116 0-0.39641-0.31329-1.2084-0.41558-1.4833-0.0575-0.14706-0.064-0.22378-0.064-0.28773 0-0.30049 0.22379-0.52427 0.52429-0.52427 0.60101 0 0.83118 0.92708 0.83118 0.97823 0 0.064-0.0575 0.064-0.0767 0.064-0.0575 0-0.0575-0.0192-0.0895-0.11509-0.12788-0.42838-0.33886-0.78643-0.65216-0.78643-0.1087 0-0.15345 0.064-0.15345 0.21099 0 0.15345 0.0575 0.30691 0.11509 0.44757 0.12147 0.33886 0.39001 1.0358 0.39001 1.4002 0 0.42199-0.27493 0.67773-0.73528 0.67773-0.57542 0-0.88872-0.4092-0.99102-0.54986-0.0384 0.35166-0.29412 0.54986-0.58823 0.54986-0.2941 0-0.41558-0.24935-0.47952-0.36444-0.10231-0.211-0.19181-0.58822-0.19181-0.61379 0-0.064 0.064-0.064 0.0767-0.064 0.064 0 0.0703 6e-3 0.10869 0.14705 0.10869 0.44755 0.23656 0.75445 0.46674 0.75445 0.12788 0 0.19821-0.0831 0.19821-0.28771 0-0.13427-0.0192-0.2046-0.10231-0.53707z"/><path d="m296.55 687.45-0.0192-6e-3h-0.0319l-0.0128-6e-3h-0.0128l-0.0192-6e-3 -0.0384-0.018-0.0192-6e-3 -0.0128-6e-3 -0.0188-0.0188-0.0256-0.012-6e-3 -0.0128-0.0316-0.0316-6e-3 -0.0128-6e-3 -6e-3 -0.03-0.064v-0.0128l-6e-3 -0.0128v-0.0128l-6e-3 -0.0128v-0.0256c0-0.0895 0.0575-0.18541 0.19821-0.18541 0.14066 0 0.29411 0.1087 0.29411 0.35804 0 0.28133-0.27493 0.54347-0.76086 0.54347-0.83757 0-1.0741-0.64576-1.0741-0.92708 0-0.49232 0.47314-0.58823 0.65216-0.62659 0.33247-0.064 0.66495-0.13426 0.66495-0.48593 0-0.16623-0.14706-0.69691-0.90791-0.69691-0.0959 0-0.58182 0-0.72888 0.33886 0.24296-0.0319 0.40281 0.15346 0.40281 0.33249 0 0.14705-0.10231 0.22378-0.23658 0.22378-0.16623 0-0.35805-0.13428-0.35805-0.42199 0-0.35806 0.36444-0.61381 0.90791-0.61381 1.0358 0 1.2851 0.76726 1.2851 1.055 0 0.23018-0.12148 0.39002-0.19821 0.46035-0.17263 0.17903-0.35805 0.211-0.63937 0.26854-0.23017 0.0512-0.48592 0.0959-0.48592 0.38362 0 0.18542 0.15345 0.56904 0.71609 0.56904 0.15984 0 0.47954-0.0448 0.57544-0.28771z"/><path d="m299.99 687.23v0.0128l6e-3 0.0128v0.0128l6e-3 0.0192v0.0128l6e-3 0.0128 6e-3 0.032 6e-3 0.0319 0.0128 0.0319 6e-3 0.032 6e-3 0.0319 6e-3 0.0128v0.0192l6e-3 0.0128v0.0128l6e-3 0.0128v0.0128l6e-3 0.0128v0.0128l6e-3 6e-3v0.0316l6e-3 6e-3v0.012c0 0.11509-0.0895 0.17263-0.18541 0.17263-0.0767 0-0.19181-0.0512-0.23656-0.17902-0.0192-0.0448-0.31969-1.2532-0.36445-1.4194-0.0448-0.19181-0.0575-0.30689-0.0575-0.42198 0-0.0704 0-0.0831 6e-3 -0.11509-0.14706-0.33247-0.33886-0.53068-0.58823-0.53068-0.50509 0-0.50509 0.46674-0.50509 0.57544 0 0.2046 0.0319 0.45396 0.33246 1.234 0.064 0.19182 0.1023 0.28133 0.1023 0.4092 0 0.28132-0.20459 0.51789-0.51789 0.51789-0.6074 0-0.84396-0.92069-0.84396-0.97824 0-0.064 0.064-0.064 0.0767-0.064 0.064 0 0.0703 0.0128 0.10229 0.11509 0.17263 0.58822 0.42198 0.78643 0.64577 0.78643 0.0575 0 0.15985-6e-3 0.15985-0.21099 0-0.15345-0.0704-0.33248-0.1151-0.44117-0.28132-0.74807-0.35804-1.0422-0.35804-1.2851 0-0.57544 0.42838-0.79921 0.90151-0.79921 0.10869 0 0.4092 0 0.66494 0.44755 0.16624-0.4092 0.6074-0.44755 0.79922-0.44755 0.47953 0 0.75445 0.40279 0.9207 0.78002 0.21738 0.49871 0.42198 1.3491 0.42198 1.6496 0 0.35165-0.17263 0.44756-0.28132 0.44756-0.15985 0-0.31969-0.16624-0.31969-0.3069 0-0.0767 0.0384-0.11509 0.0959-0.16623 0.0703-0.0704 0.23019-0.23657 0.23019-0.54347 0-0.21738-0.18543-0.83757-0.35167-1.1636-0.15984-0.33248-0.38362-0.55626-0.69691-0.55626-0.3005 0-0.47314 0.19182-0.47314 0.54987 0 0.17901 0.0448 0.37722 0.0704 0.46674z"/><path d="m302.97 686.54h0.211l0.0319 6e-3h0.0638l0.0319 6e-3h0.0319l0.064 6e-3 0.0703 6e-3 0.0704 0.0128 0.0703 0.0128 0.064 0.0128 0.0703 0.0128 0.0319 0.0128 0.0384 6e-3 0.0638 0.0256 0.0319 6e-3 0.0256 0.0128c0.44755 0.19181 0.47953 0.56904 0.47953 0.65854 0 0.27494-0.24297 0.53708-0.68414 0.53708v-0.14067c0.30051 0 0.47314-0.18541 0.47314-0.39641 0-0.66494-1.0166-0.66494-1.2787-0.66494h-0.26214c0.24935 0.96545 0.90151 1.0614 1.0677 1.0614v0.14067c-0.70329 0-1.6688-0.6138-1.6688-1.7263 0-0.65216 0.37723-1.1509 1.0038-1.1509 0.9079 0 1.445 0.67133 1.445 0.74806 0 0.0384-0.0384 0.0831-0.0767 0.0831-0.0319 0-0.0447-0.0128-0.0831-0.064-0.50511-0.62658-1.1956-0.62658-1.2724-0.62658-0.49871 0-0.55626 0.53067-0.55626 0.73527 0 0.0767 6e-3 0.27493 0.1023 0.66494z"/><path d="m305.31 685.45v-0.0128l-6e-3 -6e-3v-0.0188l-6e-3 -0.0192-6e-3 -0.0256-6e-3 -0.0192v-0.0192l-0.012-0.0384v-0.0188l-6e-3 -0.0128v-0.0188l-6e-3 -6e-3v-0.0248l-6e-3 -6e-3v-0.024c0-0.11509 0.0895-0.17263 0.18542-0.17263 0.0767 0 0.1918 0.0511 0.23656 0.17901 0.0128 0.0256 0.23018 0.88873 0.25576 1.0038 0.0448 0.21099 0.15984 0.65216 0.1982 0.82479 0.0256 0.0831 0.20459 0.38362 0.35804 0.51789 0.0512 0.0448 0.23656 0.21099 0.5115 0.21099 0.16623 0 0.26215-0.0767 0.26853-0.0767-0.19181-0.0319-0.33247-0.18542-0.33247-0.34527 0-0.10229 0.0703-0.22378 0.24297-0.22378 0.17262 0 0.35165 0.14706 0.35165 0.37084s-0.20461 0.4156-0.53068 0.4156c-0.4156 0-0.69692-0.3133-0.81839-0.48593-0.0512 0.28132-0.27493 0.48593-0.57544 0.48593-0.29412 0-0.41559-0.24936-0.47313-0.36445-0.11509-0.21099-0.19821-0.59462-0.19821-0.61379 0-0.064 0.064-0.064 0.0767-0.064 0.064 0 0.0703 6e-3 0.10869 0.14705 0.10869 0.44756 0.23658 0.75446 0.46675 0.75446 0.10869 0 0.1982-0.0512 0.1982-0.28771 0-0.13428-0.0191-0.2046-0.10229-0.53707z"/><path d="m285.73 678.13v0.043l6e-3 0.043v0.092l6e-3 0.0368v0.043l6e-3 0.043 0.03 0.184 0.0184 0.0614 0.0184 0.0675 0.0184 0.0552 0.0246 0.0613 0.0246 0.049 0.0246 0.0491 0.0246 0.049 0.0306 0.043 0.0306 0.0368 0.0307 0.0368 0.0306 0.0368 0.0307 0.0307 0.0368 0.0246 0.0307 0.0246 0.0368 0.0246 0.0307 0.0246 0.0368 0.0184 0.0368 0.0122 0.0306 0.0122 0.0736 0.0244 0.0307 6e-3 0.0368 6e-3 0.0306 6e-3h0.0613c0.0552 0 0.44159-6e-3 0.65624-0.22692-0.25145-0.0184-0.28826-0.19626-0.28826-0.27599 0-0.15946 0.1104-0.28212 0.28213-0.28212 0.15946 0 0.28212 0.10426 0.28212 0.28826 0 0.41091-0.46611 0.6501-0.93836 0.6501-0.76664 0-1.3248-0.65624-1.3248-1.4106 0-0.78504 0.60105-1.3861 1.3125-1.3861 0.82184 0 1.0181 0.72983 1.0181 0.79116 0 0.0614-0.0613 0.0614-0.0798 0.0614-0.0552 0-0.0675-0.0246-0.0797-0.0614-0.17786-0.56424-0.5765-0.63784-0.80343-0.63784-0.32505 0-0.86476 0.26373-0.86476 1.245z"/><path d="m289.78 677.26v-0.0184l6e-3 -0.0246v-0.0246l6e-3 -0.0184 0.012-0.0492 6e-3 -0.0184 0.0122-0.0246 6e-3 -0.0246 0.0244-0.0368 0.0122-0.0246 0.0488-0.0736 0.0674-0.0674 0.0368-0.0244 0.0246-0.0122 0.0184-0.0122 0.0184-6e-3 0.0246-0.0122 0.0984-0.024h0.0492c0.1288 0 0.50291 0.0859 0.50291 0.57651v0.34345h-0.15331v-0.34345c0-0.34959-0.15333-0.38639-0.2208-0.38639-0.20239 0-0.22692 0.26986-0.22692 0.30053v1.2205c0 0.25759 0 0.49678-0.2208 0.71757-0.23305 0.23919-0.5397 0.33732-0.83409 0.33732-0.50291 0-0.9261-0.28826-0.9261-0.68691 0-0.18399 0.12266-0.28825 0.28212-0.28825 0.17173 0 0.28212 0.12267 0.28212 0.28212 0 0.0736-0.0306 0.26986-0.31278 0.28213 0.16559 0.20852 0.46611 0.27598 0.66237 0.27598 0.30052 0 0.65011-0.23919 0.65011-0.7789v-0.9629c0-0.5765-0.44159-0.78503-0.71758-0.78503-0.30052 0-0.55197 0.21466-0.55197 0.51518 0 0.33732 0.25758 0.84023 1.2696 0.87703v0.12879c-0.31279-0.0184-0.74211-0.0368-1.1285-0.21465-0.45998-0.20852-0.61331-0.52744-0.61331-0.7973 0-0.49065 0.59491-0.64397 0.9813-0.64397 0.40478 0 0.6869 0.24531 0.80343 0.52744z"/><path d="m291.85 679.24h0.87703v0.19012h-0.87703v1.1224h-0.15333c-6e-3 -0.49678-0.19012-1.153-0.78503-1.1776v-0.13492h0.51517v-1.6805c0-0.74824 0.57038-0.82183 0.79117-0.82183 0.43546 0 0.60718 0.43544 0.60718 0.82183v0.34958h-0.15333v-0.33732c0-0.44771-0.18399-0.68077-0.41091-0.68077-0.41092 0-0.41092 0.55198-0.41092 0.65624z"/><path d="m293.85 678.21 6e-3 0.12266c0.0368 0.9077 0.54584 1.061 0.75437 1.061 0.63171 0 0.69304-0.82183 0.69304-1.061h-1.4474l-6e-3 -0.12266h1.705c0.13493 0 0.15333 0 0.15333 0.12266 0 0.60718-0.33119 1.196-1.0978 1.196-0.70531 0-1.2696-0.62557-1.2696-1.3922 0-0.8157 0.64397-1.4045 1.3432-1.4045 0.74823 0 1.0242 0.67464 1.0242 0.79116 0 0.0614-0.049 0.0736-0.0797 0.0736-0.0552 0-0.0675-0.0368-0.0797-0.0859-0.21466-0.62557-0.76664-0.62557-0.82797-0.62557-0.30052 0-0.54585 0.18399-0.68691 0.40478-0.184 0.29439-0.184 0.69918-0.184 0.91996z"/><path d="m296.53 677.61 6e-3 6e-3v0.0602l6e-3 0.0122v0.0182l6e-3 6e-3v0.0122l6e-3 0.0122v0.0122l6e-3 6e-3v0.0122l0.024 0.0488 6e-3 6e-3 0.018 0.0366 0.0122 0.0122 6e-3 0.0122 0.0122 0.0122c0.23305-0.17173 0.48451-0.19013 0.60104-0.19013v0.14106c-0.53358 0-0.53358 0.60718-0.53358 0.74824 0 0.16559 6e-3 0.36185 0.0981 0.50905 0.049 0.0736 0.19012 0.24532 0.43544 0.24532 0.53358 0 0.53358-0.60718 0.53358-0.74824 0-0.16559-6e-3 -0.36185-0.0981-0.50905-0.0491-0.0736-0.19012-0.24532-0.43545-0.24532v-0.14106c0.57038 0 0.99356 0.42319 0.99356 0.8893 0 0.22692-0.0981 0.45385-0.25145 0.58877 0.22078 0.20853 0.44158 0.2392 0.55197 0.2392 0.0122 0 0.043 0 0.0614-6e-3 -0.0674-0.0246-0.0981-0.092-0.0981-0.16559 0-0.10426 0.0797-0.17172 0.17173-0.17172 0.0613 0 0.17786 0.043 0.17786 0.17786 0 0.10425-0.0675 0.30052-0.30666 0.30052-0.12266 0-0.39252-0.0368-0.65011-0.28826-0.25759 0.20239-0.51517 0.22079-0.65011 0.22079-0.57037 0-0.98742-0.41705-0.98742-0.8893 0-0.26985 0.13492-0.49678 0.28825-0.62557-0.0797-0.092-0.19013-0.29439-0.19013-0.50905 0-0.18399 0.0798-0.41704 0.26986-0.53971-0.36798-0.10427-0.56424-0.36799-0.56424-0.60718 0-0.44158 0.60104-0.7789 1.3493-0.7789l6e-3 0.14106c-0.61944 0-1.0365 0.31279-1.0365 0.63784 0 0.27599 0.22694 0.50291 0.49679 0.52132h0.36185c0.52744 0 1.2144 0 1.2144-0.52132 0-0.33118-0.43545-0.63784-1.0365-0.63784l-6e-3 -0.14106c0.7237 0 1.3615 0.31279 1.3615 0.79117 0 0.20853-0.092 0.52131-0.39865 0.69304-0.32505 0.17173-0.68077 0.17173-1.0549 0.17173-0.15333 0-0.41706 0-0.45999 6e-3 -0.19626 0.0184-0.32506 0.20853-0.32506 0.40479z"/><path d="m301.82 678.11-6e-3 0.0675-0.012 0.1472-0.0244 0.135-0.0184 0.0675-0.0492 0.135-0.0246 0.0613-0.0306 0.0613-0.0368 0.0613-0.0306 0.0552-0.0368 0.0552-0.043 0.0552-0.043 0.0491-0.043 0.049-0.0491 0.049-0.049 0.049-0.049 0.0368-0.0552 0.043-0.1104 0.0736-0.0613 0.0306-0.0552 0.0306-0.0614 0.0246-0.0675 0.0184-0.0613 0.0246-0.0675 0.0122-0.0613 0.0122-0.0675 0.0122-0.0736 6e-3h-0.0675l-6e-3 -0.13492c0.26985 0 0.53358-0.13493 0.69304-0.39252 0.15946-0.26372 0.15946-0.61944 0.15946-0.84636 0-0.2208 0-0.55198-0.13493-0.82184-0.13492-0.26985-0.40478-0.44771-0.71144-0.44771-0.26372 0-0.52744 0.12879-0.69303 0.40478-0.15333 0.26985-0.15333 0.64397-0.15333 0.86477 0 0.23305 0 0.56424 0.14719 0.8341 0.16559 0.27598 0.44772 0.40478 0.69304 0.40478l6e-3 0.13492c-0.7605 0-1.3554-0.66236-1.3554-1.4229 0-0.79117 0.63784-1.3738 1.3493-1.3738 0.7421 0 1.3615 0.59491 1.3615 1.3738z"/><path d="m303.02 678.83v0.6685l-0.8525-0.0675v-0.19013c0.42932 0 0.47838-0.043 0.47838-0.33732v-1.6375c0-0.26986-0.0675-0.26986-0.47838-0.26986v-0.19013c0.2392 6e-3 0.52745 0.0184 0.69917 0.0184 0.23919 0 0.52745 0 0.77277-0.0184v0.19013h-0.12879c-0.44772 0-0.45998 0.0675-0.45998 0.28826v0.93222c0 0.60718 0.25758 1.1469 0.71756 1.1469 0.043 0 0.0552 0 0.0675-6e-3 -0.0184-6e-3 -0.14106-0.0797-0.14106-0.23305 0-0.17173 0.1288-0.26373 0.26372-0.26373 0.11041 0 0.26372 0.0736 0.26372 0.26985 0 0.19013-0.19012 0.36799-0.45384 0.36799-0.44158 0-0.66238-0.40478-0.74824-0.6685z"/><path d="m305.47 679.49-0.85863-0.0675v-0.19013c0.39864 0 0.45384-0.0368 0.45384-0.33118v-1.6437c0-0.26986-0.0675-0.26986-0.47838-0.26986v-0.19013c0.19626 6e-3 0.52745 0.0184 0.67464 0.0184 0.21465 0 0.42932-0.0122 0.63784-0.0184v0.19013c-0.40478 0-0.42931 0.0306-0.42931 0.26373z"/><path d="m305.5 680.55v0.0368l-6e-3 0.0184v0.0122l-0.012 0.0368-6e-3 0.0122-6e-3 0.0184-0.018 0.0366-0.0122 0.0184-6e-3 0.0122-0.0426 0.0426-0.0122 6e-3 -0.0122 0.0122-0.0122 6e-3 -0.0122 0.0122-0.0122 6e-3 -0.0184 6e-3 -0.0244 0.012h-0.0184l-0.0122 6e-3 -0.0184 6e-3h-0.0306l-0.0122 6e-3h-0.0184c-0.19012 0-0.32505-0.16559-0.32505-0.32505 0-0.16559 0.13493-0.32506 0.32505-0.32506 0.17173 0 0.32505 0.1288 0.32505 0.32506z"/><path d="m306.76 678.21 6e-3 0.12266c0.0368 0.9077 0.54585 1.061 0.75437 1.061 0.6317 0 0.69303-0.82183 0.69303-1.061h-1.4474l-6e-3 -0.12266h1.705c0.13492 0 0.15332 0 0.15332 0.12266 0 0.60718-0.33118 1.196-1.0978 1.196-0.7053 0-1.2696-0.62557-1.2696-1.3922 0-0.8157 0.64397-1.4045 1.3431-1.4045 0.74824 0 1.0242 0.67464 1.0242 0.79116 0 0.0614-0.049 0.0736-0.0797 0.0736-0.0552 0-0.0675-0.0368-0.0797-0.0859-0.21466-0.62557-0.76663-0.62557-0.82797-0.62557-0.30052 0-0.54584 0.18399-0.6869 0.40478-0.18399 0.29439-0.18399 0.69918-0.18399 0.91996z"/><path d="m310.07 677.99 0.0122-6e-3h0.0184l0.0184-6e-3h0.0184l0.0184-6e-3 0.0984-0.024 0.0246-0.0122 0.0246-6e-3 0.0307-0.0122 0.0246-0.0122 0.0306-0.0122 0.0246-0.0122 0.0246-0.0184 0.0306-0.0122 0.0738-0.0552 0.0246-0.0246 0.0184-0.0246 0.0246-0.0246 0.0368-0.0492 6e-3 -0.0184 6e-3 -0.0122 6e-3 -0.0184 6e-3 -0.0122 6e-3 -0.0184 6e-3 -0.0122 0.012-0.0368v-0.0184l6e-3 -0.0184v-0.0184l6e-3 -0.0184v-0.043c0-0.30666-0.21465-0.55198-0.69303-0.55198-0.51518 0-0.72984 0.34345-0.84636 0.86476-0.0184 0.0797-0.0246 0.10427-0.0859 0.10427-0.0797 0-0.0797-0.043-0.0797-0.15333v-0.80344c0-0.10426 0-0.14719 0.0675-0.14719 0.0307 0 0.0368 6e-3 0.15333 0.12266 0.0122 0.0122 0.0122 0.0246 0.12267 0.14106 0.26372-0.25759 0.53971-0.26372 0.6685-0.26372 0.7053 0 0.98743 0.41091 0.98743 0.84636 0 0.32505-0.18399 0.50905-0.25759 0.58265-0.2024 0.19012-0.44158 0.23919-0.69917 0.28825-0.34345 0.0675-0.74824 0.1472-0.74824 0.50292 0 0.20852 0.15946 0.45998 0.68078 0.45998 0.67464 0 0.7053-0.54584 0.71756-0.73597 6e-3 -0.0552 0.0613-0.0552 0.0736-0.0552 0.0797 0 0.0797 0.0307 0.0797 0.14719v0.61331c0 0.10427 0 0.14719-0.0675 0.14719-0.0307 0-0.043 0-0.12267-0.0736-0.0184-0.0246-0.0797-0.0798-0.10427-0.0981-0.23305 0.17172-0.4845 0.17172-0.5765 0.17172-0.74211 0-0.97516-0.40478-0.97516-0.74824 0-0.21466 0.0981-0.38638 0.26372-0.52131 0.19625-0.15333 0.36185-0.19013 0.80344-0.27599z"/><polygon transform="matrix(.61331 0 0 .61331 101.83 250.45)" points="343.94 702.22 343.94 702.62 342.57 702.62 342.57 692.69 343.94 692.69 343.94 693.08 342.97 693.08 342.97 702.22"/><polygon transform="matrix(.61331 0 0 .61331 101.83 250.45)" points="350.33 693.08 349.36 693.08 349.36 692.69 350.73 692.69 350.73 702.62 349.36 702.62 349.36 702.22 350.33 702.22"/><path d="m323.56 678.79h0.0916l6e-3 6e-3h0.0242l0.0122 6e-3h6e-3l6e-3 6e-3h6e-3l6e-3 6e-3 0.0122 6e-3h6e-3v6e-3l0.012 0.012v6e-3h6e-3v6e-3l6e-3 6e-3v0.012l6e-3 6e-3v0.036c0 0.12266-0.11653 0.12266-0.20239 0.12266h-3.6615c-0.0859 0-0.20239 0-0.20239-0.12266 0-0.12267 0.11653-0.12267 0.20853-0.12267z"/><path d="m323.56 677.61h0.0672l0.0122 6e-3h0.0302l0.0122 6e-3h6e-3l6e-3 6e-3h6e-3l6e-3 6e-3h0.0122l0.012 0.012v6e-3h6e-3l6e-3 6e-3v6e-3l6e-3 6e-3v6e-3h6e-3v0.012l6e-3 6e-3v0.036c0 0.12266-0.11653 0.12266-0.20853 0.12266h-3.6492c-0.092 0-0.20853 0-0.20853-0.12266s0.11653-0.12266 0.20239-0.12266z"/><g fill-rule="evenodd" stroke-miterlimit="10.433"><g transform="matrix(.61526 0 0 -.61526 121.73 1115.6)" stroke="#000" ns0:preamble="default_packages.tex" ns0:text="$$:$$\n" xml:space="preserve"><path d="m306.15 698.92v0.08l-0.01 0.03v0.03l-0.02 0.04-0.01 0.03-0.02 0.04-0.01 0.03-0.02 0.02-0.01 0.02-0.02 0.02-0.01 0.02-0.02 0.02-0.02 0.01-0.04 0.04-0.04 0.02-0.02 0.02-0.03 0.01-0.04 0.02-0.03 0.01h-0.02l-0.03 0.01h-0.02l-0.03 0.01h-0.05c-0.29 0-0.53-0.24-0.53-0.53 0-0.28 0.24-0.52 0.53-0.52s0.52 0.24 0.52 0.52z"/><path d="m306.15 695.7v0.08l-0.01 0.02v0.03l-0.01 0.02-0.01 0.03-0.02 0.04-0.01 0.03-0.01 0.02-0.02 0.02-0.01 0.02-0.02 0.02-0.01 0.02-0.04 0.04-0.02 0.01-0.02 0.02-0.02 0.01-0.02 0.02-0.02 0.01-0.03 0.01-0.04 0.02-0.03 0.01-0.02 0.01h-0.03l-0.02 0.01h-0.08c-0.29 0-0.53-0.24-0.53-0.52 0-0.29 0.24-0.53 0.53-0.53s0.52 0.24 0.52 0.53z"/></g><path d="m311.82 710.45 0.57457 0.55283c0.84561 0.74257 1.1654 1.0354 1.1654 1.572 0 0.61254-0.48245 1.0461-1.1437 1.0461-0.60711 0-1.0082-0.4987-1.0082-0.97565 0-0.30356 0.27103-0.30356 0.28729-0.30356 0.0867 0 0.27645 0.0654 0.27645 0.28722 0 0.14066-0.0976 0.28197-0.28186 0.28197h-0.0759c0.13004 0.34691 0.41739 0.54744 0.73176 0.54744 0.49329 0 0.72636-0.4336 0.72636-0.87799 0-0.42821-0.27102-0.85658-0.56916-1.1925l-1.0353-1.1492c-0.0597-0.0589-0.0597-0.0704-0.0597-0.20053h2.0056l0.14639 0.93769h-0.13003c-0.0271-0.16192-0.0651-0.40104-0.11924-0.47693-0.038-0.0442-0.39569-0.0442-0.51495-0.0442z"/><path d="m313.94 677.11 0.58027 0.5583c0.854 0.74996 1.177 1.0456 1.177 1.5876 0 0.61859-0.48723 1.0565-1.1551 1.0565-0.61313 0-1.0182-0.50368-1.0182-0.98534 0-0.30657 0.27372-0.30657 0.29014-0.30657 0.0875 0 0.27919 0.0661 0.27919 0.29011 0 0.14203-0.0985 0.28477-0.28465 0.28477h-0.0766c0.13133 0.35028 0.42153 0.55285 0.73902 0.55285 0.49819 0 0.73357-0.43795 0.73357-0.88667 0-0.43249-0.27371-0.8651-0.57481-1.2044l-1.0456-1.1606c-0.0603-0.0591-0.0603-0.0707-0.0603-0.20245h2.0255l0.14785 0.94696h-0.13133c-0.0274-0.16349-0.0657-0.40501-0.12042-0.48165-0.0384-0.0441-0.39962-0.0441-0.52006-0.0441z"/></g></g></g>
  <g fill="#000000" font-family="Verdana" font-size="8.1094px" letter-spacing="0px" stroke-width=".26458" word-spacing="0px">
   <text x="226.60779" y="386.12509" style="line-height:1.25" xml:space="preserve"><tspan x="226.60779" y="386.12509" font-family="'Courier New'" font-size="8.1094px" font-weight="bold" stroke-width=".26458" style="font-feature-settings:normal;font-variant-caps:normal;font-variant-ligatures:normal;font-variant-numeric:normal">function</tspan></text>
   <text x="-15.843955" y="324.86795" style="line-height:1.25" xml:space="preserve"><tspan x="-15.843955" y="324.86795" font-family="'Courier New'" font-size="8.1094px" font-weight="bold" stroke-width=".26458" style="font-feature-settings:normal;font-variant-caps:normal;font-variant-ligatures:normal;font-variant-numeric:normal">function</tspan></text>
   <text x="186.61838" y="214.75392" style="line-height:1.25" xml:space="preserve"><tspan x="186.61838" y="214.75392" font-family="'Courier New'" font-size="8.1094px" font-weight="bold" stroke-width=".26458" style="font-feature-settings:normal;font-variant-caps:normal;font-variant-ligatures:normal;font-variant-numeric:normal">function</tspan></text>
   <text x="138.04846" y="497.57059" style="line-height:1.25" xml:space="preserve"><tspan x="138.04846" y="497.57059" font-family="'Courier New'" font-size="8.1094px" font-weight="bold" stroke-width=".26458" style="font-feature-settings:normal;font-variant-caps:normal;font-variant-ligatures:normal;font-variant-numeric:normal">function</tspan></text>
   <text x="243.67506" y="107.72357" style="line-height:1.25" xml:space="preserve"><tspan x="243.67506" y="107.72357" font-family="'Courier New'" font-size="8.1094px" font-weight="bold" stroke-width=".26458" style="font-feature-settings:normal;font-variant-caps:normal;font-variant-ligatures:normal;font-variant-numeric:normal">function</tspan></text>
   <text x="57.329868" y="104.82969" style="line-height:1.25" xml:space="preserve"><tspan x="57.329868" y="104.82969" font-family="'Courier New'" font-size="8.1094px" font-weight="bold" stroke-width=".26458" style="font-feature-settings:normal;font-variant-caps:normal;font-variant-ligatures:normal;font-variant-numeric:normal">class</tspan></text>
  </g>
  <path d="m208.6 231.47v37.647" fill="none" marker-end="url(#l)" stroke="#000" stroke-opacity=".57971" stroke-width=".9"/>
 </g>
</svg>



##Estimating tags prior probabilities

###By hands example

$
\def\q{\unicode{x201C}}
\def\Q{\unicode{x201D}}
$
Let our classes be $C=[\q Physics \Q, \q Economics \Q, \q Religion \Q]$.
If a document falls into $\q Physics \Q$ category it's tagged $0$, if it's from $\q Economics \Q$ category it's tag is $1$
and if it's labeled as $\q Religion \Q$ it's tag is $2$.
Suppose in a corpus there are $15$ documents tagged $0$ ($C[0]=\q Physics \Q$), $25$ documents tagged $1$ (label $C[1]=\q Economics\Q$) and $30$ docs with tag $2$ ($C[2]=\q Religion \Q$).

Estimate $P(\text{tag } k)$ using MLE (emphirical frequencies).

####Solution

$$
\begin{aligned}
P(0)&=P(\q Physics \Q)=\frac{15}{15+25+30}=0.21,\\
P(1)&=P(\q Economics \Q)=\frac{25}{15+25+30}=0.36, \\
P(2)&=P(\q Religion \Q)=\frac{30}{15+25+30}=0.43.
\end{aligned}
$$

Thus if we have no further information the document would be most likely be labeled as $\q Religion \Q$.
Now let's automate this calculation. Suppose documents in the corpus are places in a mixed order. The tags are given in the separate list $tags=[1,0,2,3,\cdots]$.

###Automatic calculation

Write function that calculates $P(k)$.

**Possible input:**
```python
calculate_priors(np.array([0, 1, 1, 1]))
```

**Expected output:**
```
array([0.25, 0.75])
```


In [ ]:
import numpy as np

def calculate_priors(tags):
  # Enter your code here

calculate_priors(np.array([0, 1, 1, 1]))

####Solution

In [ ]:
import numpy as np

def calculate_priors(tags):
  _, counts = np.unique(tags, return_counts=True)
  return counts / len(tags)

calculate_priors(np.array([0, 1, 1, 1]))

array([0.25, 0.75])

##Count Vectorizer

Write your own simple `CountVectorizer` that takes a vector $corpus$ of strings, prepares $vocabulary$ and returns a table $|corpus|\times|vocabulary|$ with counts of words of the vocabulary in each document. 
Your class would be analog of ```sklearn.feature_extraction.text.CountVectorizer```.
The interface of our class should be as follows:
* ```__init__(self, binary = False)``` - defines whether the vectorizer should return simple counts or binary counts ($1$ if the word is present in the document and $0$ otherwise).
* ```fit_transform(self, corpus)``` - feeds vectorizer a list of strings and returns array $|corpus|\times|vocabulary|$ with counts.
* ```get_feature_names(self)``` - returns of lexigraphically sorted lowercase words in the vocabulary.
* ```transform(self, documents)``` - returns array $|corpus|\times|vocabulary|$ with counts.


**Note 1:** you should get rid of punctuation marks and convert your lines into lowercase in all your documents.
Feel free to use the ```re``` module for regular expressions.

**Note 2:** for simplicity return numpy arrays, not sparse matrices as the "real" vectorizer does. Thus you will not ned to call ```toarray()``` function on the result.

Test your vectorizer together with the one from the scikit.learn
**Possible input:**
```python
corpus = ['This is the first document.', 
          'This document is the second document.',
          'And this is the third one.', 
          'Is this the first document?']
document = "second is first document"
```

**Expected output:**
```
--------- STANDARD VECTORIZER -----------
Vocabulary :  ['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']
arr :
 [[0 1 1 1 0 0 1 0 1]
 [0 2 0 1 0 1 1 0 1]
 [1 0 0 1 1 0 1 1 1]
 [0 1 1 1 0 0 1 0 1]]
doc:
 [[0 1 1 1 0 1 0 0 0]]
--------- CUSTOM VECTORIZER -----------
Vocabulary :  ['and' 'document' 'first' 'is' 'one' 'second' 'the' 'third' 'this']
arr:
 [[0 1 1 1 0 0 1 0 1]
 [0 2 0 1 0 1 1 0 1]
 [1 0 0 1 1 0 1 1 1]
 [0 1 1 1 0 0 1 0 1]]
doc:
 [[0 1 1 1 0 1 0 0 0]]
 ```

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

class SimpleCountVectorizer:
  """Vectorizer that counts number of word occurrences in documents"""
  def __init__(self, binary = False):
    # Enter your code here

  def transform(self, documents):
    # Enter your code here

  def fit_transform(self, corpus):
    # Enter your code here

  def get_feature_names(self):
      # Enter your code here

      
corpus = ['This is the first document.', 
          'This document is the second document.',
          'And this is the third one.', 
          'Is this the first document?']
document = "second is first document"

print("--------- STANDARD VECTORIZER -----------")
vectorizer_C = CountVectorizer()
C = vectorizer_C.fit_transform(corpus)
print("Vocabulary : ", vectorizer_C.get_feature_names())
print("arr :\n", C.toarray())
print("doc:\n", vectorizer_C.transform([document]).toarray())


print("--------- CUSTOM VECTORIZER -----------")
SCV = SimpleCountVectorizer(binary=False)
C = SCV.fit_transform(corpus)
print("Vocabulary : ", SCV.get_feature_names())
print("arr:\n", C)
print("doc:\n", SCV.transform([document]))

###Solution

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import re

class SimpleCountVectorizer:
  """Vectorizer that counts number of word occurrences in documents"""
  def __init__(self, binary = False):
    self.binary = binary

  @staticmethod
  def _count_in_doc(document):
    return np.unique(re.sub("[^\w]", " ",  document).lower().split(), return_counts=True)

  def _transform(self, document):
    result = np.zeros_like(self.vocabulary, dtype=int)
    doc_vocabulary, doc_counts = SimpleCountVectorizer._count_in_doc(document)
    result[np.searchsorted(self.vocabulary, doc_vocabulary)] = doc_counts
    return result

  def transform(self, documents):
    if self.binary:
      return np.array([self._transform(d) > 0 for d in documents], dtype=int)
    return np.array([self._transform(d) for d in documents])

  def fit_transform(self, corpus):
    self.vocabulary, _ = SimpleCountVectorizer._count_in_doc(" ".join(corpus))
    return self.transform(corpus)
 
  def get_feature_names(self):
    return self.vocabulary
      
corpus = ['This is the first document.', 
          'This document is the second document.',
          'And this is the third one.', 
          'Is this the first document?']
document = "second is first document"

print("--------- STANDARD VECTORIZER -----------")
vectorizer_C = CountVectorizer()
C = vectorizer_C.fit_transform(corpus)
print("Vocabulary : ", vectorizer_C.get_feature_names())
print("arr :\n", C.toarray())
print("doc:\n", vectorizer_C.transform([document]).toarray())


print("--------- CUSTOM VECTORIZER -----------")
SCV = SimpleCountVectorizer(binary=False)
C = SCV.fit_transform(corpus)
print("Vocabulary : ", SCV.get_feature_names())
print("arr:\n", C)
print("doc:\n", SCV.transform([document]))

--------- STANDARD VECTORIZER -----------
Vocabulary :  ['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']
arr :
 [[0 1 1 1 0 0 1 0 1]
 [0 2 0 1 0 1 1 0 1]
 [1 0 0 1 1 0 1 1 1]
 [0 1 1 1 0 0 1 0 1]]
doc:
 [[0 1 1 1 0 1 0 0 0]]
--------- CUSTOM VECTORIZER -----------
Vocabulary :  ['and' 'document' 'first' 'is' 'one' 'second' 'the' 'third' 'this']
arr:
 [[0 1 1 1 0 0 1 0 1]
 [0 2 0 1 0 1 1 0 1]
 [1 0 0 1 1 0 1 1 1]
 [0 1 1 1 0 0 1 0 1]]
doc:
 [[0 1 1 1 0 1 0 0 0]]


In [ ]:
# Pretty print data from vectorizer
%load_ext google.colab.data_table
import pandas as pd
pd.DataFrame(C, columns=SCV.get_feature_names())

,and,document,first,is,one,second,the,third,this
0,0,1,1,1,0,0,1,0,1
1,0,2,0,1,0,1,1,0,1
2,1,0,0,1,1,0,1,1,1
3,0,1,1,1,0,0,1,0,1


In [ ]:
# Pretty print data from document
%load_ext google.colab.data_table
import pandas as pd
pd.DataFrame(SCV.transform([document]), columns=SCV.get_feature_names())

The google.colab.data_table extension is already loaded. To reload it, use:
  %reload_ext google.colab.data_table


,and,document,first,is,one,second,the,third,this
0,0,1,1,1,0,1,0,0,0


##Parameters estimation

###By hand multinomial model parameters estimation for a single tag

$$
\def\q{\unicode{x201C}}
\def\Q{\unicode{x201D}}
$$

Suppose your vocabulary contains words $\{\q position \Q, \q velocity \Q, \q stocks\Q\}$.
Suppose that in your corpus $3$ documents are tagged as $0$ (label $\q Physics \Q$).

Your 1st document contains 

* word "position" $x_1=0$ times
* word "velocity" $x_2=1$ times
* word "stocks" $x_3=0$ times.

Your 2nd document contains 

* word "position" $x_1=0$ times
* word "velocity" $x_2=1$ times
* word "stocks" $x_3=0$ times.

Your 3rd document contains 

* word "position" $x_1=3$ times
* word "velocity" $x_2=0$ times
* word "stocks" $x_3=0$ times.

Suppose you adopt multinomial model for word occurence

$$
P(\{x_1,x_2,x_3\}|\text{tag }1)=\frac{(x_1+x_2+x_3)!}{x_1!\, x_2!\, x_3!} p_{10}^{x_1}\, p_{20}^{x_2}\, p_{30}^{x_3}.
$$

With given data use emphirical frequency (with smoothing $\alpha=0.1$) to estimate model parameters $p_{10},p_{20},p_{30}$ and verify that $p_{10}+p_{20}+p_{30}=1$.

Note: You can aggregate counts in our three documents to get 

* word "position" $x_1=3$ times
* word "velocity" $x_2=2$ times
* word "stocks" $x_3=0$ times.




####Solution:

We have $V=|vocabulary|=3$ features ($3$ words in the vocabulary) and $n_0=3+2+0=5$ "trials" when we pick words from the "bag" with replacement.


$$
p_{i0}=\frac{n_{i0}+ \alpha}{n_0+V \, \alpha}.
$$

So

$$
\begin{aligned}
p_{10}&=\frac{n_{10}+ \alpha}{n_0+V\, \alpha}=\frac{3+ 0.1}{5+3 \times 0.1}=0.58, \\
p_{20}&=\frac{n_{20}+ \alpha}{n_0+V\, \alpha}=\frac{2+ 0.1}{5+3 \times 0.1}=0.4, \\
p_{30}&=\frac{n_{30}+ \alpha}{n_0+V\, \alpha}=\frac{0+ 0.1}{5+3 \times 0.1}=0.02.
\end{aligned}
$$

We see that

$$
p_{10}+p_{20}+p_{30}=0.58+0.4+0.02=1.
$$



###By hand multivariate Bernoulli model parameters estimation for a single tag

In the previos setup adopt multivariate Bernoulli model. 
Now we need to count not total number of occurencies in all documents but rather number of documents in which the word occurs.

Your 1st document contains

* word "position" is absent (bool frequency is $0$)
* word "velocity" is present (bool frequency is $1$)
* word "stocks" is absent (bool frequency is $0$)

Your 2nd document contains

* word "position" is absent (bool frequency is $0$)
* word "velocity" is present (bool frequency is $1$)
* word "stocks" is absent (bool frequency is $0$)

Your 3rd document contains

* word "position" is present (bool frequency is $1$)
* word "velocity" is absent (bool frequency is $0$)
* word "stocks" is absent (bool frequency is $0$)

We can add boolean frequencies
* word "position" is present in $1$ documents
* word "velocity" is present in $2$ documents
* word "stocks" $x_3=0$ times.

The model 

$$
P(\{x_1,x_2,x_3\}|\text{tag }0)= p_{10}^{x_1}(1-p_{10})^{1-x_1} \times p_{20}^{x_2}(1-p_{20})^{1-x_2} \times p_{30}^{x_3}(1-p_{30})^{1-x_3}.
$$

where $x_i$ is either $0$ or $1$ (i.e. the word is either present in the document or not).

With given data use emphirical frequency (with smoothing $\alpha=0.1$) to estimate model parameters $p_{10},p_{20},p_{30}$.



####Solution

We have $n_0=1+2+0=3$. Therefore

$$
\begin{aligned}
p_{10}&=\frac{n_{10}+ \alpha}{n_0+2 \, \alpha}=\frac{1+ 0.1}{3+2 \times 0.1}=0.34, \\
p_{20}&=\frac{n_{20}+ \alpha}{n_0+2 \, \alpha}=\frac{2+ 0.1}{3+2 \times 0.1}=0.66, \\
p_{30}&=\frac{n_{30}+ \alpha}{n_0+2 \, \alpha}=\frac{0+ 0.1}{3+2 \times 0.1}=0.03.
\end{aligned}
$$



###Automatic parameters estimation

Write function `get_params` that performs multinomial and multivariate parameters estimation.
The function gets array with counts from Vectorizer and a list tags returns vector with $p_i$s for each tag.
Inside it should extract counts for each tag and perform parameters estimation.
Therefore it returns an array of size $|tags|$ elements of which are arrays of parameters.

For Multinomial
$$
p_{ki}=\frac{n_{ki}+\alpha}{n_{k}+|V| \cdot \alpha},
$$
where $n_{ki}$ - number of occurences of word $i$ in the documents with tag $k$, $n_k=\sum_i {n_{ki}}$, $|V|$ - vocabulary size.

For Bernoulli
$$
p_{ki}=\frac{n_{ki}+\alpha}{n_{k}+ 2 \cdot \alpha},
$$
where $n_{ki}$ - number of documents with tag $k$ with word $i$ from vocabulary, $n_k=\sum_i {n_{ki}}$.

You can also set $\alpha$ as an additional parameter (default value is $1$). Verify your code with the example given.

**Possible input:**
```python
get_params(np.array([[1, 2, 3], [3, 4, 5],[3, 4, 5]]), np.array([1, 1, 0]))
```

**Expected output:**
```
array([[0.26666667, 0.33333333, 0.4       ],
       [0.23809524, 0.33333333, 0.42857143]])
```

In [ ]:
import numpy as np

def get_params(counts, tags, alpha=1.0):
  # Enter your code here

get_params(np.array([[1, 2, 3], [3, 4, 5],[3, 4, 5]]), np.array([1, 1, 0]))

####Solution

In [ ]:
import numpy as np

def get_params(counts, tags, alpha=1.0):
  n_ki_list = [np.sum(counts[tags == tag], axis=0) for tag in np.unique(tags)]
  return np.array([(n_ki + alpha) / (np.sum(n_ki) + len(counts[0]) * alpha) for n_ki in n_ki_list])

get_params(np.array([[1, 2, 3], [3, 4, 5],[3, 4, 5]]), np.array([1, 1, 0]))

array([[0.26666667, 0.33333333, 0.4       ],
       [0.23809524, 0.33333333, 0.42857143]])

##Calculating likelihoods in multinomial model

We have a list of estimated parameters for each tag $k$ and a vector of counts for our document. For each tag we need to apply formula

$$
L(C_k|\boldsymbol{x})=P(\boldsymbol{x}|C_k)=\frac{(\sum_i x_i)!}{\prod_{i} x_i!}\prod_i p_{ki}^{x_i}
$$

and obtain a numpy array of size $|tags|$ with likelihood of a document to have certain tag.
Feel free to use `gamma` from `scipy.special`.

**Possible input:**
```python
calculate_likelihoods(np.array([2, 4, 5]),np.array([[0.33, 0.64, 0.03],[0.2, 0.3, 0.4]]))
```

**Expected output:**
```
array([3.07671511e-06, 2.29920768e-02])
```

In [ ]:
from scipy.special import gamma

def calculate_likelihoods(counts, params):
  # Enter your code here

calculate_likelihoods(np.array([2, 4, 5]),np.array([[0.33, 0.64, 0.03],[0.2, 0.3, 0.4]]))

###Solution

In [ ]:
from scipy.special import gamma

def calculate_likelihoods(counts, params):
  return gamma(np.sum(counts) + 1.0) * np.prod(np.array(params)**counts / gamma(counts + 1.0), axis=1)

calculate_likelihoods(np.array([2, 4, 5]),np.array([[0.33, 0.64, 0.03],[0.2, 0.3, 0.4]]))

array([3.07671511e-06, 2.29920768e-02])

##Posterior calculator

Given arrays of likelihoods and prior probabilities we multiply them elementwise and get posterior probabilities.

$$
P(\text{tag }k|document)\propto L(\text{tag }k|document) P(\text{tag } k)  
$$

As a result we again obtain a numpy array of size $|tags|$.

**Possible input:**
```python
calculate_posteriors(np.array([0.6, 0.4]), np.array([0.3, 0.7]))
```

**Expected output:**
```
array([0.18, 0.28])
```

In [ ]:
def calculate_posteriors(likelihoods, priors):
  # Enter your code here

calculate_posteriors(np.array([0.6, 0.4]), np.array([0.3, 0.7]))

###Solution

In [ ]:
def calculate_posteriors(likelihoods, priors):
  return likelihoods * priors

calculate_posteriors(np.array([0.6, 0.4]), np.array([0.3, 0.7]))

array([0.18, 0.28])

##Decision Rule

We will also need code that returns index of the maximum element in an array.
Write `do_decision` function that returns index of thelarges element in a given array.
If few largest elementsare available -- return any.

**Possible input:**
```python
do_decision(np.array([1, 2, 3, 4, 25, 5, 6, 7, 8]))
```

**Expected output:**
```
4
```


In [ ]:
def do_decision(posteriors):
  # Enter your code here

do_decision(np.array([1, 2, 3, 4, 25, 5, 6, 7, 8]))

###Solution

In [ ]:
def do_decision(posteriors):
  return np.argmax(posteriors)

do_decision(np.array([1, 2, 3, 4, 25, 5, 6, 7, 8]))

4

##Collecting everything together: Text classification with our own corpus

Now it's time to put it all together, train and test the classifier. We will have $3$ labels. And a corpus of $3\times 3 = 9$ documents.

In [ ]:
import numpy as np
import re
from scipy.special import gamma

def calculate_priors(tags):
  # Enter your code here

def get_params(counts, tags, alpha=1.0):
  # Enter your code here

def calculate_likelihoods(counts, params):
  # Enter your code here

def calculate_posteriors(likelihoods, priors):
  # Enter your code here

def do_decision(posteriors):
  # Enter your code here


class SimpleCountVectorizer:
  """Vectorizer that counts number of word occurrences in documents"""
  def __init__(self, binary = False):
    # Enter your code here

  def transform(self, documents):
    # Enter your code here

  def fit_transform(self, corpus):
    # Enter your code here

  def get_feature_names(self):
      # Enter your code here


categories=["Physics", "Economy", "Religion"]

# Train set
P=["In Newtonian mechanics, linear momentum, translational momentum, or simply momentum (pl. momenta) is the product of the mass and velocity of an object.",
   "A neutrino (denoted by the Greek letter ν) is a fermion (an elementary particle with half-integer spin) that interacts only via the weak subatomic force and gravity.",
   "In physics, the center of mass of a distribution of mass in space is the unique point where the weighted relative position of the distributed mass sums to zero. This is the point to which a force may be applied to cause a linear acceleration without an angular acceleration."]

E=["Money is any item or verifiable record that is generally accepted as payment for goods and services and repayment of debts, such as taxes, in a particular country or socio-economic context.",
   "Originally money was a form of receipt, representing grain stored in temple granaries in Sumer in ancient Mesopotamia and later in Ancient Egypt. In this first stage of currency, metals were used as symbols to represent value stored in the form of commodities. ",
   "In economics, inflation is a increase in the general price level of goods and services in an economy over a period of time. When the general price level rises, each unit of currency buys fewer goods and services; consequently, inflation reflects a reduction in the purchasing power per unit of money – a loss of real value in the medium of exchange and unit of account within the economy."]

R=["Christianity is an Abrahamic monotheistic religion based on the life and teachings of Jesus of Nazareth. Its adherents, known as Christians, believe that Jesus is the Christ, the Son of God, and the savior of all people, whose coming as the Messiah was prophesied in the Hebrew Bible, called the Old Testament in Christianity, and chronicled in the New Testament.",
   "Traditionalist Catholicism is a set of religious beliefs made up of the customs, traditions, liturgical forms, public, private and group devotions, and presentations of the teaching of the Catholic Church before the Second Vatican Council",
   "Most modern scholars believe that John the Baptist performed a baptism on Jesus, and view it as a historical event to which a high degree of certainty can be assigned."]

tags = [0,0,0,1,1,1,2,2,2]
corpus = P + E + R

# Test set
document = "Physics is economics money force and gravity"

# Train and classify
SCV = SimpleCountVectorizer(binary=False)
SCV.fit(corpus)

priors = calculate_priors(tags)
params = get_params(SCV.toarray(), tags)

doc_counts = SCV.transform(document)
likelihood = calculate_likelihoods(doc_counts, params)
posterior = calculate_posteriors(likelihood, priors)
idx = do_decision(posterior)

# Result
print("Answer:", categories[idx])

###Solution

In [ ]:
import numpy as np
import re
from scipy.special import gamma

def calculate_priors(tags):
  _, counts = np.unique(tags, return_counts=True)
  return counts / len(tags)

def get_params(counts, tags, alpha=1.0):
  n_ki_list = [np.sum(counts[tags == tag], axis=0) for tag in np.unique(tags)]
  return np.array([(n_ki + alpha) / (np.sum(n_ki) + len(counts[0]) * alpha) for n_ki in n_ki_list])

def calculate_likelihoods(counts, params):
  return gamma(np.sum(counts) + 1.0) * np.prod(np.array(params)**counts / gamma(counts + 1.0), axis=1)

def calculate_posteriors(likelihoods, priors):
  return likelihoods * priors

def do_decision(posteriors):
  return np.argmax(posteriors)


class SimpleCountVectorizer:
  """Vectorizer that counts number of word occurrences in documents"""
  def __init__(self, binary = False):
    self.binary = binary

  @staticmethod
  def _count_in_doc(document):
    return np.unique(re.sub("[^\w]", " ",  document).lower().split(), return_counts=True)

  def _transform(self, document):
    result = np.zeros_like(self.vocabulary, dtype=int)
    doc_vocabulary, doc_counts = SimpleCountVectorizer._count_in_doc(document)
    result[np.searchsorted(self.vocabulary, doc_vocabulary)] = doc_counts
    return result

  def transform(self, documents):
    if self.binary:
      return np.array([self._transform(d) > 0 for d in documents], dtype=int)
    return np.array([self._transform(d) for d in documents])

  def fit_transform(self, corpus):
    self.vocabulary, _ = SimpleCountVectorizer._count_in_doc(" ".join(corpus))
    return self.transform(corpus)
 
  def get_feature_names(self):
    return self.vocabulary

categories=["Physics", "Economy", "Religion"]

# Train set
P=["In Newtonian mechanics, linear momentum, translational momentum, or simply momentum (pl. momenta) is the product of the mass and velocity of an object.",
   "A neutrino (denoted by the Greek letter ν) is a fermion (an elementary particle with half-integer spin) that interacts only via the weak subatomic force and gravity.",
   "In physics, the center of mass of a distribution of mass in space is the unique point where the weighted relative position of the distributed mass sums to zero. This is the point to which a force may be applied to cause a linear acceleration without an angular acceleration."]

E=["Money is any item or verifiable record that is generally accepted as payment for goods and services and repayment of debts, such as taxes, in a particular country or socio-economic context.",
   "Originally money was a form of receipt, representing grain stored in temple granaries in Sumer in ancient Mesopotamia and later in Ancient Egypt. In this first stage of currency, metals were used as symbols to represent value stored in the form of commodities. ",
   "In economics, inflation is a increase in the general price level of goods and services in an economy over a period of time. When the general price level rises, each unit of currency buys fewer goods and services; consequently, inflation reflects a reduction in the purchasing power per unit of money – a loss of real value in the medium of exchange and unit of account within the economy."]

R=["Christianity is an Abrahamic monotheistic religion based on the life and teachings of Jesus of Nazareth. Its adherents, known as Christians, believe that Jesus is the Christ, the Son of God, and the savior of all people, whose coming as the Messiah was prophesied in the Hebrew Bible, called the Old Testament in Christianity, and chronicled in the New Testament.",
   "Traditionalist Catholicism is a set of religious beliefs made up of the customs, traditions, liturgical forms, public, private and group devotions, and presentations of the teaching of the Catholic Church before the Second Vatican Council",
   "Most modern scholars believe that John the Baptist performed a baptism on Jesus, and view it as a historical event to which a high degree of certainty can be assigned."]

tags = [0,0,0,1,1,1,2,2,2]
corpus = P + E + R

# Test set
document = "Physics is economics money force and gravity"

# Train and classify
SCV = SimpleCountVectorizer(binary=False)
counts = SCV.fit_transform(corpus)

priors = calculate_priors(tags)
params = get_params(counts, tags)

doc_counts = SCV.transform([document])[0]
likelihood = calculate_likelihoods(doc_counts, params)
posterior = calculate_posteriors(likelihood, priors)
idx = do_decision(posterior)

# Result
print("Answer:", categories[idx])

Answer: Physics


In [ ]:
# Pretty print data from vectorizer
%load_ext google.colab.data_table
import pandas as pd
pd.DataFrame(counts, columns=SCV.get_feature_names())

The google.colab.data_table extension is already loaded. To reload it, use:
  %reload_ext google.colab.data_table


,a,abrahamic,acceleration,accepted,account,adherents,all,an,ancient,and,angular,any,applied,as,assigned,baptism,baptist,based,be,before,beliefs,believe,bible,buys,by,called,can,catholic,catholicism,cause,center,certainty,christ,christianity,christians,chronicled,church,coming,commodities,consequently,...,such,sumer,sums,symbols,taxes,teaching,teachings,temple,testament,that,the,this,time,to,traditionalist,traditions,translational,unique,unit,up,used,value,vatican,velocity,verifiable,via,view,was,weak,weighted,were,when,where,which,whose,with,within,without,zero,ν
0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,1
2,3,0,2,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,5,1,0,3,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,1,1,0
3,1,0,0,1,0,0,0,0,0,2,0,1,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,2,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,1,0,1,0,0,0,1,0,0,1,1,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0
5,4,0,0,0,1,0,0,1,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,5,0,1,0,0,0,0,0,3,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0
6,0,1,0,0,0,1,1,1,0,3,0,0,0,2,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,0,0,1,2,1,1,0,1,0,0,...,0,0,0,0,0,0,1,0,2,1,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
7,1,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0,4,0,0,0,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,3,0,0,0,0,0,0,0,0,1,0,0,0,1,1,1,1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0


In [ ]:
# Pretty print data from document
%load_ext google.colab.data_table
import pandas as pd
pd.DataFrame(doc_counts[np.newaxis, :], columns=SCV.get_feature_names())

The google.colab.data_table extension is already loaded. To reload it, use:
  %reload_ext google.colab.data_table


,a,abrahamic,acceleration,accepted,account,adherents,all,an,ancient,and,angular,any,applied,as,assigned,baptism,baptist,based,be,before,beliefs,believe,bible,buys,by,called,can,catholic,catholicism,cause,center,certainty,christ,christianity,christians,chronicled,church,coming,commodities,consequently,...,such,sumer,sums,symbols,taxes,teaching,teachings,temple,testament,that,the,this,time,to,traditionalist,traditions,translational,unique,unit,up,used,value,vatican,velocity,verifiable,via,view,was,weak,weighted,were,when,where,which,whose,with,within,without,zero,ν
0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
